# **PROYECTO SCHUTZ EN PYTHON PARA CREACIÓN DE DASHBOARD 1 - PARTE 1**

## 1.- IMPORTACIÓN DE LIBRERÍAS PARA EL PROYECTO

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from io import StringIO
from itertools import zip_longest
import io
import requests
from io import BytesIO
import pandas as pd
import chardet
import re
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from gspread_dataframe import set_with_dataframe
from urllib.request import FancyURLopener
from urllib.request import urlopen
import urllib.request
import xml.etree.ElementTree as ET
import numpy as np
import datetime
import ssl
import warnings
from datetime import datetime, timedelta


In [2]:
pd.options.display.max_rows = None
pd.options.display.float_format = '{:.2f}'.format

warnings.filterwarnings("ignore", category=FutureWarning, message="Inferring datetime64.*")

## 2.- LECTURA DE DATAFRAMES

### 2.1 LECTURA "LIBRO MAYOR"

In [3]:
url = 'http://asp.maximise.cl/query.ashx?alias=gsea&user=excel_bd&password=excel_bd&filename=queries\Libro%20Mayor%202023.txt'

# Descargar el archivo XML desde la URL
with urllib.request.urlopen(url) as response:
    xml_data = response.read()

# Parsear el archivo XML
root = ET.fromstring(xml_data)

# Crear una lista de diccionarios con los datos
data = []
for child in root:
    row = {}
    for subchild in child:
        row[subchild.tag] = subchild.text
    data.append(row)

# Convertir la lista de diccionarios en un dataframe de Pandas
df_libro_mayor = pd.DataFrame(data)

# Imprimir el dataframe
df_libro_mayor.sample(5)

,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia
6166,4101010-000,2023,6,2023-06-05T00:00:00-04:00,CIN,105,Cinta rotuladora Dymo,P-1727,I16431038,4048,0,51534
6317,4101010-000,2023,6,2023-06-07T00:00:00-04:00,CIN,206,Bandeja simple para rack,P-1706,C02030694,27771,0,51593
166,3101030-000,2023,2,2023-02-24T00:00:00-03:00,CVT,683,CONTRATO 7000026197-00010,P-1704,76525290-3,0,0,13475
760,4101010-000,2023,1,2023-01-03T00:00:00-03:00,CIN,18,Citofono GT-1D Aiphone,P-1687,C04080111,1344000,0,48667
6595,4101010-000,2023,6,2023-06-13T00:00:00-04:00,CIN,557,Transformador 100-250V CA/18V (para chap,P-1680,T14391829,22986,0,51803


In [4]:
df_libro_mayor.to_csv('archivos_respaldo/df_libro_mayor_1.csv', index = False)

### 2.2 LECTURA "DESPACHOS REALIZADOS"

In [5]:
url = 'http://asp.maximise.cl/query.ashx?alias=gsea&user=excel_bd&password=excel_bd&filename=queries\Despachos%20Realizados.txt'

# Descargar el archivo XML desde la URL
with urllib.request.urlopen(url) as response:
    xml_data = response.read()

# Parsear el archivo XML
root = ET.fromstring(xml_data)

# Crear una lista de diccionarios con los datos
data = []
for child in root:
    row = {}
    for subchild in child:
        row[subchild.tag] = subchild.text
    data.append(row)

# Convertir la lista de diccionarios en un dataframe de Pandas
df_desp_realizados = pd.DataFrame(data)

# Imprimir el dataframe
df_desp_realizados.sample(5)

,TIPO_x0020_MOVIMIENTO,FECHA_x0020_GUIA,NUMERO_x0020_GUIA,PARTIDA_x0020_PRESUPUESTARIA,CODIGO_x0020_PARTE,CANTIDAD,DESCRIPCI_xFFFD_N_x0020_PRODUCTO,UNIDAD_x0020_DE_x0020_MEDIDA,COSTO_x0020_UNITARIO,COSTO_x0020_TOTAL,COD_PROYECTO
3819,GDP,2023-03-07T00:00:00-03:00,49737,1014,T14370414,44,"Conduit Flexible 20mm Metalico con PVC (1/2"")",M,712,31328,P-1683
16920,VCO,2022-04-20T00:00:00-03:00,45056,1027,I15420994,1,Cable Bujia (rollo 50 mts),RLL,9936,9936,P-1675
19485,VCO,2023-01-31T00:00:00-03:00,49213,1015,I16430109,4,Cinta Permace 3/4,UN,625,2500,P-1694
10489,GDP,2022-12-07T00:00:00-03:00,48367,1026,P08200119,2,"Flange Manguito Ø6"" ranurado ANSI B.16.5",UN,11297,22594,P-1664
1381,VCO,2022-06-13T00:00:00-04:00,45764,None,H21570675,1,Protector auditivo reutilizable lavable,UN,220,220,P-ANALISIS PROYE


In [6]:
df_desp_realizados.to_csv('archivos_respaldo/df_desp_realizados_1.csv', index = False)

### 2.3 LECTURA "CODIGOS DE PROYECTO"

In [7]:
url = 'http://asp.maximise.cl/query.ashx?alias=gsea&user=excel_bd&password=excel_bd&filename=queries\codigos%20de%20proyectos.txt'

# Descargar el archivo XML desde la URL
with urllib.request.urlopen(url) as response:
    xml_data = response.read()

# Parsear el archivo XML
root = ET.fromstring(xml_data)

# Crear una lista de diccionarios con los datos
data = []
for child in root:
    row = {}
    for subchild in child:
        row[subchild.tag] = subchild.text
    data.append(row)

# Convertir la lista de diccionarios en un dataframe de Pandas
df_cod_proyecto = pd.DataFrame(data)

# Imprimir el dataframe
df_cod_proyecto.sample(5)

,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE
52,P-1548,Wentu,Inmobiliaria Mirador Los Trapenses S.A.
215,P-1715,CD Lo Echevers,Quezada y Boetsch
75,P-1683,CD BP-A05,EBCO S.A.
266,P-1550,Megacentro Piso 7,EBCO S.A.
176,P-1624,Fachada Oriente PUC,Rodríguez y Rodríguez Construcciones Ltda.


In [8]:
df_cod_proyecto.to_csv('archivos_respaldo/df_cod_proyecto_1.csv', index = False)

### 2.4 LECTURA "PROYECTADOS"

In [9]:
# Se crea un filtro para obtener todos aquellos que tengan valores hasta el MES anterior al mes en curso
import datetime

# Obtener el mes actual de acuerdo a la fecha actual, y el resultado se convierte en el nombre del MES para luego buscar aquel nombre en el archivo.
mes_actual = datetime.date.today().month

print(mes_actual)

if mes_actual == 1:
    mes_actual2 = "ENERO"
elif mes_actual == 2:
    mes_actual2 = "FEBRERO"
elif mes_actual == 3:
    mes_actual2 = "MARZO"
elif mes_actual == 4:
    mes_actual2 = "ABRIL"
elif mes_actual == 5:
    mes_actual2 = "MAYO"
elif mes_actual == 6:
    mes_actual2 = "JUNIO"
elif mes_actual == 7:
    mes_actual2 = "JULIO"
elif mes_actual == 8:
    mes_actual2 = "AGOSTO"
elif mes_actual == 9:
    mes_actual2 = "SEPTIEMBRE"
elif mes_actual == 10:
    mes_actual2 = "OCTUBRE"
elif mes_actual == 11:
    mes_actual2 = "NOVIEMBRE"
else:
    mes_actual2 = "DICIEMBRE"

print(mes_actual2)

6
JUNIO


In [10]:
ssl._create_default_https_context = ssl._create_unverified_context

# Lectura de presupuestos desde dropbox
df_proyectado = pd.read_excel("https://www.dropbox.com/s/m96bbvnctgmaxrz/Proyecci%C3%B3n%20mensual%20%282023%29.xlsx?dl=1",sheet_name=mes_actual2 ,engine="openpyxl")
# OJO !! El link original tiene un 0 para ser visto desde la web y un 1 para descargarlo.....
df_proyectado.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 16361,Unnamed: 16362,Unnamed: 16363,Unnamed: 16364,Unnamed: 16365,Unnamed: 16366,Unnamed: 16367,Unnamed: 16368,Unnamed: 16369,Unnamed: 16370
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4212000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,OBRA,PROYECTO,PROYECTO2,MATERIALES,DESPACHOS A LA FECHA,CONTROL MATERIALES,MANO DE OBRA,PRESENTACION EPP,PROYECCIÓN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,CERRADA,P-1600,TERRANOVA,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,CERRADA,P-1601,CUATRE CASAS,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,CERRADA,P-1615,ECO SANTIAGO,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 2.4-2 LECTURA "PROYECTADOS 2"

In [11]:
# Get the current date
import datetime

fecha_actual = datetime.datetime.now()

# Calculate the previous month
mes_anterior2 = (fecha_actual - timedelta(days=30)).month
print(mes_anterior2)

if mes_anterior2 == 1:
    mes_anterior2 = "ENERO"
elif mes_anterior2 == 2:
    mes_anterior2 = "FEBRERO"
elif mes_anterior2 == 3:
    mes_anterior2 = "MARZO"
elif mes_anterior2 == 4:
    mes_anterior2 = "ABRIL"
elif mes_anterior2 == 5:
    mes_anterior2 = "MAYO"
elif mes_anterior2 == 6:
    mes_anterior2 = "JUNIO"
elif mes_anterior2 == 7:
    mes_anterior2 = "JULIO"
elif mes_anterior2 == 8:
    mes_anterior2 = "AGOSTO"
elif mes_anterior2 == 9:
    mes_anterior2 = "SEPTIEMBRE"
elif mes_anterior2 == 10:
    mes_anterior2 = "OCTUBRE"
elif mes_anterior2 == 11:
    mes_anterior2 = "NOVIEMBRE"
else:
    mes_anterior2 = "DICIEMBRE"

print(mes_anterior2)

# Lectura de presupuestos desde dropbox
df_proyectado_1M = pd.read_excel("https://www.dropbox.com/s/m96bbvnctgmaxrz/Proyecci%C3%B3n%20mensual%20%282023%29.xlsx?dl=1",sheet_name=mes_anterior2 ,engine="openpyxl")
# OJO !! El link original tiene un 0 para ser visto desde la web y un 1 para descargarlo.....
df_proyectado_1M.head(20)

5
MAYO


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 16361,Unnamed: 16362,Unnamed: 16363,Unnamed: 16364,Unnamed: 16365,Unnamed: 16366,Unnamed: 16367,Unnamed: 16368,Unnamed: 16369,Unnamed: 16370
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,OBRA,PROYECTO,PROYECTO2,MATERIALES,DESPACHOS A LA FECHA,CONTROL MATERIALES,MANO DE OBRA,PRESENTACION EPP,PROYECCIÓN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,CERRADA,P-1600,TERRANOVA,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,CERRADA,P-1601,CUATRE CASAS,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,CERRADA,P-1615,ECO SANTIAGO,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 2.5 LECTURA "LIBRO MAYOR AÑOS ANTERIORES"

In [12]:
# Conexión a google drive

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

# Credenciales de GOOGLE API, en archivo .json
credentials = ServiceAccountCredentials.from_json_keyfile_name(
         'keen-extension-358919-9214486a06be.json', scope) # Your json file here

gc = gspread.authorize(credentials)

## SIEMPRE DAR ACCESO A ESTE USUARIO: test-321@keen-extension-358919.iam.gserviceaccount.com

In [13]:
spr = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Fx1nerTu10PZPOwknLcQuri_I6ARat5h7zuq4P3vypg/edit#gid=1097918290')
recipients = spr.sheet1.get_all_values()

# Se el string creado en un dataframe
df_libro_mayor_old = pd.DataFrame(recipients)

# se reasignan los headers en el dataframe.
new_header = df_libro_mayor_old.iloc[0] #grab the first row for the header
df_libro_mayor_old = df_libro_mayor_old[1:] #take the data less the header row
df_libro_mayor_old.columns = new_header #set the header row as the df header

# Se conservan solo las columnas de interés
#df_seguridad_v3 = df_seguridad_v3[["Nº Cédula","Nombres y apellidos","APRUEBA FASE DE CALIFICACIÓN ?","PUNTOS VIGENTES","APRUEBA FASE DE EVALUACIÓN ?","RESULTADO FINAL"]]

print("Las dimensiones del dataframe son :",df_libro_mayor_old.shape)
df_libro_mayor_old.sample(5)

Las dimensiones del dataframe son : (17368, 13)


,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario
3010,4101010-000,2021,3,3/31/21 0:00,CIN,794,Electrodo Soldadura 7018 3/32,P-1632,I15412456,41480,2529,0,
5836,4101010-000,2021,10,10/20/21 0:00,CIN,674,Amarras plasticas 250mm,P-1661,I16430011,43481,13600,0,
13867,4101010-000,2022,10,10/7/22 0:00,CIN,157,Caluga fijador amarra plastica Blanca,P-1682,I16430045,47525,5000,0,
8447,3101030-000,2022,10,10/17/22 0:00,CVT,374,EP N°0008 SERV. INSTALACION DE CORRIENTES DEBILES,P-1677,76071313-9,997,6183850,0,
10786,4101010-000,2022,5,5/23/22 0:00,CIN,449,Detector lineal humo (beam) Honeywell OSI-R-SS,P-1700,S13361386,45502,7233300,0,


In [14]:
df_libro_mayor_old.to_csv('archivos_respaldo/df_libro_mayor_old.csv', mode='a')

### 2.6 LECTURA "PLANILLA RESUMEN - PRESPUESTOS"

In [15]:
# Lectura de presupuestos desde dropbox
df_presupuesto = pd.read_excel("https://www.dropbox.com/s/miow7p9wl7t3jsr/PLANILLA%20RESUMEN.xlsb.xlsx?dl=1",sheet_name= "PROYECTOS" ,engine="openpyxl")
# OJO !! El link original tiene un 0 para ser visto desde la web y un 1 para descargarlo.....
df_presupuesto.head(5)

,PROYECTO,OBRA,RUT,CONSTRUCTORA,FECHA INICIO,FECHA TERMINO,ULTIMO EP PRESENTADO,AVANCE EN OBRA,MONTO EN $,$ POR FACTURAR,...,217465.036046312,217465.036046312.1,217465.036046312.2,217465.036046312.3,0,0.1,0.2,0.3,0.4,0.5
0,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,8111745112.45,3273128441.17,...,1,2.00,3.00,4.00,5.00,6.00,7.00,8.00,9.00,10.00
1,P-1632,Chorrillos Clorox,76.108.978-1,SuKsa S.A,2019-02-04 00:00:00,2019-05-31,NaN,0.99,138107706.03,915642.26,...,217465.04,217465.04,217465.04,217465.04,0.00,0.00,0.00,0.00,0.00,0.00
2,P-1638,Colon,78134800-7,Grevia,2019-08-13 00:00:00,2020-03-27,NaN,1.00,43206282.00,2177592.64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,P-1645,Los 3 Antonios,77.710.830-1,EDICASA,2019-08-31 00:00:00,NaT,NaN,0.98,18150000.00,208732.44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,P-1647,Bodega corrosivos TW,76.564.630-8,Inmobiliaria y Rentas Logisticas S.A,NaN,NaT,NaN,0.58,29482131.00,12437344.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df_presupuesto.to_csv('archivos_respaldo/df_presupuesto.csv', mode='a')

### 2.7 LECTURA "TABLA- PRESPUESTOS"

In [17]:
url = 'http://asp.maximise.cl/query.ashx?alias=gsea&user=excel_bd&password=excel_bd&filename=queries\Presupuesto Proyectos.txt'
response = requests.get(url)
query_result = response.text
soup = BeautifulSoup(response.content, 'xml')
rows = soup.find_all('Table')


data = []
for row in rows:
    values = [cell.text for cell in row.find_all()]
    data.append(values)

df_tabla_prespuesto = pd.DataFrame(data, columns=['CODIGO_PROYECTO','PARTIDA_PRESUPUESTARIA','PRODUCTO','UNIDAD','CANTIDAD','COSTO_UNITARIO','COSTO_TOTAL'])

# Exportación a excel
df_tabla_prespuesto.to_csv('df_tabla_prespuesto.csv', index = False)

df_tabla_prespuesto.head(5)

,CODIGO_PROYECTO,PARTIDA_PRESUPUESTARIA,PRODUCTO,UNIDAD,CANTIDAD,COSTO_UNITARIO,COSTO_TOTAL
0,P-1602,1001,DETECCIÓN-Central de detección Hochiki FNP112...,,1,428490,428490
1,P-1602,1002,DETECCIÓN-Sensor Humo Hochiki ALN-C,,49,20866,1022434
2,P-1602,1003,DETECCIÓN-Sensor de temperatura Hochiki ATJ-EA,,4,23846,95384
3,P-1602,1004,DETECCIÓN-Base Sensores Hochiki HSB-NSA-6,,53,2608,138224
4,P-1602,1005,DETECCIÓN-Palanca Manual Hochiki DCP-AMS,,13,27200,353600


## 3.- TRATAMIENTO DE DATAFRAMES

### 3.1 - TRATAMIENTO 1 - LIBRO MAYOR

#### 3.1.1 - Exploración inicial de los datos

In [18]:
df_libro_mayor.shape

(8116, 12)

In [19]:
df_libro_mayor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8116 entries, 0 to 8115
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Cta_Contable     8116 non-null   object
 1   Año              8116 non-null   object
 2   Mes              8116 non-null   object
 3   Fecha            8091 non-null   object
 4   Tipo_Comp        8116 non-null   object
 5   Num_Comp         8116 non-null   object
 6   Glosa            8116 non-null   object
 7   Primer_Analisis  8113 non-null   object
 8   Referencia       8116 non-null   object
 9   Debito           8116 non-null   object
 10  Credito          8116 non-null   object
 11  Num_Referencia   6525 non-null   object
dtypes: object(12)
memory usage: 761.0+ KB


#### 3.1.2 - Estandarización campo "Debito" y Crédito

In [20]:
# Se crean nuevos campos para trabajar "Debito" y "Credito".

df_libro_mayor['Debito_2'] = pd.to_numeric(df_libro_mayor['Debito'], errors='coerce')
df_libro_mayor['Credito_2'] = pd.to_numeric(df_libro_mayor['Credito'], errors='coerce')

# Se reemplaza el valor de los NoNe y Nan 

df_libro_mayor['Credito_2'] = df_libro_mayor['Credito_2'].fillna(0)

df_libro_mayor.head(5)

,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia,Debito_2,Credito_2
0,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,173,VENTAS DE PROYECTO,P-1677,P-1677,4081983,0,NaN,4081983,0
1,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,174,VENTAS DE PROYECTO,P-1677,P-1677,6464848,0,NaN,6464848,0
2,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,824,VENTAS DE PROYECTO,P-1700,P-1700,4673685,0,10604,4673685,0
3,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,825,VENTAS DE PROYECTO,P-1700,P-1700,1570055,0,10603,1570055,0
4,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,827,VENTAS DE PROYECTO,P-1704,P-1704,3613652,0,10602,3613652,0


#### 3.1.3 - Creación campo Codigo_proyecto

In [21]:
# Se crean una nueva columna para manejar el código del proyecto 

df_libro_mayor['Codigo_proyecto'] = df_libro_mayor['Primer_Analisis']

cols = df_libro_mayor.columns.tolist() # Obtiene el nombre de todas las columnas en una lista
cols.insert(0, cols.pop(cols.index('Codigo_proyecto'))) # Extrae la columna 'Codigo_proyecto', la inserta en la posición 0 y actualiza la lista
df_libro_mayor = df_libro_mayor.reindex(columns=cols) # Reindexa el dataframe con las columnas en el orden actualizado

# Transformación a mayúscula código del proyecto
df_libro_mayor['Codigo_proyecto'] = df_libro_mayor['Codigo_proyecto'].str.upper()

df_libro_mayor.head(5)

,Codigo_proyecto,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia,Debito_2,Credito_2
0,P-1677,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,173,VENTAS DE PROYECTO,P-1677,P-1677,4081983,0,NaN,4081983,0
1,P-1677,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,174,VENTAS DE PROYECTO,P-1677,P-1677,6464848,0,NaN,6464848,0
2,P-1700,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,824,VENTAS DE PROYECTO,P-1700,P-1700,4673685,0,10604,4673685,0
3,P-1700,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,825,VENTAS DE PROYECTO,P-1700,P-1700,1570055,0,10603,1570055,0
4,P-1704,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,827,VENTAS DE PROYECTO,P-1704,P-1704,3613652,0,10602,3613652,0


#### 3.1.4 - Creación de campo total

In [22]:
df_libro_mayor['TOTAL'] = df_libro_mayor['Credito_2'] - df_libro_mayor['Debito_2']
df_libro_mayor.head(5)

,Codigo_proyecto,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia,Debito_2,Credito_2,TOTAL
0,P-1677,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,173,VENTAS DE PROYECTO,P-1677,P-1677,4081983,0,NaN,4081983,0,-4081983
1,P-1677,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,174,VENTAS DE PROYECTO,P-1677,P-1677,6464848,0,NaN,6464848,0,-6464848
2,P-1700,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,824,VENTAS DE PROYECTO,P-1700,P-1700,4673685,0,10604,4673685,0,-4673685
3,P-1700,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,825,VENTAS DE PROYECTO,P-1700,P-1700,1570055,0,10603,1570055,0,-1570055
4,P-1704,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,827,VENTAS DE PROYECTO,P-1704,P-1704,3613652,0,10602,3613652,0,-3613652


#### 3.1.5 - Creación campo tipo_cuenta "basado en el campo cta_Contable"

In [23]:
# Función para categorizar la columna 'Cta_Contable'
def categorizar_cta_contable(cta_contable):
    if cta_contable =="3101030-000" or cta_contable == "3401002-000":
        return 'INGRESO'
    elif cta_contable == "4101010-000":
        return 'COSTO MATERIALES'
    elif cta_contable == "4150002-000":
        return 'COSTO MANO DE OBRA'
    elif cta_contable == "4150002-002":
        return 'COSTO SUPERVISIÓN'
    else:
        return 'OTRA'

# Crear la nueva columna 'Tipo_cuenta' aplicando la función
df_libro_mayor['Tipo_cuenta'] = df_libro_mayor['Cta_Contable'].apply(categorizar_cta_contable)
df_libro_mayor.sample(5)

,Codigo_proyecto,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia,Debito_2,Credito_2,TOTAL,Tipo_cuenta
164,P-1697,3101030-000,2023,2,2023-02-24T00:00:00-03:00,CVT,668,OBRA 18000 ESTOCOLMO,P-1697,96897350-9,0,0,13474,0,0,0,INGRESO
7128,P-1699,4150002-000,2023,3,2023-03-31T00:00:00-03:00,AJC,1694,REMUNERACIONES DIRECTAS PROYECTO,P-1699,16042873,137463,0,NaN,137463,0,-137463,COSTO MANO DE OBRA
5118,P-1718,4101010-000,2023,5,2023-05-05T00:00:00-03:00,CIN,161,"Perno U 3/8"" x 3"" con tuercas y golillas",P-1718,I15412727,3824,0,50931,3824,0,-3824,COSTO MATERIALES
8093,16411600-K,4201003-000,2023,1,2023-01-31T00:00:00-03:00,AJC,1606,COMISIONES COMERCIALES,16411600-K,16411600,897741,0,NaN,897741,0,-897741,OTRA
7437,P-1704,4150002-000,2023,5,2023-05-31T00:00:00-04:00,AJC,1844,REMUNERACIONES DIRECTAS PROYECTO,P-1704,20280168,244497,0,NaN,244497,0,-244497,COSTO MANO DE OBRA


In [24]:
# EXPLORACIÓN DE DATOS

df_libro_mayor_piv = pd.pivot_table(df_libro_mayor, values=['Debito_2','Credito_2','TOTAL'], index=['Tipo_cuenta','Año','Mes'], aggfunc='sum', fill_value=0, margins=True, margins_name='TOTAL')
df_libro_mayor_piv = pd.DataFrame(df_libro_mayor_piv.to_records())

#df_libro_mayor_piv.head(5)
df_libro_mayor_piv.sample(10)

,Tipo_cuenta,Año,Mes,Credito_2,Debito_2,TOTAL
15,COSTO SUPERVISIÓN,2023,5,0,4643261,-4643261
9,COSTO MATERIALES,2023,5,13252105,179084416,-165832311
8,COSTO MATERIALES,2023,4,3247244,150982901,-147735657
12,COSTO SUPERVISIÓN,2023,2,0,4016206,-4016206
16,INGRESO,2023,1,449490833,191247845,258242988
0,COSTO MANO DE OBRA,2023,1,0,72542885,-72542885
21,INGRESO,2023,6,63059808,58891059,4168749
18,INGRESO,2023,3,481925304,199901371,282023933
2,COSTO MANO DE OBRA,2023,3,0,96802066,-96802066
1,COSTO MANO DE OBRA,2023,2,7,82645544,-82645537


#### 3.1.6 - Se eliminan números de proyectos que no deben ser parte del análisis

In [25]:
# Se crea una copia del dataframe original debido que se reduciran las dimensiones
df_libro_mayor2 = df_libro_mayor.copy()

In [26]:
# Se identifican los códigos de proyecto disponibles
print("Proyectos disponibles antes de la reducción : ")
print(df_libro_mayor2["Codigo_proyecto"].unique())
print("=======================================================================")
print("Dimensiones del dataframe antes de la reducción : ")
print(df_libro_mayor2.shape)

Proyectos disponibles antes de la reducción : 
['P-1677' 'P-1700' 'P-1704' 'P-1716' 'P-1709' 'P-1687' 'P-1690' 'P-1686'
 'P-1688' 'P-1682' 'P-1694' 'P-1695' 'P-1703' 'P-1680' 'P-1697' 'P-1714'
 'P-1696' 'P-1722' 'P-1692' 'P-1718' 'P-1721' 'P-1715' 'P-1645' 'P-1699'
 'P-1701' 'P-1675' 'P-1684' 'P-1727' 'P-1723' 'P-1725' 'P-1672' 'P-1713'
 'P-1706' 'P-1726' 'P-1707' 'P-1724' 'P-1712' 'P-1705' 'P-1670' 'P-1691'
 'P-1720' 'P-1719' 'P-1681' 'P-1698' 'P-1708' 'P-PROYECTOS' 'P-1669'
 'P-1667' 'P-1668' 'P-1641' 'P-1693' 'P-1676' 'P-1689' 'P-1664' 'P-1658'
 'P-1660' 'P-1644' 'P-1592' 'P-1648' 'P-1611' 'P-1679' 'P-1642' 'P-1653'
 'P-1673' 'P-1683' 'P-1661' 'P-1612' 'P-1728' 'P-SERVICIOS' 'P-1639'
 'P-1674' 'P1718' 'P-1620' 'P-1603' 'P-1626' 'P-1649' 'P-1685' 'P-1632'
 'P-1643' 'P-1666' 'P-1656' '15314868-6' 'P-1717' 'P-1663' '78540190-5'
 '5555555-9' 'P-1729' 'P-1636' '77092384-0' '76107516-0' '96670840-9'
 'P-OFICINA' 'P-ANALISIS PROYE' 'P-1557' 'P-1591' '18800250-1'
 '13056802-5' '16411600-K' 

##### 3.1.6.1 - Se Se crea un subdataframe para COSTOS SUPERVISIÓN (separando lo que tiene P-OFICINA y cta_contable 4150002-002)

In [27]:
# Se crea un dataframe separado para tratar los que poseen proyecto "P-OFICINA" ya que los costos de supervisión solo aplican al total
df_libro_mayor2_2 = df_libro_mayor2.copy()

# Filtrar el DataFrame
df_libro_mayor2_2 = df_libro_mayor2_2[(df_libro_mayor2_2['Codigo_proyecto'] == 'P-OFICINA') & (df_libro_mayor2_2['Cta_Contable'] == '4150002-002')]

# Mostrar el DataFrame filtrado
df_libro_mayor2_2

,Codigo_proyecto,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia,Debito_2,Credito_2,TOTAL,Tipo_cuenta
7799,P-OFICINA,4150002-002,2023,1,2023-01-31T00:00:00-03:00,AJC,1606,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,2153838,0,NaN,2153838,0,-2153838,COSTO SUPERVISIÓN
7800,P-OFICINA,4150002-002,2023,1,2023-01-31T00:00:00-03:00,AJC,1606,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2182368,0,NaN,2182368,0,-2182368,COSTO SUPERVISIÓN
7801,P-OFICINA,4150002-002,2023,2,2023-02-28T00:00:00-03:00,AJC,1173,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,1833838,0,NaN,1833838,0,-1833838,COSTO SUPERVISIÓN
7802,P-OFICINA,4150002-002,2023,2,2023-02-28T00:00:00-03:00,AJC,1173,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2182368,0,NaN,2182368,0,-2182368,COSTO SUPERVISIÓN
7803,P-OFICINA,4150002-002,2023,3,2023-03-31T00:00:00-03:00,AJC,1694,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,2292504,0,NaN,2292504,0,-2292504,COSTO SUPERVISIÓN
7804,P-OFICINA,4150002-002,2023,3,2023-03-31T00:00:00-03:00,AJC,1694,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2182368,0,NaN,2182368,0,-2182368,COSTO SUPERVISIÓN
7805,P-OFICINA,4150002-002,2023,4,2023-04-30T00:00:00-03:00,AJC,1681,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,2533436,0,NaN,2533436,0,-2533436,COSTO SUPERVISIÓN
7806,P-OFICINA,4150002-002,2023,4,2023-04-30T00:00:00-03:00,AJC,1681,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2562685,0,NaN,2562685,0,-2562685,COSTO SUPERVISIÓN
7807,P-OFICINA,4150002-002,2023,5,2023-05-31T00:00:00-04:00,AJC,1844,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,2292357,0,NaN,2292357,0,-2292357,COSTO SUPERVISIÓN
7808,P-OFICINA,4150002-002,2023,5,2023-05-31T00:00:00-04:00,AJC,1844,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2350904,0,NaN,2350904,0,-2350904,COSTO SUPERVISIÓN


In [28]:
# Elimina los registros NaN en la columna Codigo_proyecto
df_libro_mayor2 = df_libro_mayor2.dropna(subset=['Codigo_proyecto'])

# Define una lista de valores a eliminar
valores_a_eliminar = ['P-PROYECTOS', 'P-SERVICIOS','P-OFICINA', 'P-ANALISIS PROYE']

# Crea una máscara booleana que identifica las filas que contienen los valores a eliminar
mask1 = df_libro_mayor2['Codigo_proyecto'].isin(valores_a_eliminar)

# Crea una máscara booleana que identifica las filas que comienzan con un número
mask2 = df_libro_mayor2['Codigo_proyecto'].str[0].str.isnumeric()

# Une las dos máscaras con el operador OR
mask = mask1 | mask2

# Crea un nuevo dataframe sin las filas que contienen los valores a eliminar
df_libro_mayor2 = df_libro_mayor2[~mask]


# Verifica si el valor 'P1718' o valore similares están presentes en la columna Codigo_proyecto
if 'P1718' in df_libro_mayor2['Codigo_proyecto'].values:
    # Reemplaza el valor 'P1718' por 'P-1718' en la columna Codigo_proyecto
    df_libro_mayor2['Codigo_proyecto'] = df_libro_mayor2['Codigo_proyecto'].replace('P1718', 'P-1718')


In [29]:
# Se identifican los códigos de proyecto disponibles
print("Proyectos disponibles antes de la reducción : ")
print(df_libro_mayor2["Codigo_proyecto"].unique())
print("=======================================================================")
print("Dimensiones del dataframe antes de la reducción : ")
print(df_libro_mayor2.shape)

Proyectos disponibles antes de la reducción : 
['P-1677' 'P-1700' 'P-1704' 'P-1716' 'P-1709' 'P-1687' 'P-1690' 'P-1686'
 'P-1688' 'P-1682' 'P-1694' 'P-1695' 'P-1703' 'P-1680' 'P-1697' 'P-1714'
 'P-1696' 'P-1722' 'P-1692' 'P-1718' 'P-1721' 'P-1715' 'P-1645' 'P-1699'
 'P-1701' 'P-1675' 'P-1684' 'P-1727' 'P-1723' 'P-1725' 'P-1672' 'P-1713'
 'P-1706' 'P-1726' 'P-1707' 'P-1724' 'P-1712' 'P-1705' 'P-1670' 'P-1691'
 'P-1720' 'P-1719' 'P-1681' 'P-1698' 'P-1708' 'P-1669' 'P-1667' 'P-1668'
 'P-1641' 'P-1693' 'P-1676' 'P-1689' 'P-1664' 'P-1658' 'P-1660' 'P-1644'
 'P-1592' 'P-1648' 'P-1611' 'P-1679' 'P-1642' 'P-1653' 'P-1673' 'P-1683'
 'P-1661' 'P-1612' 'P-1728' 'P-1639' 'P-1674' 'P-1620' 'P-1603' 'P-1626'
 'P-1649' 'P-1685' 'P-1632' 'P-1643' 'P-1666' 'P-1656' 'P-1717' 'P-1663'
 'P-1729' 'P-1636' 'P-1557' 'P-1591']
Dimensiones del dataframe antes de la reducción : 
(7596, 17)


#### 3.1.7 - Se crea un nuevo campo de fecha basado en la fecha original

In [30]:
df_libro_mayor2['Fecha2'] = pd.to_datetime(df_libro_mayor2['Año'].astype(str) + '-' + df_libro_mayor2['Mes'].astype(str) + '-1')
df_libro_mayor2.sample(5)

,Codigo_proyecto,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia,Debito_2,Credito_2,TOTAL,Tipo_cuenta,Fecha2
1952,P-1690,4101010-000,2023,2,2023-02-07T00:00:00-03:00,CIN,134,Cable UTP CAT5E Unifilar Exterior,P-1690,T14382566,363560,0,49327,363560,0,-363560,COSTO MATERIALES,2023-02-01
112,P-1690,3101030-000,2023,1,2023-01-31T00:00:00-03:00,CVT,1287,"UF 126.9833 X T/C 35.287,50",P-1690,76210676-0,0,0,13398,0,0,0,INGRESO,2023-01-01
1100,P-1714,4101010-000,2023,1,2023-01-10T00:00:00-03:00,CIN,223,DVR 16CH Hikvision Turbo HDTVI +16 IP 2H,P-1714,S09232497,281800,0,48847,281800,0,-281800,COSTO MATERIALES,2023-01-01
2792,P-1721,4101010-000,2023,3,2023-03-08T00:00:00-03:00,CIN,259,"Pulsador Panico o Salida ""Emergency""",P-1721,S11290922,10432,0,49773,10432,0,-10432,COSTO MATERIALES,2023-03-01
3595,P-1724,4101010-000,2023,4,2023-04-04T00:00:00-03:00,CIN,70,Gabinete Extintor CO2 5 Kg sobreponer pu,P-1724,P08180170,150000,0,50196,150000,0,-150000,COSTO MATERIALES,2023-04-01


##### 3.1.7.1 - Se crea un nuevo campo de fecha basado en la fecha original para  COSTOS SUPERVISIÓN

In [31]:
df_libro_mayor2_2['Fecha2'] = pd.to_datetime(df_libro_mayor2_2['Año'].astype(str) + '-' + df_libro_mayor2_2['Mes'].astype(str) + '-1')
df_libro_mayor2_2.sample(5)

,Codigo_proyecto,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia,Debito_2,Credito_2,TOTAL,Tipo_cuenta,Fecha2
7805,P-OFICINA,4150002-002,2023,4,2023-04-30T00:00:00-03:00,AJC,1681,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,2533436,0,NaN,2533436,0,-2533436,COSTO SUPERVISIÓN,2023-04-01
7806,P-OFICINA,4150002-002,2023,4,2023-04-30T00:00:00-03:00,AJC,1681,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2562685,0,NaN,2562685,0,-2562685,COSTO SUPERVISIÓN,2023-04-01
7808,P-OFICINA,4150002-002,2023,5,2023-05-31T00:00:00-04:00,AJC,1844,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2350904,0,NaN,2350904,0,-2350904,COSTO SUPERVISIÓN,2023-05-01
7801,P-OFICINA,4150002-002,2023,2,2023-02-28T00:00:00-03:00,AJC,1173,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,1833838,0,NaN,1833838,0,-1833838,COSTO SUPERVISIÓN,2023-02-01
7799,P-OFICINA,4150002-002,2023,1,2023-01-31T00:00:00-03:00,AJC,1606,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,2153838,0,NaN,2153838,0,-2153838,COSTO SUPERVISIÓN,2023-01-01


#### 3.1.8 - Se realiza pivote final del dataframe

In [32]:
# Se exploran las columnas:
df_libro_mayor2.columns

Index(['Codigo_proyecto', 'Cta_Contable', 'Año', 'Mes', 'Fecha', 'Tipo_Comp',
       'Num_Comp', 'Glosa', 'Primer_Analisis', 'Referencia', 'Debito',
       'Credito', 'Num_Referencia', 'Debito_2', 'Credito_2', 'TOTAL',
       'Tipo_cuenta', 'Fecha2'],
      dtype='object')

In [33]:
# crear pivot table sin margenes

df_libro_mayor2_piv = pd.pivot_table(df_libro_mayor2, values=['Debito_2','Credito_2','TOTAL'], index=['Codigo_proyecto','Tipo_cuenta','Fecha2','Año','Mes'], aggfunc='sum', fill_value=0, margins=True, margins_name='TOTAL')
df_libro_mayor2_piv = pd.DataFrame(df_libro_mayor2_piv.to_records())

df_libro_mayor2_piv.sample(5)

,Codigo_proyecto,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
387,P-1709,INGRESO,2023-01-01,2023,1,11346536,0,11346536
434,P-1715,COSTO MANO DE OBRA,2023-02-01,2023,2,0,2880459,-2880459
143,P-1686,INGRESO,2023-04-01,2023,4,11356986,0,11356986
474,P-1718,COSTO MATERIALES,2023-04-01,2023,4,0,5213569,-5213569
237,P-1695,INGRESO,2023-05-01,2023,5,12859444,0,12859444


In [34]:
# Estas son las dimensiones del dataframe pivoteado:
df_libro_mayor2_piv.shape

(557, 8)

##### 3.1.8.1 - Se realiza pivote final del dataframe paera COSTO SUPERVISIÓN

In [35]:
# crear pivot table sin margenes

df_libro_mayor2_2_piv = pd.pivot_table(df_libro_mayor2_2, values=['Debito_2','Credito_2','TOTAL'], index=['Codigo_proyecto','Tipo_cuenta','Fecha2','Año','Mes'], aggfunc='sum', fill_value=0, margins=True, margins_name='TOTAL')
df_libro_mayor2_2_piv = pd.DataFrame(df_libro_mayor2_2_piv.to_records())

df_libro_mayor2_2_piv = df_libro_mayor2_2_piv[df_libro_mayor2_2_piv['Codigo_proyecto'] != 'TOTAL']

# Se reemplaza el valor P-OFICINA por el valor TOTAL con lo cual se utilizará para unirlo al dataframe que crea las filas para el cálculo de los totales
df_libro_mayor2_2_piv['Codigo_proyecto'] = df_libro_mayor2_2_piv['Codigo_proyecto'].replace('P-OFICINA', 'TOTAL')


df_libro_mayor2_2_piv

,Codigo_proyecto,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
0,TOTAL,COSTO SUPERVISIÓN,2023-01-01,2023,1,0,4336206,-4336206
1,TOTAL,COSTO SUPERVISIÓN,2023-02-01,2023,2,0,4016206,-4016206
2,TOTAL,COSTO SUPERVISIÓN,2023-03-01,2023,3,0,4474872,-4474872
3,TOTAL,COSTO SUPERVISIÓN,2023-04-01,2023,4,0,5096121,-5096121
4,TOTAL,COSTO SUPERVISIÓN,2023-05-01,2023,5,0,4643261,-4643261


#### 3.1.9 - Se crea subdataframe para proyecto que considera el total con nombre "TOTAL", que se detalla por tipo_cuenta

In [36]:
df_libro_mayor_piv2 = pd.pivot_table(df_libro_mayor2_piv, values=['Debito_2','Credito_2','TOTAL'], index=['Tipo_cuenta','Fecha2','Año','Mes'], aggfunc='sum', fill_value=0)
df_libro_mayor_piv2 = pd.DataFrame(df_libro_mayor_piv2.to_records())

# Agregamos la columna Codigo_proyecto con valor "TOTAL" en la primera columna
df_libro_mayor_piv2.insert(loc=0, column='Codigo_proyecto', value='TOTAL')

# Eliminamos los registros con Tipo_cuenta igual a "TOTAL"
df_libro_mayor_piv2 = df_libro_mayor_piv2[df_libro_mayor_piv2['Tipo_cuenta'] != 'TOTAL']
df_libro_mayor_piv2 = df_libro_mayor_piv2[df_libro_mayor_piv2['Tipo_cuenta'] != '']

df_libro_mayor_piv2

,Codigo_proyecto,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
0,TOTAL,COSTO MANO DE OBRA,2023-01-01,2023,1,0,72482994,-72482994
1,TOTAL,COSTO MANO DE OBRA,2023-02-01,2023,2,0,82645544,-82645544
2,TOTAL,COSTO MANO DE OBRA,2023-03-01,2023,3,0,96760244,-96760244
3,TOTAL,COSTO MANO DE OBRA,2023-04-01,2023,4,0,111885468,-111885468
4,TOTAL,COSTO MANO DE OBRA,2023-05-01,2023,5,0,114373715,-114373715
5,TOTAL,COSTO MATERIALES,2023-01-01,2023,1,2565889,125972929,-123407040
6,TOTAL,COSTO MATERIALES,2023-02-01,2023,2,3348963,83658197,-80309234
7,TOTAL,COSTO MATERIALES,2023-03-01,2023,3,2764412,88061320,-85296908
8,TOTAL,COSTO MATERIALES,2023-04-01,2023,4,3247244,149731515,-146484271
9,TOTAL,COSTO MATERIALES,2023-05-01,2023,5,8533707,178069095,-169535388


##### 3.1.9.1 - Se anexa el dataframe creado con los totales con el dataframe creado a partir del punto 3.1.6.1 que contiene los costos de supervisión

In [37]:
df_libro_mayor_piv2 = pd.concat([df_libro_mayor_piv2, df_libro_mayor2_2_piv], ignore_index=True)
df_libro_mayor_piv2

,Codigo_proyecto,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
0,TOTAL,COSTO MANO DE OBRA,2023-01-01,2023,1,0,72482994,-72482994
1,TOTAL,COSTO MANO DE OBRA,2023-02-01,2023,2,0,82645544,-82645544
2,TOTAL,COSTO MANO DE OBRA,2023-03-01,2023,3,0,96760244,-96760244
3,TOTAL,COSTO MANO DE OBRA,2023-04-01,2023,4,0,111885468,-111885468
4,TOTAL,COSTO MANO DE OBRA,2023-05-01,2023,5,0,114373715,-114373715
5,TOTAL,COSTO MATERIALES,2023-01-01,2023,1,2565889,125972929,-123407040
6,TOTAL,COSTO MATERIALES,2023-02-01,2023,2,3348963,83658197,-80309234
7,TOTAL,COSTO MATERIALES,2023-03-01,2023,3,2764412,88061320,-85296908
8,TOTAL,COSTO MATERIALES,2023-04-01,2023,4,3247244,149731515,-146484271
9,TOTAL,COSTO MATERIALES,2023-05-01,2023,5,8533707,178069095,-169535388


#### 3.1.10 - Se concatenan los 2 dataframes resultantes

In [38]:
print("=======================================================================")
print("Dimensiones del dataframe antes de la concatenación : ")
print(df_libro_mayor2_piv.shape)

Dimensiones del dataframe antes de la concatenación : 
(557, 8)


In [39]:
df_libro_mayor3 = pd.concat([df_libro_mayor2_piv, df_libro_mayor_piv2])
df_libro_mayor3.sort_values(by=['Codigo_proyecto']).sample(5)

,Codigo_proyecto,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
467,P-1718,COSTO MANO DE OBRA,2023-02-01,2023,2,0,6094576,-6094576
5,P-1592,COSTO MATERIALES,2023-02-01,2023,2,0,102993,-102993
158,P-1688,COSTO MANO DE OBRA,2023-04-01,2023,4,0,2009744,-2009744
14,P-1636,COSTO MATERIALES,2023-06-01,2023,6,0,166850,-166850
27,P-1648,COSTO MATERIALES,2023-04-01,2023,4,176177,0,176177


In [40]:
print("=======================================================================")
print("Dimensiones del dataframe después de la concatenación : ")
print(df_libro_mayor3.shape)

Dimensiones del dataframe después de la concatenación : 
(579, 8)


#### 3.1.11 - Se crean las filas que permiten tener siempre todas las opciones de Tipo_Cuenta por cada Codigo_proyecto

In [41]:
combinaciones = []
for proyecto in df_libro_mayor3['Codigo_proyecto'].unique():
    for fecha in df_libro_mayor3['Fecha2'].unique():
        mes = pd.to_datetime(fecha).month
        año = pd.to_datetime(fecha).year
        for cuenta in ['INGRESO', 'COSTO MATERIALES', 'COSTO MANO DE OBRA', 'COSTO SUPERVISIÓN']:
            combinaciones.append((proyecto, fecha, mes, año, cuenta))

# crear un nuevo DataFrame con las combinaciones obtenidas
combinaciones_df = pd.DataFrame(combinaciones, columns=['Codigo_proyecto', 'Fecha2', 'Mes', 'Año', 'Tipo_cuenta'])

# agregar las demás columnas con valores nulos
combinaciones_df['Credito_2'] = 0
combinaciones_df['Debito_2'] = 0
combinaciones_df['TOTAL'] = 0

# Se eliminan filas con fecha en blanco.
combinaciones_df = combinaciones_df.dropna(subset=['Mes'])

combinaciones_df.sample(10)

,Codigo_proyecto,Fecha2,Mes,Año,Tipo_cuenta,Credito_2,Debito_2,TOTAL
1640,P-1701,2023-03-01,3.00,2023.00,INGRESO,0,0,0
1293,P-1689,2023-05-01,5.00,2023.00,COSTO MATERIALES,0,0,0
1388,P-1692,2023-03-01,3.00,2023.00,INGRESO,0,0,0
1138,P-1683,2023-03-01,3.00,2023.00,COSTO MANO DE OBRA,0,0,0
264,P-1636,2023-02-01,2.00,2023.00,INGRESO,0,0,0
468,P-1648,2023-06-01,6.00,2023.00,INGRESO,0,0,0
551,P-1656,2023-03-01,3.00,2023.00,COSTO SUPERVISIÓN,0,0,0
1948,P-1715,2023-03-01,3.00,2023.00,INGRESO,0,0,0
2353,TOTAL,2023-04-01,4.00,2023.00,COSTO MATERIALES,0,0,0
970,P-1676,2023-03-01,3.00,2023.00,COSTO MANO DE OBRA,0,0,0


#### 3.1.12 - Se concatenan los 2 dataframes resultantes

In [42]:
print("=======================================================================")
print("Dimensiones del dataframe antes de la concatenación : ")
print(df_libro_mayor3.shape)

Dimensiones del dataframe antes de la concatenación : 
(579, 8)


In [43]:
df_libro_mayor4 = pd.concat([df_libro_mayor3, combinaciones_df])
df_libro_mayor4.sort_values(by=['Codigo_proyecto']).sample(5)

,Codigo_proyecto,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
162,P-1688,COSTO MATERIALES,2023-03-01,2023,3,620196,4747994,-4127798
415,P-1644,COSTO SUPERVISIÓN,2023-06-01,2023.00,6.00,0,0,0
525,P-1653,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,0,0
771,P-1668,COSTO SUPERVISIÓN,2023-02-01,2023.00,2.00,0,0,0
2034,P-1718,COSTO MANO DE OBRA,2023-03-01,2023.00,3.00,0,0,0


In [44]:
print("=======================================================================")
print("Dimensiones del dataframe después de la concatenación : ")
print(df_libro_mayor4.shape)

Dimensiones del dataframe después de la concatenación : 
(2619, 8)


#### 3.1.13 - Estandarización de campos 

In [45]:
# Se estandarizan los campos con sus nombres y formatos, para un mayor entendimiento.

df_libro_mayor4 = df_libro_mayor4.rename(columns={
    'Codigo_proyecto': 'CODIGO_PROYECTO',
    'Tipo_cuenta': 'TIPO_CUENTA',
    'Fecha2': 'FECHA',
    'Año': 'AÑO',
    'Mes': 'MES',
    'Credito_2': 'CREDITO',
    'Debito_2': 'DEBITO'
})

df_libro_mayor4['AÑO'] = pd.to_numeric(df_libro_mayor4['AÑO'], downcast='integer')
df_libro_mayor4['MES'] = pd.to_numeric(df_libro_mayor4['MES'], downcast='integer')

df_libro_mayor4.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL
719,P-1666,COSTO SUPERVISIÓN,2023-03-01,2023.00,3.00,0,0,0
1121,P-1683,COSTO MATERIALES,2023-04-01,2023.00,4.00,0,0,0
606,P-1660,COSTO MANO DE OBRA,2023-03-01,2023.00,3.00,0,0,0
1497,P-1696,COSTO MATERIALES,2023-02-01,2023.00,2.00,0,0,0
1995,P-1717,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,0,0
1840,P-1709,INGRESO,2023-06-01,2023.00,6.00,0,0,0
1489,P-1696,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,0,0
556,TOTAL,,NaT,NaN,NaN,2290563418,1931316000,359247418
224,P-1632,INGRESO,2023-04-01,2023.00,4.00,0,0,0
2130,P-1722,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,0,0


In [46]:
#df_libro_mayor4[df_libro_mayor4["CODIGO_PROYECTO"]=="P-1706"]
df_libro_mayor4[df_libro_mayor4["CODIGO_PROYECTO"]=="TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL
556,TOTAL,,NaT,NaN,NaN,2290563418,1931316000,359247418
0,TOTAL,COSTO MANO DE OBRA,2023-01-01,2023.00,1.00,0,72482994,-72482994
1,TOTAL,COSTO MANO DE OBRA,2023-02-01,2023.00,2.00,0,82645544,-82645544
2,TOTAL,COSTO MANO DE OBRA,2023-03-01,2023.00,3.00,0,96760244,-96760244
3,TOTAL,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,111885468,-111885468
4,TOTAL,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,114373715,-114373715
5,TOTAL,COSTO MATERIALES,2023-01-01,2023.00,1.00,2565889,125972929,-123407040
6,TOTAL,COSTO MATERIALES,2023-02-01,2023.00,2.00,3348963,83658197,-80309234
7,TOTAL,COSTO MATERIALES,2023-03-01,2023.00,3.00,2764412,88061320,-85296908
8,TOTAL,COSTO MATERIALES,2023-04-01,2023.00,4.00,3247244,149731515,-146484271


#### 3.1.14 Creación de columna MARGEN

In [47]:
df_libro_mayor4_2 = df_libro_mayor4.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_libro_mayor4_2.groupby(['CODIGO_PROYECTO','MES']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

    
df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(5)

,CODIGO_PROYECTO,MES,MARGEN
350,P-1701,3.00,0.00
473,P-1724,6.00,0.00
318,P-1696,1.00,0.00
129,P-1660,4.00,0.00
325,P-1697,2.00,0.00


In [48]:
print("=======================================================================")
print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor4_2  : ", df_libro_mayor4.shape)
print("=======================================================================")
print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_margen          : ", df_margen.shape)
print(df_margen.shape)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor4_2  :  (2619, 8)
DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_margen          :  (510, 3)
(510, 3)


In [49]:
# CONCATENACIÓN DE DATAFRAMES

df_libro_mayor5 = pd.merge(df_libro_mayor4, df_margen, on=['CODIGO_PROYECTO', 'MES'], how='left')

# Se estandariza el valor TOTAL a 1.TOTAL para que pueda ser seleccionado desde los filtros del Dashboard.
df_libro_mayor5['CODIGO_PROYECTO'] = df_libro_mayor5['CODIGO_PROYECTO'].apply(lambda x: '1. TOTAL' if x == 'TOTAL' else x)

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5  : ", df_libro_mayor5.shape)
print(df_libro_mayor5.shape)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5  :  (2619, 9)
(2619, 9)


In [50]:
# INSPECCIÓN VISUAL 1
df_libro_mayor5[df_libro_mayor5["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
556,1. TOTAL,,NaT,NaN,NaN,2290563418,1931316000,359247418,NaN
557,1. TOTAL,COSTO MANO DE OBRA,2023-01-01,2023.00,1.00,0,72482994,-72482994,0.00
558,1. TOTAL,COSTO MANO DE OBRA,2023-02-01,2023.00,2.00,0,82645544,-82645544,0.00
559,1. TOTAL,COSTO MANO DE OBRA,2023-03-01,2023.00,3.00,0,96760244,-96760244,0.00
560,1. TOTAL,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,111885468,-111885468,0.00
561,1. TOTAL,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,114373715,-114373715,0.00
562,1. TOTAL,COSTO MATERIALES,2023-01-01,2023.00,1.00,2565889,125972929,-123407040,0.00
563,1. TOTAL,COSTO MATERIALES,2023-02-01,2023.00,2.00,3348963,83658197,-80309234,0.00
564,1. TOTAL,COSTO MATERIALES,2023-03-01,2023.00,3.00,2764412,88061320,-85296908,0.00
565,1. TOTAL,COSTO MATERIALES,2023-04-01,2023.00,4.00,3247244,149731515,-146484271,0.00


In [51]:
# INSPECCIÓN VISUAL 2
df_libro_mayor5[df_libro_mayor5["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
319,P-1704,COSTO MANO DE OBRA,2023-01-01,2023.00,1.00,0,7375493,-7375493,0.00
320,P-1704,COSTO MANO DE OBRA,2023-02-01,2023.00,2.00,0,10751091,-10751091,0.00
321,P-1704,COSTO MANO DE OBRA,2023-03-01,2023.00,3.00,0,12332472,-12332472,0.00
322,P-1704,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,14242535,-14242535,0.00
323,P-1704,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,12890883,-12890883,0.00
324,P-1704,COSTO MATERIALES,2023-01-01,2023.00,1.00,0,20073319,-20073319,0.00
325,P-1704,COSTO MATERIALES,2023-02-01,2023.00,2.00,139976,3388760,-3248784,0.00
326,P-1704,COSTO MATERIALES,2023-03-01,2023.00,3.00,0,14768626,-14768626,0.00
327,P-1704,COSTO MATERIALES,2023-04-01,2023.00,4.00,0,20031042,-20031042,0.00
328,P-1704,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,13967551,-13967551,0.00


In [52]:
df_libro_mayor5["CODIGO_PROYECTO"].unique()

array(['P-1557', 'P-1591', 'P-1592', 'P-1603', 'P-1611', 'P-1612',
       'P-1620', 'P-1626', 'P-1632', 'P-1636', 'P-1639', 'P-1641',
       'P-1642', 'P-1643', 'P-1644', 'P-1645', 'P-1648', 'P-1649',
       'P-1653', 'P-1656', 'P-1658', 'P-1660', 'P-1661', 'P-1663',
       'P-1664', 'P-1666', 'P-1667', 'P-1668', 'P-1669', 'P-1670',
       'P-1672', 'P-1673', 'P-1674', 'P-1675', 'P-1676', 'P-1677',
       'P-1679', 'P-1680', 'P-1681', 'P-1682', 'P-1683', 'P-1684',
       'P-1685', 'P-1686', 'P-1687', 'P-1688', 'P-1689', 'P-1690',
       'P-1691', 'P-1692', 'P-1693', 'P-1694', 'P-1695', 'P-1696',
       'P-1697', 'P-1698', 'P-1699', 'P-1700', 'P-1701', 'P-1703',
       'P-1704', 'P-1705', 'P-1706', 'P-1707', 'P-1708', 'P-1709',
       'P-1712', 'P-1713', 'P-1714', 'P-1715', 'P-1716', 'P-1717',
       'P-1718', 'P-1719', 'P-1720', 'P-1721', 'P-1722', 'P-1723',
       'P-1724', 'P-1725', 'P-1726', 'P-1727', 'P-1728', 'P-1729',
       '1. TOTAL'], dtype=object)

In [53]:
df_libro_mayor5.to_csv('archivos_respaldo/df_libro_mayor5.csv', index = False)

### 3.1 - TRATAMIENTO 1 - LIBRO MAYOR SUBSETS

#### 3.1.16 - CREACIÓN DF **"YTD AÑO EN CURSO HASTA MES CERRADO"**

In [54]:
fecha_actual = datetime.date.today()
fecha_actual

datetime.date(2023, 6, 19)

In [55]:
# Se crea una copia del dataframe 

df_libro_mayor_ytd = df_libro_mayor5.copy()
df_libro_mayor_ytd.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
1018,P-1653,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,0,0,0.00
1782,P-1693,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,0,0,0.00
1585,P-1684,COSTO MANO DE OBRA,2023-06-01,2023.00,6.00,0,0,0,0.00
1660,P-1688,COSTO MATERIALES,2023-04-01,2023.00,4.00,0,0,0,0.00
1547,P-1683,INGRESO,2023-01-01,2023.00,1.00,0,0,0,0.00


In [56]:
# Se crea un filtro para obtener todos aquellos que tengan valores hasta el MES anterior al mes en curso

# Obtener el mes actual
mes_actual = datetime.date.today().month
print(mes_actual)

# Filtrar los registros del dataframe que corresponden al mes actual y a todos los meses anteriores
df_libro_mayor_ytd["FECHA"] = pd.to_datetime(df_libro_mayor_ytd["FECHA"])
df_libro_mayor_ytd_closed = df_libro_mayor_ytd[df_libro_mayor_ytd["FECHA"].dt.month < mes_actual]

df_libro_mayor_ytd_closed.sample(10)

6


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
261,P-1697,INGRESO,2023-01-01,2023.00,1.00,3575993,0,3575993,0.00
619,P-1591,INGRESO,2023-03-01,2023.00,3.00,0,0,0,0.00
2507,P-1726,INGRESO,2023-01-01,2023.00,1.00,0,0,0,0.00
2215,P-1714,INGRESO,2023-05-01,2023.00,5.00,0,0,0,0.00
350,P-1706,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,2951357,-2951357,0.00
71,P-1676,COSTO MATERIALES,2023-01-01,2023.00,1.00,93720,2125358,-2031638,0.00
1143,P-1663,INGRESO,2023-02-01,2023.00,2.00,0,0,0,0.00
136,P-1686,COSTO MATERIALES,2023-02-01,2023.00,2.00,144000,1048173,-904173,0.00
2382,P-1721,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,0,0,0.00
1029,P-1653,COSTO MANO DE OBRA,2023-03-01,2023.00,3.00,0,0,0,0.00


In [57]:
# crear pivot table sin margenes

df_libro_mayor_ytd_closed_piv = pd.pivot_table(df_libro_mayor_ytd_closed, values=['TOTAL'], index=['CODIGO_PROYECTO', 'TIPO_CUENTA'], aggfunc='sum', fill_value=0)
#df_libro_mayor4_piv = pd.pivot_table(df_libro_mayor4, values=['Debito_2','Credito_2','TOTAL'], index=['Codigo_proyecto','Tipo_cuenta'], columns = ['Fecha2'], aggfunc='sum', fill_value=0, margins=True, margins_name='TOTAL')

df_libro_mayor_ytd_closed_piv = pd.DataFrame(df_libro_mayor_ytd_closed_piv.to_records())

df_libro_mayor_ytd_closed_piv.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
66,P-1645,COSTO SUPERVISIÓN,0
232,P-1700,COSTO MANO DE OBRA,-4264241
260,P-1708,COSTO MANO DE OBRA,-1679752
182,P-1687,COSTO SUPERVISIÓN,0
107,P-1666,INGRESO,0
307,P-1721,INGRESO,40331329
168,P-1684,COSTO MANO DE OBRA,-6707469
84,P-1658,COSTO MANO DE OBRA,0
130,P-1673,COSTO SUPERVISIÓN,0
270,P-1712,COSTO SUPERVISIÓN,0


#### 3.1.16-1 - CREACIÓN DF **"YTD AÑO EN CURSO HASTA MES CERRADO 2"**
Este dataframe es para considerar como mes cerrado 2 meses atrás, de esta forma se puede evaluar el mes cerrado.

In [58]:
# Obtener el mes actual
mes_actual = datetime.date.today().month
print(mes_actual)

# Calcular el mes que es dos meses antes del actual
mes_filtrado = mes_actual - 2 if mes_actual > 2 else (12 + mes_actual - 2) 

# Filtrar los registros del dataframe que corresponden al mes filtrado y a todos los meses anteriores
df_libro_mayor_ytd["FECHA"] = pd.to_datetime(df_libro_mayor_ytd["FECHA"])
df_libro_mayor_ytd_closed_2M = df_libro_mayor_ytd[df_libro_mayor_ytd["FECHA"].dt.month <= mes_filtrado]

df_libro_mayor_ytd_closed_2M.sample(10)

6


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
452,P-1716,COSTO MANO DE OBRA,2023-03-01,2023.00,3.00,0,5896052,-5896052,0.00
45,P-1667,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,967228,-967228,0.00
764,P-1626,COSTO MATERIALES,2023-03-01,2023.00,3.00,0,0,0,0.00
1995,P-1703,INGRESO,2023-04-01,2023.00,4.00,0,0,0,0.00
1030,P-1653,COSTO SUPERVISIÓN,2023-03-01,2023.00,3.00,0,0,0,0.00
2357,P-1720,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,0,0,0.00
1396,P-1676,COSTO MATERIALES,2023-04-01,2023.00,4.00,0,0,0,0.00
1579,P-1684,INGRESO,2023-03-01,2023.00,3.00,0,0,0,0.00
1950,P-1700,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,0,0,0.00
1085,P-1660,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,0,0,0.00


In [59]:
# crear pivot table sin margenes

df_libro_mayor_ytd_closed_2M = pd.pivot_table(df_libro_mayor_ytd_closed_2M, values=['TOTAL'], index=['CODIGO_PROYECTO', 'TIPO_CUENTA'], aggfunc='sum', fill_value=0)

df_libro_mayor_ytd_closed_2M = pd.DataFrame(df_libro_mayor_ytd_closed_2M.to_records())

df_libro_mayor_ytd_closed_2M.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
212,P-1695,COSTO MANO DE OBRA,-5042385
220,P-1697,COSTO MANO DE OBRA,-6902710
158,P-1681,COSTO SUPERVISIÓN,0
46,P-1639,COSTO SUPERVISIÓN,0
205,P-1693,COSTO MATERIALES,103024


#### 3.1.17 - CREACIÓN DF **LIBRO MAYOR PARA MES EN CURSO**

In [60]:
# Se crea un nuevo dataframe basado en df_libro_mayor5, la palabra "mec" significa mes en curso.
 
df_libro_mayor5_mec = df_libro_mayor5.copy()
df_libro_mayor5_mec.shape
print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec  : ", df_libro_mayor5_mec.shape)
df_libro_mayor5_mec.sample(3)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec  :  (2619, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
1533,P-1682,COSTO MANO DE OBRA,2023-03-01,2023.00,3.00,0,0,0,0.00
2003,P-1703,INGRESO,2023-01-01,2023.00,1.00,0,0,0,0.00
2363,P-1720,INGRESO,2023-01-01,2023.00,1.00,0,0,0,0.00


In [61]:
# Se crea una nueva columna para categorizar el origen de los datos.

df_libro_mayor5_mec["ORIGEN"]="LIBRO MAYOR"
df_libro_mayor5_mec.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN,ORIGEN
22,P-1645,COSTO MATERIALES,2023-05-01,2023.00,5.00,19980,0,19980,0.00,LIBRO MAYOR
2390,P-1721,COSTO SUPERVISIÓN,2023-01-01,2023.00,1.00,0,0,0,0.00,LIBRO MAYOR
365,P-1707,COSTO MATERIALES,2023-04-01,2023.00,4.00,0,2057186,-2057186,0.00,LIBRO MAYOR
104,P-1682,COSTO MANO DE OBRA,2023-02-01,2023.00,2.00,0,3110959,-3110959,0.00,LIBRO MAYOR
2030,P-1704,COSTO SUPERVISIÓN,2023-01-01,2023.00,1.00,0,0,0,0.00,LIBRO MAYOR


In [62]:
# Se seleccionan los meses que son utiles para crear la vista de mes en curso.
# Para el real se necesita: INGRESO, COSTO MATERIALES, Y SUPERVISIÓN (ÉSTE ÚLTIMO DEL MES ANTERIOR)
# Para el proyectado se necesita: SUPERVISIÓN (DEL MES ANTERIOR)

In [63]:
# Filtro para TIPO_CUENTA y FECHA
from datetime import datetime, timedelta


df_libro_mayor5_mec['FECHA'] = pd.to_datetime(df_libro_mayor5_mec['FECHA'])

filtro_ingreso_materiales = ((df_libro_mayor5_mec['TIPO_CUENTA'] == 'INGRESO') | 
                             (df_libro_mayor5_mec['TIPO_CUENTA'] == 'COSTO MATERIALES')) & \
                            (df_libro_mayor5_mec['FECHA'].dt.month == datetime.now().month)

# Filtro para SUPERVISIÓN del mes anterior
mes_anterior = (datetime.now() - timedelta(days=32)).month
filtro_supervision_mes_anterior = (df_libro_mayor5_mec['TIPO_CUENTA'] == 'COSTO SUPERVISIÓN') & \
                                  (df_libro_mayor5_mec['FECHA'].dt.month == mes_anterior)


# Filtro para COSTO_MANO_OBRA del mes anterior
mes_anterior = (datetime.now() - timedelta(days=32)).month
filtro_mo_mes_anterior = (df_libro_mayor5_mec['TIPO_CUENTA'] == 'COSTO MANO DE OBRA') & \
                                  (df_libro_mayor5_mec['FECHA'].dt.month == mes_anterior)

# Combinar los filtros en un filtro general
filtro_general = filtro_ingreso_materiales | filtro_supervision_mes_anterior  | filtro_mo_mes_anterior

# Crear el sub-dataframe filtrado
df_libro_mayor5_mec2 = df_libro_mayor5_mec[filtro_general]

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec2  : ", df_libro_mayor5_mec2.shape)

df_libro_mayor5_mec2.sample(10)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec2  :  (424, 10)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN,ORIGEN
242,P-1696,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,310358,-310358,0.00,LIBRO MAYOR
567,1. TOTAL,COSTO MATERIALES,2023-06-01,2023.00,6.00,4154758,145257692,-141102934,0.00,LIBRO MAYOR
2375,P-1720,INGRESO,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR
1185,P-1666,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
922,P-1644,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
16,P-1639,COSTO MATERIALES,2023-06-01,2023.00,6.00,15100,0,15100,0.00,LIBRO MAYOR
101,P-1680,INGRESO,2023-06-01,2023.00,6.00,0,6000000,-6000000,0.00,LIBRO MAYOR
682,P-1611,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
1210,P-1667,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
2280,P-1716,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR


In [64]:
# INSPECCIÓN VISUAL 1
df_libro_mayor5_mec2[df_libro_mayor5_mec2["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN,ORIGEN
323,P-1704,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,12890883,-12890883,0.00,LIBRO MAYOR
329,P-1704,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,24160596,-24160596,0.00,LIBRO MAYOR
2025,P-1704,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
2026,P-1704,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
2039,P-1704,INGRESO,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR
2040,P-1704,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR


In [65]:
# INSPECCIÓN VISUAL 2
df_libro_mayor5_mec2[df_libro_mayor5_mec2["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN,ORIGEN
561,1. TOTAL,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,114373715,-114373715,0.00,LIBRO MAYOR
567,1. TOTAL,COSTO MATERIALES,2023-06-01,2023.00,6.00,4154758,145257692,-141102934,0.00,LIBRO MAYOR
573,1. TOTAL,INGRESO,2023-06-01,2023.00,6.00,63059808,58891059,4168749,0.00,LIBRO MAYOR
578,1. TOTAL,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,4643261,-4643261,0.00,LIBRO MAYOR
2601,1. TOTAL,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
2602,1. TOTAL,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
2615,1. TOTAL,INGRESO,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR
2616,1. TOTAL,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR


#### 3.1.17-1 - CREACIÓN DF **LIBRO MAYOR PARA MES EN CURSO 2**
Este dataframe es para considerar como mes en curso de 2 meses atrás, de esta forma se puede evaluar el mes cerrado.

In [66]:
# Se crea un nuevo dataframe basado en df_libro_mayor5, la palabra "mec" significa mes en curso.

df_libro_mayor5_mec_2M = df_libro_mayor5.copy()
df_libro_mayor5_mec_2M.shape
print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec_2M  : ", df_libro_mayor5_mec_2M.shape)
df_libro_mayor5_mec_2M.sample(3)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec_2M  :  (2619, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
299,P-1701,COSTO MATERIALES,2023-01-01,2023.00,1.00,0,1733676,-1733676,0.00
809,P-1636,COSTO MANO DE OBRA,2023-02-01,2023.00,2.00,0,0,0,0.00
115,P-1682,INGRESO,2023-05-01,2023.00,5.00,18808581,0,18808581,0.00


In [67]:
# Se crea una nueva columna para categorizar el origen de los datos.

df_libro_mayor5_mec_2M["ORIGEN"]="LIBRO MAYOR"
df_libro_mayor5_mec_2M.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN,ORIGEN
1245,P-1668,COSTO MANO DE OBRA,2023-03-01,2023.00,3.00,0,0,0,0.00,LIBRO MAYOR
1976,P-1701,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
1002,P-1649,COSTO SUPERVISIÓN,2023-02-01,2023.00,2.00,0,0,0,0.00,LIBRO MAYOR
1522,P-1682,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
1,P-1557,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,301065,-301065,0.00,LIBRO MAYOR


In [68]:
# Filtro para TIPO_CUENTA y FECHA
from datetime import datetime, timedelta

df_libro_mayor5_mec_2M['FECHA'] = pd.to_datetime(df_libro_mayor5_mec_2M['FECHA'])

# Filtro para INGRESO Y COSTO DE MATERIALES
mes_anterior = (datetime.now() - timedelta(days=30)).month

filtro_ingreso_materiales = ((df_libro_mayor5_mec_2M['TIPO_CUENTA'] == 'INGRESO') | 
                             (df_libro_mayor5_mec_2M['TIPO_CUENTA'] == 'COSTO MATERIALES')) & \
                            (df_libro_mayor5_mec_2M['FECHA'].dt.month == mes_anterior)

# Filtro para SUPERVISIÓN del mes anterior
mes_anterior2 = (datetime.now() - timedelta(days=60)).month

filtro_supervision_mes_anterior = (df_libro_mayor5_mec_2M['TIPO_CUENTA'] == 'COSTO SUPERVISIÓN') & \
                                  (df_libro_mayor5_mec_2M['FECHA'].dt.month == mes_anterior2)


# Filtro para COSTO_MANO_OBRA del mes anterior
mes_anterior2 = (datetime.now() - timedelta(days=60)).month

filtro_mo_mes_anterior = (df_libro_mayor5_mec_2M['TIPO_CUENTA'] == 'COSTO MANO DE OBRA') & \
                                  (df_libro_mayor5_mec_2M['FECHA'].dt.month == mes_anterior2)

# Combinar los filtros en un filtro general
filtro_general = filtro_ingreso_materiales | filtro_supervision_mes_anterior  | filtro_mo_mes_anterior

# Crear el sub-dataframe filtrado
df_libro_mayor5_mec_2M = df_libro_mayor5_mec_2M[filtro_general]

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec_2M  : ", df_libro_mayor5_mec_2M.shape)

df_libro_mayor5_mec_2M.sample(5)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec_2M  :  (456, 10)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN,ORIGEN
1663,P-1688,INGRESO,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
134,P-1686,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,4634424,-4634424,0.00,LIBRO MAYOR
2141,P-1709,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,0,0,0.00,LIBRO MAYOR
1639,P-1687,INGRESO,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
1352,P-1674,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR


In [69]:
# INSPECCIÓN VISUAL 1
df_libro_mayor5_mec_2M[df_libro_mayor5_mec_2M["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN,ORIGEN
322,P-1704,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,14242535,-14242535,0.00,LIBRO MAYOR
328,P-1704,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,13967551,-13967551,0.00,LIBRO MAYOR
334,P-1704,INGRESO,2023-05-01,2023.00,5.00,132309997,53250000,79059997,0.00,LIBRO MAYOR
2021,P-1704,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,0,0,0.00,LIBRO MAYOR
2022,P-1704,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,0,0,0.00,LIBRO MAYOR
2023,P-1704,INGRESO,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
2024,P-1704,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR


In [70]:
# INSPECCIÓN VISUAL 2
df_libro_mayor5_mec_2M[df_libro_mayor5_mec_2M["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN,ORIGEN
560,1. TOTAL,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,111885468,-111885468,0.00,LIBRO MAYOR
566,1. TOTAL,COSTO MATERIALES,2023-05-01,2023.00,5.00,8533707,178069095,-169535388,0.00,LIBRO MAYOR
572,1. TOTAL,INGRESO,2023-05-01,2023.00,5.00,631496823,170698310,460798513,0.00,LIBRO MAYOR
577,1. TOTAL,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,5096121,-5096121,0.00,LIBRO MAYOR
2597,1. TOTAL,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,0,0,0.00,LIBRO MAYOR
2598,1. TOTAL,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,0,0,0.00,LIBRO MAYOR
2599,1. TOTAL,INGRESO,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
2600,1. TOTAL,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR


#### 3.1.18 - CREACIÓN DF **LIBRO MAYOR PARA YTD MES PROYECTADO (Incluyendo mes actual)**

In [71]:
# Se crea una copia del dataframe 

df_libro_mayor_ytd2 = df_libro_mayor5.copy()
df_libro_mayor_ytd2.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
796,P-1636,COSTO MATERIALES,2023-04-01,2023.00,4.00,0,0,0,0.00
1798,P-1693,COSTO SUPERVISIÓN,2023-03-01,2023.00,3.00,0,0,0,0.00
653,P-1603,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,0,0,0.00
1554,P-1683,COSTO SUPERVISIÓN,2023-02-01,2023.00,2.00,0,0,0,0.00
2376,P-1720,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,0,0,0.00
962,P-1645,COSTO SUPERVISIÓN,2023-06-01,2023.00,6.00,0,0,0,0.00
1308,P-1672,COSTO MATERIALES,2023-01-01,2023.00,1.00,0,0,0,0.00
1068,P-1658,COSTO MATERIALES,2023-01-01,2023.00,1.00,0,0,0,0.00
2071,P-1706,INGRESO,2023-05-01,2023.00,5.00,0,0,0,0.00
1865,P-1696,COSTO MANO DE OBRA,2023-02-01,2023.00,2.00,0,0,0,0.00


In [72]:
# crear pivot table sin margenes

df_libro_mayor_ytd2_closed_piv = pd.pivot_table(df_libro_mayor_ytd2, values=['TOTAL'], index=['CODIGO_PROYECTO', 'TIPO_CUENTA'], aggfunc='sum', fill_value=0)

df_libro_mayor_ytd2_closed_piv = pd.DataFrame(df_libro_mayor_ytd2_closed_piv.to_records())

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor_ytd2_closed_piv  : ", df_libro_mayor_ytd2_closed_piv.shape)

df_libro_mayor_ytd2_closed_piv.sample(10)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor_ytd2_closed_piv  :  (341, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
215,P-1695,COSTO SUPERVISIÓN,0
213,P-1695,COSTO MANO DE OBRA,-5042385
115,P-1668,COSTO SUPERVISIÓN,0
141,P-1676,COSTO MANO DE OBRA,-1798488
39,P-1632,COSTO SUPERVISIÓN,0
337,P-1729,COSTO MANO DE OBRA,0
214,P-1695,COSTO MATERIALES,-20588695
151,P-1679,COSTO SUPERVISIÓN,0
219,P-1696,COSTO SUPERVISIÓN,0
201,P-1692,COSTO MANO DE OBRA,-27116720


#### 3.1.18-2 - CREACIÓN DF **LIBRO MAYOR PARA YTD MES PROYECTADO (Incluyendo mes actual) 2**

In [73]:
# Obtener el mes actual
mes_actual = datetime.now().month

# Filtrar los registros hasta el mes anterior al mes actual
df_libro_mayor_ytd2_1M = df_libro_mayor_ytd2[df_libro_mayor_ytd2['MES'] < mes_actual]

# Mostrar el nuevo DataFrame
print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor_ytd2_1M  : ", df_libro_mayor_ytd2_1M.shape)
df_libro_mayor_ytd2_1M.sample(5)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor_ytd2_1M  :  (2230, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
2468,P-1724,COSTO MATERIALES,2023-03-01,2023.00,3.00,0,0,0,0.00
565,1. TOTAL,COSTO MATERIALES,2023-04-01,2023.00,4.00,3247244,149731515,-146484271,0.00
2252,P-1715,COSTO MATERIALES,2023-03-01,2023.00,3.00,0,0,0,0.00
2100,P-1707,COSTO MATERIALES,2023-01-01,2023.00,1.00,0,0,0,0.00
2243,P-1715,INGRESO,2023-01-01,2023.00,1.00,0,0,0,0.00


In [74]:
# crear pivot table sin margenes

df_libro_mayor_ytd2_closed_piv_1M = pd.pivot_table(df_libro_mayor_ytd2_1M, values=['TOTAL'], index=['CODIGO_PROYECTO', 'TIPO_CUENTA'], aggfunc='sum', fill_value=0)

df_libro_mayor_ytd2_closed_piv_1M = pd.DataFrame(df_libro_mayor_ytd2_closed_piv_1M.to_records())

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor_ytd2_closed_piv_1M  : ", df_libro_mayor_ytd2_closed_piv_1M.shape)

df_libro_mayor_ytd2_closed_piv_1M.sample(10)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor_ytd2_closed_piv_1M  :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
293,P-1718,COSTO MATERIALES,-13961366
121,P-1670,COSTO MATERIALES,6945
84,P-1658,COSTO MANO DE OBRA,0
62,P-1644,COSTO SUPERVISIÓN,0
318,P-1724,COSTO SUPERVISIÓN,0
98,P-1663,COSTO SUPERVISIÓN,0
260,P-1708,COSTO MANO DE OBRA,-1679752
115,P-1668,INGRESO,0
48,P-1641,COSTO MANO DE OBRA,0
22,P-1611,COSTO SUPERVISIÓN,0


#### 3.1.19 - CREACIÓN DE DATAFRAME BASE LIBRO MAYOR 2023 PARA EL CRUCE OTROS DATAFRAMES

##### NOTA IMPORTANTE
##### Se considera que todos los dataframes que pintan los dashboards deben tener como base los proyectos abiertos en el Libro Mayor 2023. Para esto se crea el dataframe base con el cual debe comenzar cualquier cruce de datos y que considera el código del proyecto, todos los tipos de cuenta por cada proyecto y con valor en cero, además se cruza con la tabla codigo de proyecto.

In [75]:
df_libro_mayor5_base = df_libro_mayor5.copy()

In [76]:
print(df_libro_mayor5_base["CODIGO_PROYECTO"].unique())

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_libro_mayor5_base   : ", df_libro_mayor5_base.shape)

df_libro_mayor5_base = df_libro_mayor5_base[['CODIGO_PROYECTO','TIPO_CUENTA']].drop_duplicates()

# Se elimina fila de TOTAL
df_libro_mayor5_base = df_libro_mayor5_base[df_libro_mayor5_base['CODIGO_PROYECTO'] != '1. TOTAL']

# Se Reemplazan valores de la column TIPO_CUENTA
df_libro_mayor5_base['TIPO_CUENTA'] = df_libro_mayor5_base['TIPO_CUENTA'].replace({
    'INGRESO': '1. INGRESO',
    'COSTO MATERIALES': '2. COSTO MATERIALES',
    'COSTO MANO DE OBRA': '3. COSTO MANO DE OBRA',
    'COSTO SUPERVISIÓN': '4. COSTO SUPERVISIÓN'
})

df_libro_mayor5_base['TOTAL_LB']= 0

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor5_base : ", df_libro_mayor5_base.shape)
df_libro_mayor5_base.sample(10)

['P-1557' 'P-1591' 'P-1592' 'P-1603' 'P-1611' 'P-1612' 'P-1620' 'P-1626'
 'P-1632' 'P-1636' 'P-1639' 'P-1641' 'P-1642' 'P-1643' 'P-1644' 'P-1645'
 'P-1648' 'P-1649' 'P-1653' 'P-1656' 'P-1658' 'P-1660' 'P-1661' 'P-1663'
 'P-1664' 'P-1666' 'P-1667' 'P-1668' 'P-1669' 'P-1670' 'P-1672' 'P-1673'
 'P-1674' 'P-1675' 'P-1676' 'P-1677' 'P-1679' 'P-1680' 'P-1681' 'P-1682'
 'P-1683' 'P-1684' 'P-1685' 'P-1686' 'P-1687' 'P-1688' 'P-1689' 'P-1690'
 'P-1691' 'P-1692' 'P-1693' 'P-1694' 'P-1695' 'P-1696' 'P-1697' 'P-1698'
 'P-1699' 'P-1700' 'P-1701' 'P-1703' 'P-1704' 'P-1705' 'P-1706' 'P-1707'
 'P-1708' 'P-1709' 'P-1712' 'P-1713' 'P-1714' 'P-1715' 'P-1716' 'P-1717'
 'P-1718' 'P-1719' 'P-1720' 'P-1721' 'P-1722' 'P-1723' 'P-1724' 'P-1725'
 'P-1726' 'P-1727' 'P-1728' 'P-1729' '1. TOTAL']
DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_libro_mayor5_base   :  (2619, 9)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor5_base :  (336, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB
360,P-1707,3. COSTO MANO DE OBRA,0
193,P-1692,2. COSTO MATERIALES,0
18,P-1642,2. COSTO MATERIALES,0
1373,P-1675,3. COSTO MANO DE OBRA,0
251,P-1697,3. COSTO MANO DE OBRA,0
845,P-1641,3. COSTO MANO DE OBRA,0
1566,P-1684,4. COSTO SUPERVISIÓN,0
30,P-1653,2. COSTO MATERIALES,0
407,P-1713,3. COSTO MANO DE OBRA,0
299,P-1701,2. COSTO MATERIALES,0


In [77]:
# Se Agrega COSTO DE SUPERVISIÓN

nueva_fila = pd.DataFrame({'CODIGO_PROYECTO': ['1. TOTAL'],
                           'TIPO_CUENTA': ['4. COSTO SUPERVISIÓN'],
                           'TOTAL_LB': [0]})

df_libro_mayor5_base = pd.concat([df_libro_mayor5_base, nueva_fila], ignore_index=True)

df_libro_mayor5_base[df_libro_mayor5_base["CODIGO_PROYECTO"]=="1. TOTAL"]
print(df_libro_mayor5_base["CODIGO_PROYECTO"].unique())
print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor5_base : ", df_libro_mayor5_base.shape)

['P-1557' 'P-1591' 'P-1592' 'P-1603' 'P-1611' 'P-1612' 'P-1620' 'P-1626'
 'P-1632' 'P-1636' 'P-1639' 'P-1641' 'P-1642' 'P-1643' 'P-1644' 'P-1645'
 'P-1648' 'P-1649' 'P-1653' 'P-1656' 'P-1658' 'P-1660' 'P-1661' 'P-1663'
 'P-1664' 'P-1666' 'P-1667' 'P-1668' 'P-1669' 'P-1670' 'P-1672' 'P-1673'
 'P-1674' 'P-1675' 'P-1676' 'P-1677' 'P-1679' 'P-1680' 'P-1681' 'P-1682'
 'P-1683' 'P-1684' 'P-1685' 'P-1686' 'P-1687' 'P-1688' 'P-1689' 'P-1690'
 'P-1691' 'P-1692' 'P-1693' 'P-1694' 'P-1695' 'P-1696' 'P-1697' 'P-1698'
 'P-1699' 'P-1700' 'P-1701' 'P-1703' 'P-1704' 'P-1705' 'P-1706' 'P-1707'
 'P-1708' 'P-1709' 'P-1712' 'P-1713' 'P-1714' 'P-1715' 'P-1716' 'P-1717'
 'P-1718' 'P-1719' 'P-1720' 'P-1721' 'P-1722' 'P-1723' 'P-1724' 'P-1725'
 'P-1726' 'P-1727' 'P-1728' 'P-1729' '1. TOTAL']
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor5_base :  (337, 3)


In [78]:
# Crear DataFrame con TIPO_CUENTA y valor cero para cada combinación única

df_nuevo = pd.DataFrame({
    'CODIGO_PROYECTO': df_libro_mayor5_base['CODIGO_PROYECTO'].repeat(4),
    'TIPO_CUENTA': ['1. INGRESO', '2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN'] * len(df_libro_mayor5_base),
    'TOTAL_LB': 0
})

print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_libro_mayor5_base : ", df_libro_mayor5_base.shape)
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_nuevo             : ", df_nuevo.shape)

df_libro_mayor5_base = pd.concat([df_libro_mayor5_base, df_nuevo], ignore_index=True)

# Se remueve cualquier duplicado.
df_libro_mayor5_base.drop_duplicates(inplace=True)

print("DIMENSIONES ANTERIOR DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor5_base : ", df_libro_mayor5_base.shape)
df_libro_mayor5_base.sample(5)

DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_libro_mayor5_base :  (337, 3)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_nuevo             :  (1348, 3)
DIMENSIONES ANTERIOR DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor5_base :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB
61,P-1686,1. INGRESO,0
115,P-1707,3. COSTO MANO DE OBRA,0
324,P-1722,3. COSTO MANO DE OBRA,0
170,P-1557,4. COSTO SUPERVISIÓN,0
103,P-1703,3. COSTO MANO DE OBRA,0


In [79]:
# INSPECCIÓN VISUAL 1
df_libro_mayor5_base.sort_values(by=['CODIGO_PROYECTO','TIPO_CUENTA']).head(10)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB
1681,1. TOTAL,1. INGRESO,0
1682,1. TOTAL,2. COSTO MATERIALES,0
1683,1. TOTAL,3. COSTO MANO DE OBRA,0
336,1. TOTAL,4. COSTO SUPERVISIÓN,0
168,P-1557,1. INGRESO,0
169,P-1557,2. COSTO MATERIALES,0
0,P-1557,3. COSTO MANO DE OBRA,0
170,P-1557,4. COSTO SUPERVISIÓN,0
171,P-1591,1. INGRESO,0
172,P-1591,2. COSTO MATERIALES,0


In [80]:
# MERGE CON CODIGO_PROYECTO

# Se crea dataframe de codigos de proyecto y se aplica mayúscula
df_cod_proyecto_base = df_cod_proyecto.applymap(lambda s: s.upper() if isinstance(s, str) else s)

# Se hace merge con los codigos de proyecto.

print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_libro_mayor5_base : ", df_libro_mayor5_base.shape)

df_libro_mayor5_base2 = pd.merge(df_libro_mayor5_base, df_cod_proyecto_base, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor5_base : ", df_libro_mayor5_base.shape)

df_libro_mayor5_base2.sample(10)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_libro_mayor5_base :  (340, 3)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor5_base :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
328,P-1725,4. COSTO SUPERVISIÓN,0,LO CAÑAS,CONSTRUCTORA BRISAS DE BATUCO
207,P-1643,1. INGRESO,0,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.
133,P-1715,3. COSTO MANO DE OBRA,0,CD LO ECHEVERS,QUEZADA Y BOETSCH
48,P-1680,2. COSTO MATERIALES,0,NUEVA UNO,CONSTRUCTORA TECSA S.A.
125,P-1712,2. COSTO MATERIALES,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.
285,P-1689,3. COSTO MANO DE OBRA,0,CASINO BUENAVENTURA,EBCO S.A.
283,P-1688,4. COSTO SUPERVISIÓN,0,SUIZA,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...
0,P-1557,3. COSTO MANO DE OBRA,0,ROSARIO ROSALES,EBCO S.A.
334,P-1729,3. COSTO MANO DE OBRA,0,NaN,NaN
92,P-1698,2. COSTO MATERIALES,0,CD ANDES,EBCO S.A.


In [81]:
# Modificar las columnas "NOMBRE_OBRA" y "NOMBRE_CLIENTE" para "1. TOTAL"

df_libro_mayor5_base2.loc[df_libro_mayor5_base2['CODIGO_PROYECTO'] == '1. TOTAL', ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'

df_libro_mayor5_base2.sample(10) 

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
139,P-1717,2. COSTO MATERIALES,0,ECO QUILÍN I,CONSTRUCTORA ICF SPA
237,P-1663,3. COSTO MANO DE OBRA,0,OFICINAS EQUIFAX,CONSTRUCTORA VRK INGENIERIA LIMITADA
187,P-1620,3. COSTO MANO DE OBRA,0,"OF APOQUINDO- HABILITACIONES, INTEGRACION Y AD...",CONSTRUCTORA INARCO S.A.
45,P-1679,3. COSTO MANO DE OBRA,0,JUAN SABAJ,CONSTRUCTORA GREVIA
160,P-1726,2. COSTO MATERIALES,0,NaN,NaN
221,P-1653,1. INGRESO,0,EDUARDO DE LA BARRA,CONSTRUCTORA E INVERSIONES VITAL SPA.
271,P-1681,3. COSTO MANO DE OBRA,0,CD ROSEN (CCTV PROVISORIO),COLCHONES ROSEN SAIC
90,P-1697,2. COSTO MATERIALES,0,ESTOCOLMO,CONSTRUCTORA FORTALEZA S.A
330,P-1727,4. COSTO SUPERVISIÓN,0,NaN,NaN
117,P-1707,1. INGRESO,0,LAS TRANQUERAS,CONSTRUCTORA SANTOLAYA LTDA.


In [82]:
# INSPECCIÓN VISUAL 1

df_libro_mayor5_base2[df_libro_mayor5_base2['CODIGO_PROYECTO']=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL


In [83]:
# INSPECCIÓN VISUAL 2

df_libro_mayor5_base2[df_libro_mayor5_base2['CODIGO_PROYECTO']=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
106,P-1704,3. COSTO MANO DE OBRA,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.
107,P-1704,2. COSTO MATERIALES,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.
108,P-1704,1. INGRESO,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.
302,P-1704,4. COSTO SUPERVISIÓN,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.


### 3.2 - TRATAMIENTO DE DESPACHOS REALIZADOS

#### 3.2.1 - Creación de dataframe copia

In [84]:
df_desp_realizados2 = df_desp_realizados.copy()
df_desp_realizados2.head(5)

,TIPO_x0020_MOVIMIENTO,FECHA_x0020_GUIA,NUMERO_x0020_GUIA,PARTIDA_x0020_PRESUPUESTARIA,CODIGO_x0020_PARTE,CANTIDAD,DESCRIPCI_xFFFD_N_x0020_PRODUCTO,UNIDAD_x0020_DE_x0020_MEDIDA,COSTO_x0020_UNITARIO,COSTO_x0020_TOTAL,COD_PROYECTO
0,STT,2022-01-14T00:00:00-03:00,44313,None,C02040808,9,Conector Macho RJ45 Cat5 para crimpar,UN,47,423,NaN
1,VCO,2022-01-19T00:00:00-03:00,44345,1012,S09240029,22,Conector de poder 3.5mm macho (para cama,UN,324,7128,P-1691
2,VCO,2022-01-19T00:00:00-03:00,44345,1001,S09231993,1,DVR 32CH (analogo o IP) DAHUA DH-XVR523,UN,259000,259000,P-1691
3,VCO,2022-01-19T00:00:00-03:00,44345,1008,S09241106,1,Disco duro SATA 4 TB,UN,65793,65793,P-1691
4,VCO,2022-01-24T00:00:00-03:00,44377,1017,T14380057,400,Cable pin 4x24,M,74,29600,P-1676


#### 3.2.2 - Se eliminan proyectos relacionados a STT, P-Proyectos, P-Servicios, P-Mantención

In [85]:
df_desp_realizados2["COD_PROYECTO"].unique()

array([nan, 'P-1691', 'P-1676', 'P-1669', 'P-Servicios', 'P-Proyectos',
       'P-1677', 'P-1668', 'P-1685', 'P-1698', 'P-1683', 'P-1672',
       'P-1689', 'P-1681', 'P-1645', 'P-1693', 'P-1686', 'P-1680',
       'P-1704', 'P-1701', 'P-1700', 'P-1694', 'P-PROYECTOS', 'P-1690',
       'P-1696', 'P-1692', 'P-1667', 'P-1682', 'P-1712', 'P-1670',
       'P-1703', 'P-1707', 'P-1714', 'P-1625', 'P-1661', 'P-1688',
       'P-1675', 'P-1702', 'P-1684', 'P-1697', 'P-1687',
       'P-ANALISIS PROYE', 'P-1673', 'P-1615', 'P-Mantención', 'P-1648',
       'P-1639', 'P-1716', 'P-1718', 'P-1715', 'P-1695', 'P-1721',
       'P-1709', 'P-1623', 'P-1642', 'P-1724', 'P-1592', 'P-1699',
       'P-1723', 'P-1620', 'P-1726', 'P-1664', 'P-1728', 'P-1705',
       'P-1713', 'P-1727', 'P-1706', 'P-1708', 'P-1725', 'P-1636',
       'P-1652', 'P-1671', 'P-1660', 'P-1627', 'P-1649', 'P-1656',
       'P-1626', 'P-1678', 'P-1717', 'P-1658', 'P-1638', 'P-Oficina',
       'P-1653', 'S-1601', 'P-1629', 'P-1663', 'P-166

In [86]:
valores_eliminar = ['STT', 'P-Proyectos', 'P-Servicios', 'P-Mantención','P-Oficina''P-ANALISIS PROYE','P-ANALISIS PROYE','P-PROYECTOS','P-Oficina','nan']

# Convertir los valores a eliminar y la columna 'PROYECTO' a mayúsculas
valores_eliminar_mayusculas = [valor.upper() for valor in valores_eliminar]
df_desp_realizados2['PROYECTO_MAYUS'] = df_desp_realizados2['COD_PROYECTO'].str.upper()

# Filtrar las filas que no contengan los valores a eliminar
df_desp_realizados3 = df_desp_realizados2[~df_desp_realizados2['PROYECTO_MAYUS'].isin(valores_eliminar_mayusculas)]

# Eliminar la columna auxiliar 'PROYECTO_MAYUS'
df_desp_realizados3 = df_desp_realizados3.drop(columns=['PROYECTO_MAYUS'])

df_desp_realizados3.sample(10)

,TIPO_x0020_MOVIMIENTO,FECHA_x0020_GUIA,NUMERO_x0020_GUIA,PARTIDA_x0020_PRESUPUESTARIA,CODIGO_x0020_PARTE,CANTIDAD,DESCRIPCI_xFFFD_N_x0020_PRODUCTO,UNIDAD_x0020_DE_x0020_MEDIDA,COSTO_x0020_UNITARIO,COSTO_x0020_TOTAL,COD_PROYECTO
10137,VCO,2022-12-07T00:00:00-03:00,48368,1206,P08200583,2,"Flange Lap Joint Ø6"" p/stub end ANSI 150",UN,23800,47600,P-1712
156,VCO,2022-03-02T00:00:00-03:00,44655,1060,T14380048,1220,Cable coaxial RG6,M,143,174460,P-1672
16329,VCO,2022-12-14T00:00:00-03:00,48444,1031,T14390027,1,Bateria 12V 5amp,UN,8091,8091,P-1686
20270,VCO,2023-05-23T00:00:00-04:00,51380,1080,C02031044,9,Rack Gabinete Mural 06U 600x450mm,UN,65085,585765,P-1723
9716,GDP,2022-11-16T00:00:00-03:00,48059,1011,T14382428,610,Cable Incendio 2x18 AWG Sin Pantalla UL,M,304,185440,P-1702
15918,GDP,2022-12-09T00:00:00-03:00,48369,1087,P08202222,41,"Soporte pera 3"" UL/FM",UN,1626,66666,P-1700
3857,VCO,2023-03-06T00:00:00-03:00,49700,1016,S11280105,5,Centralita Bosch DS3MX,UN,22000,110000,P-1695
11105,VCO,2022-10-03T00:00:00-03:00,47425,1026,P08202341,32,"Tee Mecanica 4"" x 2"" ranurada A-536",UN,5022,160704,P-1704
15491,VCO,2023-04-13T00:00:00-03:00,50375,1102,C08712674,50,Mezclador PAU con Distribuidor F 4 salid,UN,4747,237350,P-1694
4290,VCO,2023-03-14T00:00:00-03:00,49853,1045,I17442581,4,Punta Phillips,UN,882,3528,P-1713


#### 3.2.2 - Se eliminan Tipos de movimientos diferentes a VCO y GDP

In [87]:
# Reemplazar los valores NaN en la columna "TIPO MOVIMIENTO" con una cadena vacía
df_desp_realizados3['TIPO_x0020_MOVIMIENTO'] = df_desp_realizados3['TIPO_x0020_MOVIMIENTO'].fillna('')

# Filtrar las filas donde "TIPO MOVIMIENTO" sea igual a "VCO" o "GDP" y excluir otros valores

mask = df_desp_realizados3['TIPO_x0020_MOVIMIENTO'].isin(['VCO', 'GDP'])
df_desp_realizados3 = df_desp_realizados3[mask]

In [88]:
# Se realiza la evaluación de dimensiones después de la eliminación de filas
print("\nDataframe Original antes de los cambios:")
print(df_desp_realizados.shape)
print("Dataframe después de eliminar registros:")
print(df_desp_realizados3.shape)

df_desp_realizados3.to_csv('df_desp_realizados3.csv', index = False)


Dataframe Original antes de los cambios:
(20580, 11)
Dataframe después de eliminar registros:
(12220, 11)


In [89]:
# Convertir la columna 'FECHA GUIA' a objetos de fecha y hora, manejando campos en blanco o no válidos como fechas
df_desp_realizados3['FECHA GUIA 2'] = pd.to_datetime(df_desp_realizados3['FECHA_x0020_GUIA'], errors='coerce')

# Extraer el MES y el AÑO en nuevas columnas utilizando funciones lambda y apply()
df_desp_realizados3['MES'] = df_desp_realizados3['FECHA GUIA 2'].apply(lambda x: x.month if pd.notnull(x) else None)
df_desp_realizados3['AÑO'] = df_desp_realizados3['FECHA GUIA 2'].apply(lambda x: x.year if pd.notnull(x) else None)

df_desp_realizados3.sample(5)

,TIPO_x0020_MOVIMIENTO,FECHA_x0020_GUIA,NUMERO_x0020_GUIA,PARTIDA_x0020_PRESUPUESTARIA,CODIGO_x0020_PARTE,CANTIDAD,DESCRIPCI_xFFFD_N_x0020_PRODUCTO,UNIDAD_x0020_DE_x0020_MEDIDA,COSTO_x0020_UNITARIO,COSTO_x0020_TOTAL,COD_PROYECTO,FECHA GUIA 2,MES,AÑO
19237,VCO,2022-12-26T00:00:00-03:00,48571,1035,T14370737,10,Conector recto 25mm p/ flexible metalico,UN,1172,11720,P-1682,2022-12-26 00:00:00-03:00,12,2022
2414,VCO,2022-12-06T00:00:00-03:00,48356,1066,T14380048,500,Cable coaxial RG6 77% malla Libre de Hal,M,376,188000,P-1707,2022-12-06 00:00:00-03:00,12,2022
11835,VCO,2023-03-07T00:00:00-03:00,49752,1010,I16430109,4,Cinta Permace 3/4,UN,625,2500,P-1696,2023-03-07 00:00:00-03:00,3,2023
9840,VCO,2022-09-23T00:00:00-03:00,47287,1013,I16431048,400,Amarras plasticas 300mm,UN,17,6800,P-1667,2022-09-23 00:00:00-03:00,9,2022
8575,VCO,2022-06-06T00:00:00-04:00,45659,1041,S10271527,1,Pulsador Salida Aluminio (Boton salida/e,UN,7540,7540,P-1691,2022-06-06 00:00:00-04:00,6,2022


#### 3.2.4 - Se realizan tablas pivot

In [90]:
df_desp_realizados3['COSTO_x0020_TOTAL'] = df_desp_realizados3['COSTO_x0020_TOTAL'].astype(int)
df_desp_realizados3['CANTIDAD'] = df_desp_realizados3['CANTIDAD'].astype(float)

In [91]:
df_desp_realizados3.columns

Index(['TIPO_x0020_MOVIMIENTO', 'FECHA_x0020_GUIA', 'NUMERO_x0020_GUIA',
       'PARTIDA_x0020_PRESUPUESTARIA', 'CODIGO_x0020_PARTE', 'CANTIDAD',
       'DESCRIPCI_xFFFD_N_x0020_PRODUCTO', 'UNIDAD_x0020_DE_x0020_MEDIDA',
       'COSTO_x0020_UNITARIO', 'COSTO_x0020_TOTAL', 'COD_PROYECTO',
       'FECHA GUIA 2', 'MES', 'AÑO'],
      dtype='object')

In [92]:
# Crear el pivot dataframe
df_desp_realizados4 = pd.pivot_table(df_desp_realizados3, 
                          values=['CANTIDAD', 'COSTO_x0020_TOTAL'], 
                          index=['COD_PROYECTO', 'AÑO', 'MES'], 
                          aggfunc='sum', 
                          columns='TIPO_x0020_MOVIMIENTO')

df_desp_realizados4 = pd.DataFrame(df_desp_realizados4.to_records())

df_desp_realizados4.sample(10)

,COD_PROYECTO,AÑO,MES,"('CANTIDAD', 'GDP')","('CANTIDAD', 'VCO')","('COSTO_x0020_TOTAL', 'GDP')","('COSTO_x0020_TOTAL', 'VCO')"
346,P-1691,2022,8,NaN,954.06,NaN,1531899.00
327,P-1690,2022,5,3.00,5013.00,59100.00,2753224.00
525,P-1712,2023,3,NaN,2370.00,NaN,2941143.00
279,P-1685,2022,7,NaN,469.00,NaN,1947896.00
367,P-1693,2022,5,1.00,555.00,44855.00,9659613.00
39,P-1639,2022,11,NaN,207.00,NaN,46455.00
581,P-1725,2023,2,NaN,4335.00,NaN,741299.00
239,P-1680,2023,4,1.00,9095.00,9807.00,9659871.00
160,P-1670,2022,7,1.00,NaN,773.00,NaN
257,P-1683,2022,6,7.00,1.00,66976.00,21139.00


#### 3.2.5 - Se renombran columnas 

In [93]:
df_desp_realizados4 = df_desp_realizados4.rename(columns={'COD_PROYECTO': 'CODIGO_PROYECTO'})
df_desp_realizados4 = df_desp_realizados4.rename(columns={"('CANTIDAD', 'GDP')": "CANT. GDP"})
df_desp_realizados4 = df_desp_realizados4.rename(columns={"('CANTIDAD', 'VCO')": "CANT. VCO"})
df_desp_realizados4 = df_desp_realizados4.rename(columns={"('COSTO_x0020_TOTAL', 'GDP')": "TOTAL. GDP"})
df_desp_realizados4 = df_desp_realizados4.rename(columns={"('COSTO_x0020_TOTAL', 'VCO')": "TOTAL. VCO"})

df_desp_realizados4.sample(5)

,CODIGO_PROYECTO,AÑO,MES,CANT. GDP,CANT. VCO,TOTAL. GDP,TOTAL. VCO
92,P-1658,2022,1,NaN,4.00,NaN,440000.00
188,P-1674,2023,4,2.00,NaN,23048.00,NaN
365,P-1693,2022,3,NaN,2827.00,NaN,4156719.00
346,P-1691,2022,8,NaN,954.06,NaN,1531899.00
458,P-1701,2023,6,7.00,NaN,70215.00,NaN


#### 3.2.6 - Se reemplazas columnas NaN por ceros y se crea columna de TOTAL GPD-VCO

In [94]:
df_desp_realizados4.fillna(0, inplace=True)
df_desp_realizados4['TOTAL. VCO - GDP'] = df_desp_realizados4['TOTAL. VCO'] - df_desp_realizados4['TOTAL. GDP']
df_desp_realizados4.sample(5)

,CODIGO_PROYECTO,AÑO,MES,CANT. GDP,CANT. VCO,TOTAL. GDP,TOTAL. VCO,TOTAL. VCO - GDP
405,P-1696,2023,3,0.00,3049.00,0.00,1679529.00,1679529.00
194,P-1675,2022,9,1.00,29.00,75654.00,227766.00,152112.00
175,P-1672,2022,8,0.00,5910.00,0.00,557347.00,557347.00
30,P-1632,2022,4,4.00,0.00,15229.00,0.00,-15229.00
119,P-1664,2022,10,1.00,0.00,1616.00,0.00,-1616.00


#### 3.2.7 - Se seleccionan solamente los registros del mes y año en curso 

In [95]:
# Obtener el año y mes actual
#now = datetime.datetime.now()

now = datetime.now()
current_year = now.year
print(current_year)
current_month = now.month
print(current_month)
print("Dimensiones antes del filtro", df_desp_realizados4.shape)

# Filtrar el dataframe por año y mes
df_desp_realizados5 = df_desp_realizados4[(df_desp_realizados4['AÑO'] == current_year) & (df_desp_realizados4['MES'] == current_month)]
print("Dimensiones después del filtro", df_desp_realizados5.shape)
df_desp_realizados5.sample(5)

2023
6
Dimensiones antes del filtro (597, 8)
Dimensiones después del filtro (40, 8)


,CODIGO_PROYECTO,AÑO,MES,CANT. GDP,CANT. VCO,TOTAL. GDP,TOTAL. VCO,TOTAL. VCO - GDP
297,P-1686,2023,6,0.00,1444.00,0.00,2063949.00,2063949.00
165,P-1670,2023,6,5.00,0.00,3410.00,0.00,-3410.00
183,P-1672,2023,6,24.00,0.00,21744.00,0.00,-21744.00
550,P-1715,2023,6,0.00,726.00,0.00,3303377.00,3303377.00
272,P-1684,2023,6,0.00,17558.00,0.00,10477112.00,10477112.00


#### 3.2.7-2 - Se seleccionan solamente los registros del mes y año en curso 2
Se considera esta línea para poblar la vista del mes anterior.

In [96]:
# Obtener el año y mes actual
#now = datetime.datetime.now()

now = datetime.now()
current_year = now.year
print(current_year)
current_month = now.month
print(current_month)
print("Dimensiones antes del filtro", df_desp_realizados4.shape)

mes_anterior = (datetime.now() - timedelta(days=30)).month

# Filtrar el dataframe por año y mes
df_desp_realizados5_1M = df_desp_realizados4[(df_desp_realizados4['AÑO'] == current_year) & (df_desp_realizados4['MES'] == mes_anterior)]
print("Dimensiones después del filtro", df_desp_realizados5_1M.shape)
df_desp_realizados5_1M.sample(5)

2023
6
Dimensiones antes del filtro (597, 8)
Dimensiones después del filtro (46, 8)


,CODIGO_PROYECTO,AÑO,MES,CANT. GDP,CANT. VCO,TOTAL. GDP,TOTAL. VCO,TOTAL. VCO - GDP
574,P-1723,2023,5,18.00,62814.00,3285765.00,37324477.00,34038712.00
406,P-1696,2023,5,0.00,1.00,0.00,2304440.00,2304440.00
446,P-1700,2023,5,52.00,60.00,13104.00,38040.00,24936.00
474,P-1703,2023,5,0.00,318.00,0.00,200940.00,200940.00
139,P-1667,2023,5,0.00,513.00,0.00,90852.00,90852.00


#### 3.2.8 - Se agrega la fila TOTAL

In [97]:
# Obtener la suma de cada columna y guardarla en un diccionario

print("Dimensiones antes del filtro", df_desp_realizados5.shape)

total_row = {
    'CODIGO_PROYECTO': 'TOTAL',
    'AÑO': df_desp_realizados5['AÑO'].iloc[0], # Obtener el valor del año de la primera fila
    'MES': df_desp_realizados5['MES'].iloc[0], # Obtener el valor del mes de la primera fila
    'CANT. GDP': df_desp_realizados5['CANT. GDP'].sum(),
    'CANT. VCO': df_desp_realizados5['CANT. VCO'].sum(),
    'TOTAL. GDP': df_desp_realizados5['TOTAL. GDP'].sum(),
    'TOTAL. VCO': df_desp_realizados5['TOTAL. VCO'].sum(),
    'TOTAL. VCO - GDP': df_desp_realizados5['TOTAL. VCO - GDP'].sum()
}

total_df = pd.DataFrame(total_row, index=[0]) # Crear un nuevo DataFrame con la fila 'TOTAL'
df_desp_realizados5 = pd.concat([df_desp_realizados5, total_df], ignore_index=True) # Concatenar el nuevo DataFrame con el original

#df_desp_realizados5["TIPO_CUENTA"]= df_desp_realizados5["TOTAL. VCO - GDP"]
df_desp_realizados5["TOTAL"]= df_desp_realizados5["TOTAL. VCO - GDP"]
df_desp_realizados5["TIPO_CUENTA"]= "COSTO MATERIALES"

df_desp_realizados5.sample()

print("Dimensiones después del filtro", df_desp_realizados5.shape)

df_desp_realizados5.sample(5)

Dimensiones antes del filtro (40, 8)
Dimensiones después del filtro (41, 10)


,CODIGO_PROYECTO,AÑO,MES,CANT. GDP,CANT. VCO,TOTAL. GDP,TOTAL. VCO,TOTAL. VCO - GDP,TOTAL,TIPO_CUENTA
0,P-1636,2023,6,0.00,1.00,0.00,166850.00,166850.00,166850.00,COSTO MATERIALES
24,P-1707,2023,6,0.00,5118.00,0.00,3247552.00,3247552.00,3247552.00,COSTO MATERIALES
1,P-1639,2023,6,100.00,0.00,15100.00,0.00,-15100.00,-15100.00,COSTO MATERIALES
16,P-1694,2023,6,0.00,15589.00,0.00,8964664.00,8964664.00,8964664.00,COSTO MATERIALES
37,P-1726,2023,6,0.00,565.00,0.00,1227062.00,1227062.00,1227062.00,COSTO MATERIALES


#### 3.2.8-2 - Se agrega la fila TOTAL 2

In [98]:
# Obtener la suma de cada columna y guardarla en un diccionario

print("Dimensiones antes del filtro", df_desp_realizados5_1M.shape)

total_row = {
    'CODIGO_PROYECTO': 'TOTAL',
    'AÑO': df_desp_realizados5_1M['AÑO'].iloc[0], # Obtener el valor del año de la primera fila
    'MES': df_desp_realizados5_1M['MES'].iloc[0], # Obtener el valor del mes de la primera fila
    'CANT. GDP': df_desp_realizados5_1M['CANT. GDP'].sum(),
    'CANT. VCO': df_desp_realizados5_1M['CANT. VCO'].sum(),
    'TOTAL. GDP': df_desp_realizados5_1M['TOTAL. GDP'].sum(),
    'TOTAL. VCO': df_desp_realizados5_1M['TOTAL. VCO'].sum(),
    'TOTAL. VCO - GDP': df_desp_realizados5_1M['TOTAL. VCO - GDP'].sum()
}

total_df = pd.DataFrame(total_row, index=[0]) # Crear un nuevo DataFrame con la fila 'TOTAL'
df_desp_realizados5_1M = pd.concat([df_desp_realizados5_1M, total_df], ignore_index=True) # Concatenar el nuevo DataFrame con el original

#df_desp_realizados5["TIPO_CUENTA"]= df_desp_realizados5["TOTAL. VCO - GDP"]
df_desp_realizados5_1M["TOTAL"]= df_desp_realizados5_1M["TOTAL. VCO - GDP"]
df_desp_realizados5_1M["TIPO_CUENTA"]= "COSTO MATERIALES"

print("Dimensiones después del filtro", df_desp_realizados5_1M.shape)

df_desp_realizados5_1M.sample(5)

Dimensiones antes del filtro (46, 8)
Dimensiones después del filtro (47, 10)


,CODIGO_PROYECTO,AÑO,MES,CANT. GDP,CANT. VCO,TOTAL. GDP,TOTAL. VCO,TOTAL. VCO - GDP,TOTAL,TIPO_CUENTA
29,P-1704,2023,5,0.00,2549.00,0.00,13398585.00,13398585.00,13398585.00,COSTO MATERIALES
43,P-1725,2023,5,0.00,8020.00,0.00,6708545.00,6708545.00,6708545.00,COSTO MATERIALES
17,P-1688,2023,5,3.00,316.00,135337.00,217172.00,81835.00,81835.00,COSTO MATERIALES
25,P-1699,2023,5,0.00,17026.00,0.00,8687386.00,8687386.00,8687386.00,COSTO MATERIALES
20,P-1692,2023,5,250.00,7060.00,360000.00,1837143.00,1477143.00,1477143.00,COSTO MATERIALES


In [99]:
df_desp_realizados5.to_csv('archivos_respaldo/df_desp_realizados5.csv', mode='a')
df_desp_realizados5_1M.to_csv('archivos_respaldo/df_desp_realizados5_1M.csv', mode='a')

### 3.4 TRATAMIENTO "PROYECTADO"

#### 3.4.1 CREACIÓN NUEVO DATAFRAME DE COPIA.

In [100]:
df_proyectado2 = df_proyectado
df_proyectado2.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 16361,Unnamed: 16362,Unnamed: 16363,Unnamed: 16364,Unnamed: 16365,Unnamed: 16366,Unnamed: 16367,Unnamed: 16368,Unnamed: 16369,Unnamed: 16370
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4212000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,OBRA,PROYECTO,PROYECTO2,MATERIALES,DESPACHOS A LA FECHA,CONTROL MATERIALES,MANO DE OBRA,PRESENTACION EPP,PROYECCIÓN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,CERRADA,P-1600,TERRANOVA,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,CERRADA,P-1601,CUATRE CASAS,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,CERRADA,P-1615,ECO SANTIAGO,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 3.4.1-2 CREACIÓN NUEVO DATAFRAME DE COPIA 2.

In [101]:
df_proyectado2_1M = df_proyectado_1M
df_proyectado2_1M.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 16361,Unnamed: 16362,Unnamed: 16363,Unnamed: 16364,Unnamed: 16365,Unnamed: 16366,Unnamed: 16367,Unnamed: 16368,Unnamed: 16369,Unnamed: 16370
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,OBRA,PROYECTO,PROYECTO2,MATERIALES,DESPACHOS A LA FECHA,CONTROL MATERIALES,MANO DE OBRA,PRESENTACION EPP,PROYECCIÓN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,CERRADA,P-1600,TERRANOVA,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,CERRADA,P-1601,CUATRE CASAS,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,CERRADA,P-1615,ECO SANTIAGO,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 3.4.2 LIMPIEZA DE DATAFRAME Y SELECCIÓN DE COLUMNAS UTILES

In [102]:
df_proyectado2 = df_proyectado2.drop(range(0, 6))
new_header = df_proyectado2.iloc[0]
df_proyectado2 = df_proyectado2[1:]
df_proyectado2.columns = new_header
df_proyectado2 = df_proyectado2.loc[:, ["PROYECTO", "PROYECTO2", "INGRESO","MATERIALES","MANO DE OBRA"]]
df_proyectado2.sample(10)

6,PROYECTO,PROYECTO2,INGRESO,MATERIALES,MANO DE OBRA
33,P-1679,JUAN SABAJ,NaN,NaN,NaN
27,P-1672,SANTA CLARA,NaN,NaN,NaN
43,P-1690,ONGOLMO,5000000,2000000,NaN
8,P-1601,CUATRE CASAS,NaN,NaN,NaN
11,P-1652,ABDON CIFUENTES,NaN,NaN,NaN
85,NaN,NaN,138800000,NaN,NaN
34,P-1680,NUEVA UNO,15000000,5000000,NaN
84,NaN,NaN,130000000,NaN,0.32
41,P-1688,SUIZA,2500000,1000000,NaN
82,NaN,NaN,423800000,176535806,0


#### 3.4.2-2 LIMPIEZA DE DATAFRAME Y SELECCIÓN DE COLUMNAS UTILES 2

In [103]:
df_proyectado2_1M = df_proyectado2_1M.drop(range(0, 6))
new_header = df_proyectado2_1M.iloc[0]
df_proyectado2_1M = df_proyectado2_1M[1:]
df_proyectado2_1M.columns = new_header
df_proyectado2_1M = df_proyectado2_1M.loc[:, ["PROYECTO", "PROYECTO2", "INGRESO","MATERIALES","MANO DE OBRA"]]
df_proyectado2_1M.sample(10)

6,PROYECTO,PROYECTO2,INGRESO,MATERIALES,MANO DE OBRA
42,P-1689,CASINO,NaN,NaN,NaN
32,P-1678,PIRER,NaN,NaN,NaN
19,P-1660,CORONEL ALVARADO,NaN,NaN,NaN
54,P-1701,ZENTENO,NaN,NaN,NaN
89,NaN,NaN,NaN,NaN,NaN
83,NaN,NaN,-327717,NaN,NaN
66,P-1715,Lo Echevers,18230799.58,6000000,NaN
49,P-1696,SANTA VICTORIA,NaN,NaN,NaN
56,P-1703,SANTA VICTORIA DANACORP,NaN,NaN,NaN
37,P-1684,GOYCOLEA,3335671.64,1100000,NaN


In [104]:
# INSPECCIÓN VISUAL 1
df_proyectado2_1M[df_proyectado2_1M["PROYECTO"]=="P-1726"]

6,PROYECTO,PROYECTO2,INGRESO,MATERIALES,MANO DE OBRA
77,P-1726,LA GLORIA,25429159,10000000,NaN


#### 3.4.3 TRATAMIENTO DE DATOS NAN

In [105]:
# seleccionar las columnas deseadas
df_proyectado3 = df_proyectado2[["PROYECTO", "PROYECTO2", "INGRESO","MATERIALES","MANO DE OBRA"]]
# eliminar la columna índice
df_proyectado3 = df_proyectado3.reset_index(drop=True)
# eliminar las filas que contengan NaN en la columna P
df_proyectado3 = df_proyectado3.dropna(subset=['PROYECTO'])
# reemplazar los valores NaN en la columna INGRESO por cero
df_proyectado3['INGRESO'] = df_proyectado3['INGRESO'].fillna(0)
df_proyectado3['MATERIALES'] = df_proyectado3['MATERIALES'].fillna(0)
df_proyectado3['MANO DE OBRA'] = df_proyectado3['MANO DE OBRA'].fillna(0)

# renombrar la columna P a CODIGO_PROYECTO
df_proyectado3 = df_proyectado3.rename(columns={'PROYECTO': 'CODIGO_PROYECTO'})
df_proyectado3 = df_proyectado3.rename(columns={'PROYECTO2': 'PROYECTO'})

# poner en mayúscula el contenido de las columnas CODIGO_PROYECTO y PROYECTO
df_proyectado3['CODIGO_PROYECTO'] = df_proyectado3['CODIGO_PROYECTO'].str.upper()
df_proyectado3['PROYECTO'] = df_proyectado3['PROYECTO'].str.upper()

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_proyectado3  : ", df_proyectado3.shape)

df_proyectado3.sample(10)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_proyectado3  :  (74, 5)


6,CODIGO_PROYECTO,PROYECTO,INGRESO,MATERIALES,MANO DE OBRA
40,P-1694,ECO HUEMUL,18500000,8000000,0
30,P-1684,GOYCOLEA,25000000,12035806,0
21,P-1674,BEIERSDORF,0,0,0
36,P-1690,ONGOLMO,5000000,2000000,0
71,P-1727,BUDNIK,26500000,10000000,0
32,P-1686,ICALMA,5000000,2000000,0
68,P-1724,185 VOLTA,0,0,0
59,P-1715,LO ECHEVERS,11500000,6000000,0
34,P-1688,SUIZA,2500000,1000000,0
2,P-1615,ECO SANTIAGO,0,0,0


In [106]:
print(df_proyectado3.columns)

Index(['CODIGO_PROYECTO', 'PROYECTO', 'INGRESO', 'MATERIALES', 'MANO DE OBRA'], dtype='object', name=6)


In [107]:
# Eliminar la columna '6'
df_proyectado3 = df_proyectado3.rename_axis(None, axis=1)
df_proyectado3 = df_proyectado3.dropna(subset=['CODIGO_PROYECTO'])

print(df_proyectado3.shape)
df_proyectado3.sample(15)

(72, 5)


,CODIGO_PROYECTO,PROYECTO,INGRESO,MATERIALES,MANO DE OBRA
28,P-1682,PLAZA CEPPI,0,0,0
18,P-1670,MK SEGURIDAS,0,0,0
61,P-1717,ECO QUILIN,1000000,200000,0
71,P-1727,BUDNIK,26500000,10000000,0
13,P-1661,SAN VICENTE,0,0,0
46,P-1700,ORIZON,0,0,0
50,P-1704,MERCADO LIBRE,62000000,27000000,0
19,P-1671,BARRIO PARQUE 8 Y 9,0,0,0
40,P-1694,ECO HUEMUL,18500000,8000000,0
0,P-1600,TERRANOVA,0,0,0


#### 3.4.3-2 TRATAMIENTO DE DATOS NAN 2

In [108]:
# seleccionar las columnas deseadas
df_proyectado3_1M = df_proyectado2_1M[["PROYECTO", "PROYECTO2", "INGRESO","MATERIALES","MANO DE OBRA"]]
# eliminar la columna índice
df_proyectado3_1M = df_proyectado3_1M.reset_index(drop=True)
# eliminar las filas que contengan NaN en la columna P
df_proyectado3_1M = df_proyectado3_1M.dropna(subset=['PROYECTO'])
# reemplazar los valores NaN en la columna INGRESO por cero
df_proyectado3_1M['INGRESO'] = df_proyectado3_1M['INGRESO'].fillna(0)
df_proyectado3_1M['MATERIALES'] = df_proyectado3_1M['MATERIALES'].fillna(0)
df_proyectado3_1M['MANO DE OBRA'] = df_proyectado3_1M['MANO DE OBRA'].fillna(0)

# renombrar la columna P a CODIGO_PROYECTO
df_proyectado3_1M = df_proyectado3_1M.rename(columns={'PROYECTO': 'CODIGO_PROYECTO'})
df_proyectado3_1M = df_proyectado3_1M.rename(columns={'PROYECTO2': 'PROYECTO'})

# poner en mayúscula el contenido de las columnas CODIGO_PROYECTO y PROYECTO
df_proyectado3_1M['CODIGO_PROYECTO'] = df_proyectado3_1M['CODIGO_PROYECTO'].str.upper()
df_proyectado3_1M['PROYECTO'] = df_proyectado3_1M['PROYECTO'].str.upper()

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_proyectado3_1M  : ", df_proyectado3_1M.shape)

df_proyectado3_1M.sample(10)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_proyectado3_1M  :  (73, 5)


6,CODIGO_PROYECTO,PROYECTO,INGRESO,MATERIALES,MANO DE OBRA
42,P-1696,SANTA VICTORIA,0.00,0,0
41,P-1695,ESTORIL,13003381.00,10500000,0
15,P-1667,ECO PACILLA,0.00,0,0
44,P-1698,PCI ANDES,0.00,0,0
16,P-1668,QUILLAYES,0.00,0,0
32,P-1686,ICALMA,10669236.17,4000000,0
21,P-1674,BEIERSDORF,0.00,0,0
5,P-1632,CHORILLOS CLOROX,0.00,0,0
6,P-1638,COLON,0.00,0,0
7,P-1642,CORINTOS,0.00,0,0


In [109]:
# INSPECCIÓN VISUAL 1
df_proyectado3_1M[df_proyectado3_1M["CODIGO_PROYECTO"]=="P-1726"]

6,CODIGO_PROYECTO,PROYECTO,INGRESO,MATERIALES,MANO DE OBRA
70,P-1726,LA GLORIA,25429159.00,10000000,0


#### 3.4.4 SE AGREGA LA COLUMNA TOTAL PARA EL DATAFRAME Y SE FILTRA POR EL MES EN CURSO

In [110]:
# Obtener el año y mes actual

import datetime

current_year = datetime.datetime.now().year
current_month = datetime.datetime.now().month

# Calcula la sumatoria de la columna 'INGRESO'
total_ingreso = df_proyectado3['INGRESO'].sum()
total_materiales = df_proyectado3['MATERIALES'].sum()
total_mano_obra = df_proyectado3['MANO DE OBRA'].sum()

# Crea la fila 'TOTAL'
total_row = {'CODIGO_PROYECTO': 'TOTAL', 'PROYECTO': '', 'INGRESO': total_ingreso,'MATERIALES': total_materiales,'MANO DE OBRA': total_mano_obra}

# Crea un DataFrame con la fila 'TOTAL'
df_total = pd.DataFrame([total_row])

# Concatena el DataFrame original con el DataFrame que contiene la fila 'TOTAL'
df_proyectado3 = pd.concat([df_proyectado3, df_total], ignore_index=True)

#Agreación de columnas para estandarización
df_proyectado3["TOTAL"] = df_proyectado3["INGRESO"]
df_proyectado3["TIPO_CUENTA"] = "INGRESO"
df_proyectado3["MES"] = current_month 
df_proyectado3["AÑO"] = current_year 
df_proyectado3["ORIGEN"] = "PROYECTADO" 

# Se reemplazan valores NaN en 0 
df_proyectado3 = df_proyectado3.fillna(value=0)

# Multiplicar las columnas MATERIALES y MANO DE OBRA por -1 para obtener valores negativos
df_proyectado3[['MATERIALES', 'MANO DE OBRA']] = df_proyectado3[['MATERIALES', 'MANO DE OBRA']].mul(-1)

# Se calcula la columna TOTAL como la suma de INGRESO, MATERIALES y MANO DE OBRA
df_proyectado3['TOTAL'] = df_proyectado3[['INGRESO', 'MATERIALES', 'MANO DE OBRA']].sum(axis=1)

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_proyectado3  : ", df_proyectado3.shape)
df_proyectado3.sample(5)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_proyectado3  :  (73, 10)


,CODIGO_PROYECTO,PROYECTO,INGRESO,MATERIALES,MANO DE OBRA,TOTAL,TIPO_CUENTA,MES,AÑO,ORIGEN
45,P-1701,ZENTENO,0,0,0,0,INGRESO,6,2023,PROYECTADO
35,P-1691,ROSARIO ROSALES,0,0,0,0,INGRESO,6,2023,PROYECTADO
1,P-1601,CUATRE CASAS,0,0,0,0,INGRESO,6,2023,PROYECTADO
68,P-1726,LA GLORIA,12000000,-2000000,0,10000000,INGRESO,6,2023,PROYECTADO
37,P-1693,VOLTA,0,0,0,0,INGRESO,6,2023,PROYECTADO


In [111]:
df_proyectado3['INGRESO'].sum()

847600000

#### 3.4.4-2 SE AGREGA LA COLUMNA TOTAL PARA EL DATAFRAME Y SE FILTRA POR EL MES EN CURSO 2

In [112]:
# Obtener el año y mes actual

import datetime

current_year = datetime.datetime.now().year
last_month = (fecha_actual - timedelta(days=30)).month

# Calcula la sumatoria de la columna 'INGRESO'
total_ingreso    = df_proyectado3_1M['INGRESO'].sum()
total_materiales = df_proyectado3_1M['MATERIALES'].sum()
total_mano_obra  = df_proyectado3_1M['MANO DE OBRA'].sum()

# Crea la fila 'TOTAL'
total_row = {'CODIGO_PROYECTO': 'TOTAL', 'PROYECTO': '', 'INGRESO': total_ingreso,'MATERIALES': total_materiales,'MANO DE OBRA': total_mano_obra}

# Crea un DataFrame con la fila 'TOTAL'
df_total = pd.DataFrame([total_row])

# Concatena el DataFrame original con el DataFrame que contiene la fila 'TOTAL'
df_proyectado3_1M = pd.concat([df_proyectado3_1M, df_total], ignore_index=True)

#Agreación de columnas para estandarización
df_proyectado3_1M["TOTAL"] = df_proyectado3["INGRESO"]
df_proyectado3_1M["TIPO_CUENTA"] = "INGRESO"
df_proyectado3_1M["MES"] = last_month 
df_proyectado3_1M["AÑO"] = current_year 
df_proyectado3_1M["ORIGEN"] = "PROYECTADO" 

# Se reemplazan valores NaN en 0 
df_proyectado3_1M = df_proyectado3_1M.fillna(value=0)

# Multiplicar las columnas MATERIALES y MANO DE OBRA por -1 para obtener valores negativos
df_proyectado3_1M[['MATERIALES', 'MANO DE OBRA']] = df_proyectado3_1M[['MATERIALES', 'MANO DE OBRA']].mul(-1)

# Se calcula la columna TOTAL como la suma de INGRESO, MATERIALES y MANO DE OBRA
df_proyectado3_1M['TOTAL'] = df_proyectado3_1M[['INGRESO', 'MATERIALES', 'MANO DE OBRA']].sum(axis=1)

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_proyectado3_1M  : ", df_proyectado3_1M.shape)
df_proyectado3_1M.sample(5)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_proyectado3_1M  :  (74, 10)


,CODIGO_PROYECTO,PROYECTO,INGRESO,MATERIALES,MANO DE OBRA,TOTAL,TIPO_CUENTA,MES,AÑO,ORIGEN
12,P-1660,CORONEL ALVARADO,0.00,0,0,0.00,INGRESO,5,2023,PROYECTADO
43,P-1697,FORTALEZA ESTOCOLMO,16105752.69,-2500000,0,13605752.69,INGRESO,5,2023,PROYECTADO
47,P-1701,ZENTENO,0.00,0,0,0.00,INGRESO,5,2023,PROYECTADO
6,P-1638,COLON,0.00,0,0,0.00,INGRESO,5,2023,PROYECTADO
60,P-1716,ASTARA,10082518.00,0,0,10082518.00,INGRESO,5,2023,PROYECTADO


In [113]:
# INSPECCIÓN VISUAL 1
df_proyectado3_1M[df_proyectado3_1M["CODIGO_PROYECTO"]=="P-1726"]

,CODIGO_PROYECTO,PROYECTO,INGRESO,MATERIALES,MANO DE OBRA,TOTAL,TIPO_CUENTA,MES,AÑO,ORIGEN
70,P-1726,LA GLORIA,25429159.00,-10000000,0,15429159.00,INGRESO,5,2023,PROYECTADO


#### 3.4.5 ESTRATEGIA PARA TRANSFORMAR LAS COLUMNAS DE MANO DE OBRA, MATERIALES E INGRESOS EN FILAS. 

In [114]:
print("Dimensiones solo df_proyectado3 antes de cambios",df_proyectado3.shape)

# Crear el subDataFrame sin las columnas MANO DE OBRA, MATERIALES, TOTAL y TIPO_CUENTA
df_sub = df_proyectado3.drop(['MANO DE OBRA', 'MATERIALES', 'TOTAL', 'TIPO_CUENTA'], axis=1)

# Agregar la columna TIPO_CUENTA con el valor INGRESO
df_sub['TIPO_CUENTA'] = 'INGRESO'

# Renombrar la columna INGRESO como TOTAL
df_sub = df_sub.rename(columns={'INGRESO': 'TOTAL'})
print("Dimensiones solo INGRESO",df_sub.shape)

# Crear el subDataFrame sin las columnas INGRESO, MATERIALES, TOTAL y TIPO_CUENTA
df_sub2 = df_proyectado3.drop(['INGRESO', 'MATERIALES', 'TOTAL', 'TIPO_CUENTA'], axis=1)

# Agregar la columna TIPO_CUENTA con el valor MANO DE OBRA
df_sub2['TIPO_CUENTA'] = 'MANO DE OBRA'

# Renombrar la columna MANO DE OBRA como TOTAL
df_sub2 = df_sub2.rename(columns={'MANO DE OBRA': 'TOTAL'})
print("Dimensiones solo MANO DE OBRA",df_sub2.shape)


# Crear el subDataFrame sin las columnas INGRESO, MANO DE OBRA, TOTAL y TIPO_CUENTA
df_sub3 = df_proyectado3.drop(['INGRESO', 'MANO DE OBRA', 'TOTAL', 'TIPO_CUENTA'], axis=1)

# Agregar la columna TIPO_CUENTA con el valor MANO DE OBRA
df_sub3['TIPO_CUENTA'] = 'MATERIALES'

# Renombrar la columna MANO DE OBRA como TOTAL
df_sub3 = df_sub3.rename(columns={'MATERIALES': 'TOTAL'})
print("Dimensiones solo MATERIALES",df_sub3.shape)

# Concatenar los DataFrames verticalmente
df_proyectado4 = pd.concat([df_sub, df_sub2, df_sub3])

# Mostrar el nuevo DataFrame concatenado

print("Dimensiones solo df_proyectado4 después de cambios",df_proyectado4.shape)

df_proyectado4.sample(10)

Dimensiones solo df_proyectado3 antes de cambios (73, 10)
Dimensiones solo INGRESO (73, 7)
Dimensiones solo MANO DE OBRA (73, 7)
Dimensiones solo MATERIALES (73, 7)
Dimensiones solo df_proyectado4 después de cambios (219, 7)


,CODIGO_PROYECTO,PROYECTO,TOTAL,MES,AÑO,ORIGEN,TIPO_CUENTA
70,P-1728,VOLTA SEGREGACIÓN,0,6,2023,PROYECTADO,MANO DE OBRA
68,P-1726,LA GLORIA,0,6,2023,PROYECTADO,MANO DE OBRA
37,P-1693,VOLTA,0,6,2023,PROYECTADO,MANO DE OBRA
60,P-1718,PATIO DE COMIDA,19500000,6,2023,PROYECTADO,INGRESO
19,P-1671,BARRIO PARQUE 8 Y 9,0,6,2023,PROYECTADO,MATERIALES
63,P-1721,LUIS PASTER,-2700000,6,2023,PROYECTADO,MATERIALES
59,P-1717,ECO QUILIN,-200000,6,2023,PROYECTADO,MATERIALES
17,P-1669,DEPARTAMENTAL II,0,6,2023,PROYECTADO,INGRESO
33,P-1689,CASINO,0,6,2023,PROYECTADO,MATERIALES
43,P-1699,VILLA PANAMERICANA LOTE B,0,6,2023,PROYECTADO,MANO DE OBRA


#### 3.4.5-2 ESTRATEGIA PARA TRANSFORMAR LAS COLUMNAS DE MANO DE OBRA, MATERIALES E INGRESOS EN FILAS 2  

In [115]:
print("Dimensiones solo df_proyectado3 antes de cambios",df_proyectado3_1M.shape)

# Crear el subDataFrame sin las columnas MANO DE OBRA, MATERIALES, TOTAL y TIPO_CUENTA
df_sub = df_proyectado3_1M.drop(['MANO DE OBRA', 'MATERIALES', 'TOTAL', 'TIPO_CUENTA'], axis=1)

# Agregar la columna TIPO_CUENTA con el valor INGRESO
df_sub['TIPO_CUENTA'] = 'INGRESO'

# Renombrar la columna INGRESO como TOTAL
df_sub = df_sub.rename(columns={'INGRESO': 'TOTAL'})
print("Dimensiones solo INGRESO",df_sub.shape)

# Crear el subDataFrame sin las columnas INGRESO, MATERIALES, TOTAL y TIPO_CUENTA
df_sub2 = df_proyectado3_1M.drop(['INGRESO', 'MATERIALES', 'TOTAL', 'TIPO_CUENTA'], axis=1)

# Agregar la columna TIPO_CUENTA con el valor MANO DE OBRA
df_sub2['TIPO_CUENTA'] = 'MANO DE OBRA'

# Renombrar la columna MANO DE OBRA como TOTAL
df_sub2 = df_sub2.rename(columns={'MANO DE OBRA': 'TOTAL'})
print("Dimensiones solo MANO DE OBRA",df_sub2.shape)


# Crear el subDataFrame sin las columnas INGRESO, MANO DE OBRA, TOTAL y TIPO_CUENTA
df_sub3 = df_proyectado3_1M.drop(['INGRESO', 'MANO DE OBRA', 'TOTAL', 'TIPO_CUENTA'], axis=1)

# Agregar la columna TIPO_CUENTA con el valor MANO DE OBRA
df_sub3['TIPO_CUENTA'] = 'MATERIALES'

# Renombrar la columna MANO DE OBRA como TOTAL
df_sub3 = df_sub3.rename(columns={'MATERIALES': 'TOTAL'})
print("Dimensiones solo MATERIALES",df_sub3.shape)

# Concatenar los DataFrames verticalmente
df_proyectado4_1M = pd.concat([df_sub, df_sub2, df_sub3])

# Mostrar el nuevo DataFrame concatenado

print("Dimensiones solo df_proyectado4 después de cambios",df_proyectado4_1M.shape)

df_proyectado4_1M.sample(10)

Dimensiones solo df_proyectado3 antes de cambios (74, 10)
Dimensiones solo INGRESO (74, 7)
Dimensiones solo MANO DE OBRA (74, 7)
Dimensiones solo MATERIALES (74, 7)
Dimensiones solo df_proyectado4 después de cambios (222, 7)


,CODIGO_PROYECTO,PROYECTO,TOTAL,MES,AÑO,ORIGEN,TIPO_CUENTA
2,P-1615,ECO SANTIAGO,0.00,5,2023,PROYECTADO,MANO DE OBRA
41,P-1695,ESTORIL,0.00,5,2023,PROYECTADO,MANO DE OBRA
52,P-1706,CALIFORNIA,0.00,5,2023,PROYECTADO,MANO DE OBRA
38,P-1692,JOSE PEREZ COTAPO,-500000.00,5,2023,PROYECTADO,MATERIALES
62,P-1718,PATIO DE COMIDA,-12000000.00,5,2023,PROYECTADO,MATERIALES
46,P-1700,ORIZON,0.00,5,2023,PROYECTADO,MANO DE OBRA
29,0,BPA05,0.00,5,2023,PROYECTADO,MANO DE OBRA
70,P-1726,LA GLORIA,25429159.00,5,2023,PROYECTADO,INGRESO
62,P-1718,PATIO DE COMIDA,0.00,5,2023,PROYECTADO,MANO DE OBRA
28,P-1682,PLAZA CEPPI,18808581.00,5,2023,PROYECTADO,INGRESO


In [116]:
# INSPECCIÓN VISUAL 1
df_proyectado4_1M[df_proyectado4_1M["CODIGO_PROYECTO"]=="P-1726"]

,CODIGO_PROYECTO,PROYECTO,TOTAL,MES,AÑO,ORIGEN,TIPO_CUENTA
70,P-1726,LA GLORIA,25429159.00,5,2023,PROYECTADO,INGRESO
70,P-1726,LA GLORIA,0.00,5,2023,PROYECTADO,MANO DE OBRA
70,P-1726,LA GLORIA,-10000000.00,5,2023,PROYECTADO,MATERIALES


#### 3.4.6 CAMBIO DE VALORES EN COLUMNA TIPO_CUENTA

In [117]:
# Cambiar los valores de la columna TIPO_CUENTA
df_proyectado4['TIPO_CUENTA'] = df_proyectado4['TIPO_CUENTA'].replace({'MATERIALES': '2. COSTO MATERIALES', 'MANO DE OBRA': '3. COSTO MANO DE OBRA', 'INGRESO': '1. INGRESO'})

# Mostrar el DataFrame con los valores cambiados
print("Dimensiones solo df_proyectado4 después de cambios",df_proyectado4.shape)
df_proyectado4.sample(10)

Dimensiones solo df_proyectado4 después de cambios (219, 7)


,CODIGO_PROYECTO,PROYECTO,TOTAL,MES,AÑO,ORIGEN,TIPO_CUENTA
6,P-1638,COLON,0,6,2023,PROYECTADO,2. COSTO MATERIALES
52,P-1708,ECO ESPAÑA,-4200000,6,2023,PROYECTADO,2. COSTO MATERIALES
71,P-1729,ESTACION CENTRAL,0,6,2023,PROYECTADO,3. COSTO MANO DE OBRA
66,P-1724,185 VOLTA,0,6,2023,PROYECTADO,1. INGRESO
51,P-1707,LAS TRANQUERAS,11000000,6,2023,PROYECTADO,1. INGRESO
67,P-1725,LO CAÑA,0,6,2023,PROYECTADO,3. COSTO MANO DE OBRA
14,P-1663,OFICINAS EQUIFAX,0,6,2023,PROYECTADO,2. COSTO MATERIALES
48,P-1704,MERCADO LIBRE,62000000,6,2023,PROYECTADO,1. INGRESO
41,P-1697,FORTALEZA ESTOCOLMO,5000000,6,2023,PROYECTADO,1. INGRESO
12,P-1660,CORONEL ALVARADO,0,6,2023,PROYECTADO,1. INGRESO


In [118]:
# TEST TOTAL "INGRESO" 

df_proyectado4_test = df_proyectado4[df_proyectado4['TIPO_CUENTA'] == '1. INGRESO']
df_proyectado4_test                                     

,CODIGO_PROYECTO,PROYECTO,TOTAL,MES,AÑO,ORIGEN,TIPO_CUENTA
0,P-1600,TERRANOVA,0,6,2023,PROYECTADO,1. INGRESO
1,P-1601,CUATRE CASAS,0,6,2023,PROYECTADO,1. INGRESO
2,P-1615,ECO SANTIAGO,0,6,2023,PROYECTADO,1. INGRESO
3,P-1621,RENAULT,0,6,2023,PROYECTADO,1. INGRESO
4,P-1652,ABDON CIFUENTES,0,6,2023,PROYECTADO,1. INGRESO
5,P-1632,CHORILLOS CLOROX,0,6,2023,PROYECTADO,1. INGRESO
6,P-1638,COLON,0,6,2023,PROYECTADO,1. INGRESO
7,P-1642,CORINTOS,0,6,2023,PROYECTADO,1. INGRESO
8,P-1645,LOS TRES ANTONIO,0,6,2023,PROYECTADO,1. INGRESO
9,P-1654,EXEQUIEL FERNANDEZ 1670,0,6,2023,PROYECTADO,1. INGRESO


#### 3.4.6-2 CAMBIO DE VALORES EN COLUMNA TIPO_CUENTA 2

In [119]:
# Cambiar los valores de la columna TIPO_CUENTA
df_proyectado4_1M['TIPO_CUENTA'] = df_proyectado4_1M['TIPO_CUENTA'].replace({'MATERIALES': '2. COSTO MATERIALES', 'MANO DE OBRA': '3. COSTO MANO DE OBRA', 'INGRESO': '1. INGRESO'})

# Mostrar el DataFrame con los valores cambiados
print("Dimensiones solo df_proyectado4 después de cambios",df_proyectado4_1M.shape)
df_proyectado4_1M.sample(10)

Dimensiones solo df_proyectado4 después de cambios (222, 7)


,CODIGO_PROYECTO,PROYECTO,TOTAL,MES,AÑO,ORIGEN,TIPO_CUENTA
39,P-1693,VOLTA,0.00,5,2023,PROYECTADO,1. INGRESO
70,P-1726,LA GLORIA,0.00,5,2023,PROYECTADO,3. COSTO MANO DE OBRA
25,P-1678,PIRER,0.00,5,2023,PROYECTADO,1. INGRESO
48,P-1702,KARUN VITACURA,0.00,5,2023,PROYECTADO,1. INGRESO
28,P-1682,PLAZA CEPPI,-200000.00,5,2023,PROYECTADO,2. COSTO MATERIALES
21,P-1674,BEIERSDORF,0.00,5,2023,PROYECTADO,2. COSTO MATERIALES
58,P-1714,DEPARTAMENTAL II,0.00,5,2023,PROYECTADO,3. COSTO MANO DE OBRA
68,P-1724,185 VOLTA,0.00,5,2023,PROYECTADO,3. COSTO MANO DE OBRA
20,P-1672,SANTA CLARA,0.00,5,2023,PROYECTADO,2. COSTO MATERIALES
55,P-1709,CC PASEO SAN PEDRO,11127770.00,5,2023,PROYECTADO,1. INGRESO


In [120]:
df_proyectado4_1M[df_proyectado4_1M["CODIGO_PROYECTO"]=="P-1726"]

,CODIGO_PROYECTO,PROYECTO,TOTAL,MES,AÑO,ORIGEN,TIPO_CUENTA
70,P-1726,LA GLORIA,25429159.00,5,2023,PROYECTADO,1. INGRESO
70,P-1726,LA GLORIA,0.00,5,2023,PROYECTADO,3. COSTO MANO DE OBRA
70,P-1726,LA GLORIA,-10000000.00,5,2023,PROYECTADO,2. COSTO MATERIALES


In [121]:
df_proyectado4.to_csv('archivos_respaldo/df_proyectado4.csv', mode='a')
df_proyectado4_1M.to_csv('archivos_respaldo/df_proyectado4_1M.csv', mode='a')

### 3.5 TRATAMIENTO LIBRO MAYOR AÑOS ANTERIORES

#### 3.5.1 - REVISIÓN DE LOS DATOS

In [122]:
df_libro_mayor_old.shape

(17368, 13)

In [123]:
df_libro_mayor_old.describe()

,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario
count,17368,17368,17368,17368,17368,17368,17368,17368,17368,17368,17368,17368,17368
unique,10,2,12,531,5,1312,3453,134,1500,5119,8462,1714,2
top,4101010-000,2022,9,12/31/22 0:00,CIN,1252,REMUNERACIONES DIRECTAS PROYECTO,,76525290-3,,0,0,
freq,12133,9797,1775,252,11773,251,1359,699,406,1958,3199,15265,17359


#### 3.5.2 - Estandarización campo "Debito" y Crédito

In [124]:
# Se crean nuevos campos para trabajar "Debito" y "Credito".

df_libro_mayor_old['Debito_2'] = pd.to_numeric(df_libro_mayor_old['Debito'], errors='coerce')
df_libro_mayor_old['Credito_2'] = pd.to_numeric(df_libro_mayor_old['Credito'], errors='coerce')

# Se reemplaza el valor de los NoNe y Nan 

df_libro_mayor_old['Credito_2'] = df_libro_mayor_old['Credito_2'].fillna(0)

df_libro_mayor_old.head(5)

,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario,Debito_2,Credito_2
1,3101030-000,2021,1,1/6/21 0:00,CVT,47,EP N°26 - OBRA: S-0333 EDIFICIO NC 3000,P-1592,76525290-3,10756,0,6481068,,0.00,6481068.00
2,3101030-000,2021,1,1/6/21 0:00,CVT,47,"UF 222,93 x T/C $29.072,21",P-1592,76525290-3,10756,0,0,,0.00,0.00
3,3101030-000,2021,1,1/6/21 0:00,CVT,47,SEGÚN CONTRATO N°8000033317-00010 - HES N°1000...,P-1592,76525290-3,10756,0,0,,0.00,0.00
4,3101030-000,2021,1,1/6/21 0:00,CVT,60,EP N°4 - OBRA: VIVIPRA,P-1658,79606430-7,10763,0,4301576,,0.00,4301576.00
5,3101030-000,2021,1,1/6/21 0:00,CVT,60,"UF 148 x T/C $29.064,70",P-1658,79606430-7,10763,0,0,,0.00,0.00


#### 3.5.3 - Creación campo Codigo_proyecto

In [125]:
# Se crean una nueva columna para manejar el código del proyecto 

df_libro_mayor_old['CODIGO_PROYECTO'] = df_libro_mayor_old['Primer_Analisis']

cols = df_libro_mayor_old.columns.tolist() # Obtiene el nombre de todas las columnas en una lista
cols.insert(0, cols.pop(cols.index('CODIGO_PROYECTO'))) # Extrae la columna 'Codigo_proyecto', la inserta en la posición 0 y actualiza la lista
df_libro_mayor_old = df_libro_mayor_old.reindex(columns=cols) # Reindexa el dataframe con las columnas en el orden actualizado

# Transformación a mayúscula código del proyecto
df_libro_mayor_old['CODIGO_PROYECTO'] = df_libro_mayor_old['CODIGO_PROYECTO'].str.upper()

df_libro_mayor_old.head(5)

,CODIGO_PROYECTO,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario,Debito_2,Credito_2
1,P-1592,3101030-000,2021,1,1/6/21 0:00,CVT,47,EP N°26 - OBRA: S-0333 EDIFICIO NC 3000,P-1592,76525290-3,10756,0,6481068,,0.00,6481068.00
2,P-1592,3101030-000,2021,1,1/6/21 0:00,CVT,47,"UF 222,93 x T/C $29.072,21",P-1592,76525290-3,10756,0,0,,0.00,0.00
3,P-1592,3101030-000,2021,1,1/6/21 0:00,CVT,47,SEGÚN CONTRATO N°8000033317-00010 - HES N°1000...,P-1592,76525290-3,10756,0,0,,0.00,0.00
4,P-1658,3101030-000,2021,1,1/6/21 0:00,CVT,60,EP N°4 - OBRA: VIVIPRA,P-1658,79606430-7,10763,0,4301576,,0.00,4301576.00
5,P-1658,3101030-000,2021,1,1/6/21 0:00,CVT,60,"UF 148 x T/C $29.064,70",P-1658,79606430-7,10763,0,0,,0.00,0.00


#### 3.5.4 - Creación de campo total

In [126]:
df_libro_mayor_old['TOTAL'] = df_libro_mayor_old['Credito_2'] - df_libro_mayor_old['Debito_2']
df_libro_mayor_old.head(5)

,CODIGO_PROYECTO,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario,Debito_2,Credito_2,TOTAL
1,P-1592,3101030-000,2021,1,1/6/21 0:00,CVT,47,EP N°26 - OBRA: S-0333 EDIFICIO NC 3000,P-1592,76525290-3,10756,0,6481068,,0.00,6481068.00,6481068.00
2,P-1592,3101030-000,2021,1,1/6/21 0:00,CVT,47,"UF 222,93 x T/C $29.072,21",P-1592,76525290-3,10756,0,0,,0.00,0.00,0.00
3,P-1592,3101030-000,2021,1,1/6/21 0:00,CVT,47,SEGÚN CONTRATO N°8000033317-00010 - HES N°1000...,P-1592,76525290-3,10756,0,0,,0.00,0.00,0.00
4,P-1658,3101030-000,2021,1,1/6/21 0:00,CVT,60,EP N°4 - OBRA: VIVIPRA,P-1658,79606430-7,10763,0,4301576,,0.00,4301576.00,4301576.00
5,P-1658,3101030-000,2021,1,1/6/21 0:00,CVT,60,"UF 148 x T/C $29.064,70",P-1658,79606430-7,10763,0,0,,0.00,0.00,0.00


#### 3.5.5 - Creación campo tipo_cuenta "basado en el campo cta_Contable"

In [127]:
# Función para categorizar la columna 'Cta_Contable'
def categorizar_cta_contable(cta_contable):
    if cta_contable =="3101030-000" or cta_contable == "3401002-000":
        return '1. INGRESO'
    elif cta_contable == "4101010-000":
        return '2. COSTO MATERIALES'
    elif cta_contable == "4150002-000":
        return '3. COSTO MANO DE OBRA'
    elif cta_contable == "4150002-002":
        return '4. COSTO SUPERVISIÓN'
    else:
        return 'OTRA'

# Crear la nueva columna 'Tipo_cuenta' aplicando la función
df_libro_mayor_old['Tipo_cuenta'] = df_libro_mayor_old['Cta_Contable'].apply(categorizar_cta_contable)
df_libro_mayor_old.sample(5)

,CODIGO_PROYECTO,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario,Debito_2,Credito_2,TOTAL,Tipo_cuenta
16747,P-PROYECTOS,4150002-001,2022,1,1/31/22 0:00,AJC,1232,REMUNERACIONES INDIRECTAS PROYECTO,P-Proyectos,P-Proyectos,27155507,809441,0,,809441.00,0.00,-809441.00,OTRA
5971,,4101010-000,2021,10,10/31/21 0:00,CRE,1187,VIA-100,,11474055,8,44000,0,,44000.00,0.00,-44000.00,2. COSTO MATERIALES
13234,P-1712,4101010-000,2022,9,9/12/22 0:00,CCO,480,COSTOS DE PROYECTOS,p-1712,76821330-5,29384,6370,0,,6370.00,0.00,-6370.00,2. COSTO MATERIALES
12101,P-1700,4101010-000,2022,7,7/26/22 0:00,CIN,440,Central Incendio Notifier NFS320,P-1700,P07120949,46452,1369000,0,,1369000.00,0.00,-1369000.00,2. COSTO MATERIALES
16920,P-OFICINA,4150002-003,2022,7,7/31/22 0:00,AJC,1508,REMUNERACIONES DIRECTAS PROYECTO POR DISTRIBUIR,P-Oficina,11474055,,523149,0,,523149.00,0.00,-523149.00,OTRA


In [128]:
df_libro_mayor_old['Tipo_cuenta'].unique()

array(['1. INGRESO', '2. COSTO MATERIALES', 'OTRA',
       '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN'], dtype=object)

In [129]:
# Revisión de los datos creados

df_libro_mayor_old_piv = pd.pivot_table(df_libro_mayor_old, values=['Debito_2','Credito_2','TOTAL'], index=['Tipo_cuenta','Año','Mes'], aggfunc='sum', fill_value=0, margins=True, margins_name='TOTAL')
df_libro_mayor_old_piv = pd.DataFrame(df_libro_mayor_old_piv.to_records())

#df_libro_mayor_piv.head(5)
df_libro_mayor_old_piv.sample(10)

,Tipo_cuenta,Año,Mes,Credito_2,Debito_2,TOTAL
26,2. COSTO MATERIALES,2021,11,2334550,23327140,-20992590
78,4. COSTO SUPERVISIÓN,2021,4,0,2990002,-2990002
75,4. COSTO SUPERVISIÓN,2021,12,0,2799714,-2799714
84,4. COSTO SUPERVISIÓN,2022,1,0,3549389,-3549389
18,1. INGRESO,2022,4,177303564,80863736,96439828
45,2. COSTO MATERIALES,2022,7,1279821,91618849,-90339028
106,OTRA,2021,8,0,10668933,-10668933
1,1. INGRESO,2021,10,188858961,118021388,70837573
69,3. COSTO MANO DE OBRA,2022,7,0,65074622,-65074622
11,1. INGRESO,2021,9,138688902,66847588,71841314


#### 3.5.6 - Se eliminan números de proyectos que no deben ser parte del análisis

In [130]:
# Se crea una copia del dataframe original debido que se reduciran las dimensiones
df_libro_mayor_old2 = df_libro_mayor_old.copy()

# Se identifican los códigos de proyecto disponibles
print("Proyectos disponibles antes de la reducción : ")
print(df_libro_mayor_old2["CODIGO_PROYECTO"].unique())
print("=======================================================================")
print("Dimensiones del dataframe antes de la reducción : ")
print(df_libro_mayor_old2.shape)


Proyectos disponibles antes de la reducción : 
['P-1592' 'P-1658' 'P-1638' 'P-1615' 'P-1639' 'S-1601' 'P-1644' 'P-1628'
 'P-1652' 'P-1666' 'P-1653' 'P-1654' 'P-1656' 'P-1665' 'P-1664' 'P-1670'
 'P-1629' 'P-1649' 'P-1657' 'P-1661' 'P-1671' 'P-1645' 'P-1626' 'P-1611'
 'P-1642' 'P-1648' 'P-1659' 'P-1617' 'P-1627' 'P-1641' 'P-1660' 'P-1636'
 'P-1625' 'P-1667' 'P-1672' 'P-1637' 'P-1623' 'P-1674' 'P-1668' 'P-1676'
 'P-1646' 'P-1675' 'P-1678' 'P-1643' 'P-1632' 'P-1673' 'P-1669' 'P-1677'
 'P-1681' 'P-1683' 'P-1679' 'P-1680' 'P-1684' 'P-1689' 'P-1686' 'P-1685'
 'P-1690' 'P-1691' 'P-OFICINA' 'P-PROYECTOS' 'P-1663' 'P-1640' 'P-1655'
 'P-1631' 'P-1647' 'P-1610' 'P-1612' 'P-1527' 'P-1590' 'P-1616' 'P-1622'
 'P-1633' '' 'P-1692' 'P-1687' 'P-1693' 'P-1694' 'P-1695' 'P-1698'
 'P-1696' 'P-1688' 'P-1697' 'P-1700' 'P-1701' 'P-1702' 'P-1703' 'P-1709'
 'P-1707' 'P-1704' 'P-1712' 'P-1713' 'P-1708' 'P-1682' 'P-1714' 'P-1706'
 'P-1705' 'P-1699' 'P-1715' 'P-1716' 'P-1718' 'P-1717' 'P-1613'
 'P-ANALISIS PROYE' 

##### 3.5.6.1 - Se crea un subdataframe para COSTOS SUPERVISIÓN (separando lo que tiene P-OFICINA y cta_contable 4150002-002)

In [131]:
# Se crea un dataframe separado para tratar los que poseen proyecto "P-OFICINA" ya que los costos de supervisión solo aplican al total
df_libro_mayor_old2_2 = df_libro_mayor_old2.copy()

# Filtrar el DataFrame
df_libro_mayor_old2_2 = df_libro_mayor_old2_2[(df_libro_mayor_old2_2['CODIGO_PROYECTO'] == 'P-OFICINA') & (df_libro_mayor_old2_2['Cta_Contable'] == '4150002-002')]

# Mostrar el DataFrame filtrado
df_libro_mayor_old2_2


,CODIGO_PROYECTO,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario,Debito_2,Credito_2,TOTAL,Tipo_cuenta
16836,P-OFICINA,4150002-002,2022,9,9/30/22 0:00,AJC,1325,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,0,1724280,0,,1724280.00,0.00,-1724280.00,4. COSTO SUPERVISIÓN
16837,P-OFICINA,4150002-002,2022,9,9/30/22 0:00,AJC,1325,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,0,2183185,0,,2183185.00,0.00,-2183185.00,4. COSTO SUPERVISIÓN
16838,P-OFICINA,4150002-002,2022,10,10/31/22 0:00,AJC,1220,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,,2234754,0,,2234754.00,0.00,-2234754.00,4. COSTO SUPERVISIÓN
16839,P-OFICINA,4150002-002,2022,10,10/31/22 0:00,AJC,1220,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,,2584134,0,,2584134.00,0.00,-2584134.00,4. COSTO SUPERVISIÓN
16840,P-OFICINA,4150002-002,2022,11,11/30/22 0:00,AJC,1385,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,,2369840,0,,2369840.00,0.00,-2369840.00,4. COSTO SUPERVISIÓN
16841,P-OFICINA,4150002-002,2022,11,11/30/22 0:00,AJC,1385,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,,2719503,0,,2719503.00,0.00,-2719503.00,4. COSTO SUPERVISIÓN
16842,P-OFICINA,4150002-002,2022,12,12/31/22 0:00,AJC,1252,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,,2153705,0,,2153705.00,0.00,-2153705.00,4. COSTO SUPERVISIÓN
16843,P-OFICINA,4150002-002,2022,12,12/31/22 0:00,AJC,1252,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,,2502914,0,,2502914.00,0.00,-2502914.00,4. COSTO SUPERVISIÓN


In [132]:
# Elimina los registros NaN en la columna Codigo_proyecto
df_libro_mayor_old2 = df_libro_mayor_old2.dropna(subset=['CODIGO_PROYECTO'])

# Define una lista de valores a eliminar
valores_a_eliminar = ['P-PROYECTOS', 'P-SERVICIOS','P-OFICINA', 'P-ANALISIS PROYE', '13056802-5', '16411600-K', '18800250-1','P-MANTENCIÓN','18800250','13056802','16411600']

# Crea una máscara booleana que identifica las filas que contienen los valores a eliminar
mask = df_libro_mayor_old2['CODIGO_PROYECTO'].isin(valores_a_eliminar)

# Crea un nuevo dataframe sin las filas que contienen los valores a eliminar
df_libro_mayor_old2 = df_libro_mayor_old2[~mask]


In [133]:
# Se identifican los códigos de proyecto disponibles
print("Proyectos disponibles antes de la reducción : ")
print(df_libro_mayor_old.shape)
print(df_libro_mayor_old["CODIGO_PROYECTO"].unique())
print("=======================================================================")
print("Dimensiones del dataframe antes de la reducción : ")
print(df_libro_mayor_old2.shape)
print(df_libro_mayor_old2["CODIGO_PROYECTO"].unique())


Proyectos disponibles antes de la reducción : 
(17368, 18)
['P-1592' 'P-1658' 'P-1638' 'P-1615' 'P-1639' 'S-1601' 'P-1644' 'P-1628'
 'P-1652' 'P-1666' 'P-1653' 'P-1654' 'P-1656' 'P-1665' 'P-1664' 'P-1670'
 'P-1629' 'P-1649' 'P-1657' 'P-1661' 'P-1671' 'P-1645' 'P-1626' 'P-1611'
 'P-1642' 'P-1648' 'P-1659' 'P-1617' 'P-1627' 'P-1641' 'P-1660' 'P-1636'
 'P-1625' 'P-1667' 'P-1672' 'P-1637' 'P-1623' 'P-1674' 'P-1668' 'P-1676'
 'P-1646' 'P-1675' 'P-1678' 'P-1643' 'P-1632' 'P-1673' 'P-1669' 'P-1677'
 'P-1681' 'P-1683' 'P-1679' 'P-1680' 'P-1684' 'P-1689' 'P-1686' 'P-1685'
 'P-1690' 'P-1691' 'P-OFICINA' 'P-PROYECTOS' 'P-1663' 'P-1640' 'P-1655'
 'P-1631' 'P-1647' 'P-1610' 'P-1612' 'P-1527' 'P-1590' 'P-1616' 'P-1622'
 'P-1633' '' 'P-1692' 'P-1687' 'P-1693' 'P-1694' 'P-1695' 'P-1698'
 'P-1696' 'P-1688' 'P-1697' 'P-1700' 'P-1701' 'P-1702' 'P-1703' 'P-1709'
 'P-1707' 'P-1704' 'P-1712' 'P-1713' 'P-1708' 'P-1682' 'P-1714' 'P-1706'
 'P-1705' 'P-1699' 'P-1715' 'P-1716' 'P-1718' 'P-1717' 'P-1613'
 'P-ANAL

#### 3.5.7 - Se crea un nuevo campo de fecha basado en la fecha original

In [134]:
df_libro_mayor_old2['Fecha2'] = pd.to_datetime(df_libro_mayor_old2['Año'].astype(str) + '-' + df_libro_mayor_old2['Mes'].astype(str) + '-1')
df_libro_mayor_old2.sample(5)

,CODIGO_PROYECTO,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario,Debito_2,Credito_2,TOTAL,Tipo_cuenta,Fecha2
13641,P-1696,4101010-000,2022,9,9/28/22 0:00,CIN,794,Cable paralelo 2x18 color,P-1696,T14381073,47361,33800,0,,33800.00,0.00,-33800.00,2. COSTO MATERIALES,2022-09-01
6961,,4150002-001,2021,6,6/29/21 0:00,CRE,763,SBA-100,,12291050,1,881204,0,,881204.00,0.00,-881204.00,OTRA,2021-06-01
13150,P-1667,4101010-000,2022,9,9/7/22 0:00,CIN,212,Cable UTP CAT5E SFTP Multifilar exterior Blindado,P-1667,T14380918,47115,46970,0,,46970.00,0.00,-46970.00,2. COSTO MATERIALES,2022-09-01
1047,P-1666,3401002-000,2021,6,6/30/21 0:00,AJC,970,Ajuste por anticipos y retenciones,P-1666,P-1666,,0,6254300,,0.00,6254300.00,6254300.00,1. INGRESO,2021-06-01
16393,P-1694,4150002-000,2022,10,10/31/22 0:00,AJC,1220,REMUNERACIONES DIRECTAS PROYECTO,P-1694,12764949,,183949,0,,183949.00,0.00,-183949.00,3. COSTO MANO DE OBRA,2022-10-01


##### 3.5.7.1 - Se crea un nuevo campo de fecha basado en la fecha original para  COSTOS SUPERVISIÓN

In [135]:
df_libro_mayor_old2_2['Fecha2'] = pd.to_datetime(df_libro_mayor_old2_2['Año'].astype(str) + '-' + df_libro_mayor_old2_2['Mes'].astype(str) + '-1')
df_libro_mayor_old2_2.sample(5)

,CODIGO_PROYECTO,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario,Debito_2,Credito_2,TOTAL,Tipo_cuenta,Fecha2
16836,P-OFICINA,4150002-002,2022,9,9/30/22 0:00,AJC,1325,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,0,1724280,0,,1724280.00,0.00,-1724280.00,4. COSTO SUPERVISIÓN,2022-09-01
16839,P-OFICINA,4150002-002,2022,10,10/31/22 0:00,AJC,1220,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,,2584134,0,,2584134.00,0.00,-2584134.00,4. COSTO SUPERVISIÓN,2022-10-01
16843,P-OFICINA,4150002-002,2022,12,12/31/22 0:00,AJC,1252,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,,2502914,0,,2502914.00,0.00,-2502914.00,4. COSTO SUPERVISIÓN,2022-12-01
16841,P-OFICINA,4150002-002,2022,11,11/30/22 0:00,AJC,1385,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,,2719503,0,,2719503.00,0.00,-2719503.00,4. COSTO SUPERVISIÓN,2022-11-01
16837,P-OFICINA,4150002-002,2022,9,9/30/22 0:00,AJC,1325,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,0,2183185,0,,2183185.00,0.00,-2183185.00,4. COSTO SUPERVISIÓN,2022-09-01


In [136]:
df_libro_mayor_old2_2['Tipo_cuenta'].unique()

array(['4. COSTO SUPERVISIÓN'], dtype=object)

In [137]:
df_libro_mayor_old2_2[df_libro_mayor_old2_2["Primer_Analisis"] == "P-1673"]

,CODIGO_PROYECTO,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario,Debito_2,Credito_2,TOTAL,Tipo_cuenta,Fecha2


#### 3.5.8 - Se realiza pivote final del dataframe

In [138]:
# Se exploran las columnas:
df_libro_mayor_old2.columns

Index(['CODIGO_PROYECTO', 'Cta_Contable', 'Año', 'Mes', 'Fecha', 'Tipo_Comp',
       'Num_Comp', 'Glosa', 'Primer_Analisis', 'Referencia', 'Num_Referencia',
       'Debito', 'Credito', 'Comentario ', 'Debito_2', 'Credito_2', 'TOTAL',
       'Tipo_cuenta', 'Fecha2'],
      dtype='object', name=0)

In [139]:
# crear pivot table sin margenes

df_libro_mayor_old2_piv = pd.pivot_table(df_libro_mayor_old2, values=['Debito_2','Credito_2','TOTAL'], index=['CODIGO_PROYECTO','Tipo_cuenta','Fecha2','Año','Mes'], aggfunc='sum', fill_value=0, margins=True, margins_name='TOTAL')
df_libro_mayor_old2_piv = pd.DataFrame(df_libro_mayor_old2_piv.to_records())

df_libro_mayor_old2_piv.sample(5)

,CODIGO_PROYECTO,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
1598,P-1714,3. COSTO MANO DE OBRA,2022-08-01,2022,8,0,328169,-328169
928,P-1674,1. INGRESO,2021-09-01,2021,9,1084477,0,1084477
334,P-1645,3. COSTO MANO DE OBRA,2022-02-01,2022,2,0,257346,-257346
1563,P-1707,3. COSTO MANO DE OBRA,2022-12-01,2022,12,0,1407421,-1407421
1511,P-1703,2. COSTO MATERIALES,2022-05-01,2022,5,0,328789,-328789


In [140]:
df_libro_mayor_old2_piv[df_libro_mayor_old2_piv["CODIGO_PROYECTO"] == "P-1704"]

,CODIGO_PROYECTO,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
1523,P-1704,1. INGRESO,2022-07-01,2022,7,179643612,0,179643612
1524,P-1704,1. INGRESO,2022-08-01,2022,8,84292194,0,84292194
1525,P-1704,1. INGRESO,2022-09-01,2022,9,97802473,34754970,63047503
1526,P-1704,1. INGRESO,2022-10-01,2022,10,49425495,0,49425495
1527,P-1704,1. INGRESO,2022-11-01,2022,11,42952357,0,42952357
1528,P-1704,1. INGRESO,2022-12-01,2022,12,89246807,42952357,46294450
1529,P-1704,2. COSTO MATERIALES,2022-05-01,2022,5,0,273115,-273115
1530,P-1704,2. COSTO MATERIALES,2022-06-01,2022,6,518865,27455060,-26936195
1531,P-1704,2. COSTO MATERIALES,2022-07-01,2022,7,0,24099085,-24099085
1532,P-1704,2. COSTO MATERIALES,2022-08-01,2022,8,0,26228061,-26228061


In [141]:
# Estas son las dimensiones del dataframe pivoteado:
df_libro_mayor_old2_piv.shape

(1632, 8)

##### 3.5.8.1 - Se realiza pivote final del dataframe paera COSTO SUPERVISIÓN

In [142]:
# crear pivot table sin margenes

df_libro_mayor_old2_2_piv = pd.pivot_table(df_libro_mayor_old2_2, values=['Debito_2','Credito_2','TOTAL'], index=['CODIGO_PROYECTO','Tipo_cuenta','Fecha2','Año','Mes'], aggfunc='sum', fill_value=0, margins=True, margins_name='TOTAL')
df_libro_mayor_old2_2_piv = pd.DataFrame(df_libro_mayor_old2_2_piv.to_records())

df_libro_mayor_old2_2_piv = df_libro_mayor_old2_2_piv[df_libro_mayor_old2_2_piv['CODIGO_PROYECTO'] != 'TOTAL']

# Se reemplaza el valor P-OFICINA por el valor TOTAL con lo cual se utilizará para unirlo al dataframe que crea las filas para el cálculo de los totales
df_libro_mayor_old2_2_piv['CODIGO_PROYECTO'] = df_libro_mayor_old2_2_piv['CODIGO_PROYECTO'].replace('P-OFICINA', 'TOTAL')

df_libro_mayor_old2_2_piv.sample()

,CODIGO_PROYECTO,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
1,TOTAL,4. COSTO SUPERVISIÓN,2022-10-01,2022,10,0,4818888,-4818888


#### 3.5.9 - Se crea subdataframe para proyecto que considera el total con nombre "TOTAL", que se detalla por tipo_cuenta


In [143]:
df_libro_mayor_old2_piv2 = pd.pivot_table(df_libro_mayor_old2_piv, values=['Debito_2','Credito_2','TOTAL'], index=['Tipo_cuenta','Fecha2','Año','Mes'], aggfunc='sum', fill_value=0)
df_libro_mayor_old2_piv2 = pd.DataFrame(df_libro_mayor_old2_piv2.to_records())

# Agregamos la columna Codigo_proyecto con valor "TOTAL" en la primera columna
df_libro_mayor_old2_piv2.insert(loc=0, column='CODIGO_PROYECTO', value='TOTAL')

# Eliminamos los registros con Tipo_cuenta igual a "TOTAL"
df_libro_mayor_old2_piv2 = df_libro_mayor_old2_piv2[df_libro_mayor_old2_piv2['Tipo_cuenta'] != 'TOTAL']
df_libro_mayor_old2_piv2 = df_libro_mayor_old2_piv2[df_libro_mayor_old2_piv2['Tipo_cuenta'] != '']

df_libro_mayor_old2_piv2.sample(5)

,CODIGO_PROYECTO,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
57,TOTAL,3. COSTO MANO DE OBRA,2021-10-01,2021,10,0,1677696,-1677696
68,TOTAL,3. COSTO MANO DE OBRA,2022-09-01,2022,9,0,74019234,-74019234
25,TOTAL,2. COSTO MATERIALES,2021-02-01,2021,2,394704,40981843,-40587139
34,TOTAL,2. COSTO MATERIALES,2021-11-01,2021,11,2334550,23327140,-20992590
8,TOTAL,1. INGRESO,2021-09-01,2021,9,138688902,66847588,71841314


##### 3.5.9.1 - Se anexa el dataframe creado con los totales con el dataframe creado a partir del punto 3.1.6.1 que contiene los costos de supervisión

In [144]:
df_libro_mayor_old2_piv_fin = pd.concat([df_libro_mayor_old2_piv2, df_libro_mayor_old2_2_piv], ignore_index=True)
df_libro_mayor_old2_piv_fin.sample(10)

,CODIGO_PROYECTO,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
0,TOTAL,1. INGRESO,2021-01-01,2021,1,153082726,51479131,101603595
106,TOTAL,4. COSTO SUPERVISIÓN,2022-10-01,2022,10,0,4818888,-4818888
90,TOTAL,OTRA,2021-07-01,2021,7,0,10093565,-10093565
96,TOTAL,OTRA,2022-01-01,2022,1,0,1000000,-1000000
2,TOTAL,1. INGRESO,2021-03-01,2021,3,227194679,39876627,187318052
45,TOTAL,2. COSTO MATERIALES,2022-10-01,2022,10,1727688,78591009,-76863321
36,TOTAL,2. COSTO MATERIALES,2022-01-01,2022,1,870354,24254206,-23383852
11,TOTAL,1. INGRESO,2021-12-01,2021,12,98684388,25304241,73380147
31,TOTAL,2. COSTO MATERIALES,2021-08-01,2021,8,454260,21287509,-20833249
74,TOTAL,4. COSTO SUPERVISIÓN,2021-03-01,2021,3,0,3079552,-3079552


#### 3.5.10 - Se concatenan los 2 dataframes resultantes

In [145]:
print("=======================================================================")
print("Dimensiones del dataframe antes de la concatenación : ")
print(df_libro_mayor_old2_piv_fin.shape)

Dimensiones del dataframe antes de la concatenación : 
(109, 8)


In [146]:
df_libro_mayor_old3 = pd.concat([df_libro_mayor_old2_piv, df_libro_mayor_old2_piv_fin])
df_libro_mayor_old3.sort_values(by=['CODIGO_PROYECTO']).sample(5)
print(df_libro_mayor_old3.shape)

(1741, 8)


#### 3.5.11 - Estandarización de campos 

In [147]:
# Se estandarizan los campos con sus nombres y formatos, para un mayor entendimiento.

df_libro_mayor_old3 = df_libro_mayor_old3.rename(columns={
    'Codigo_proyecto': 'CODIGO_PROYECTO',
    'Tipo_cuenta': 'TIPO_CUENTA',
    'Fecha2': 'FECHA',
    'Año': 'AÑO',
    'Mes': 'MES',
    'Credito_2': 'CREDITO',
    'Debito_2': 'DEBITO'
})

df_libro_mayor_old3['AÑO'] = pd.to_numeric(df_libro_mayor_old3['AÑO'], downcast='integer')
df_libro_mayor_old3['MES'] = pd.to_numeric(df_libro_mayor_old3['MES'], downcast='integer')

df_libro_mayor_old3.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL
952,P-1675,2. COSTO MATERIALES,2022-03-01,2022.00,3.00,0,2123769,-2123769
61,TOTAL,3. COSTO MANO DE OBRA,2022-02-01,2022.00,2.00,0,14497984,-14497984
322,P-1645,2. COSTO MATERIALES,2022-08-01,2022.00,8.00,0,88574,-88574
963,P-1675,3. COSTO MANO DE OBRA,2022-04-01,2022.00,4.00,0,822349,-822349
79,TOTAL,4. COSTO SUPERVISIÓN,2021-08-01,2021.00,8.00,0,2767200,-2767200
128,P-1626,2. COSTO MATERIALES,2021-01-01,2021.00,1.00,0,290022,-290022
1300,P-1691,1. INGRESO,2022-09-01,2022.00,9.00,4368636,0,4368636
824,P-1670,3. COSTO MANO DE OBRA,2021-06-01,2021.00,6.00,0,0,0
253,P-1642,1. INGRESO,2021-06-01,2021.00,6.00,1036817,0,1036817
89,TOTAL,OTRA,2021-06-01,2021.00,6.00,0,11751595,-11751595


#### 3.5.14 Creación de columna MARGEN

In [148]:
df_libro_mayor_old3_2 = df_libro_mayor_old3.copy()

#df_libro_mayor4_2['TOTAL'] = pd.to_numeric(df_libro_mayor4_2['AÑO'], downcast='float')

df_margen = df_libro_mayor_old3_2.groupby(['CODIGO_PROYECTO', 'MES']).apply(lambda x: pd.Series({
    'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['INGRESO','COSTO SUPERVISIÓN', 'COSTO MANO DE OBRA', 'COSTO MATERIALES']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == 'INGRESO', 'TOTAL'].sum()
}))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)

/var/folders/50/1s4nc051355gh33rchwmpfkr0000gp/T/ipykernel_1581/3531166699.py:6: RuntimeWarning: invalid value encountered in long_scalars
  'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['INGRESO','COSTO SUPERVISIÓN', 'COSTO MANO DE OBRA', 'COSTO MATERIALES']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == 'INGRESO', 'TOTAL'].sum()
/var/folders/50/1s4nc051355gh33rchwmpfkr0000gp/T/ipykernel_1581/3531166699.py:6: RuntimeWarning: invalid value encountered in long_scalars
  'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['INGRESO','COSTO SUPERVISIÓN', 'COSTO MANO DE OBRA', 'COSTO MATERIALES']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == 'INGRESO', 'TOTAL'].sum()
/var/folders/50/1s4nc051355gh33rchwmpfkr0000gp/T/ipykernel_1581/3531166699.py:6: RuntimeWarning: invalid value encountered in long_scalars
  'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['INGRESO','COSTO SUPERVISIÓN', 'COSTO MANO DE OBRA', 'COSTO MATERIALES']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == 'INGRESO', 'TOTAL'].sum()
/var/folders/50/1s4nc051355g

,CODIGO_PROYECTO,MES,MARGEN
470,P-1683,12.00,0.00
432,P-1677,11.00,0.00
252,P-1658,9.00,0.00
596,P-1698,5.00,0.00
681,S-1601,7.00,0.00
169,P-1645,9.00,0.00
341,P-1669,1.00,0.00
248,P-1658,5.00,0.00
688,TOTAL,7.00,0.00
161,P-1645,1.00,0.00


In [149]:
print("=======================================================================")
print("Dimensiones del dataframe antes del merge : ")
print(df_libro_mayor_old3.shape)
print("=======================================================================")
print("Dimensiones del dataframe antes del merge : ")
print(df_margen.shape)

Dimensiones del dataframe antes del merge : 
(1741, 8)
Dimensiones del dataframe antes del merge : 
(694, 3)


In [150]:
df_libro_mayor_old3 = pd.merge(df_libro_mayor_old3, df_margen, on=['CODIGO_PROYECTO', 'MES'], how='left')
#EJEMPLO
#df_libro_mayor5[df_libro_mayor5["CODIGO_PROYECTO"]=="P-1706"]
df_libro_mayor_old3[df_libro_mayor_old3["CODIGO_PROYECTO"]=="TOTAL"]

# Se estandariza el valor TOTAL a 1.TOTAL para que pueda ser seleccionado desde los filtros del Dashboard.
df_libro_mayor_old3['CODIGO_PROYECTO'] = df_libro_mayor_old3['CODIGO_PROYECTO'].apply(lambda x: '1. TOTAL' if x == 'TOTAL' else x)

In [151]:
print("=======================================================================")
print("Dimensiones del dataframe después de la concatenación : ")
print(df_libro_mayor_old3.shape)

Dimensiones del dataframe después de la concatenación : 
(1741, 9)


In [152]:
df_libro_mayor_old3.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
780,P-1669,2. COSTO MATERIALES,2022-02-01,2022.00,2.00,65853,0,65853,0.00
1530,P-1704,2. COSTO MATERIALES,2022-06-01,2022.00,6.00,518865,27455060,-26936195,0.00
1112,P-1682,1. INGRESO,2022-11-01,2022.00,11.00,17611400,0,17611400,0.00
993,P-1676,2. COSTO MATERIALES,2022-09-01,2022.00,9.00,0,153738,-153738,0.00
1218,P-1686,3. COSTO MANO DE OBRA,2022-12-01,2022.00,12.00,0,3565686,-3565686,0.00
733,P-1668,1. INGRESO,2021-12-01,2021.00,12.00,4610905,0,4610905,0.00
874,P-1672,2. COSTO MATERIALES,2021-12-01,2021.00,12.00,27600,4049499,-4021899,0.00
637,P-1665,2. COSTO MATERIALES,2021-01-01,2021.00,1.00,0,4049500,-4049500,0.00
941,P-1675,1. INGRESO,2021-11-01,2021.00,11.00,1991650,0,1991650,0.00
1424,P-1697,2. COSTO MATERIALES,2022-09-01,2022.00,9.00,0,871004,-871004,0.00


In [153]:
df_libro_mayor_old3.to_csv('archivos_respaldo/df_libro_mayor_old3.csv', mode='a')

### 3.6 TRATAMIENTO DE "PLANILLA PRESUPUESTO"

In [154]:
# Se crea una copia del dataframe llamado "presupuesto"
df_presupuesto2 = df_presupuesto.copy()

In [155]:
# Eliminar primera fila completa
df_presupuesto2 = df_presupuesto2.iloc[1:]

# Filtrar y mantener solo las columnas deseadas
df_presupuesto2 = df_presupuesto2[['PROYECTO', 'MONTO EN $']]

df_presupuesto2 = df_presupuesto2.rename(columns={'PROYECTO': 'CODIGO_PROYECTO', 'MONTO EN $': 'VALOR'})

df_presupuesto2.sample(10)

,CODIGO_PROYECTO,VALOR
33,P-1689,20986779.50
9,P-1661,48586986.00
34,P-1690,64872885.78
46,P-1702,0.00
62,P-1720,10690929.98
30,P-1686,105081270.13
3,P-1645,18150000.00
50,P-1706,104786220.54
4,P-1647,29482131.00
19,P-1675,17376021.17


In [156]:
# Se agregan nuevas columnas tipo de cuenta y Origen.

df_presupuesto2["TIPO_CUENTA"]= "1. INGRESO"
df_presupuesto2["ORIGEN"]= "PLANILLA PRESUPUESTO"

df_presupuesto2.sample(10)

,CODIGO_PROYECTO,VALOR,TIPO_CUENTA,ORIGEN
51,P-1707,67945807.16,1. INGRESO,PLANILLA PRESUPUESTO
31,P-1687,25285885.04,1. INGRESO,PLANILLA PRESUPUESTO
11,P-1667,85409828.00,1. INGRESO,PLANILLA PRESUPUESTO
47,P-1703,56316406.63,1. INGRESO,PLANILLA PRESUPUESTO
14,P-1670,202553946.00,1. INGRESO,PLANILLA PRESUPUESTO
60,P-1718,184014731.01,1. INGRESO,PLANILLA PRESUPUESTO
15,P-1671,125912115.00,1. INGRESO,PLANILLA PRESUPUESTO
54,P-1712,215321946.83,1. INGRESO,PLANILLA PRESUPUESTO
5,P-1654,23880352.49,1. INGRESO,PLANILLA PRESUPUESTO
52,P-1708,213452539.86,1. INGRESO,PLANILLA PRESUPUESTO


### 3.7 TRATAMIENTO DE "TABLA PRESUPUESTO"

In [157]:
# Se crea una copia del dataframe para realizar el tratamiento.

df_tabla_presupuesto2 = df_tabla_prespuesto.copy()
print("DIMENSIONES DATAFRAME df_tabla_prespuesto2", df_tabla_presupuesto2.shape)
df_tabla_presupuesto2.sample(5)

DIMENSIONES DATAFRAME df_tabla_prespuesto2 (13831, 7)


,CODIGO_PROYECTO,PARTIDA_PRESUPUESTARIA,PRODUCTO,UNIDAD,CANTIDAD,COSTO_UNITARIO,COSTO_TOTAL
6137,P-1658,1114,SUPERVISION,,1,71132,71132
2346,P-1626,1639,"""An13 Spk 2do Piso-Sprinkler 1 Colgante Semiem...",,23,3659,84157
7459,P-1674,1100,"Soportes y Antisismicos-Perno U 6"""" """,UNIDA,1,1812,1812
10960,P-1708,1092,TC-COPLA SC/APC-SC/APC,,600,226,135600
7207,P-1671,1081,Control de Acceso 9-PC I5,,1,480000,480000


In [158]:
# Se estandarizan los tipos de campo y se reemplazan valores NaN

df_tabla_presupuesto2['COSTO_UNITARIO'] = pd.to_numeric(df_tabla_presupuesto2['COSTO_UNITARIO'], errors='coerce')
df_tabla_presupuesto2['COSTO_TOTAL'] = pd.to_numeric(df_tabla_presupuesto2['COSTO_TOTAL'], errors='coerce')

# Se reemplaza el valor de los NoNe y Nan 

df_tabla_presupuesto2['COSTO_UNITARIO'] = df_tabla_presupuesto2['COSTO_UNITARIO'].fillna(0)
df_tabla_presupuesto2['COSTO_TOTAL'] = df_tabla_presupuesto2['COSTO_TOTAL'].fillna(0)

print("DIMENSIONES DATAFRAME df_tabla_prespuesto2", df_tabla_presupuesto2.shape)
df_tabla_presupuesto2.head(5)

DIMENSIONES DATAFRAME df_tabla_prespuesto2 (13831, 7)


,CODIGO_PROYECTO,PARTIDA_PRESUPUESTARIA,PRODUCTO,UNIDAD,CANTIDAD,COSTO_UNITARIO,COSTO_TOTAL
0,P-1602,1001,DETECCIÓN-Central de detección Hochiki FNP112...,,1,428490.00,428490.00
1,P-1602,1002,DETECCIÓN-Sensor Humo Hochiki ALN-C,,49,20866.00,1022434.00
2,P-1602,1003,DETECCIÓN-Sensor de temperatura Hochiki ATJ-EA,,4,23846.00,95384.00
3,P-1602,1004,DETECCIÓN-Base Sensores Hochiki HSB-NSA-6,,53,2608.00,138224.00
4,P-1602,1005,DETECCIÓN-Palanca Manual Hochiki DCP-AMS,,13,27200.00,353600.00


In [159]:
#### Creación de campo tipo_costo basado en el valor del a columna "PRODUCTO"

def clasificar_tipo_costo(producto):
    if   producto.upper().startswith("MANO DE OBRA"):
        return "3. COSTO MANO DE OBRA"
    elif producto.upper().startswith("SUPER"):
        return "4. COSTO SUPERVISIÓN"
    else:
        return "2. COSTO MATERIALES"

df_tabla_presupuesto2['TIPO_COSTO'] = df_tabla_presupuesto2['PRODUCTO'].apply(clasificar_tipo_costo)

df_tabla_presupuesto2.head(5)

,CODIGO_PROYECTO,PARTIDA_PRESUPUESTARIA,PRODUCTO,UNIDAD,CANTIDAD,COSTO_UNITARIO,COSTO_TOTAL,TIPO_COSTO
0,P-1602,1001,DETECCIÓN-Central de detección Hochiki FNP112...,,1,428490.00,428490.00,2. COSTO MATERIALES
1,P-1602,1002,DETECCIÓN-Sensor Humo Hochiki ALN-C,,49,20866.00,1022434.00,2. COSTO MATERIALES
2,P-1602,1003,DETECCIÓN-Sensor de temperatura Hochiki ATJ-EA,,4,23846.00,95384.00,2. COSTO MATERIALES
3,P-1602,1004,DETECCIÓN-Base Sensores Hochiki HSB-NSA-6,,53,2608.00,138224.00,2. COSTO MATERIALES
4,P-1602,1005,DETECCIÓN-Palanca Manual Hochiki DCP-AMS,,13,27200.00,353600.00,2. COSTO MATERIALES


In [160]:
# crear pivot table sin margenes

df_tabla_presupuesto3 = pd.pivot_table(df_tabla_presupuesto2, values=['COSTO_TOTAL'], index=['CODIGO_PROYECTO','TIPO_COSTO'], aggfunc='sum', fill_value=0)

df_tabla_presupuesto3 = pd.DataFrame(df_tabla_presupuesto3.to_records())
df_tabla_presupuesto3.sample(10)


,CODIGO_PROYECTO,TIPO_COSTO,COSTO_TOTAL
283,P-1697,3. COSTO MANO DE OBRA,0.00
289,P-1700,2. COSTO MATERIALES,72938784.00
162,P-1656,2. COSTO MATERIALES,12184301.00
263,P-1689,4. COSTO SUPERVISIÓN,491478.00
58,P-1621,3. COSTO MANO DE OBRA,5890760.00
368,P-838,2. COSTO MATERIALES,10000.00
159,P-1655,2. COSTO MATERIALES,22164917.00
35,P-1613,4. COSTO SUPERVISIÓN,1346292.00
318,P-1710,2. COSTO MATERIALES,77603774.00
292,P-1701,2. COSTO MATERIALES,42377348.00


In [161]:

# Se crea columna VALOR y se considera el valor en negativo.
df_tabla_presupuesto3['TOTAL'] = df_tabla_presupuesto3['COSTO_TOTAL'].apply(lambda x: x * -1 if x != 0 else 0)

# Se crea la columna de ORIGEN del archivo
df_tabla_presupuesto3["ORIGEN"] = "TABLA_PRESUPUESTO"

print("DIMENSIÓN DE LA TABLA ES", df_tabla_presupuesto3.shape)
df_tabla_presupuesto3.sample(5)

DIMENSIÓN DE LA TABLA ES (388, 5)


,CODIGO_PROYECTO,TIPO_COSTO,COSTO_TOTAL,TOTAL,ORIGEN
62,P-1622,4. COSTO SUPERVISIÓN,642985.00,-642985.00,TABLA_PRESUPUESTO
243,P-1683,2. COSTO MATERIALES,29098536.60,-29098536.60,TABLA_PRESUPUESTO
237,P-1681,2. COSTO MATERIALES,4132408.00,-4132408.00,TABLA_PRESUPUESTO
179,P-1661,4. COSTO SUPERVISIÓN,2015626.00,-2015626.00,TABLA_PRESUPUESTO
353,P-1723,4. COSTO SUPERVISIÓN,0.00,0.00,TABLA_PRESUPUESTO


In [162]:
# INSPECCIÓN VISUAL
df_tabla_presupuesto_testing = pd.pivot_table(df_tabla_presupuesto3, values=['TOTAL'], index=['TIPO_COSTO'], aggfunc='sum', fill_value=0)
df_tabla_presupuesto_testing

,TOTAL
TIPO_COSTO,
2. COSTO MATERIALES,-5745413332.04
3. COSTO MANO DE OBRA,-1482885459.00
4. COSTO SUPERVISIÓN,-311477854.00


In [163]:
df_tabla_presupuesto3.to_csv('archivos_respaldo/df_tabla_presupuesto3.csv', mode='a')

## 4.- **MERGE Y TRATAMIENTO DE DATAFRAMES 

### 4.1 - **MERGE PARA -> CUADRO FINANCIERO PROYECTOS 
Merge entre Dataframes: df_libro_mayor5 y df_cod_proyecto

In [164]:
print("Dimensiones antes del merge:")
print(df_libro_mayor5.shape)

df_libro_mayor6 = pd.merge(df_libro_mayor5, df_cod_proyecto, on=['CODIGO_PROYECTO'], how='left')

print("Dimensiones después del merge:")
print(df_libro_mayor6.shape)

Dimensiones antes del merge:
(2619, 9)
Dimensiones después del merge:
(2619, 11)


In [165]:
# Reemplazar los valores NaN por "TOTAL" cuando el CODIGO_PROYECTO sea "TOTAL"

cols_to_replace = ['CODIGO_PROYECTO', 'NOMBRE_OBRA', 'NOMBRE_CLIENTE']
df_libro_mayor6[cols_to_replace] = df_libro_mayor6[cols_to_replace].replace('TOTAL', '1. TOTAL')

# Quizás el código anterior sea redundante.
mask = df_libro_mayor6['CODIGO_PROYECTO'] == '1. TOTAL'
df_libro_mayor6.loc[mask, ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'

# Se elimina la columna MERGE la que es innecesaria tener hasta ahora
df_libro_mayor6 = df_libro_mayor6.drop('MARGEN', axis=1)

df_libro_mayor6[df_libro_mayor6["CODIGO_PROYECTO"]== "1. TOTAL"].head(5)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
556,1. TOTAL,,NaT,NaN,NaN,2290563418,1931316000,359247418,1. TOTAL,1. TOTAL
557,1. TOTAL,COSTO MANO DE OBRA,2023-01-01,2023.00,1.00,0,72482994,-72482994,1. TOTAL,1. TOTAL
558,1. TOTAL,COSTO MANO DE OBRA,2023-02-01,2023.00,2.00,0,82645544,-82645544,1. TOTAL,1. TOTAL
559,1. TOTAL,COSTO MANO DE OBRA,2023-03-01,2023.00,3.00,0,96760244,-96760244,1. TOTAL,1. TOTAL
560,1. TOTAL,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,111885468,-111885468,1. TOTAL,1. TOTAL


#### 4.1.1 - AGREGACIÓN DE NUEVA COLUMNA DE TOTAL PARA AGREGAR EN GRÁFICO.
Se agrega nueva columna de total pero considerando el valor de los tipos de costo con valores positivos.

In [166]:
# Crear la nueva columna 'TOTAL_2' con los valores de 'TOTAL' positivos
df_libro_mayor6['TOTAL_2'] = np.where(df_libro_mayor6['TIPO_CUENTA'] != 'INGRESO', np.abs(df_libro_mayor6['TOTAL']), df_libro_mayor6['TOTAL'])
df_libro_mayor6.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_2
739,P-1620,INGRESO,2023-03-01,2023.00,3.00,0,0,0,"OF Apoquindo- Habilitaciones, Integracion y ad...",CONSTRUCTORA INARCO S.A.,0
5,P-1592,COSTO MATERIALES,2023-02-01,2023.00,2.00,0,102993,-102993,Nueva Costanera 3300,EBCO S.A.,102993
1384,P-1675,COSTO MATERIALES,2023-02-01,2023.00,2.00,0,0,0,Palena II,CONSTRUCTORA MYA SpA.,0
673,P-1603,COSTO MANO DE OBRA,2023-06-01,2023.00,6.00,0,0,0,Farellones,Empresa Constructora DLP S.A.,0
114,P-1682,INGRESO,2023-01-01,2023.00,1.00,21858931,0,21858931,Plaza Ceppi,EMP. CONSTRUCTORA INGENIEROS S.A.,21858931
2373,P-1720,COSTO MANO DE OBRA,2023-03-01,2023.00,3.00,0,0,0,NaN,NaN,0
1864,P-1696,COSTO MATERIALES,2023-02-01,2023.00,2.00,0,0,0,Santa Victoria - Terracorp,Constructora Terracorp S.A.,0
754,P-1626,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,0,0,Clinica Bio Bio,Constructora LYD S.A.,0
2171,P-1712,INGRESO,2023-01-01,2023.00,1.00,0,0,0,Centro Comercial Viña Outlet Park,EBCO S.A.,0
64,P-1672,COSTO MATERIALES,2023-06-01,2023.00,6.00,21744,0,21744,Sta Clara (ccdd-telecom),Constructora UPC S.A.,21744


#### 4.1.2 - AJUSTES DE COLUMNAS 

In [167]:
# Renombrar los valores de la columna CODIGO_PROYECTO
df_libro_mayor6['TIPO_CUENTA'] = df_libro_mayor6['TIPO_CUENTA'].replace({
    'INGRESO': '1. INGRESO',
    'COSTO MATERIALES': '2. COSTO MATERIALES',
    'COSTO MANO DE OBRA': '3. COSTO MANO DE OBRA',
    'COSTO SUPERVISIÓN': '4. COSTO SUPERVISIÓN'
})

df_libro_mayor6['CODIGO_PROYECTO'] = df_libro_mayor6['CODIGO_PROYECTO'].replace({
    'TOTAL': '1. TOTAL'})

In [168]:
# Ordenar el DataFrame por la columna CODIGO_PROYECTO
df_libro_mayor6 = df_libro_mayor6.sort_values('CODIGO_PROYECTO')

# Mover la fila con CODIGO_PROYECTO = '1. TOTAL' al primer lugar
df_libro_mayor6 = pd.concat([df_libro_mayor6[df_libro_mayor6['CODIGO_PROYECTO'] == '1. TOTAL'],
                            df_libro_mayor6[df_libro_mayor6['CODIGO_PROYECTO'] != '1. TOTAL']])

df_libro_mayor6['NOMBRE_OBRA'] = df_libro_mayor6['NOMBRE_OBRA'].str.upper()
df_libro_mayor6['NOMBRE_CLIENTE'] = df_libro_mayor6['NOMBRE_CLIENTE'].str.upper()

# Reiniciar los índices del DataFrame
df_libro_mayor6 = df_libro_mayor6.reset_index(drop=True)
df_libro_mayor6.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_2
1607,P-1697,2. COSTO MATERIALES,2023-01-01,2023.00,1.00,0,0,0,ESTOCOLMO,CONSTRUCTORA FORTALEZA S.A,0
769,P-1668,2. COSTO MATERIALES,2023-01-01,2023.00,1.00,78500,0,78500,QUILLAYES PLAZA,E.MOLINA MOREL CONSTRUCTORA S.A.,78500
18,1. TOTAL,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,111885468,-111885468,1. TOTAL,1. TOTAL,111885468
1473,P-1693,3. COSTO MANO DE OBRA,2023-02-01,2023.00,2.00,0,525201,-525201,VOLTA V0800,ECOBIO SPA,525201
1353,P-1690,3. COSTO MANO DE OBRA,2023-03-01,2023.00,3.00,0,0,0,ONGOLMO - HOME CONDOMINIO,CANTAUCO CONSTRUCCION LTDA.,0


#### 4.1.3 - CREACIÓN DE COLUMNA MARGEN

In [169]:
df_margen = df_libro_mayor6.groupby(['CODIGO_PROYECTO', 'MES']).apply(lambda x: pd.Series({
    'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA','4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
}))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)

#warnings.filterwarnings("ignore", category=FutureWarning, message="Inferring datetime64.*")

# mostrar el DataFrame resultante
df_margen.sample(10)

df_libro_mayor6 = pd.merge(df_libro_mayor6, df_margen, on=['CODIGO_PROYECTO', 'MES'], how='left')

# INSPECCIÓN VISUAL 1
df_libro_mayor6[df_libro_mayor6["CODIGO_PROYECTO"]=="P-1704"]


/var/folders/50/1s4nc051355gh33rchwmpfkr0000gp/T/ipykernel_1581/1823500159.py:2: RuntimeWarning: invalid value encountered in long_scalars
  'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA','4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
/var/folders/50/1s4nc051355gh33rchwmpfkr0000gp/T/ipykernel_1581/1823500159.py:2: RuntimeWarning: invalid value encountered in long_scalars
  'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA','4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
/var/folders/50/1s4nc051355gh33rchwmpfkr0000gp/T/ipykernel_1581/1823500159.py:2: RuntimeWarning: invalid value encountered in long_scalars
  'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA','4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', '

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_2,MARGEN
1806,P-1704,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.00
1807,P-1704,3. COSTO MANO DE OBRA,2023-06-01,2023.00,6.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.00
1808,P-1704,4. COSTO SUPERVISIÓN,2023-06-01,2023.00,6.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.00
1809,P-1704,1. INGRESO,2023-06-01,2023.00,6.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.00
1810,P-1704,4. COSTO SUPERVISIÓN,2023-03-01,2023.00,3.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.04
1811,P-1704,2. COSTO MATERIALES,2023-01-01,2023.00,1.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.25
1812,P-1704,1. INGRESO,2023-01-01,2023.00,1.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.25
1813,P-1704,4. COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.66
1814,P-1704,4. COSTO SUPERVISIÓN,2023-01-01,2023.00,1.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.25
1815,P-1704,3. COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.66


In [170]:
df_libro_mayor6.to_csv('archivos_respaldo/df_libro_mayor6.csv', mode='a')

### 4.2 - **MERGE PARA -> YTD AÑO EN CURSO HASTA MES CERRADO
MERGE ENTRE df_libro_mayor_ytd_closed_piv y df_cod_proyecto

In [171]:
print("Dimensiones antes del merge:")
df_libro_mayor_ytd_closed_piv.shape

Dimensiones antes del merge:


(340, 3)

In [172]:
df_ytd_cerrado = df_libro_mayor_ytd_closed_piv.copy()

In [173]:
# Se realiza merge con Nombre de Obra y Cliente.

print("Dimensiones mes_en_curso antes de cambios: ", df_ytd_cerrado.shape)
df_ytd_cerrado = df_ytd_cerrado.merge(df_cod_proyecto, on='CODIGO_PROYECTO', how='left')
print("Dimensiones mes_en_curso2 antes de cambios: ", df_ytd_cerrado.shape)
df_ytd_cerrado.sample(10)

Dimensiones mes_en_curso antes de cambios:  (340, 3)
Dimensiones mes_en_curso2 antes de cambios:  (340, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
170,P-1684,COSTO SUPERVISIÓN,0,Edificio Goycolea,EBCO S.A.
319,P-1724,INGRESO,28935273,NaN,NaN
306,P-1721,COSTO SUPERVISIÓN,0,Strip Center Luis Pasteur,Inmobiliaria Luis Pasteur S.A.
41,P-1636,COSTO MATERIALES,0,Santo Domingo 1410,DANACORP S.A
267,P-1709,INGRESO,76827615,Centro Comercial Paseo San Pedro,EBCO S.A.
108,P-1667,COSTO MANO DE OBRA,-6159129,Eco Placilla,Constructora ICF SPA
243,P-1703,INGRESO,35296641,Santa Victoria Danacorp,DANACORP S.A
99,P-1663,INGRESO,0,Oficinas Equifax,Constructora VRK Ingenieria Limitada
66,P-1645,COSTO SUPERVISIÓN,0,Los tres Antonios,Sociedad Constructora Echavarri Hermanos Limitada
189,P-1689,COSTO MATERIALES,26522,Casino Buenaventura,EBCO S.A.


In [174]:
# Se reemplazan nombres de TIPO_CUENTA

df_ytd_cerrado['TIPO_CUENTA'] = df_ytd_cerrado['TIPO_CUENTA'].replace({
    'INGRESO': '1. INGRESO',
    'COSTO MATERIALES': '2. COSTO MATERIALES',
    'COSTO MANO DE OBRA': '3. COSTO MANO DE OBRA',
    'COSTO SUPERVISIÓN': '4. COSTO SUPERVISIÓN'
})
df_ytd_cerrado.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
61,P-1644,2. COSTO MATERIALES,204032,Mirador de San Cristobal,EBCO S.A.
57,P-1643,2. COSTO MATERIALES,-330968,Chorrillos M4,Constructora Suksa S.A.
1,1. TOTAL,2. COSTO MATERIALES,-605032841,NaN,NaN
290,P-1717,4. COSTO SUPERVISIÓN,0,Eco Quilín I,Constructora ICF SPA
91,P-1660,1. INGRESO,0,Coronel Alvarado,Larrain Prieto Risopatron S.A.
13,P-1592,2. COSTO MATERIALES,41716,Nueva Costanera 3300,EBCO S.A.
49,P-1641,2. COSTO MATERIALES,-369720,Inacap Temuco,Desarrollos Constructivos Axis Sociedad Anonima
124,P-1672,3. COSTO MANO DE OBRA,0,Sta Clara (ccdd-telecom),Constructora UPC S.A.
205,P-1693,2. COSTO MATERIALES,103024,Volta V0800,ECOBIO SPA
223,P-1697,1. INGRESO,53923452,Estocolmo,Constructora Fortaleza S.A


In [175]:
# INSPECCIÓN VISUAL 1 
df_ytd_cerrado[df_ytd_cerrado["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
0,1. TOTAL,3. COSTO MANO DE OBRA,-478147965,NaN,NaN
1,1. TOTAL,2. COSTO MATERIALES,-605032841,NaN,NaN
2,1. TOTAL,4. COSTO SUPERVISIÓN,-22566666,NaN,NaN
3,1. TOTAL,1. INGRESO,1579362409,NaN,NaN


In [176]:
#Se renombra "TOTAL" a "1. TOTAL" para que se pueda visualizar en el dashboard.

cols_to_replace = ['CODIGO_PROYECTO', 'NOMBRE_OBRA', 'NOMBRE_CLIENTE']
df_ytd_cerrado[cols_to_replace] = df_ytd_cerrado[cols_to_replace].replace('TOTAL', '1. TOTAL')
mask = df_ytd_cerrado['CODIGO_PROYECTO'] == '1. TOTAL'
df_ytd_cerrado.loc[mask, ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'
df_ytd_cerrado.sample(4)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
189,P-1689,2. COSTO MATERIALES,26522,Casino Buenaventura,EBCO S.A.
274,P-1713,4. COSTO SUPERVISIÓN,0,Reserva San Francisco,EMPRESA NACIONAL DE CONSTRUCCIÓN LAS VERTIENTE...
195,P-1690,1. INGRESO,16662806,Ongolmo - Home Condominio,CANTAUCO CONSTRUCCION LTDA.
97,P-1663,2. COSTO MATERIALES,0,Oficinas Equifax,Constructora VRK Ingenieria Limitada


In [177]:
# INSPECCIÓN VISUAL 1
df_ytd_cerrado[df_ytd_cerrado["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
0,1. TOTAL,3. COSTO MANO DE OBRA,-478147965,1. TOTAL,1. TOTAL
1,1. TOTAL,2. COSTO MATERIALES,-605032841,1. TOTAL,1. TOTAL
2,1. TOTAL,4. COSTO SUPERVISIÓN,-22566666,1. TOTAL,1. TOTAL
3,1. TOTAL,1. INGRESO,1579362409,1. TOTAL,1. TOTAL


#### 4.2.1 - MERGE CON "df_libro_mayor5_base2" PARA CREAR DATAFRAME FINAL

In [178]:
print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado   : ", df_ytd_cerrado.shape)

df_ytd_cerrado2 = df_ytd_cerrado.filter(["CODIGO_PROYECTO", "TIPO_CUENTA", "TOTAL"])

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado2 : ", df_ytd_cerrado2.shape)

df_ytd_cerrado2.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado   :  (340, 5)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado2 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
263,P-1708,1. INGRESO,392191
30,P-1620,4. COSTO SUPERVISIÓN,0
154,P-1680,4. COSTO SUPERVISIÓN,0
300,P-1720,3. COSTO MANO DE OBRA,-65665
256,P-1707,3. COSTO MANO DE OBRA,-7528867


In [179]:
# Exploración dataframe base

df_libro_mayor5_base2.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
269,P-1680,4. COSTO SUPERVISIÓN,0,NUEVA UNO,CONSTRUCTORA TECSA S.A.
92,P-1698,2. COSTO MATERIALES,0,CD ANDES,EBCO S.A.
147,P-1721,2. COSTO MATERIALES,0,STRIP CENTER LUIS PASTEUR,INMOBILIARIA LUIS PASTEUR S.A.
263,P-1675,4. COSTO SUPERVISIÓN,0,PALENA II,CONSTRUCTORA MYA SPA.
160,P-1726,2. COSTO MATERIALES,0,NaN,NaN


In [180]:
# Merge Dataframe BASE 

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2   : ", df_libro_mayor5_base2.shape)

df_ytd_cerrado3 = df_libro_mayor5_base2.merge(df_ytd_cerrado2, on=['CODIGO_PROYECTO', 'TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado3       : ", df_ytd_cerrado3.shape)

df_ytd_cerrado3.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2   :  (340, 5)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado3       :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
278,P-1685,1. INGRESO,0,MAPA EDIFICIOS NO OPERACIONALES,CLARO VICUÑA VALENZUELA S.A.,0
51,P-1682,3. COSTO MANO DE OBRA,0,PLAZA CEPPI,EMP. CONSTRUCTORA INGENIEROS S.A.,-10329193
35,P-1672,1. INGRESO,0,STA CLARA (CCDD-TELECOM),CONSTRUCTORA UPC S.A.,9435519
156,P-1725,3. COSTO MANO DE OBRA,0,LO CAÑAS,CONSTRUCTORA BRISAS DE BATUCO,-5242316
288,P-1691,4. COSTO SUPERVISIÓN,0,ROSARIO ROSALES,CONSTRUCTORA E INMOBILIARIA PRADOVERDE LTDA.,0


In [181]:
# INSPECCIÓN VISUAL 1

df_ytd_cerrado3[df_ytd_cerrado3["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-22566666
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,1579362409
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-605032841
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-478147965


In [182]:
# INSPECCIÓN VISUAL 2

df_ytd_cerrado3[df_ytd_cerrado3["CODIGO_PROYECTO"]=="P-1724"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
153,P-1724,3. COSTO MANO DE OBRA,0,NaN,NaN,-506066
154,P-1724,2. COSTO MATERIALES,0,NaN,NaN,-7660814
155,P-1724,1. INGRESO,0,NaN,NaN,28935273
327,P-1724,4. COSTO SUPERVISIÓN,0,NaN,NaN,0


#### 4.2.2 - SE REALIZA CREACIÓN DE COLUMNA MARGEN

In [183]:
# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_ytd_cerrado3.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES','3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_margen       : ", df_margen.shape)

# mostrar el DataFrame resultante
df_margen.sample(10)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_margen       :  (85, 2)


,CODIGO_PROYECTO,MARGEN
32,P-1673,0.00
29,P-1669,0.00
54,P-1696,0.03
38,P-1680,0.11
81,P-1726,0.31
52,P-1694,0.27
43,P-1685,0.00
34,P-1675,1.00
39,P-1681,1.00
3,P-1592,0.00


In [184]:
df_margen[df_margen["CODIGO_PROYECTO"]=="P-1724"]

,CODIGO_PROYECTO,MARGEN
79,P-1724,0.72


In [185]:
# Merge Dataframe para agregar MARGEN

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado3   : ", df_ytd_cerrado3.shape)

df_ytd_cerrado3 = df_ytd_cerrado3.merge(df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado3 : ", df_ytd_cerrado3.shape)

df_ytd_cerrado3.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado3   :  (340, 6)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado3 :  (340, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
250,P-1669,1. INGRESO,0,DEPARTAMENTAL,EMP. CONSTRUCTORA INGENIEROS S.A.,0,0.00
274,P-1683,1. INGRESO,0,CD BP-A05,EBCO S.A.,0,0.00
52,P-1682,2. COSTO MATERIALES,0,PLAZA CEPPI,EMP. CONSTRUCTORA INGENIEROS S.A.,-10525711,0.49
318,P-1719,3. COSTO MANO DE OBRA,0,CD METALPAR,EBCO S.A.,0,1.00
249,P-1668,4. COSTO SUPERVISIÓN,0,QUILLAYES PLAZA,E.MOLINA MOREL CONSTRUCTORA S.A.,0,0.00


In [186]:
df_ytd_cerrado3.to_csv('archivos_respaldo/df_ytd_cerrado3.csv', index = False)

### 4.2-1 - MERGE PARA -> YTD AÑO EN CURSO HASTA MES CERRADO (PERSPECTIVA MES ANTERIOR)
MERGE ENTRE df_libro_mayor_ytd_closed_2M y df_cod_proyecto

In [187]:
print("Dimensiones antes del merge:")
df_libro_mayor_ytd_closed_2M.shape

Dimensiones antes del merge:


(340, 3)

In [188]:
df_ytd_cerrado_2M = df_libro_mayor_ytd_closed_2M.copy()

In [189]:
# Se realiza merge con Nombre de Obra y Cliente.

print("Dimensiones df_ytd_cerrado_2M antes de cambios: ", df_ytd_cerrado_2M.shape)
df_ytd_cerrado_2M = df_ytd_cerrado_2M.merge(df_cod_proyecto, on='CODIGO_PROYECTO', how='left')
print("Dimensiones df_ytd_cerrado_2M antes de cambios: ", df_ytd_cerrado_2M.shape)
df_ytd_cerrado_2M.sample(10)

Dimensiones df_ytd_cerrado_2M antes de cambios:  (340, 3)
Dimensiones df_ytd_cerrado_2M antes de cambios:  (340, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
243,P-1703,INGRESO,35296641,Santa Victoria Danacorp,DANACORP S.A
223,P-1697,INGRESO,37817699,Estocolmo,Constructora Fortaleza S.A
141,P-1676,COSTO MATERIALES,-2031638,Lo Campino,EBCO S.A.
60,P-1644,COSTO MANO DE OBRA,0,Mirador de San Cristobal,EBCO S.A.
325,P-1726,COSTO MATERIALES,-14374046,NaN,NaN
112,P-1668,COSTO MANO DE OBRA,0,Quillayes Plaza,E.MOLINA MOREL CONSTRUCTORA S.A.
23,P-1611,INGRESO,0,Edificio Plaza Egaña,EBCO S.A.
135,P-1674,INGRESO,0,Oficinas Beiersdorf Chorrillos,Constructora Suksa S.A.
178,P-1686,COSTO SUPERVISIÓN,0,Macul Icalma,EMPRESA CONSTRUCTORA INGEVEC S.A.
324,P-1726,COSTO MANO DE OBRA,-2654140,NaN,NaN


In [190]:
# Se reemplazan nombres de TIPO_CUENTA

df_ytd_cerrado_2M['TIPO_CUENTA'] = df_ytd_cerrado_2M['TIPO_CUENTA'].replace({
    'INGRESO': '1. INGRESO',
    'COSTO MATERIALES': '2. COSTO MATERIALES',
    'COSTO MANO DE OBRA': '3. COSTO MANO DE OBRA',
    'COSTO SUPERVISIÓN': '4. COSTO SUPERVISIÓN'
})
df_ytd_cerrado_2M.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
136,P-1675,3. COSTO MANO DE OBRA,0,Palena II,CONSTRUCTORA MYA SpA.
190,P-1689,4. COSTO SUPERVISIÓN,0,Casino Buenaventura,EBCO S.A.
9,P-1591,2. COSTO MATERIALES,0,Edificio Estoril,EBCO S.A.
293,P-1718,2. COSTO MATERIALES,-9887658,Patio Comidas,EBCO S.A.
284,P-1716,3. COSTO MANO DE OBRA,-20322771,CD Astara,Astara Logística Chile SpA.
338,P-1729,4. COSTO SUPERVISIÓN,0,NaN,NaN
173,P-1685,2. COSTO MATERIALES,88900,Mapa Edificios No Operacionales,Claro Vicuña Valenzuela S.A.
272,P-1713,3. COSTO MANO DE OBRA,-3277788,Reserva San Francisco,EMPRESA NACIONAL DE CONSTRUCCIÓN LAS VERTIENTE...
90,P-1660,4. COSTO SUPERVISIÓN,0,Coronel Alvarado,Larrain Prieto Risopatron S.A.
161,P-1682,2. COSTO MATERIALES,-11527560,Plaza Ceppi,EMP. CONSTRUCTORA INGENIEROS S.A.


In [191]:
# INSPECCIÓN VISUAL 1 
df_ytd_cerrado_2M[df_ytd_cerrado_2M["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
0,1. TOTAL,3. COSTO MANO DE OBRA,-363774250,NaN,NaN
1,1. TOTAL,2. COSTO MATERIALES,-435497453,NaN,NaN
2,1. TOTAL,4. COSTO SUPERVISIÓN,-17923405,NaN,NaN
3,1. TOTAL,1. INGRESO,1118563896,NaN,NaN


In [192]:
#Se renombra "TOTAL" a "1. TOTAL" para que se pueda visualizar en el dashboard.

cols_to_replace = ['CODIGO_PROYECTO', 'NOMBRE_OBRA', 'NOMBRE_CLIENTE']
df_ytd_cerrado_2M[cols_to_replace] = df_ytd_cerrado_2M[cols_to_replace].replace('TOTAL', '1. TOTAL')
mask = df_ytd_cerrado['CODIGO_PROYECTO'] == '1. TOTAL'
df_ytd_cerrado_2M.loc[mask, ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'
df_ytd_cerrado_2M.sample(4)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
241,P-1703,2. COSTO MATERIALES,-12417197,Santa Victoria Danacorp,DANACORP S.A
302,P-1720,4. COSTO SUPERVISIÓN,0,NaN,NaN
150,P-1679,4. COSTO SUPERVISIÓN,0,Juan Sabaj,Constructora Grevia
200,P-1692,3. COSTO MANO DE OBRA,-21813347,José Perez Cotapos,EBCO S.A.


In [193]:
# INSPECCIÓN VISUAL 1
df_ytd_cerrado_2M[df_ytd_cerrado_2M["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
0,1. TOTAL,3. COSTO MANO DE OBRA,-363774250,1. TOTAL,1. TOTAL
1,1. TOTAL,2. COSTO MATERIALES,-435497453,1. TOTAL,1. TOTAL
2,1. TOTAL,4. COSTO SUPERVISIÓN,-17923405,1. TOTAL,1. TOTAL
3,1. TOTAL,1. INGRESO,1118563896,1. TOTAL,1. TOTAL


#### 4.2-1.1 - MERGE CON "df_libro_mayor5_base2" PARA CREAR DATAFRAME FINAL

In [194]:
print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M   : ", df_ytd_cerrado_2M.shape)

df_ytd_cerrado_2M_2 = df_ytd_cerrado_2M.filter(["CODIGO_PROYECTO", "TIPO_CUENTA", "TOTAL"])

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M_2 : ", df_ytd_cerrado_2M_2.shape)

df_ytd_cerrado_2M_2.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M   :  (340, 5)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M_2 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
89,P-1660,2. COSTO MATERIALES,158753
309,P-1722,2. COSTO MATERIALES,0
161,P-1682,2. COSTO MATERIALES,-11527560
319,P-1724,1. INGRESO,28844553
112,P-1668,3. COSTO MANO DE OBRA,0


In [195]:
df_margen[df_margen["CODIGO_PROYECTO"]=="P-1724"]

,CODIGO_PROYECTO,MARGEN
79,P-1724,0.72


In [196]:
# Exploración dataframe base

df_libro_mayor5_base2.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
132,P-1714,1. INGRESO,0,DEPARTAMENTAL II,CONSTRUCTORA FUERZA SUR DOS SPA
217,P-1648,4. COSTO SUPERVISIÓN,0,ETAPA 2 CD PERALILLO TW,INMOBILIARIA Y RENTAS LOGÍSTICA S.A.
72,P-1691,3. COSTO MANO DE OBRA,0,ROSARIO ROSALES,CONSTRUCTORA E INMOBILIARIA PRADOVERDE LTDA.
59,P-1686,3. COSTO MANO DE OBRA,0,MACUL ICALMA,EMPRESA CONSTRUCTORA INGEVEC S.A.
206,P-1642,4. COSTO SUPERVISIÓN,0,CORINTO,EMP. CONSTRUCTORA INGENIEROS S.A.


In [197]:
# Merge Dataframe BASE 

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2   : ", df_libro_mayor5_base2.shape)

df_ytd_cerrado_2M_3 = df_libro_mayor5_base2.merge(df_ytd_cerrado_2M_2, on=['CODIGO_PROYECTO', 'TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M_3       : ", df_ytd_cerrado_2M_3.shape)

df_ytd_cerrado_2M_3.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2   :  (340, 5)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M_3       :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
132,P-1714,1. INGRESO,0,DEPARTAMENTAL II,CONSTRUCTORA FUERZA SUR DOS SPA,18147027
253,P-1670,4. COSTO SUPERVISIÓN,0,MK II SEGURIDAD Y REDES,COMERCIAL K SPA.,0
95,P-1699,2. COSTO MATERIALES,0,VILLA PANAMERICANA LOTE B,EMPRESA CONSTRUCTORA DLP S.A.,-22557868
36,P-1673,2. COSTO MATERIALES,0,INACAP TEMUCO - ETAPA A,UNIVERSIDAD TECNOLOGICA DE CHILE INACAP,68000
231,P-1660,3. COSTO MANO DE OBRA,0,CORONEL ALVARADO,LARRAIN PRIETO RISOPATRON S.A.,0


In [198]:
# INSPECCIÓN VISUAL 1

df_ytd_cerrado_2M_3[df_ytd_cerrado_2M_3["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-17923405
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,1118563896
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-435497453
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-363774250


#### 4.2-1.2 - SE REALIZA CREACIÓN DE COLUMNA MARGEN

In [199]:
# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_ytd_cerrado_2M_3.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES','3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_margen       : ", df_margen.shape)

# mostrar el DataFrame resultante
df_margen.sample(10)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_margen       :  (85, 2)


,CODIGO_PROYECTO,MARGEN
60,P-1703,0.40
50,P-1692,0.08
57,P-1699,0.44
4,P-1603,0.00
38,P-1680,0.21
79,P-1724,0.72
52,P-1694,0.26
62,P-1705,-0.81
43,P-1685,0.00
32,P-1673,0.00


In [200]:
df_margen[df_margen["CODIGO_PROYECTO"]=="P-1724"]

,CODIGO_PROYECTO,MARGEN
79,P-1724,0.72


In [201]:
# Merge Dataframe para agregar MARGEN

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M_3   : ", df_ytd_cerrado_2M_3.shape)

df_ytd_cerrado_2M_3 = df_ytd_cerrado_2M_3.merge(df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M_3 : ", df_ytd_cerrado_2M_3.shape)

df_ytd_cerrado_2M_3.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M_3   :  (340, 6)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M_3 :  (340, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
74,P-1691,1. INGRESO,0,ROSARIO ROSALES,CONSTRUCTORA E INMOBILIARIA PRADOVERDE LTDA.,2124157,0.32
192,P-1632,1. INGRESO,0,CHORRILLOS CLOROX,CONSTRUCTORA SUKSA S.A.,0,0.00
141,P-1718,2. COSTO MATERIALES,0,PATIO COMIDAS,EBCO S.A.,-9887658,0.59
50,P-1681,1. INGRESO,0,CD ROSEN (CCTV PROVISORIO),COLCHONES ROSEN SAIC,765714,1.00
213,P-1645,3. COSTO MANO DE OBRA,0,LOS TRES ANTONIOS,SOCIEDAD CONSTRUCTORA ECHAVARRI HERMANOS LIMITADA,0,0.00


In [202]:
df_ytd_cerrado_2M_3.to_csv('archivos_respaldo/df_ytd_cerrado_2M_3.csv', index = False)

### 4.3 - **MERGE PARA -> MES EN CURSO ENTRE LIBRO MAYOR + PROYECTADO 
(DATAFRAMES: df_libro_mayor5_mec2 y df_proyectado4)

#### 4.3.1 - CREACIÓN DE COPIA DE DATAFRAME "df_libro_mayor5_mec2" Y AJUSTES INICIALES

In [203]:
# Ajuste del dataframe proveniente de LIBRO MAYOR

df_libro_mayor5_mec2.sample(5)
df_libro_mayor5_mec3 = df_libro_mayor5_mec2.drop(['CREDITO', 'DEBITO', 'MARGEN'], axis=1)
df_libro_mayor5_mec3.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN
555,P-1729,COSTO MATERIALES,2023-06-01,2023.00,6.00,-114402,LIBRO MAYOR
341,P-1705,COSTO MATERIALES,2023-06-01,2023.00,6.00,-2027108,LIBRO MAYOR
1017,P-1653,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,LIBRO MAYOR
1666,P-1688,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,LIBRO MAYOR
1464,P-1679,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,LIBRO MAYOR


In [204]:
# INSPECCIÓN VISUAL INICIAL 1

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec3  : ", df_libro_mayor5_mec3.shape)
df_libro_mayor5_mec3[df_libro_mayor5_mec3["CODIGO_PROYECTO"]=="P-1704"]

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec3  :  (424, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN
323,P-1704,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,-12890883,LIBRO MAYOR
329,P-1704,COSTO MATERIALES,2023-06-01,2023.00,6.00,-24160596,LIBRO MAYOR
2025,P-1704,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,LIBRO MAYOR
2026,P-1704,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,LIBRO MAYOR
2039,P-1704,INGRESO,2023-06-01,2023.00,6.00,0,LIBRO MAYOR
2040,P-1704,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,LIBRO MAYOR


In [205]:
# Se crea nueva columna que define el destino del uso del trozo del DATAFRAME proveniente del libro mayor

def determinar_origen(row):
    if row['TIPO_CUENTA'] in ['INGRESO', 'COSTO MATERIALES']:
        return 'REAL'
    elif row['TIPO_CUENTA'] in ['COSTO SUPERVISIÓN','COSTO MANO DE OBRA']:
        return 'REAL Y PROYECTADO'

df_libro_mayor5_mec3['DESTINO'] = df_libro_mayor5_mec3.apply(determinar_origen, axis=1)
print(df_libro_mayor5_mec3.shape)
df_libro_mayor5_mec3.sample(10)


(424, 8)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
504,P-1723,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,-15564904,LIBRO MAYOR,REAL Y PROYECTADO
1593,P-1685,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO
768,P-1626,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,LIBRO MAYOR,REAL
1953,P-1700,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO
1607,P-1685,INGRESO,2023-06-01,2023.00,6.00,0,LIBRO MAYOR,REAL
2472,P-1724,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,LIBRO MAYOR,REAL
1833,P-1695,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO
705,P-1612,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO
159,P-1688,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,-1621597,LIBRO MAYOR,REAL Y PROYECTADO
767,P-1626,INGRESO,2023-06-01,2023.00,6.00,0,LIBRO MAYOR,REAL


In [206]:
# Cambiar los valores de la columna TIPO_CUENTA
df_libro_mayor5_mec3['TIPO_CUENTA'] = df_libro_mayor5_mec3['TIPO_CUENTA'].replace({'COSTO MATERIALES': '2. COSTO MATERIALES', 'COSTO MANO DE OBRA': '3. COSTO MANO DE OBRA', 'INGRESO': '1. INGRESO','COSTO SUPERVISIÓN': '4. COSTO SUPERVISIÓN'})

print(df_libro_mayor5_mec3.shape)
df_libro_mayor5_mec3["TIPO_CUENTA"].unique()

(424, 8)


array(['3. COSTO MANO DE OBRA', '2. COSTO MATERIALES', '1. INGRESO',
       '4. COSTO SUPERVISIÓN'], dtype=object)

#### 4.3.2 - CREACIÓN DE COPIA DE DATAFRAME "df_proyectado4" Y AJUSTES INICIALES

In [207]:
# Ajuste del dataframe proveniente de PROYECTADO

df_proyectado4.sample(5)
df_proyectado5 = df_proyectado4.drop(['PROYECTO'], axis=1)

# Se crea la columna Fecha
df_proyectado5['FECHA'] = pd.to_datetime(df_proyectado5['MES'].astype(str) + '-1-' + df_proyectado5['AÑO'].astype(str))

df_proyectado5.sample(5)

,CODIGO_PROYECTO,TOTAL,MES,AÑO,ORIGEN,TIPO_CUENTA,FECHA
18,P-1670,0,6,2023,PROYECTADO,1. INGRESO,2023-06-01
15,P-1667,0,6,2023,PROYECTADO,1. INGRESO,2023-06-01
68,P-1726,0,6,2023,PROYECTADO,3. COSTO MANO DE OBRA,2023-06-01
8,P-1645,0,6,2023,PROYECTADO,3. COSTO MANO DE OBRA,2023-06-01
27,P-1680,-5000000,6,2023,PROYECTADO,2. COSTO MATERIALES,2023-06-01


In [208]:
# Se crea nueva columna que define el destino del uso del trozo del DATAFRAME proveniente del proyectado

def determinar_origen(row):
    if row['TIPO_CUENTA'] in ['1. INGRESO', '2. COSTO MATERIALES']:
        return 'PROYECTADO'

# Se realiza 
df_proyectado5['DESTINO'] = df_proyectado5.apply(determinar_origen, axis=1)
df_proyectado5 = df_proyectado5[df_proyectado5['TIPO_CUENTA'] != '3. COSTO MANO DE OBRA']


# Se reordenan las columnas para estandarizarlo como la que viene desde Libro Mayot
df_proyectado5 = df_proyectado5.reindex(columns=['CODIGO_PROYECTO', 'TIPO_CUENTA', 'FECHA', 'AÑO', 'MES', 'TOTAL', 'ORIGEN', 'DESTINO'])

df_proyectado5.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
34,P-1690,2. COSTO MATERIALES,2023-06-01,2023,6,-2000000,PROYECTADO,PROYECTADO
66,P-1724,1. INGRESO,2023-06-01,2023,6,0,PROYECTADO,PROYECTADO
64,P-1722,1. INGRESO,2023-06-01,2023,6,0,PROYECTADO,PROYECTADO
39,P-1695,2. COSTO MATERIALES,2023-06-01,2023,6,-3000000,PROYECTADO,PROYECTADO
39,P-1695,1. INGRESO,2023-06-01,2023,6,0,PROYECTADO,PROYECTADO


#### 4.3.3 - CONCATENACIÓN ENTRE DATAFRAMES DEL MES EN CURSO LIBRO MAYOR Y PROYECTADO

In [209]:
# Se realiza concatenación entre ambos dataframes

print("Dimensiones df_libro_mayor5_mec3 antes de cambios: ", df_libro_mayor5_mec3.shape)
print("Dimensiones df_proyectado5 antes de cambios      : ", df_proyectado5.shape)

mes_en_curso = pd.concat([df_libro_mayor5_mec3, df_proyectado5])

print("Dimensiones dataframe final mes_en_curso después de concatenación  : ", mes_en_curso.shape)
mes_en_curso.sample(10)

Dimensiones df_libro_mayor5_mec3 antes de cambios:  (424, 8)
Dimensiones df_proyectado5 antes de cambios      :  (146, 8)
Dimensiones dataframe final mes_en_curso después de concatenación  :  (570, 8)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
402,P-1712,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,-7219006,LIBRO MAYOR,REAL
2351,P-1719,1. INGRESO,2023-06-01,2023.00,6.00,0,LIBRO MAYOR,REAL
1752,P-1691,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,0,LIBRO MAYOR,REAL
1536,P-1682,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,0,LIBRO MAYOR,REAL
46,P-1667,3. COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,-382370,LIBRO MAYOR,REAL Y PROYECTADO
1906,P-1698,4. COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO
41,P-1697,1. INGRESO,2023-06-01,2023.00,6.00,5000000,PROYECTADO,PROYECTADO
1161,P-1664,3. COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO
9,P-1654,1. INGRESO,2023-06-01,2023.00,6.00,0,PROYECTADO,PROYECTADO
50,P-1706,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,-4200000,PROYECTADO,PROYECTADO


#### 4.3.4 - MERGE CON "df_cod_proyecto" PARA CREAR DATAFRAME FINAL

In [210]:
print("Dimensiones mes_en_curso antes de cambios: ", mes_en_curso.shape)
mes_en_curso2 = mes_en_curso.merge(df_cod_proyecto, on='CODIGO_PROYECTO', how='left')
mes_en_curso2 = mes_en_curso2[['CODIGO_PROYECTO', 'NOMBRE_OBRA', 'NOMBRE_CLIENTE', 'TIPO_CUENTA', 'FECHA', 'AÑO', 'MES', 'TOTAL', 'ORIGEN', 'DESTINO']]
print("Dimensiones mes_en_curso2 antes de cambios: ", mes_en_curso2.shape)
mes_en_curso2.sample(10)

Dimensiones mes_en_curso antes de cambios:  (570, 8)
Dimensiones mes_en_curso2 antes de cambios:  (570, 10)


,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
167,P-1658,VIVIPRA,Vicherat y Pradenas Ltda.,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,0,LIBRO MAYOR,REAL
312,P-1700,CD Orizon,EBCO S.A.,3. COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO
233,P-1680,Nueva Uno,Constructora Tecsa S.A.,4. COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO
77,P-1727,NaN,NaN,3. COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,-3455213,LIBRO MAYOR,REAL Y PROYECTADO
235,P-1680,Nueva Uno,Constructora Tecsa S.A.,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,0,LIBRO MAYOR,REAL
268,P-1689,Casino Buenaventura,EBCO S.A.,3. COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO
315,P-1700,CD Orizon,EBCO S.A.,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,0,LIBRO MAYOR,REAL
336,P-1707,Las Tranqueras,Constructora Santolaya Ltda.,3. COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO
458,P-1690,Ongolmo - Home Condominio,CANTAUCO CONSTRUCCION LTDA.,1. INGRESO,2023-06-01,2023.00,6.00,5000000,PROYECTADO,PROYECTADO
93,P-1592,Nueva Costanera 3300,EBCO S.A.,4. COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO


In [211]:
#Para reemplazar los valores en blanco en las columnas "NOMBRE_OBRA" y "NOMBRE_CLIENTE" por el valor "TOTAL" cada vez que el valor de la columna "CODIGO_PROYECTO" es igual a "TOTAL", puedes hacer lo siguiente:

mes_en_curso2.loc[(mes_en_curso2['CODIGO_PROYECTO'] == '1. TOTAL') & (mes_en_curso2['NOMBRE_OBRA'].isnull()), 'NOMBRE_OBRA'] = '1. TOTAL'
mes_en_curso2.loc[(mes_en_curso2['CODIGO_PROYECTO'] == '1. TOTAL') & (mes_en_curso2['NOMBRE_CLIENTE'].isnull()), 'NOMBRE_CLIENTE'] = '1. TOTAL'
mes_en_curso2.sample(5)

,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
526,P-1684,Edificio Goycolea,EBCO S.A.,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,-12035806,PROYECTADO,PROYECTADO
258,P-1686,Macul Icalma,EMPRESA CONSTRUCTORA INGEVEC S.A.,1. INGRESO,2023-06-01,2023.00,6.00,0,LIBRO MAYOR,REAL
481,P-1715,CD Lo Echevers,Quezada y Boetsch,1. INGRESO,2023-06-01,2023.00,6.00,11500000,PROYECTADO,PROYECTADO
40,P-1705,Los Clarines,CONSTRUCTORA INARCO S.A.,3. COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,-5256672,LIBRO MAYOR,REAL Y PROYECTADO
491,P-1725,Lo Cañas,Constructora Brisas de Batuco,1. INGRESO,2023-06-01,2023.00,6.00,14000000,PROYECTADO,PROYECTADO


In [212]:
# INSPECCIÓN VISUAL 1

mes_en_curso2[mes_en_curso2["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
38,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,3. COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,-12890883,LIBRO MAYOR,REAL Y PROYECTADO
39,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,-24160596,LIBRO MAYOR,REAL
324,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,3. COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO
325,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,4. COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO
326,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,1. INGRESO,2023-06-01,2023.00,6.00,0,LIBRO MAYOR,REAL
327,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,0,LIBRO MAYOR,REAL
472,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,1. INGRESO,2023-06-01,2023.00,6.00,62000000,PROYECTADO,PROYECTADO
545,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,-27000000,PROYECTADO,PROYECTADO


#### 4.3.5 - AGREGACIÓN DE TODOS TIPOS DE CUENTA

In [213]:
print("Dimensiones Dataframe antes de cambios", mes_en_curso2.shape)

# definir nuevas filas
df_rows_to_add = []

for _, row in mes_en_curso2.iterrows():
    if row["TIPO_CUENTA"] == "1. INGRESO" and row["TOTAL"] == 0:
        row1 = row.copy()
        row1["TIPO_CUENTA"] = "1. INGRESO"
        row1["TOTAL"] = 0
        row1["ORIGEN"] = "AGREGACION"
        row1["DESTINO"] = "REAL"
        df_rows_to_add.append(row1)

        row2 = row.copy()
        row2["TIPO_CUENTA"] = "1. INGRESO"
        row2["TOTAL"] = 0
        row2["ORIGEN"] = "AGREGACION"
        row2["DESTINO"] = "PROYECTADO"
        df_rows_to_add.append(row2)

    elif row["TIPO_CUENTA"] == "2. COSTO MATERIALES" and row["TOTAL"] == 0:
        row1 = row.copy()
        row1["TIPO_CUENTA"] = "2. COSTO MATERIALES"
        row1["TOTAL"] = 0
        row1["ORIGEN"] = "AGREGACION"
        row1["DESTINO"] = "REAL"
        df_rows_to_add.append(row1)

        row2 = row.copy()
        row2["TIPO_CUENTA"] = "2. COSTO MATERIALES"
        row2["TOTAL"] = 0
        row2["ORIGEN"] = "AGREGACION"
        row2["DESTINO"] = "PROYECTADO"
        df_rows_to_add.append(row2)

    elif row["TIPO_CUENTA"] == "3. COSTO MANO DE OBRA" and row["TOTAL"] == 0:
        row1 = row.copy()
        row1["TIPO_CUENTA"] = "3. COSTO MANO DE OBRA"
        row1["TOTAL"] = 0
        row1["ORIGEN"] = "AGREGACION"
        row1["DESTINO"] = "REAL Y PROYECTADO"
        df_rows_to_add.append(row1)

  
    elif row["TIPO_CUENTA"] == "4. COSTO SUPERVISIÓN" and row["TOTAL"] == 0:
        row1 = row.copy()
        row1["TIPO_CUENTA"] = "4. COSTO SUPERVISIÓN"
        row1["TOTAL"] = 0
        row1["ORIGEN"] = "AGREGACION"
        row1["DESTINO"] = "REAL Y PROYECTADO"
        df_rows_to_add.append(row1)
        
        row2 = row.copy()
        row2["TIPO_CUENTA"] = "3. COSTO MANO DE OBRA"
        row2["TOTAL"] = 0
        row2["ORIGEN"] = "AGREGACION"
        row2["DESTINO"] = "REAL Y PROYECTADO"
        df_rows_to_add.append(row2)

# concatenar nuevas filas al DataFrame original
mes_en_curso2 = pd.concat([mes_en_curso2, pd.DataFrame(df_rows_to_add)], ignore_index=True)

# Se renombra "TOTAL" a "1. TOTAL" para que se pueda visualizar como primera opción en el dashboard.
mes_en_curso2['CODIGO_PROYECTO'] = mes_en_curso2['CODIGO_PROYECTO'].apply(lambda x: '1. TOTAL' if x == 'TOTAL' else x)

cols_to_replace = ['CODIGO_PROYECTO', 'NOMBRE_OBRA', 'NOMBRE_CLIENTE']
mes_en_curso2[cols_to_replace] = mes_en_curso2[cols_to_replace].replace('TOTAL', '1. TOTAL')
mask = mes_en_curso2['CODIGO_PROYECTO'] == '1. TOTAL'
mes_en_curso2.loc[mask, ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'

# Cambiar los valores de la columna TIPO_CUENTA

print("Dimensiones Dataframe después de cambios", mes_en_curso2.shape)


Dimensiones Dataframe antes de cambios (570, 10)
Dimensiones Dataframe después de cambios (1357, 10)


In [214]:
mes_en_curso2["TIPO_CUENTA"].unique()

array(['3. COSTO MANO DE OBRA', '2. COSTO MATERIALES', '1. INGRESO',
       '4. COSTO SUPERVISIÓN'], dtype=object)

In [215]:
# INSPECCIÓN VISUAL 1
mes_en_curso2[mes_en_curso2["CODIGO_PROYECTO"]=="P-1644"]

,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
140,P-1644,Mirador de San Cristobal,EBCO S.A.,3. COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO
141,P-1644,Mirador de San Cristobal,EBCO S.A.,4. COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO
142,P-1644,Mirador de San Cristobal,EBCO S.A.,1. INGRESO,2023-06-01,2023.00,6.00,0,LIBRO MAYOR,REAL
143,P-1644,Mirador de San Cristobal,EBCO S.A.,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,0,LIBRO MAYOR,REAL
668,P-1644,Mirador de San Cristobal,EBCO S.A.,3. COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,AGREGACION,REAL Y PROYECTADO
669,P-1644,Mirador de San Cristobal,EBCO S.A.,4. COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,AGREGACION,REAL Y PROYECTADO
670,P-1644,Mirador de San Cristobal,EBCO S.A.,3. COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,AGREGACION,REAL Y PROYECTADO
671,P-1644,Mirador de San Cristobal,EBCO S.A.,1. INGRESO,2023-06-01,2023.00,6.00,0,AGREGACION,REAL
672,P-1644,Mirador de San Cristobal,EBCO S.A.,1. INGRESO,2023-06-01,2023.00,6.00,0,AGREGACION,PROYECTADO
673,P-1644,Mirador de San Cristobal,EBCO S.A.,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,0,AGREGACION,REAL


In [216]:
mes_en_curso2["TIPO_CUENTA"].unique()

array(['3. COSTO MANO DE OBRA', '2. COSTO MATERIALES', '1. INGRESO',
       '4. COSTO SUPERVISIÓN'], dtype=object)

#### 4.3.6 - **CREACIÓN DATAFRAME PARA -> PROYECTADO

In [217]:
# Se crea una copia del dataframe original mes_en_curso2

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso2       : ", mes_en_curso2.shape)

mes_en_curso2_1 = mes_en_curso2.copy()
mes_en_curso2_1 = mes_en_curso2[mes_en_curso2['DESTINO'].str.contains('PROYECTADO')]

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso2_1   : ", mes_en_curso2_1.shape)


print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso2_1     : ", mes_en_curso2_1.shape)

mes_en_curso_proy = pd.pivot_table(mes_en_curso2_1, values=['TOTAL'], index=['CODIGO_PROYECTO','TIPO_CUENTA'], aggfunc='sum', fill_value=0)
mes_en_curso_proy = pd.DataFrame(mes_en_curso_proy.to_records())

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy  : ", mes_en_curso_proy.shape)

mes_en_curso_proy.sample(10)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso2       :  (1357, 10)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso2_1   :  (873, 10)
DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso2_1     :  (873, 10)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy  :  (362, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
332,P-1722,3. COSTO MANO DE OBRA,0
9,P-1591,2. COSTO MATERIALES,0
284,P-1708,3. COSTO MANO DE OBRA,-208532
43,P-1626,4. COSTO SUPERVISIÓN,0
94,P-1655,1. INGRESO,0
225,P-1693,2. COSTO MATERIALES,0
324,P-1720,3. COSTO MANO DE OBRA,-65665
320,P-1719,3. COSTO MANO DE OBRA,0
292,P-1712,3. COSTO MANO DE OBRA,-5436047
66,P-1643,1. INGRESO,0


In [218]:
# TESTING VISUAL 1
mes_en_curso_proy[mes_en_curso_proy["CODIGO_PROYECTO"]=="P-1712"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
290,P-1712,1. INGRESO,12500000
291,P-1712,2. COSTO MATERIALES,-7700000
292,P-1712,3. COSTO MANO DE OBRA,-5436047
293,P-1712,4. COSTO SUPERVISIÓN,0


In [219]:
# TESTING VISUAL 2
mes_en_curso_proy[mes_en_curso_proy["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
0,1. TOTAL,1. INGRESO,423800000
1,1. TOTAL,2. COSTO MATERIALES,-176535806
2,1. TOTAL,3. COSTO MANO DE OBRA,-114373715
3,1. TOTAL,4. COSTO SUPERVISIÓN,-4643261


##### 4.3.6.1 - MERGE CON DATAFRAME LIBRO MAYOR 2023 BASE 

In [220]:
# Merge Dataframe BASE 

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy   : ", mes_en_curso_proy.shape)
print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2   : ", df_libro_mayor5_base2.shape)

mes_en_curso_proy2 = df_libro_mayor5_base2.merge(mes_en_curso_proy, on=['CODIGO_PROYECTO', 'TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy2       : ", mes_en_curso_proy2.shape)

mes_en_curso_proy2['TOTAL'] = mes_en_curso_proy2['TOTAL'].fillna(0)

mes_en_curso_proy2.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy   :  (362, 3)
DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2   :  (340, 5)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy2       :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
95,P-1699,2. COSTO MATERIALES,0,VILLA PANAMERICANA LOTE B,EMPRESA CONSTRUCTORA DLP S.A.,-22000000
321,P-1720,4. COSTO SUPERVISIÓN,0,NaN,NaN,0
251,P-1669,4. COSTO SUPERVISIÓN,0,DEPARTAMENTAL,EMP. CONSTRUCTORA INGENIEROS S.A.,0
290,P-1693,1. INGRESO,0,VOLTA V0800,ECOBIO SPA,0
149,P-1722,1. INGRESO,0,ECO CERRILLOS,CONSTRUCTORA ICF SPA,0


In [221]:
# TESTING VISUAL 1

mes_en_curso_proy2[mes_en_curso_proy2["CODIGO_PROYECTO"]=="P-1712"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
124,P-1712,3. COSTO MANO DE OBRA,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,-5436047
125,P-1712,2. COSTO MATERIALES,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,-7700000
126,P-1712,1. INGRESO,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,12500000
308,P-1712,4. COSTO SUPERVISIÓN,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,0


In [222]:
# TESTING VISUAL 2
mes_en_curso_proy2[mes_en_curso_proy2["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-4643261
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,423800000
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-176535806
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-114373715


##### 4.3.6.2 - SE RECALCULAN LOS TOTALES

In [223]:
# Calcular la suma de los valores totales para cada tipo de cuenta

# Se eliminan totales excepto supervisión, para re hacer el calculo de totales

mes_en_curso_proy2 = mes_en_curso_proy2.drop(mes_en_curso_proy2[(mes_en_curso_proy2['CODIGO_PROYECTO'] == '1. TOTAL') & (mes_en_curso_proy2['TIPO_CUENTA'] != '4. COSTO SUPERVISIÓN')].index)

print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_proy2 : ", mes_en_curso_proy2.shape)

df_total = mes_en_curso_proy2.groupby('TIPO_CUENTA')['TOTAL'].sum().reset_index()
df_total['CODIGO_PROYECTO'] = '1. TOTAL'

# Agregar las filas adicionales al DataFrame
mes_en_curso_proy3 = pd.concat([mes_en_curso_proy2, df_total], ignore_index=True)

# Modificar las columnas "NOMBRE_OBRA" y "NOMBRE_CLIENTE" para "1. TOTAL"
mes_en_curso_proy3.loc[mes_en_curso_proy3['CODIGO_PROYECTO'] == '1. TOTAL', ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_proy3 : ", mes_en_curso_proy3.shape)

mes_en_curso_proy3.sample(10) 

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_proy2 :  (337, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_proy3 :  (341, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
56,P-1684,2. COSTO MATERIALES,0.00,EDIFICIO GOYCOLEA,EBCO S.A.,-12035806
188,P-1620,4. COSTO SUPERVISIÓN,0.00,"OF APOQUINDO- HABILITACIONES, INTEGRACION Y AD...",CONSTRUCTORA INARCO S.A.,0
73,P-1691,2. COSTO MATERIALES,0.00,ROSARIO ROSALES,CONSTRUCTORA E INMOBILIARIA PRADOVERDE LTDA.,0
166,P-1728,2. COSTO MATERIALES,0.00,NaN,NaN,0
221,P-1653,1. INGRESO,0.00,EDUARDO DE LA BARRA,CONSTRUCTORA E INVERSIONES VITAL SPA.,0
130,P-1714,3. COSTO MANO DE OBRA,0.00,DEPARTAMENTAL II,CONSTRUCTORA FUERZA SUR DOS SPA,-205777
38,P-1675,2. COSTO MATERIALES,0.00,PALENA II,CONSTRUCTORA MYA SPA.,0
238,P-1663,4. COSTO SUPERVISIÓN,0.00,OFICINAS EQUIFAX,CONSTRUCTORA VRK INGENIERIA LIMITADA,0
46,P-1679,2. COSTO MATERIALES,0.00,JUAN SABAJ,CONSTRUCTORA GREVIA,0
129,P-1713,1. INGRESO,0.00,RESERVA SAN FRANCISCO,EMPRESA NACIONAL DE CONSTRUCCIÓN LAS VERTIENTE...,16500000


In [224]:
# TESTING VISUAL 2
mes_en_curso_proy3[mes_en_curso_proy3["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-4643261
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,423800000
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-176535806
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-114373715
340,1. TOTAL,4. COSTO SUPERVISIÓN,NaN,1. TOTAL,1. TOTAL,-4643261


In [225]:
# SE ELIMINA COSTO SUPERVISIÓN DUPLICADO.

print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_proy3 : ", mes_en_curso_proy3.shape)

mes_en_curso_proy3 = mes_en_curso_proy3.drop(mes_en_curso_proy3[(mes_en_curso_proy3['CODIGO_PROYECTO'] == '1. TOTAL') & (mes_en_curso_proy3['TIPO_CUENTA'] == '4. COSTO SUPERVISIÓN') & (mes_en_curso_proy3['TOTAL_LB'].isnull())].index)

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_proy3 : ", mes_en_curso_proy3.shape)

mes_en_curso_proy3[mes_en_curso_proy3["CODIGO_PROYECTO"]=="1. TOTAL"]

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_proy3 :  (341, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_proy3 :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-4643261
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,423800000
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-176535806
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-114373715


In [226]:
# TESTING VISUAL 1
mes_en_curso_proy3[mes_en_curso_proy3["CODIGO_PROYECTO"]=="P-1723"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
150,P-1723,3. COSTO MANO DE OBRA,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,-15564904
151,P-1723,2. COSTO MATERIALES,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,-31000000
152,P-1723,1. INGRESO,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,75000000
326,P-1723,4. COSTO SUPERVISIÓN,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,0


In [227]:
# TESTING VISUAL 2
mes_en_curso_proy3[mes_en_curso_proy3["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-4643261
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,423800000
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-176535806
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-114373715


##### 4.3.6.3 - CREACIÓN DE MARGEN

In [228]:
#CREACIÓN DE MARGENES

mes_en_curso_proy4 = mes_en_curso_proy3.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = mes_en_curso_proy4.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)

,CODIGO_PROYECTO,MARGEN
49,P-1691,0.00
17,P-1648,0.00
65,P-1708,0.00
80,P-1725,0.43
45,P-1687,0.00
12,P-1641,0.00
29,P-1669,0.00
0,1. TOTAL,0.30
6,P-1612,0.00
21,P-1658,0.00


In [229]:
print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_proy4 : ", mes_en_curso_proy4.shape)

mes_en_curso_proy5 = pd.merge(mes_en_curso_proy4, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_proy5 : ", mes_en_curso_proy5.shape)
mes_en_curso_proy5.sample(10)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_proy4 :  (340, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_proy5 :  (340, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
4,P-1611,2. COSTO MATERIALES,0.00,EDIFICIO PLAZA EGAÑA,EBCO S.A.,0,0.00
268,P-1679,4. COSTO SUPERVISIÓN,0.00,JUAN SABAJ,CONSTRUCTORA GREVIA,0,0.00
53,P-1682,1. INGRESO,0.00,PLAZA CEPPI,EMP. CONSTRUCTORA INGENIEROS S.A.,0,0.00
97,P-1700,3. COSTO MANO DE OBRA,0.00,CD ORIZON,EBCO S.A.,0,0.00
288,P-1691,4. COSTO SUPERVISIÓN,0.00,ROSARIO ROSALES,CONSTRUCTORA E INMOBILIARIA PRADOVERDE LTDA.,0,0.00
136,P-1716,3. COSTO MANO DE OBRA,0.00,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,-1666000,0.00
31,P-1669,2. COSTO MATERIALES,0.00,DEPARTAMENTAL,EMP. CONSTRUCTORA INGENIEROS S.A.,0,0.00
89,P-1697,3. COSTO MANO DE OBRA,0.00,ESTOCOLMO,CONSTRUCTORA FORTALEZA S.A,-4485402,-0.10
115,P-1707,3. COSTO MANO DE OBRA,0.00,LAS TRANQUERAS,CONSTRUCTORA SANTOLAYA LTDA.,-3715026,0.39
152,P-1723,1. INGRESO,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,75000000,0.38


In [230]:
# TESTING VISUAL 1
mes_en_curso_proy5[mes_en_curso_proy5["CODIGO_PROYECTO"]=="P-1723"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
150,P-1723,3. COSTO MANO DE OBRA,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,-15564904,0.38
151,P-1723,2. COSTO MATERIALES,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,-31000000,0.38
152,P-1723,1. INGRESO,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,75000000,0.38
326,P-1723,4. COSTO SUPERVISIÓN,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,0,0.38


In [231]:
# TESTING VISUAL 2
mes_en_curso_proy5[mes_en_curso_proy5["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-4643261,0.30
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,423800000,0.30
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-176535806,0.30
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-114373715,0.30


In [232]:
# Respaldo de Dataframe mes en curso antes de abrirrlo
mes_en_curso_proy5.to_csv('archivos_respaldo/mes_en_curso_proy5.csv', mode='a')

#### 4.3.7 - **CREACIÓN DATAFRAME PARA -> MES EN CURSO

##### 4.3.7.1 - TRATAMIENTO mes_en_curso2

In [233]:
# Se crea una copia del dataframe original mes_en_curso2

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso2       : ", mes_en_curso2.shape)

mes_en_curso2_1 = mes_en_curso2.copy()
mes_en_curso2_1 = mes_en_curso2[mes_en_curso2['DESTINO'].str.contains('REAL')]

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso2_1   : ", mes_en_curso2_1.shape)


print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso2_1     : ", mes_en_curso2_1.shape)

mes_en_curso_real = pd.pivot_table(mes_en_curso2_1, values=['TOTAL'], index=['CODIGO_PROYECTO','TIPO_CUENTA'], aggfunc='sum', fill_value=0)
mes_en_curso_real = pd.DataFrame(mes_en_curso_real.to_records())

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_real  : ", mes_en_curso_real.shape)

mes_en_curso_real.sample(10)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso2       :  (1357, 10)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso2_1   :  (945, 10)
DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso2_1     :  (945, 10)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_real  :  (362, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
317,P-1718,4. COSTO SUPERVISIÓN,0
57,P-1639,4. COSTO SUPERVISIÓN,0
105,P-1660,2. COSTO MATERIALES,0
241,P-1697,2. COSTO MATERIALES,-1229403
149,P-1673,4. COSTO SUPERVISIÓN,0
210,P-1689,3. COSTO MANO DE OBRA,0
84,P-1649,3. COSTO MANO DE OBRA,0
191,P-1684,4. COSTO SUPERVISIÓN,0
238,P-1696,3. COSTO MANO DE OBRA,-310358
252,P-1700,1. INGRESO,0


In [234]:
# TESTING VISUAL 1

mes_en_curso_real[mes_en_curso_real["CODIGO_PROYECTO"]=="P-1712"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
290,P-1712,1. INGRESO,-3823676
291,P-1712,2. COSTO MATERIALES,-7219006
292,P-1712,3. COSTO MANO DE OBRA,-5436047
293,P-1712,4. COSTO SUPERVISIÓN,0


In [235]:
# TESTING VISUAL 2

mes_en_curso_real[mes_en_curso_real["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
0,1. TOTAL,1. INGRESO,4168749
1,1. TOTAL,2. COSTO MATERIALES,-141102934
2,1. TOTAL,3. COSTO MANO DE OBRA,-114373715
3,1. TOTAL,4. COSTO SUPERVISIÓN,-4643261


##### 4.3.7.2 - MERGE CON DATAFRAME LIBRO MAYOR 2023 BASE 

In [236]:
# Merge Dataframe BASE 

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_real   : ", mes_en_curso_real.shape)
print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2   : ", df_libro_mayor5_base2.shape)

mes_en_curso_real2 = df_libro_mayor5_base2.merge(mes_en_curso_real, on=['CODIGO_PROYECTO', 'TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_real       : ", mes_en_curso_real.shape)

mes_en_curso_real2.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_real   :  (362, 3)
DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2   :  (340, 5)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_real       :  (362, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
238,P-1663,4. COSTO SUPERVISIÓN,0,OFICINAS EQUIFAX,CONSTRUCTORA VRK INGENIERIA LIMITADA,0
243,P-1666,3. COSTO MANO DE OBRA,0,CD AXIONLONG,EBCO S.A.,0
37,P-1674,2. COSTO MATERIALES,0,OFICINAS BEIERSDORF CHORRILLOS,CONSTRUCTORA SUKSA S.A.,0
207,P-1643,1. INGRESO,0,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,0
53,P-1682,1. INGRESO,0,PLAZA CEPPI,EMP. CONSTRUCTORA INGENIEROS S.A.,0


In [237]:
# TESTING VISUAL 1

mes_en_curso_real2[mes_en_curso_real2["CODIGO_PROYECTO"]=="P-1712"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
124,P-1712,3. COSTO MANO DE OBRA,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,-5436047
125,P-1712,2. COSTO MATERIALES,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,-7219006
126,P-1712,1. INGRESO,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,-3823676
308,P-1712,4. COSTO SUPERVISIÓN,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,0


In [238]:
# TESTING VISUAL 2

mes_en_curso_real2[mes_en_curso_real2["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-4643261
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,4168749
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-141102934
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-114373715


##### 4.3.7.3 - SE RECALCULAN LOS TOTALES

In [239]:
# Calcular la suma de los valores totales para cada tipo de cuenta

# Se eliminan totales excepto supervisión, para re hacer el calculo de totales

mes_en_curso_real2 = mes_en_curso_real2.drop(mes_en_curso_real2[(mes_en_curso_real2['CODIGO_PROYECTO'] == '1. TOTAL') & (mes_en_curso_real2['TIPO_CUENTA'] != '4. COSTO SUPERVISIÓN')].index)

print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_real2 : ", mes_en_curso_real2.shape)

df_total = mes_en_curso_real2.groupby('TIPO_CUENTA')['TOTAL'].sum().reset_index()
df_total['CODIGO_PROYECTO'] = '1. TOTAL'

# Agregar las filas adicionales al DataFrame
mes_en_curso_real3 = pd.concat([mes_en_curso_real2, df_total], ignore_index=True)

# Modificar las columnas "NOMBRE_OBRA" y "NOMBRE_CLIENTE" para "1. TOTAL"
mes_en_curso_real3.loc[mes_en_curso_real3['CODIGO_PROYECTO'] == '1. TOTAL', ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_real3 : ", mes_en_curso_real3.shape)

mes_en_curso_real3.sample(10)   

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_real2 :  (337, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_real3 :  (341, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
256,P-1673,1. INGRESO,0.00,INACAP TEMUCO - ETAPA A,UNIVERSIDAD TECNOLOGICA DE CHILE INACAP,0
160,P-1726,2. COSTO MATERIALES,0.00,NaN,NaN,-1227062
242,P-1666,1. INGRESO,0.00,CD AXIONLONG,EBCO S.A.,0
94,P-1699,3. COSTO MANO DE OBRA,0.00,VILLA PANAMERICANA LOTE B,EMPRESA CONSTRUCTORA DLP S.A.,-3166568
23,P-1661,2. COSTO MATERIALES,0.00,SAN VICENTE DE PAUL,DANACORP S.A,11176
292,P-1694,4. COSTO SUPERVISIÓN,0.00,ECO HUEMUL,CONSTRUCTORA ICF SPA,0
176,P-1592,4. COSTO SUPERVISIÓN,0.00,NUEVA COSTANERA 3300,EBCO S.A.,0
50,P-1681,1. INGRESO,0.00,CD ROSEN (CCTV PROVISORIO),COLCHONES ROSEN SAIC,0
278,P-1685,1. INGRESO,0.00,MAPA EDIFICIOS NO OPERACIONALES,CLARO VICUÑA VALENZUELA S.A.,0
195,P-1636,1. INGRESO,0.00,SANTO DOMINGO 1410,DANACORP S.A,0


In [240]:
# TESTING VISUAL 1

mes_en_curso_real3[mes_en_curso_real3["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-4643261
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,4168749
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-141102934
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-114373715
340,1. TOTAL,4. COSTO SUPERVISIÓN,NaN,1. TOTAL,1. TOTAL,-4643261


In [241]:
# SE ELIMINA COSTO SUPERVISIÓN DUPLICADO.

print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_real3 : ", mes_en_curso_real3.shape)

mes_en_curso_real3 = mes_en_curso_real3.drop(mes_en_curso_real3[(mes_en_curso_real3['CODIGO_PROYECTO'] == '1. TOTAL') & (mes_en_curso_real3['TIPO_CUENTA'] == '4. COSTO SUPERVISIÓN') & (mes_en_curso_real3['TOTAL_LB'].isnull())].index)

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_real3 : ", mes_en_curso_real3.shape)

mes_en_curso_real3[mes_en_curso_real3["CODIGO_PROYECTO"]=="1. TOTAL"]

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_real3 :  (341, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_real3 :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-4643261
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,4168749
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-141102934
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-114373715


##### 4.3.7.4 - CREACIÓN DE MARGEN

In [242]:
#CREACIÓN DE MARGENES

mes_en_curso_real4 = mes_en_curso_real3.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = mes_en_curso_real4.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)

,CODIGO_PROYECTO,MARGEN
73,P-1718,-0.13
13,P-1642,0.00
77,P-1722,0.00
59,P-1701,0.00
52,P-1694,0.00
14,P-1643,0.00
53,P-1695,0.00
65,P-1708,0.00
72,P-1717,0.00
39,P-1681,0.00


In [243]:
print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_real4 : ", mes_en_curso_real4.shape)

mes_en_curso_real5 = pd.merge(mes_en_curso_real4, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_real5 : ", mes_en_curso_real5.shape)
mes_en_curso_real5.sample(10)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_real4 :  (340, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_real5 :  (340, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
271,P-1681,3. COSTO MANO DE OBRA,0.00,CD ROSEN (CCTV PROVISORIO),COLCHONES ROSEN SAIC,0,0.00
165,P-1728,3. COSTO MANO DE OBRA,0.00,NaN,NaN,0,0.00
327,P-1724,4. COSTO SUPERVISIÓN,0.00,NaN,NaN,0,0.00
246,P-1667,4. COSTO SUPERVISIÓN,0.00,ECO PLACILLA,CONSTRUCTORA ICF SPA,0,0.00
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-141102934,-61.40
199,P-1639,3. COSTO MANO DE OBRA,0.00,LYON 2546,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...,0,0.00
306,P-1708,4. COSTO SUPERVISIÓN,0.00,ECO ESPAÑA,CONSTRUCTORA ICF SPA,0,0.00
292,P-1694,4. COSTO SUPERVISIÓN,0.00,ECO HUEMUL,CONSTRUCTORA ICF SPA,0,0.00
98,P-1700,2. COSTO MATERIALES,0.00,CD ORIZON,EBCO S.A.,0,0.00
3,P-1603,2. COSTO MATERIALES,0.00,FARELLONES,EMPRESA CONSTRUCTORA DLP S.A.,0,0.00


In [244]:
# TESTING VISUAL 1

mes_en_curso_real5[mes_en_curso_real5["CODIGO_PROYECTO"]=="P-1716"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
136,P-1716,3. COSTO MANO DE OBRA,0.00,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,-1666000,0.00
137,P-1716,2. COSTO MATERIALES,0.00,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,-2308672,0.00
138,P-1716,1. INGRESO,0.00,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,0,0.00
312,P-1716,4. COSTO SUPERVISIÓN,0.00,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,0,0.00


In [245]:
# TESTING VISUAL 3
mes_en_curso_real5[mes_en_curso_real5["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
106,P-1704,3. COSTO MANO DE OBRA,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-12890883,0.00
107,P-1704,2. COSTO MATERIALES,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-24160596,0.00
108,P-1704,1. INGRESO,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.00
302,P-1704,4. COSTO SUPERVISIÓN,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.00


In [246]:
# Respaldo de Dataframe mes en curso real
mes_en_curso_real5.to_csv('archivos_respaldo/mes_en_curso_real5.csv', mode='a')

### 4.4 - **MERGE PARA -> MES EN CURSO ENTRE LIBRO MAYOR + PROYECTADO (PERSPECTIVA MES ANTERIOR)
(DATAFRAMES: df_libro_mayor5_mec_2M y df_proyectado4) 


#### 4.4.1 - CREACIÓN DE COPIA DE DATAFRAME "df_libro_mayor5_mec_2M" Y AJUSTES INICIALES
Ajuste del dataframe proveniente de LIBRO MAYOR

In [247]:
# Ajuste del dataframe proveniente de LIBRO MAYOR

df_libro_mayor5_mec_2M.sample(5)
df_libro_mayor5_mec_2M_3 = df_libro_mayor5_mec_2M.drop(['CREDITO', 'DEBITO', 'MARGEN'], axis=1)
df_libro_mayor5_mec_2M_3.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN
1520,P-1682,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,LIBRO MAYOR
540,P-1726,INGRESO,2023-05-01,2023.00,5.00,25429159,LIBRO MAYOR
702,P-1612,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,LIBRO MAYOR
1181,P-1666,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,LIBRO MAYOR
2574,P-1729,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,LIBRO MAYOR


In [248]:
# Ajuste del dataframe proveniente de LIBRO MAYOR

df_libro_mayor5_mec_2M.sample(5)
df_libro_mayor5_mec_2M_3 = df_libro_mayor5_mec_2M.drop(['CREDITO', 'DEBITO', 'MARGEN'], axis=1)
df_libro_mayor5_mec_2M_3.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN
52,P-1669,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,-156861,LIBRO MAYOR
344,P-1705,INGRESO,2023-05-01,2023.00,5.00,20190289,LIBRO MAYOR
584,P-1557,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,LIBRO MAYOR
293,P-1700,COSTO MATERIALES,2023-05-01,2023.00,5.00,-24936,LIBRO MAYOR
2599,1. TOTAL,INGRESO,2023-05-01,2023.00,5.00,0,LIBRO MAYOR


In [249]:
# INSPECCIÓN VISUAL INICIAL 1

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec_2M  : ", df_libro_mayor5_mec_2M.shape)
df_libro_mayor5_mec_2M_3[df_libro_mayor5_mec_2M_3["CODIGO_PROYECTO"]=="P-1704"]

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec_2M  :  (456, 10)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN
322,P-1704,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,-14242535,LIBRO MAYOR
328,P-1704,COSTO MATERIALES,2023-05-01,2023.00,5.00,-13967551,LIBRO MAYOR
334,P-1704,INGRESO,2023-05-01,2023.00,5.00,79059997,LIBRO MAYOR
2021,P-1704,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,LIBRO MAYOR
2022,P-1704,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,LIBRO MAYOR
2023,P-1704,INGRESO,2023-05-01,2023.00,5.00,0,LIBRO MAYOR
2024,P-1704,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,LIBRO MAYOR


In [250]:
# INSPECCIÓN VISUAL INICIAL

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec_2M_3  : ", df_libro_mayor5_mec_2M_3.shape)
df_libro_mayor5_mec_2M_3[df_libro_mayor5_mec_2M_3["CODIGO_PROYECTO"]=="P-1704"]

# Se crea nueva columna que define el destino del uso del trozo del DATAFRAME proveniente del libro mayor

def determinar_origen(row):
    if row['TIPO_CUENTA'] in ['INGRESO', 'COSTO MATERIALES']:
        return 'REAL'
    elif row['TIPO_CUENTA'] in ['COSTO SUPERVISIÓN','COSTO MANO DE OBRA']:
        return 'REAL Y PROYECTADO'

df_libro_mayor5_mec_2M_3['DESTINO'] = df_libro_mayor5_mec_2M_3.apply(determinar_origen, axis=1)
print(df_libro_mayor5_mec_2M_3.shape)
df_libro_mayor5_mec_2M_3.sample(10)


DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec_2M_3  :  (456, 7)
(456, 8)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
158,P-1688,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,-2009744,LIBRO MAYOR,REAL Y PROYECTADO
1133,P-1663,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,LIBRO MAYOR,REAL Y PROYECTADO
871,P-1642,INGRESO,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL
1205,P-1667,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,LIBRO MAYOR,REAL Y PROYECTADO
1567,P-1684,INGRESO,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL
1686,P-1689,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,LIBRO MAYOR,REAL Y PROYECTADO
1565,P-1684,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,LIBRO MAYOR,REAL Y PROYECTADO
1736,P-1691,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL
1904,P-1698,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL
349,P-1706,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,-1197155,LIBRO MAYOR,REAL Y PROYECTADO


In [251]:
# Cambiar los valores de la columna TIPO_CUENTA
df_libro_mayor5_mec_2M_3['TIPO_CUENTA'] = df_libro_mayor5_mec_2M_3['TIPO_CUENTA'].replace({'COSTO MATERIALES': '2. COSTO MATERIALES', 'COSTO MANO DE OBRA': '3. COSTO MANO DE OBRA', 'INGRESO': '1. INGRESO','COSTO SUPERVISIÓN': '4. COSTO SUPERVISIÓN'})

print(df_libro_mayor5_mec_2M_3.shape)
df_libro_mayor5_mec_2M_3["TIPO_CUENTA"].unique()

(456, 8)


array(['3. COSTO MANO DE OBRA', '2. COSTO MATERIALES', '1. INGRESO',
       '4. COSTO SUPERVISIÓN'], dtype=object)

#### 4.4.2 - CREACIÓN DE COPIA DE DATAFRAME "df_proyectado4" Y AJUSTES INICIALES
Ajuste del dataframe proveniente de PROYECTADO

In [252]:
df_proyectado4_1M.sample(5)
df_proyectado5_1M = df_proyectado4_1M.drop(['PROYECTO'], axis=1)

# Se crea la columna Fecha
df_proyectado5_1M['FECHA'] = pd.to_datetime(df_proyectado5_1M['MES'].astype(str) + '-1-' + df_proyectado5_1M['AÑO'].astype(str))

df_proyectado5_1M.sample(5)

,CODIGO_PROYECTO,TOTAL,MES,AÑO,ORIGEN,TIPO_CUENTA,FECHA
28,P-1682,0.00,5,2023,PROYECTADO,3. COSTO MANO DE OBRA,2023-05-01
27,P-1680,-11500000.00,5,2023,PROYECTADO,2. COSTO MATERIALES,2023-05-01
61,P-1717,0.00,5,2023,PROYECTADO,3. COSTO MANO DE OBRA,2023-05-01
20,P-1672,0.00,5,2023,PROYECTADO,2. COSTO MATERIALES,2023-05-01
24,P-1677,0.00,5,2023,PROYECTADO,2. COSTO MATERIALES,2023-05-01


In [253]:
# Se crea nueva columna que define el destino del uso del trozo del DATAFRAME proveniente del proyectado

def determinar_origen(row):
    if row['TIPO_CUENTA'] in ['1. INGRESO', '2. COSTO MATERIALES']:
        return 'PROYECTADO'

# Se realiza 
df_proyectado5_1M['DESTINO'] = df_proyectado5_1M.apply(determinar_origen, axis=1)
df_proyectado5_1M = df_proyectado5_1M[df_proyectado5_1M['TIPO_CUENTA'] != '3. COSTO MANO DE OBRA']


# Se reordenan las columnas para estandarizarlo como la que viene desde Libro Mayot
df_proyectado5_1M = df_proyectado5_1M.reindex(columns=['CODIGO_PROYECTO', 'TIPO_CUENTA', 'FECHA', 'AÑO', 'MES', 'TOTAL', 'ORIGEN', 'DESTINO'])

df_proyectado5_1M.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
38,P-1692,1. INGRESO,2023-05-01,2023,5,0.00,PROYECTADO,PROYECTADO
50,P-1704,2. COSTO MATERIALES,2023-05-01,2023,5,-24000000.00,PROYECTADO,PROYECTADO
48,P-1702,2. COSTO MATERIALES,2023-05-01,2023,5,0.00,PROYECTADO,PROYECTADO
21,P-1674,1. INGRESO,2023-05-01,2023,5,0.00,PROYECTADO,PROYECTADO
70,P-1726,2. COSTO MATERIALES,2023-05-01,2023,5,-10000000.00,PROYECTADO,PROYECTADO


#### 4.4.3 - CONCATENACIÓN ENTRE DATAFRAMES DEL MES EN CURSO LIBRO MAYOR Y PROYECTADO
Se realiza concatenación entre ambos dataframes

In [254]:
print("Dimensiones df_libro_mayor5_mec_2M_3 antes de cambios: ", df_libro_mayor5_mec_2M_3.shape)
print("Dimensiones df_proyectado5_1M antes de cambios       : ", df_proyectado5_1M.shape)

mes_en_curso_2M = pd.concat([df_libro_mayor5_mec_2M_3, df_proyectado5_1M])

print("Dimensiones dataframe final mes_en_curso después de concatenación  : ", mes_en_curso_2M.shape)
mes_en_curso_2M.sample(10)


Dimensiones df_libro_mayor5_mec_2M_3 antes de cambios:  (456, 8)
Dimensiones df_proyectado5_1M antes de cambios       :  (148, 8)
Dimensiones dataframe final mes_en_curso después de concatenación  :  (604, 8)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
848,P-1641,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
17,P-1669,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,0.00,PROYECTADO,PROYECTADO
46,P-1700,1. INGRESO,2023-05-01,2023.00,5.00,0.00,PROYECTADO,PROYECTADO
2096,P-1707,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
2237,P-1715,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
55,P-1709,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,-500000.00,PROYECTADO,PROYECTADO
1181,P-1666,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
1277,P-1670,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
1422,P-1677,4. COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
1781,P-1693,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO


#### 4.4.4 - MERGE CON "df_cod_proyecto" PARA CREAR DATAFRAME FINAL

In [255]:
print("Dimensiones mes_en_curso antes de cambios: ", mes_en_curso_2M.shape)
mes_en_curso_2M = mes_en_curso_2M.merge(df_cod_proyecto, on='CODIGO_PROYECTO', how='left')
mes_en_curso_2M = mes_en_curso_2M[['CODIGO_PROYECTO', 'NOMBRE_OBRA', 'NOMBRE_CLIENTE', 'TIPO_CUENTA', 'FECHA', 'AÑO', 'MES', 'TOTAL', 'ORIGEN', 'DESTINO']]
print("Dimensiones mes_en_curso_2M antes de cambios: ", mes_en_curso_2M.shape)
mes_en_curso_2M.sample(10)

Dimensiones mes_en_curso antes de cambios:  (604, 8)
Dimensiones mes_en_curso_2M antes de cambios:  (604, 10)


,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
96,P-1721,Strip Center Luis Pasteur,Inmobiliaria Luis Pasteur S.A.,1. INGRESO,2023-05-01,2023.00,5.00,17010449.00,LIBRO MAYOR,REAL
550,P-1672,Sta Clara (ccdd-telecom),Constructora UPC S.A.,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,0.00,PROYECTADO,PROYECTADO
364,P-1706,California,CONSTRUCTORA E INVERSIONES VITAL SPA.,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
375,P-1708,Eco España,Constructora ICF SPA,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
486,P-1684,Edificio Goycolea,EBCO S.A.,1. INGRESO,2023-05-01,2023.00,5.00,3335671.64,PROYECTADO,PROYECTADO
194,P-1656,Mirador Club Hípico,Constructora Vital Edificacion SPA,1. INGRESO,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
338,P-1698,CD Andes,EBCO S.A.,1. INGRESO,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
206,P-1661,San Vicente de Paul,DANACORP S.A,1. INGRESO,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
14,P-1672,Sta Clara (ccdd-telecom),Constructora UPC S.A.,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,-548892.00,LIBRO MAYOR,REAL
356,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO


In [256]:
#Para reemplazar los valores en blanco en las columnas "NOMBRE_OBRA" y "NOMBRE_CLIENTE" por el valor "TOTAL" cada vez que el valor de la columna "CODIGO_PROYECTO" es igual a "TOTAL", puedes hacer lo siguiente:

mes_en_curso_2M.loc[(mes_en_curso_2M['CODIGO_PROYECTO'] == '1. TOTAL') & (mes_en_curso_2M['NOMBRE_OBRA'].isnull()), 'NOMBRE_OBRA'] = '1. TOTAL'
mes_en_curso_2M.loc[(mes_en_curso_2M['CODIGO_PROYECTO'] == '1. TOTAL') & (mes_en_curso_2M['NOMBRE_CLIENTE'].isnull()), 'NOMBRE_CLIENTE'] = '1. TOTAL'
mes_en_curso_2M.sample(5)

,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
602,P-1728,NaN,NaN,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,0.00,PROYECTADO,PROYECTADO
0,P-1557,Rosario Rosales,EBCO S.A.,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,-945949.00,LIBRO MAYOR,REAL Y PROYECTADO
493,P-1691,Rosario Rosales,CONSTRUCTORA E INMOBILIARIA PRADOVERDE LTDA.,1. INGRESO,2023-05-01,2023.00,5.00,0.00,PROYECTADO,PROYECTADO
260,P-1679,Juan Sabaj,Constructora Grevia,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
374,P-1708,Eco España,Constructora ICF SPA,1. INGRESO,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL


In [257]:
# INSPECCIÓN VISUAL 1

mes_en_curso_2M[mes_en_curso_2M["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
58,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,-14242535.00,LIBRO MAYOR,REAL Y PROYECTADO
59,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,-13967551.00,LIBRO MAYOR,REAL
60,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,1. INGRESO,2023-05-01,2023.00,5.00,79059997.00,LIBRO MAYOR,REAL
356,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
357,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,4. COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
358,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,1. INGRESO,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
359,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
506,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,1. INGRESO,2023-05-01,2023.00,5.00,68606659.28,PROYECTADO,PROYECTADO
580,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,-24000000.00,PROYECTADO,PROYECTADO


#### 4.3.4 - AGREGACIÓN DE TODOS TIPOS DE CUENTA

In [258]:
print("Dimensiones Dataframe antes de cambios", mes_en_curso_2M.shape)

Dimensiones Dataframe antes de cambios (604, 10)


In [259]:
# definir nuevas filas
df_rows_to_add = []

for _, row in mes_en_curso_2M.iterrows():
    if row["TIPO_CUENTA"] == "1. INGRESO" and row["TOTAL"] == 0:
        row1 = row.copy()
        row1["TIPO_CUENTA"] = "1. INGRESO"
        row1["TOTAL"] = 0
        row1["ORIGEN"] = "AGREGACION"
        row1["DESTINO"] = "REAL"
        df_rows_to_add.append(row1)

        row2 = row.copy()
        row2["TIPO_CUENTA"] = "1. INGRESO"
        row2["TOTAL"] = 0
        row2["ORIGEN"] = "AGREGACION"
        row2["DESTINO"] = "PROYECTADO"
        df_rows_to_add.append(row2)

    elif row["TIPO_CUENTA"] == "2. COSTO MATERIALES" and row["TOTAL"] == 0:
        row1 = row.copy()
        row1["TIPO_CUENTA"] = "2. COSTO MATERIALES"
        row1["TOTAL"] = 0
        row1["ORIGEN"] = "AGREGACION"
        row1["DESTINO"] = "REAL"
        df_rows_to_add.append(row1)

        row2 = row.copy()
        row2["TIPO_CUENTA"] = "2. COSTO MATERIALES"
        row2["TOTAL"] = 0
        row2["ORIGEN"] = "AGREGACION"
        row2["DESTINO"] = "PROYECTADO"
        df_rows_to_add.append(row2)

    elif row["TIPO_CUENTA"] == "3. COSTO MANO DE OBRA" and row["TOTAL"] == 0:
        row1 = row.copy()
        row1["TIPO_CUENTA"] = "3. COSTO MANO DE OBRA"
        row1["TOTAL"] = 0
        row1["ORIGEN"] = "AGREGACION"
        row1["DESTINO"] = "REAL Y PROYECTADO"
        df_rows_to_add.append(row1)

  
    elif row["TIPO_CUENTA"] == "4. COSTO SUPERVISIÓN" and row["TOTAL"] == 0:
        row1 = row.copy()
        row1["TIPO_CUENTA"] = "4. COSTO SUPERVISIÓN"
        row1["TOTAL"] = 0
        row1["ORIGEN"] = "AGREGACION"
        row1["DESTINO"] = "REAL Y PROYECTADO"
        df_rows_to_add.append(row1)
        
        row2 = row.copy()
        row2["TIPO_CUENTA"] = "3. COSTO MANO DE OBRA"
        row2["TOTAL"] = 0
        row2["ORIGEN"] = "AGREGACION"
        row2["DESTINO"] = "REAL Y PROYECTADO"
        df_rows_to_add.append(row2)

In [260]:
# concatenar nuevas filas al DataFrame original
mes_en_curso_2M = pd.concat([mes_en_curso_2M, pd.DataFrame(df_rows_to_add)], ignore_index=True)


In [261]:
# Se renombra "TOTAL" a "1. TOTAL" para que se pueda visualizar como primera opción en el dashboard.
mes_en_curso_2M['CODIGO_PROYECTO'] = mes_en_curso_2M['CODIGO_PROYECTO'].apply(lambda x: '1. TOTAL' if x == 'TOTAL' else x)

cols_to_replace = ['CODIGO_PROYECTO', 'NOMBRE_OBRA', 'NOMBRE_CLIENTE']
mes_en_curso_2M[cols_to_replace] = mes_en_curso_2M[cols_to_replace].replace('TOTAL', '1. TOTAL')
mask = mes_en_curso_2M['CODIGO_PROYECTO'] == '1. TOTAL'
mes_en_curso_2M.loc[mask, ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'

print("Dimensiones Dataframe después de cambios", mes_en_curso_2M.shape)

mes_en_curso_2M.sample(10)

Dimensiones Dataframe después de cambios (1395, 10)


,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
86,P-1715,CD Lo Echevers,Quezada y Boetsch,1. INGRESO,2023-05-01,2023.00,5.00,18230800.00,LIBRO MAYOR,REAL
697,P-1642,Corinto,EMP. CONSTRUCTORA INGENIEROS S.A.,1. INGRESO,2023-05-01,2023.00,5.00,0.00,AGREGACION,REAL
879,P-1681,CD Rosen (CCTV Provisorio),Colchones ROSEN SAIC,1. INGRESO,2023-05-01,2023.00,5.00,0.00,AGREGACION,REAL
1255,P-1678,Habilitación Oficinas Pirer SPA,SOC. CONSTRUCTORA MANSILLA Y SALGADO LIMITADA,1. INGRESO,2023-05-01,2023.00,5.00,0.00,AGREGACION,REAL
123,P-1591,Edificio Estoril,EBCO S.A.,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
573,P-1697,Estocolmo,Constructora Fortaleza S.A,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,-2500000.00,PROYECTADO,PROYECTADO
981,P-1696,Santa Victoria - Terracorp,Constructora Terracorp S.A.,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0.00,AGREGACION,REAL Y PROYECTADO
987,P-1696,Santa Victoria - Terracorp,Constructora Terracorp S.A.,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,0.00,AGREGACION,PROYECTADO
419,P-1721,Strip Center Luis Pasteur,Inmobiliaria Luis Pasteur S.A.,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
605,P-1677,Galvarino Gallardo,Moller y Perez - Cotapos Construcciones Indust...,1. INGRESO,2023-05-01,2023.00,5.00,0.00,AGREGACION,PROYECTADO


In [262]:
# INSPECCIÓN VISUAL 1
mes_en_curso_2M[mes_en_curso_2M["CODIGO_PROYECTO"]=="P-1726"]


,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
105,P-1726,NaN,NaN,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,-2654140.00,LIBRO MAYOR,REAL Y PROYECTADO
106,P-1726,NaN,NaN,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,-9165547.00,LIBRO MAYOR,REAL
107,P-1726,NaN,NaN,1. INGRESO,2023-05-01,2023.00,5.00,25429159.00,LIBRO MAYOR,REAL
436,P-1726,NaN,NaN,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
437,P-1726,NaN,NaN,4. COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
438,P-1726,NaN,NaN,1. INGRESO,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
439,P-1726,NaN,NaN,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
526,P-1726,NaN,NaN,1. INGRESO,2023-05-01,2023.00,5.00,25429159.00,PROYECTADO,PROYECTADO
600,P-1726,NaN,NaN,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,-10000000.00,PROYECTADO,PROYECTADO
1170,P-1726,NaN,NaN,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0.00,AGREGACION,REAL Y PROYECTADO


In [263]:
mes_en_curso_2M["TIPO_CUENTA"].unique()

array(['3. COSTO MANO DE OBRA', '2. COSTO MATERIALES', '1. INGRESO',
       '4. COSTO SUPERVISIÓN'], dtype=object)

In [264]:
# INSPECCIÓN VISUAL 1
mes_en_curso_2M[mes_en_curso_2M["CODIGO_PROYECTO"]=="1. TOTAL"]


,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
112,1. TOTAL,1. TOTAL,1. TOTAL,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,-111885468.00,LIBRO MAYOR,REAL Y PROYECTADO
113,1. TOTAL,1. TOTAL,1. TOTAL,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,-169535388.00,LIBRO MAYOR,REAL
114,1. TOTAL,1. TOTAL,1. TOTAL,1. INGRESO,2023-05-01,2023.00,5.00,460798513.00,LIBRO MAYOR,REAL
115,1. TOTAL,1. TOTAL,1. TOTAL,4. COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,-5096121.00,LIBRO MAYOR,REAL Y PROYECTADO
452,1. TOTAL,1. TOTAL,1. TOTAL,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
453,1. TOTAL,1. TOTAL,1. TOTAL,4. COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
454,1. TOTAL,1. TOTAL,1. TOTAL,1. INGRESO,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
455,1. TOTAL,1. TOTAL,1. TOTAL,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
529,1. TOTAL,1. TOTAL,1. TOTAL,1. INGRESO,2023-05-01,2023.00,5.00,461955807.36,PROYECTADO,PROYECTADO
603,1. TOTAL,1. TOTAL,1. TOTAL,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,-212300000.00,PROYECTADO,PROYECTADO


#### 4.4.7 - **CREACIÓN DATAFRAME PARA -> PROYECTADO

In [265]:
# Se crea una copia del dataframe original mes_en_curso2

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_2M       : ", mes_en_curso_2M.shape)

mes_en_curso_2M_1 = mes_en_curso_2M.copy()
mes_en_curso_2M_1 = mes_en_curso_2M_1[mes_en_curso_2M_1['DESTINO'].str.contains('PROYECTADO')]

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_2M_1   : ", mes_en_curso_2M_1.shape)


print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_2M_1     : ", mes_en_curso_2M_1.shape)

mes_en_curso_proy_2M = pd.pivot_table(mes_en_curso_2M_1, values=['TOTAL'], index=['CODIGO_PROYECTO','TIPO_CUENTA'], aggfunc='sum', fill_value=0)
mes_en_curso_proy_2M = pd.DataFrame(mes_en_curso_proy_2M.to_records())

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy_2M  : ", mes_en_curso_proy_2M.shape)

mes_en_curso_proy_2M.sample(10)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_2M       :  (1395, 10)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_2M_1   :  (878, 10)
DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_2M_1     :  (878, 10)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy_2M  :  (364, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
308,P-1716,1. INGRESO,10082518.00
62,P-1641,3. COSTO MANO DE OBRA,0.00
112,P-1661,3. COSTO MANO DE OBRA,0.00
151,P-1673,4. COSTO SUPERVISIÓN,0.00
325,P-1720,2. COSTO MATERIALES,0.00
335,P-1722,4. COSTO SUPERVISIÓN,0.00
259,P-1701,2. COSTO MATERIALES,0.00
178,P-1681,1. INGRESO,0.00
346,P-1725,3. COSTO MANO DE OBRA,-2120143.00
22,P-1603,1. INGRESO,0.00


In [266]:
# TESTING VISUAL 1
mes_en_curso_proy_2M[mes_en_curso_proy_2M["CODIGO_PROYECTO"]=="P-1726"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
348,P-1726,1. INGRESO,25429159.00
349,P-1726,2. COSTO MATERIALES,-10000000.00
350,P-1726,3. COSTO MANO DE OBRA,-2654140.00
351,P-1726,4. COSTO SUPERVISIÓN,0.00


In [267]:
# TESTING VISUAL 2
mes_en_curso_proy_2M[mes_en_curso_proy_2M["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
2,1. TOTAL,1. INGRESO,461955807.36
3,1. TOTAL,2. COSTO MATERIALES,-212300000.00
4,1. TOTAL,3. COSTO MANO DE OBRA,-111885468.00
5,1. TOTAL,4. COSTO SUPERVISIÓN,-5096121.00


##### 4.4.7.1 - MERGE CON DATAFRAME LIBRO MAYOR 2023 BASE 

In [268]:
# Merge Dataframe BASE 

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy_2M     : ", mes_en_curso_proy_2M.shape)
print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2    : ", df_libro_mayor5_base2.shape)

mes_en_curso_proy_2M_2 = df_libro_mayor5_base2.merge(mes_en_curso_proy_2M, on=['CODIGO_PROYECTO', 'TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy_2M_2  : ", mes_en_curso_proy_2M_2.shape)

mes_en_curso_proy_2M_2['TOTAL'] = mes_en_curso_proy_2M_2['TOTAL'].fillna(0)

mes_en_curso_proy_2M_2.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy_2M     :  (364, 3)
DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2    :  (340, 5)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy_2M_2  :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
45,P-1679,3. COSTO MANO DE OBRA,0,JUAN SABAJ,CONSTRUCTORA GREVIA,0.00
3,P-1603,2. COSTO MATERIALES,0,FARELLONES,EMPRESA CONSTRUCTORA DLP S.A.,0.00
121,P-1709,3. COSTO MANO DE OBRA,0,CENTRO COMERCIAL PASEO SAN PEDRO,EBCO S.A.,-2333919.00
294,P-1696,4. COSTO SUPERVISIÓN,0,SANTA VICTORIA - TERRACORP,CONSTRUCTORA TERRACORP S.A.,0.00
173,P-1591,4. COSTO SUPERVISIÓN,0,EDIFICIO ESTORIL,EBCO S.A.,0.00


In [269]:
# TESTING VISUAL 1

mes_en_curso_proy_2M_2[mes_en_curso_proy_2M_2["CODIGO_PROYECTO"]=="P-1726"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
159,P-1726,3. COSTO MANO DE OBRA,0,NaN,NaN,-2654140.00
160,P-1726,2. COSTO MATERIALES,0,NaN,NaN,-10000000.00
161,P-1726,1. INGRESO,0,NaN,NaN,25429159.00
329,P-1726,4. COSTO SUPERVISIÓN,0,NaN,NaN,0.00


In [270]:
# TESTING VISUAL 2
mes_en_curso_proy_2M_2[mes_en_curso_proy_2M_2["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-5096121.00
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,461955807.36
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-212300000.00
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-111885468.00


##### 4.4.7.2 - SE RECALCULAN LOS TOTALES

In [271]:
# Calcular la suma de los valores totales para cada tipo de cuenta

# Se eliminan totales excepto supervisión, para re hacer el calculo de totales

mes_en_curso_proy_2M_2 = mes_en_curso_proy_2M_2.drop(mes_en_curso_proy_2M_2[(mes_en_curso_proy_2M_2['CODIGO_PROYECTO'] == '1. TOTAL') & (mes_en_curso_proy_2M_2['TIPO_CUENTA'] != '4. COSTO SUPERVISIÓN')].index)

print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_proy_2M_2 : ", mes_en_curso_proy_2M_2.shape)

df_total = mes_en_curso_proy_2M_2.groupby('TIPO_CUENTA')['TOTAL'].sum().reset_index()
df_total['CODIGO_PROYECTO'] = '1. TOTAL'

# Agregar las filas adicionales al DataFrame
mes_en_curso_proy_2M_3 = pd.concat([mes_en_curso_proy_2M_2, df_total], ignore_index=True)

# Modificar las columnas "NOMBRE_OBRA" y "NOMBRE_CLIENTE" para "1. TOTAL"
mes_en_curso_proy_2M_3.loc[mes_en_curso_proy_2M_3['CODIGO_PROYECTO'] == '1. TOTAL', ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_proy_2M_3 : ", mes_en_curso_proy_2M_3.shape)

mes_en_curso_proy_2M_3.sample(10) 

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_proy_2M_2 :  (337, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_proy_2M_3 :  (341, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
96,P-1699,1. INGRESO,0.00,VILLA PANAMERICANA LOTE B,EMPRESA CONSTRUCTORA DLP S.A.,21598229.00
251,P-1669,4. COSTO SUPERVISIÓN,0.00,DEPARTAMENTAL,EMP. CONSTRUCTORA INGENIEROS S.A.,0.00
335,P-1729,4. COSTO SUPERVISIÓN,0.00,NaN,NaN,0.00
301,P-1703,4. COSTO SUPERVISIÓN,0.00,SANTA VICTORIA DANACORP,DANACORP S.A,0.00
317,P-1719,2. COSTO MATERIALES,0.00,CD METALPAR,EBCO S.A.,0.00
183,P-1612,1. INGRESO,0.00,BARRIO PARQUE SANTIAGO 7,CONSTRUCTORA ALTURAS DEL SUR S.A,0.00
262,P-1675,3. COSTO MANO DE OBRA,0.00,PALENA II,CONSTRUCTORA MYA SPA.,0.00
2,P-1592,2. COSTO MATERIALES,0.00,NUEVA COSTANERA 3300,EBCO S.A.,0.00
87,P-1696,2. COSTO MATERIALES,0.00,SANTA VICTORIA - TERRACORP,CONSTRUCTORA TERRACORP S.A.,0.00
143,P-1719,1. INGRESO,0.00,CD METALPAR,EBCO S.A.,0.00


In [272]:
# TESTING VISUAL 2
mes_en_curso_proy_2M_3[mes_en_curso_proy_2M_3["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-5096121.00
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,461955807.36
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-212300000.00
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-111885468.00
340,1. TOTAL,4. COSTO SUPERVISIÓN,NaN,1. TOTAL,1. TOTAL,-5096121.00


In [273]:
# SE ELIMINA COSTO SUPERVISIÓN DUPLICADO.

print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_proy_2M_3 : ", mes_en_curso_proy_2M_3.shape)

mes_en_curso_proy_2M_3 = mes_en_curso_proy_2M_3.drop(mes_en_curso_proy_2M_3[(mes_en_curso_proy_2M_3['CODIGO_PROYECTO'] == '1. TOTAL') & (mes_en_curso_proy_2M_3['TIPO_CUENTA'] == '4. COSTO SUPERVISIÓN') & (mes_en_curso_proy_2M_3['TOTAL_LB'].isnull())].index)

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_proy_2M_3 : ", mes_en_curso_proy_2M_3.shape)

mes_en_curso_proy_2M_3[mes_en_curso_proy_2M_3["CODIGO_PROYECTO"]=="1. TOTAL"]

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_proy_2M_3 :  (341, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_proy_2M_3 :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-5096121.00
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,461955807.36
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-212300000.00
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-111885468.00


In [274]:
# TESTING VISUAL 1
mes_en_curso_proy_2M_3[mes_en_curso_proy_2M_3["CODIGO_PROYECTO"]=="P-1726"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
159,P-1726,3. COSTO MANO DE OBRA,0.00,NaN,NaN,-2654140.00
160,P-1726,2. COSTO MATERIALES,0.00,NaN,NaN,-10000000.00
161,P-1726,1. INGRESO,0.00,NaN,NaN,25429159.00
329,P-1726,4. COSTO SUPERVISIÓN,0.00,NaN,NaN,0.00


In [275]:
# TESTING VISUAL 2
mes_en_curso_proy_2M_3[mes_en_curso_proy_2M_3["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-5096121.00
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,461955807.36
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-212300000.00
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-111885468.00


##### 4.4.7.3 - CREACIÓN DE MARGEN

In [276]:
#CREACIÓN DE MARGENES

mes_en_curso_proy_2M_4 = mes_en_curso_proy_2M_3.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = mes_en_curso_proy_2M_4.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)

,CODIGO_PROYECTO,MARGEN
74,P-1719,0.00
18,P-1649,0.00
82,P-1727,0.26
35,P-1676,0.00
19,P-1653,0.00
77,P-1722,0.00
81,P-1726,0.50
25,P-1664,0.00
80,P-1725,0.58
52,P-1694,0.31


In [277]:
print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_proy_2M_4 : ", mes_en_curso_proy_2M_4.shape)

mes_en_curso_proy_2M_5 = pd.merge(mes_en_curso_proy_2M_4, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_proy_2M_5 : ", mes_en_curso_proy_2M_5.shape)
mes_en_curso_proy_2M_5.sample(10)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_proy_2M_4 :  (340, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_proy_2M_5 :  (340, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
270,P-1681,2. COSTO MATERIALES,0.00,CD ROSEN (CCTV PROVISORIO),COLCHONES ROSEN SAIC,0.00,0.00
88,P-1696,1. INGRESO,0.00,SANTA VICTORIA - TERRACORP,CONSTRUCTORA TERRACORP S.A.,0.00,0.00
127,P-1713,3. COSTO MANO DE OBRA,0.00,RESERVA SAN FRANCISCO,EMPRESA NACIONAL DE CONSTRUCCIÓN LAS VERTIENTE...,-172081.00,0.00
40,P-1676,3. COSTO MANO DE OBRA,0.00,LO CAMPINO,EBCO S.A.,0.00,0.00
196,P-1636,3. COSTO MANO DE OBRA,0.00,SANTO DOMINGO 1410,DANACORP S.A,0.00,0.00
5,P-1612,2. COSTO MATERIALES,0.00,BARRIO PARQUE SANTIAGO 7,CONSTRUCTORA ALTURAS DEL SUR S.A,0.00,0.00
308,P-1712,4. COSTO SUPERVISIÓN,0.00,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,0.00,0.25
189,P-1626,1. INGRESO,0.00,CLINICA BIO BIO,CONSTRUCTORA LYD S.A.,0.00,0.00
334,P-1729,3. COSTO MANO DE OBRA,0.00,NaN,NaN,0.00,0.00
169,P-1557,2. COSTO MATERIALES,0.00,ROSARIO ROSALES,EBCO S.A.,0.00,0.00


In [278]:
# TESTING VISUAL 1
mes_en_curso_proy_2M_5[mes_en_curso_proy_2M_5["CODIGO_PROYECTO"]=="P-1723"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
150,P-1723,3. COSTO MANO DE OBRA,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,-7889395.00,0.11
151,P-1723,2. COSTO MATERIALES,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,-53000000.00,0.11
152,P-1723,1. INGRESO,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,68515939.00,0.11
326,P-1723,4. COSTO SUPERVISIÓN,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,0.00,0.11


In [279]:
# TESTING VISUAL 2
mes_en_curso_proy_2M_5[mes_en_curso_proy_2M_5["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-5096121.00,0.29
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,461955807.36,0.29
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-212300000.00,0.29
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-111885468.00,0.29


In [280]:
# Respaldo de Dataframe mes en curso antes de abrirrlo
mes_en_curso_proy_2M_5.to_csv('archivos_respaldo/mes_en_curso_proy_2M_5.csv', mode='a')

In [281]:
mes_en_curso_proy_2M_5[mes_en_curso_proy_2M_5["CODIGO_PROYECTO"]=="P-1726"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
159,P-1726,3. COSTO MANO DE OBRA,0.00,NaN,NaN,-2654140.00,0.50
160,P-1726,2. COSTO MATERIALES,0.00,NaN,NaN,-10000000.00,0.50
161,P-1726,1. INGRESO,0.00,NaN,NaN,25429159.00,0.50
329,P-1726,4. COSTO SUPERVISIÓN,0.00,NaN,NaN,0.00,0.50


#### 4.4.8 - **CREACIÓN DATAFRAME PARA -> MES EN CURSO (PERSPECTIVA MES ANTERIOR)

##### 4.4.8.1 - TRATAMIENTO mes_en_curso2

In [282]:
# Se crea una copia del dataframe original mes_en_curso2

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_2M       : ", mes_en_curso_2M.shape)

mes_en_curso_2M_1 = mes_en_curso_2M.copy()
mes_en_curso_2M_1 = mes_en_curso_2M_1[mes_en_curso_2M_1['DESTINO'].str.contains('REAL')]

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_2M_1     : ", mes_en_curso_2M_1.shape)

mes_en_curso_real_2M = pd.pivot_table(mes_en_curso_2M_1, values=['TOTAL'], index=['CODIGO_PROYECTO','TIPO_CUENTA'], aggfunc='sum', fill_value=0)
mes_en_curso_real_2M = pd.DataFrame(mes_en_curso_real_2M.to_records())

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_real_2M  : ", mes_en_curso_real_2M.shape)

mes_en_curso_real_2M.sample(10)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_2M       :  (1395, 10)
DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_2M_1     :  (979, 10)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_real_2M  :  (364, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
21,P-1601,2. COSTO MATERIALES,0
339,P-1723,4. COSTO SUPERVISIÓN,0
215,P-1690,2. COSTO MATERIALES,-167190
247,P-1698,2. COSTO MATERIALES,0
346,P-1725,3. COSTO MANO DE OBRA,-2120143
212,P-1689,3. COSTO MANO DE OBRA,0
278,P-1706,3. COSTO MANO DE OBRA,-1197155
2,1. TOTAL,1. INGRESO,460798513
231,P-1694,2. COSTO MATERIALES,-7641345
175,P-1680,2. COSTO MATERIALES,-11122532


In [283]:
# TESTING VISUAL 1

mes_en_curso_real_2M[mes_en_curso_real_2M["CODIGO_PROYECTO"]=="P-1712"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
292,P-1712,1. INGRESO,15000000
293,P-1712,2. COSTO MATERIALES,-5124213
294,P-1712,3. COSTO MANO DE OBRA,-2791400
295,P-1712,4. COSTO SUPERVISIÓN,0


In [284]:
# TESTING VISUAL 2

mes_en_curso_real_2M[mes_en_curso_real_2M["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
2,1. TOTAL,1. INGRESO,460798513
3,1. TOTAL,2. COSTO MATERIALES,-169535388
4,1. TOTAL,3. COSTO MANO DE OBRA,-111885468
5,1. TOTAL,4. COSTO SUPERVISIÓN,-5096121


##### 4.4.8.2 - MERGE CON DATAFRAME LIBRO MAYOR 2023 BASE 

In [285]:
# Merge Dataframe BASE 

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_real_2M    : ", mes_en_curso_real_2M.shape)
print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2   : ", df_libro_mayor5_base2.shape)

mes_en_curso_real_2M = df_libro_mayor5_base2.merge(mes_en_curso_real_2M, on=['CODIGO_PROYECTO', 'TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_real_2M   : ", mes_en_curso_real_2M.shape)

mes_en_curso_real_2M.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_real_2M    :  (364, 3)
DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2   :  (340, 5)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_real_2M   :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
182,P-1611,4. COSTO SUPERVISIÓN,0,EDIFICIO PLAZA EGAÑA,EBCO S.A.,0
299,P-1700,4. COSTO SUPERVISIÓN,0,CD ORIZON,EBCO S.A.,0
150,P-1723,3. COSTO MANO DE OBRA,0,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,-7889395
77,P-1692,1. INGRESO,0,JOSÉ PEREZ COTAPOS,EBCO S.A.,142397
323,P-1722,2. COSTO MATERIALES,0,ECO CERRILLOS,CONSTRUCTORA ICF SPA,0


In [286]:
# TESTING VISUAL 1

mes_en_curso_real_2M[mes_en_curso_real_2M["CODIGO_PROYECTO"]=="P-1712"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
124,P-1712,3. COSTO MANO DE OBRA,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,-2791400
125,P-1712,2. COSTO MATERIALES,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,-5124213
126,P-1712,1. INGRESO,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,15000000
308,P-1712,4. COSTO SUPERVISIÓN,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,0


In [287]:
# TESTING VISUAL 2

mes_en_curso_real_2M[mes_en_curso_real_2M["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-5096121
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,460798513
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-169535388
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-111885468


##### 4.4.8.3 - SE RECALCULAN LOS TOTALES

In [288]:
# Calcular la suma de los valores totales para cada tipo de cuenta

# Se eliminan totales excepto supervisión, para re hacer el calculo de totales

mes_en_curso_real_2M = mes_en_curso_real_2M.drop(mes_en_curso_real_2M[(mes_en_curso_real_2M['CODIGO_PROYECTO'] == '1. TOTAL') & (mes_en_curso_real_2M['TIPO_CUENTA'] != '4. COSTO SUPERVISIÓN')].index)

print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_real_2M : ", mes_en_curso_real_2M.shape)

df_total = mes_en_curso_real_2M.groupby('TIPO_CUENTA')['TOTAL'].sum().reset_index()
df_total['CODIGO_PROYECTO'] = '1. TOTAL'

# Agregar las filas adicionales al DataFrame
mes_en_curso_real_2M_3 = pd.concat([mes_en_curso_real_2M, df_total], ignore_index=True)

# Modificar las columnas "NOMBRE_OBRA" y "NOMBRE_CLIENTE" para "1. TOTAL"
mes_en_curso_real_2M_3.loc[mes_en_curso_real_2M_3['CODIGO_PROYECTO'] == '1. TOTAL', ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_real_2M_3 : ", mes_en_curso_real_2M_3.shape)

mes_en_curso_real_2M_3.sample(10)  

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_real_2M :  (337, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_real_2M_3 :  (341, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
50,P-1681,1. INGRESO,0.00,CD ROSEN (CCTV PROVISORIO),COLCHONES ROSEN SAIC,0
269,P-1680,4. COSTO SUPERVISIÓN,0.00,NUEVA UNO,CONSTRUCTORA TECSA S.A.,0
17,P-1648,2. COSTO MATERIALES,0.00,ETAPA 2 CD PERALILLO TW,INMOBILIARIA Y RENTAS LOGÍSTICA S.A.,0
277,P-1684,4. COSTO SUPERVISIÓN,0.00,EDIFICIO GOYCOLEA,EBCO S.A.,0
100,P-1701,3. COSTO MANO DE OBRA,0.00,EDIFICIO ZENTENO,CONSTRUCTORA GREVIA SPA.,0
152,P-1723,1. INGRESO,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,68515940
306,P-1708,4. COSTO SUPERVISIÓN,0.00,ECO ESPAÑA,CONSTRUCTORA ICF SPA,0
69,P-1690,3. COSTO MANO DE OBRA,0.00,ONGOLMO - HOME CONDOMINIO,CANTAUCO CONSTRUCCION LTDA.,0
19,P-1653,2. COSTO MATERIALES,0.00,EDUARDO DE LA BARRA,CONSTRUCTORA E INVERSIONES VITAL SPA.,0
99,P-1700,1. INGRESO,0.00,CD ORIZON,EBCO S.A.,0


In [289]:
# TESTING VISUAL 1

mes_en_curso_real_2M_3[mes_en_curso_real_2M_3["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-5096121
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,460798513
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-169535388
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-111885468
340,1. TOTAL,4. COSTO SUPERVISIÓN,NaN,1. TOTAL,1. TOTAL,-5096121


In [290]:
# SE ELIMINA COSTO SUPERVISIÓN DUPLICADO.

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: mes_en_curso_real_2M_3 : ", mes_en_curso_real_2M_3.shape)

mes_en_curso_real_2M_3 = mes_en_curso_real_2M_3.drop(mes_en_curso_real_2M_3[(mes_en_curso_real_2M_3['CODIGO_PROYECTO'] == '1. TOTAL') & (mes_en_curso_real_2M_3['TIPO_CUENTA'] == '4. COSTO SUPERVISIÓN') & (mes_en_curso_real_2M_3['TOTAL_LB'].isnull())].index)

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_real_2M_3 : ", mes_en_curso_real_2M_3.shape)

mes_en_curso_real_2M_3[mes_en_curso_real_2M_3["CODIGO_PROYECTO"]=="1. TOTAL"]

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: mes_en_curso_real_2M_3 :  (341, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_real_2M_3 :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-5096121
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,460798513
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-169535388
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-111885468


##### 4.4.8.4 - CREACIÓN DE MARGEN

In [291]:
#CREACIÓN DE MARGENES

mes_en_curso_real_2M_4 = mes_en_curso_real_2M_3.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = mes_en_curso_real_2M_4.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)

,CODIGO_PROYECTO,MARGEN
54,P-1696,0.00
33,P-1674,0.00
1,P-1557,0.00
10,P-1636,0.00
55,P-1697,0.57
27,P-1667,0.00
67,P-1712,0.47
45,P-1687,0.00
17,P-1648,0.00
41,P-1683,0.00


In [292]:
print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_real_2M_4 : ", mes_en_curso_real_2M_4.shape)

mes_en_curso_real_2M_5 = pd.merge(mes_en_curso_real_2M_4, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_real_2M_5 : ", mes_en_curso_real_2M_5.shape)
mes_en_curso_real_2M_5.sample(10)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_real_2M_4 :  (340, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_real_2M_5 :  (340, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
4,P-1611,2. COSTO MATERIALES,0.00,EDIFICIO PLAZA EGAÑA,EBCO S.A.,0,0.00
273,P-1682,4. COSTO SUPERVISIÓN,0.00,PLAZA CEPPI,EMP. CONSTRUCTORA INGENIEROS S.A.,0,1.00
288,P-1691,4. COSTO SUPERVISIÓN,0.00,ROSARIO ROSALES,CONSTRUCTORA E INMOBILIARIA PRADOVERDE LTDA.,0,0.00
246,P-1667,4. COSTO SUPERVISIÓN,0.00,ECO PLACILLA,CONSTRUCTORA ICF SPA,0,0.00
81,P-1694,2. COSTO MATERIALES,0.00,ECO HUEMUL,CONSTRUCTORA ICF SPA,-7641345,0.32
88,P-1696,1. INGRESO,0.00,SANTA VICTORIA - TERRACORP,CONSTRUCTORA TERRACORP S.A.,0,0.00
53,P-1682,1. INGRESO,0.00,PLAZA CEPPI,EMP. CONSTRUCTORA INGENIEROS S.A.,18808581,1.00
271,P-1681,3. COSTO MANO DE OBRA,0.00,CD ROSEN (CCTV PROVISORIO),COLCHONES ROSEN SAIC,0,0.00
275,P-1683,3. COSTO MANO DE OBRA,0.00,CD BP-A05,EBCO S.A.,0,0.00
169,P-1557,2. COSTO MATERIALES,0.00,ROSARIO ROSALES,EBCO S.A.,0,0.00


In [293]:
# TESTING VISUAL 1
mes_en_curso_real_2M_5[mes_en_curso_real_2M_5["CODIGO_PROYECTO"]=="P-1716"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
136,P-1716,3. COSTO MANO DE OBRA,0.00,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,-895428,0.61
137,P-1716,2. COSTO MATERIALES,0.00,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,-2986507,0.61
138,P-1716,1. INGRESO,0.00,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,10082518,0.61
312,P-1716,4. COSTO SUPERVISIÓN,0.00,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,0,0.61


In [294]:
# TESTING VISUAL 2
mes_en_curso_real_2M_5[mes_en_curso_real_2M_5["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
106,P-1704,3. COSTO MANO DE OBRA,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-14242535,0.64
107,P-1704,2. COSTO MATERIALES,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-13967551,0.64
108,P-1704,1. INGRESO,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,79059997,0.64
302,P-1704,4. COSTO SUPERVISIÓN,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.64


In [295]:
# Respaldo de Dataframe mes en curso real
mes_en_curso_real_2M_5.to_csv('archivos_respaldo/mes_en_curso_real_2M_5.csv', mode='a')

### 4.5 - **MERGE PARA -> YTD PROYECTADO
YTD Proyectado = YTD + el mes en curso del proyectado

#### 4.5.1 - EXPLORACIÓN Y PREPARACIÓN DE DATAFRAMES


In [296]:
# DATAFRAME PROVENIENTE DEL LIBRO MAYOR 2023 HASTA EL MES CERRADO.

print(df_ytd_cerrado.shape)
df_ytd_cerrado.sample(10)

(340, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
311,P-1722,1. INGRESO,0,Eco Cerrillos,Constructora ICF SPA
120,P-1670,3. COSTO MANO DE OBRA,0,MK II Seguridad y Redes,Comercial K SpA.
245,P-1704,2. COSTO MATERIALES,-72089322,CD Mercado Libre Mezzanina,EBCO S.A.
43,P-1636,1. INGRESO,0,Santo Domingo 1410,DANACORP S.A
175,P-1685,1. INGRESO,0,Mapa Edificios No Operacionales,Claro Vicuña Valenzuela S.A.
249,P-1705,2. COSTO MATERIALES,-15060169,Los Clarines,CONSTRUCTORA INARCO S.A.
329,P-1727,2. COSTO MATERIALES,-13655419,NaN,NaN
46,P-1639,4. COSTO SUPERVISIÓN,0,Lyon 2546,Moller y Perez - Cotapos Construcciones Indust...
125,P-1672,2. COSTO MATERIALES,-229982,Sta Clara (ccdd-telecom),Constructora UPC S.A.
24,P-1612,3. COSTO MANO DE OBRA,0,Barrio Parque Santiago 7,CONSTRUCTORA ALTURAS DEL SUR S.A


In [297]:
# DATAFRAME PROVENIENTE DEL LIBRO PERO SOO CONSIDERA MES EN CURSO.

print(mes_en_curso2.shape)
mes_en_curso2.head(5)

# Se filtra por la columna DESTINO != REAL (para considerar solo lo proyectado)
mes_en_curso2_1 = mes_en_curso2[mes_en_curso2['DESTINO'] != 'REAL']

# Eliminar las columnas ORIGEN y DESTINO
mes_en_curso2_1 = mes_en_curso2_1.drop(columns=['ORIGEN', 'DESTINO','MES','AÑO','FECHA'])

mes_en_curso2_1.sample(5)

(1357, 10)


,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,TOTAL
57,P-1715,CD Lo Echevers,Quezada y Boetsch,3. COSTO MANO DE OBRA,-4177681
486,P-1720,NaN,NaN,1. INGRESO,0
555,P-1716,CD Astara,Astara Logística Chile SpA.,2. COSTO MATERIALES,0
733,P-1663,Oficinas Equifax,Constructora VRK Ingenieria Limitada,3. COSTO MANO DE OBRA,0
173,P-1661,San Vicente de Paul,DANACORP S.A,4. COSTO SUPERVISIÓN,0


#### 4.5.2 - Concatenación de Dataframes

In [298]:
# Se concatenan los dataframes.

print("Dataframes antes de concatenar   : df_ytd_cerrado   ", df_ytd_cerrado.shape)
print("Dataframes antes de concatenar   : mes_en_curso2_1  ", mes_en_curso2_1.shape)

df_ytd_proyectado = pd.concat([df_ytd_cerrado, mes_en_curso2_1], axis=0)
print("Dataframes después de concatenar : df_ytd_proyectado  ", df_ytd_proyectado.shape)

df_ytd_proyectado.sample(5)

Dataframes antes de concatenar   : df_ytd_cerrado    (340, 5)
Dataframes antes de concatenar   : mes_en_curso2_1   (873, 5)
Dataframes después de concatenar : df_ytd_proyectado   (1213, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
1132,P-1726,3. COSTO MANO DE OBRA,0,NaN,NaN
712,P-1658,3. COSTO MANO DE OBRA,0,VIVIPRA,Vicherat y Pradenas Ltda.
859,P-1684,3. COSTO MANO DE OBRA,0,Edificio Goycolea,EBCO S.A.
690,P-1649,4. COSTO SUPERVISIÓN,0,Inacap Vitacura E2,CONSTRUCTORA CUEVAS Y PURCELL S.A.
618,P-1620,2. COSTO MATERIALES,0,"OF Apoquindo- Habilitaciones, Integracion y ad...",CONSTRUCTORA INARCO S.A.


In [299]:
df_ytd_proyectado["CODIGO_PROYECTO"].unique()

array(['1. TOTAL', 'P-1557', 'P-1591', 'P-1592', 'P-1603', 'P-1611',
       'P-1612', 'P-1620', 'P-1626', 'P-1632', 'P-1636', 'P-1639',
       'P-1641', 'P-1642', 'P-1643', 'P-1644', 'P-1645', 'P-1648',
       'P-1649', 'P-1653', 'P-1656', 'P-1658', 'P-1660', 'P-1661',
       'P-1663', 'P-1664', 'P-1666', 'P-1667', 'P-1668', 'P-1669',
       'P-1670', 'P-1672', 'P-1673', 'P-1674', 'P-1675', 'P-1676',
       'P-1677', 'P-1679', 'P-1680', 'P-1681', 'P-1682', 'P-1683',
       'P-1684', 'P-1685', 'P-1686', 'P-1687', 'P-1688', 'P-1689',
       'P-1690', 'P-1691', 'P-1692', 'P-1693', 'P-1694', 'P-1695',
       'P-1696', 'P-1697', 'P-1698', 'P-1699', 'P-1700', 'P-1701',
       'P-1703', 'P-1704', 'P-1705', 'P-1706', 'P-1707', 'P-1708',
       'P-1709', 'P-1712', 'P-1713', 'P-1714', 'P-1715', 'P-1716',
       'P-1717', 'P-1718', 'P-1719', 'P-1720', 'P-1721', 'P-1722',
       'P-1723', 'P-1724', 'P-1725', 'P-1726', 'P-1727', 'P-1728',
       'P-1729', 'P-1600', 'P-1601', 'P-1615', 'P-1621', 'P-

In [300]:
# TESTING VISUAL 1 

df_ytd_proyectado[df_ytd_proyectado["CODIGO_PROYECTO"] == "P-1727"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
328,P-1727,3. COSTO MANO DE OBRA,-8169893,NaN,NaN
329,P-1727,2. COSTO MATERIALES,-13655419,NaN,NaN
330,P-1727,4. COSTO SUPERVISIÓN,0,NaN,NaN
331,P-1727,1. INGRESO,28570952,NaN,NaN
77,P-1727,3. COSTO MANO DE OBRA,-3455213,NaN,NaN
408,P-1727,3. COSTO MANO DE OBRA,0,NaN,NaN
409,P-1727,4. COSTO SUPERVISIÓN,0,NaN,NaN
493,P-1727,1. INGRESO,26500000,NaN,NaN
566,P-1727,2. COSTO MATERIALES,-10000000,NaN,NaN
1137,P-1727,3. COSTO MANO DE OBRA,0,NaN,NaN


In [301]:
# TESTING VISUAL 2

df_ytd_proyectado[df_ytd_proyectado["CODIGO_PROYECTO"] == "1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
0,1. TOTAL,3. COSTO MANO DE OBRA,-478147965,1. TOTAL,1. TOTAL
1,1. TOTAL,2. COSTO MATERIALES,-605032841,1. TOTAL,1. TOTAL
2,1. TOTAL,4. COSTO SUPERVISIÓN,-22566666,1. TOTAL,1. TOTAL
3,1. TOTAL,1. INGRESO,1579362409,1. TOTAL,1. TOTAL
80,1. TOTAL,3. COSTO MANO DE OBRA,-114373715,1. TOTAL,1. TOTAL
83,1. TOTAL,4. COSTO SUPERVISIÓN,-4643261,1. TOTAL,1. TOTAL
420,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL
421,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL
496,1. TOTAL,1. INGRESO,423800000,1. TOTAL,1. TOTAL
569,1. TOTAL,2. COSTO MATERIALES,-176535806,1. TOTAL,1. TOTAL


#### 4.5.3 - AGREGACIÓN COLUMNA MARGEN

In [302]:
#CREACIÓN DE MARGENES

df_ytd_proyectado2 = df_ytd_proyectado.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_ytd_proyectado2.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)


print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_real4 : ", df_ytd_proyectado2.shape)

df_ytd_proyectado2 = pd.merge(df_ytd_proyectado2, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ytd_proyectado2 : ", df_ytd_proyectado2.shape)
df_ytd_proyectado2.sample(10)


DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_real4 :  (1213, 5)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ytd_proyectado2 :  (1213, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE,MARGEN
683,P-1722,2. COSTO MATERIALES,0,Eco Cerrillos,Constructora ICF SPA,0.00
216,P-1696,3. COSTO MANO DE OBRA,-15747680,Santa Victoria - Terracorp,Constructora Terracorp S.A.,0.02
510,P-1713,3. COSTO MANO DE OBRA,0,Reserva San Francisco,EMPRESA NACIONAL DE CONSTRUCCIÓN LAS VERTIENTE...,0.36
973,P-1699,4. COSTO SUPERVISIÓN,0,Villa Panamericana Lote B,Empresa Constructora DLP S.A.,0.47
388,P-1620,3. COSTO MANO DE OBRA,0,"OF Apoquindo- Habilitaciones, Integracion y ad...",CONSTRUCTORA INARCO S.A.,0.00
290,P-1717,4. COSTO SUPERVISIÓN,0,Eco Quilín I,Constructora ICF SPA,0.80
688,P-1727,2. COSTO MATERIALES,-10000000,NaN,NaN,0.36
640,P-1674,2. COSTO MATERIALES,0,Oficinas Beiersdorf Chorrillos,Constructora Suksa S.A.,0.00
272,P-1713,3. COSTO MANO DE OBRA,-4283233,Reserva San Francisco,EMPRESA NACIONAL DE CONSTRUCCIÓN LAS VERTIENTE...,0.36
521,P-1718,4. COSTO SUPERVISIÓN,0,Patio Comidas,EBCO S.A.,0.55


In [303]:
# TESTING VISUAL 1 

df_ytd_proyectado2[df_ytd_proyectado2["CODIGO_PROYECTO"] == "P-1727"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE,MARGEN
328,P-1727,3. COSTO MANO DE OBRA,-8169893,NaN,NaN,0.36
329,P-1727,2. COSTO MATERIALES,-13655419,NaN,NaN,0.36
330,P-1727,4. COSTO SUPERVISIÓN,0,NaN,NaN,0.36
331,P-1727,1. INGRESO,28570952,NaN,NaN,0.36
373,P-1727,3. COSTO MANO DE OBRA,-3455213,NaN,NaN,0.36
538,P-1727,3. COSTO MANO DE OBRA,0,NaN,NaN,0.36
539,P-1727,4. COSTO SUPERVISIÓN,0,NaN,NaN,0.36
615,P-1727,1. INGRESO,26500000,NaN,NaN,0.36
688,P-1727,2. COSTO MATERIALES,-10000000,NaN,NaN,0.36
1097,P-1727,3. COSTO MANO DE OBRA,0,NaN,NaN,0.36


In [304]:
# TESTING VISUAL 2

df_ytd_proyectado2[df_ytd_proyectado2["CODIGO_PROYECTO"] == "1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE,MARGEN
0,1. TOTAL,3. COSTO MANO DE OBRA,-478147965,1. TOTAL,1. TOTAL,0.30
1,1. TOTAL,2. COSTO MATERIALES,-605032841,1. TOTAL,1. TOTAL,0.30
2,1. TOTAL,4. COSTO SUPERVISIÓN,-22566666,1. TOTAL,1. TOTAL,0.30
3,1. TOTAL,1. INGRESO,1579362409,1. TOTAL,1. TOTAL,0.30
374,1. TOTAL,3. COSTO MANO DE OBRA,-114373715,1. TOTAL,1. TOTAL,0.30
375,1. TOTAL,4. COSTO SUPERVISIÓN,-4643261,1. TOTAL,1. TOTAL,0.30
544,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,0.30
545,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,0.30
618,1. TOTAL,1. INGRESO,423800000,1. TOTAL,1. TOTAL,0.30
691,1. TOTAL,2. COSTO MATERIALES,-176535806,1. TOTAL,1. TOTAL,0.30


In [305]:
# Respaldo de Dataframe mes en curso real

df_ytd_proyectado2.to_csv('archivos_respaldo/df_ytd_proyectado2.csv', mode='a')

### 4.6 - **MERGE PARA -> YTD PROYECTADO (PERSPECTIVA MES ANTERIOR)
YTD Proyectado = YTD + el mes en curso del proyectado

#### 4.6.1 - EXPLORACIÓN Y PREPARACIÓN DE DATAFRAMES


In [306]:
# DATAFRAME PROVENIENTE DEL LIBRO MAYOR 2023 HASTA EL MES CERRADO.

print(df_ytd_cerrado_2M.shape)
df_ytd_cerrado_2M.sample(10)

(340, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
95,P-1661,1. INGRESO,0,San Vicente de Paul,DANACORP S.A
263,P-1708,1. INGRESO,0,Eco España,Constructora ICF SPA
150,P-1679,4. COSTO SUPERVISIÓN,0,Juan Sabaj,Constructora Grevia
86,P-1658,4. COSTO SUPERVISIÓN,0,VIVIPRA,Vicherat y Pradenas Ltda.
214,P-1695,4. COSTO SUPERVISIÓN,0,Edificio Estoril,EMPRESA CONSTRUCTORA BRAVO E IZQUIERDO
317,P-1724,2. COSTO MATERIALES,-7634256,NaN,NaN
23,P-1611,1. INGRESO,0,Edificio Plaza Egaña,EBCO S.A.
173,P-1685,2. COSTO MATERIALES,88900,Mapa Edificios No Operacionales,Claro Vicuña Valenzuela S.A.
89,P-1660,2. COSTO MATERIALES,158753,Coronel Alvarado,Larrain Prieto Risopatron S.A.
167,P-1683,1. INGRESO,0,CD BP-A05,EBCO S.A.


In [307]:
# DATAFRAME PROVENIENTE DEL LIBRO PERO SOO CONSIDERA MES EN CURSO.

print(mes_en_curso_2M.shape)
mes_en_curso_2M.head(5)

# Se filtra por la columna DESTINO != REAL (para considerar solo lo proyectado)#
mes_en_curso_proy_2M_2_1 = mes_en_curso_2M[mes_en_curso_2M['DESTINO'] != 'REAL']

# Eliminar las columnas ORIGEN y DESTINO
mes_en_curso_proy_2M_2_1 = mes_en_curso_proy_2M_2_1.drop(columns=['ORIGEN', 'DESTINO','MES','AÑO','FECHA'])

mes_en_curso_proy_2M_2_1.sample(5)

(1395, 10)


,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,TOTAL
87,P-1716,CD Astara,Astara Logística Chile SpA.,3. COSTO MANO DE OBRA,-895428.00
1045,P-1706,California,CONSTRUCTORA E INVERSIONES VITAL SPA.,4. COSTO SUPERVISIÓN,0.00
128,P-1603,Farellones,Empresa Constructora DLP S.A.,3. COSTO MANO DE OBRA,0.00
667,P-1632,Chorrillos Clorox,Constructora Suksa S.A.,4. COSTO SUPERVISIÓN,0.00
1174,P-1726,NaN,NaN,1. INGRESO,0.00


In [308]:
# TESTING VISUAL 1 

mes_en_curso_proy_2M_2_1[mes_en_curso_proy_2M_2_1["CODIGO_PROYECTO"] == "1. TOTAL"]

,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,TOTAL
112,1. TOTAL,1. TOTAL,1. TOTAL,3. COSTO MANO DE OBRA,-111885468.00
115,1. TOTAL,1. TOTAL,1. TOTAL,4. COSTO SUPERVISIÓN,-5096121.00
452,1. TOTAL,1. TOTAL,1. TOTAL,3. COSTO MANO DE OBRA,0.00
453,1. TOTAL,1. TOTAL,1. TOTAL,4. COSTO SUPERVISIÓN,0.00
529,1. TOTAL,1. TOTAL,1. TOTAL,1. INGRESO,461955807.36
603,1. TOTAL,1. TOTAL,1. TOTAL,2. COSTO MATERIALES,-212300000.00
1198,1. TOTAL,1. TOTAL,1. TOTAL,3. COSTO MANO DE OBRA,0.00
1199,1. TOTAL,1. TOTAL,1. TOTAL,4. COSTO SUPERVISIÓN,0.00
1200,1. TOTAL,1. TOTAL,1. TOTAL,3. COSTO MANO DE OBRA,0.00
1202,1. TOTAL,1. TOTAL,1. TOTAL,1. INGRESO,0.00


#### 4.6.2 - Concatenación de Dataframes

In [309]:
# Se concatenan los dataframes.

print("Dataframes antes de concatenar   : df_ytd_cerrado_2M       :", df_ytd_cerrado_2M.shape)
print("Dataframes antes de concatenar   : mes_en_curso_proy_2M_2_1:", mes_en_curso_proy_2M_2_1.shape)

df_ytd_proyectado_2M = pd.concat([df_ytd_cerrado_2M, mes_en_curso_proy_2M_2_1], axis=0)
print("Dataframes después de concatenar : df_ytd_proyectado_2M  ", df_ytd_proyectado_2M.shape)

df_ytd_proyectado_2M.sample(5)

Dataframes antes de concatenar   : df_ytd_cerrado_2M       : (340, 5)
Dataframes antes de concatenar   : mes_en_curso_proy_2M_2_1: (878, 5)
Dataframes después de concatenar : df_ytd_proyectado_2M   (1218, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
29,P-1620,2. COSTO MATERIALES,-16400.00,"OF Apoquindo- Habilitaciones, Integracion y ad...",CONSTRUCTORA INARCO S.A.
892,P-1683,3. COSTO MANO DE OBRA,0.00,CD BP-A05,EBCO S.A.
341,P-1699,4. COSTO SUPERVISIÓN,0.00,Villa Panamericana Lote B,Empresa Constructora DLP S.A.
397,P-1716,4. COSTO SUPERVISIÓN,0.00,CD Astara,Astara Logística Chile SpA.
30,P-1688,3. COSTO MANO DE OBRA,-2009744.00,Suiza,Moller y Perez - Cotapos Construcciones Indust...


In [310]:
df_ytd_proyectado_2M["CODIGO_PROYECTO"].unique()

array(['1. TOTAL', 'P-1557', 'P-1591', 'P-1592', 'P-1603', 'P-1611',
       'P-1612', 'P-1620', 'P-1626', 'P-1632', 'P-1636', 'P-1639',
       'P-1641', 'P-1642', 'P-1643', 'P-1644', 'P-1645', 'P-1648',
       'P-1649', 'P-1653', 'P-1656', 'P-1658', 'P-1660', 'P-1661',
       'P-1663', 'P-1664', 'P-1666', 'P-1667', 'P-1668', 'P-1669',
       'P-1670', 'P-1672', 'P-1673', 'P-1674', 'P-1675', 'P-1676',
       'P-1677', 'P-1679', 'P-1680', 'P-1681', 'P-1682', 'P-1683',
       'P-1684', 'P-1685', 'P-1686', 'P-1687', 'P-1688', 'P-1689',
       'P-1690', 'P-1691', 'P-1692', 'P-1693', 'P-1694', 'P-1695',
       'P-1696', 'P-1697', 'P-1698', 'P-1699', 'P-1700', 'P-1701',
       'P-1703', 'P-1704', 'P-1705', 'P-1706', 'P-1707', 'P-1708',
       'P-1709', 'P-1712', 'P-1713', 'P-1714', 'P-1715', 'P-1716',
       'P-1717', 'P-1718', 'P-1719', 'P-1720', 'P-1721', 'P-1722',
       'P-1723', 'P-1724', 'P-1725', 'P-1726', 'P-1727', 'P-1728',
       'P-1729', 'P-1600', 'P-1601', 'P-1615', 'P-1621', 'P-

In [311]:
# TESTING VISUAL 1 

df_ytd_proyectado_2M[df_ytd_proyectado_2M["CODIGO_PROYECTO"] == "P-1727"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
328,P-1727,3. COSTO MANO DE OBRA,-4714680.00,NaN,NaN
329,P-1727,2. COSTO MATERIALES,-3753865.00,NaN,NaN
330,P-1727,4. COSTO SUPERVISIÓN,0.00,NaN,NaN
331,P-1727,1. INGRESO,11005000.00,NaN,NaN
108,P-1727,3. COSTO MANO DE OBRA,-3205300.00,NaN,NaN
440,P-1727,3. COSTO MANO DE OBRA,0.00,NaN,NaN
441,P-1727,4. COSTO SUPERVISIÓN,0.00,NaN,NaN
527,P-1727,1. INGRESO,17255460.00,NaN,NaN
601,P-1727,2. COSTO MATERIALES,-9500000.00,NaN,NaN
1177,P-1727,3. COSTO MANO DE OBRA,0.00,NaN,NaN


In [312]:
# TESTING VISUAL 2

df_ytd_proyectado_2M[df_ytd_proyectado_2M["CODIGO_PROYECTO"] == "1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
0,1. TOTAL,3. COSTO MANO DE OBRA,-363774250.00,1. TOTAL,1. TOTAL
1,1. TOTAL,2. COSTO MATERIALES,-435497453.00,1. TOTAL,1. TOTAL
2,1. TOTAL,4. COSTO SUPERVISIÓN,-17923405.00,1. TOTAL,1. TOTAL
3,1. TOTAL,1. INGRESO,1118563896.00,1. TOTAL,1. TOTAL
112,1. TOTAL,3. COSTO MANO DE OBRA,-111885468.00,1. TOTAL,1. TOTAL
115,1. TOTAL,4. COSTO SUPERVISIÓN,-5096121.00,1. TOTAL,1. TOTAL
452,1. TOTAL,3. COSTO MANO DE OBRA,0.00,1. TOTAL,1. TOTAL
453,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL
529,1. TOTAL,1. INGRESO,461955807.36,1. TOTAL,1. TOTAL
603,1. TOTAL,2. COSTO MATERIALES,-212300000.00,1. TOTAL,1. TOTAL


#### 4.6.3 - AGREGACIÓN COLUMNA MARGEN

In [313]:
#CREACIÓN DE MARGENES

df_ytd_proyectado_2M_2 = df_ytd_proyectado_2M.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_ytd_proyectado_2M_2.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)


print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_ytd_proyectado_2M_2 : ", df_ytd_proyectado_2M_2.shape)

df_ytd_proyectado_2M_2 = pd.merge(df_ytd_proyectado_2M_2, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ytd_proyectado_2M_2 : ", df_ytd_proyectado_2M_2.shape)
df_ytd_proyectado_2M_2.sample(10)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_ytd_proyectado_2M_2 :  (1218, 5)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ytd_proyectado_2M_2 :  (1218, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE,MARGEN
782,P-1648,2. COSTO MATERIALES,0.00,Etapa 2 CD Peralillo TW,Inmobiliaria y Rentas Logística S.A.,0.00
511,P-1713,3. COSTO MANO DE OBRA,0.00,Reserva San Francisco,EMPRESA NACIONAL DE CONSTRUCCIÓN LAS VERTIENTE...,0.02
989,P-1701,4. COSTO SUPERVISIÓN,0.00,Edificio Zenteno,CONSTRUCTORA GREVIA SpA.,-0.56
834,P-1668,4. COSTO SUPERVISIÓN,0.00,Quillayes Plaza,E.MOLINA MOREL CONSTRUCTORA S.A.,0.00
1116,P-1729,1. INGRESO,0.00,NaN,NaN,0.00
364,P-1713,3. COSTO MANO DE OBRA,-172081.00,Reserva San Francisco,EMPRESA NACIONAL DE CONSTRUCCIÓN LAS VERTIENTE...,0.02
299,P-1719,1. INGRESO,131348.00,CD METALPAR,EBCO S.A.,1.00
1216,P-1724,2. COSTO MATERIALES,0.00,NaN,NaN,0.72
658,P-1691,2. COSTO MATERIALES,0.00,Rosario Rosales,CONSTRUCTORA E INMOBILIARIA PRADOVERDE LTDA.,0.32
1082,P-1722,2. COSTO MATERIALES,0.00,Eco Cerrillos,Constructora ICF SPA,0.00


In [314]:
# TESTING VISUAL 1 

df_ytd_proyectado_2M_2[df_ytd_proyectado_2M_2["CODIGO_PROYECTO"] == "P-1727"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE,MARGEN
328,P-1727,3. COSTO MANO DE OBRA,-4714680.00,NaN,NaN,0.25
329,P-1727,2. COSTO MATERIALES,-3753865.00,NaN,NaN,0.25
330,P-1727,4. COSTO SUPERVISIÓN,0.00,NaN,NaN,0.25
331,P-1727,1. INGRESO,11005000.00,NaN,NaN,0.25
373,P-1727,3. COSTO MANO DE OBRA,-3205300.00,NaN,NaN,0.25
539,P-1727,3. COSTO MANO DE OBRA,0.00,NaN,NaN,0.25
540,P-1727,4. COSTO SUPERVISIÓN,0.00,NaN,NaN,0.25
618,P-1727,1. INGRESO,17255460.00,NaN,NaN,0.25
692,P-1727,2. COSTO MATERIALES,-9500000.00,NaN,NaN,0.25
1103,P-1727,3. COSTO MANO DE OBRA,0.00,NaN,NaN,0.25


In [315]:
# Respaldo de Dataframe mes en curso real

df_ytd_proyectado_2M_2.to_csv('archivos_respaldo/df_ytd_proyectado_2M_2.csv', mode='a')

### 4.7 - **MERGE PARA -> PTD CERRADO

Cruce entre YTD_Cerrado + Libros mayores de años anteriores.

#### 4.7.1 - EXPLORACIÓN Y PREPARACIÓN DE DATAFRAMES

In [316]:
# Exploración Dataframe para el dataframe que proviene del Libro Mayor 2023.
# Dataframe : df_ytd_cerrado2 proveniente del punto 4.2 - MERGE PARA -> YTD AÑO EN CURSO HASTA MES CERRADO

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ytd_cerrado2 : ", df_ytd_cerrado2.shape)

df_ytd_cerrado_1 = df_ytd_cerrado2.copy()
df_ytd_cerrado_1.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ytd_cerrado2 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
174,P-1685,4. COSTO SUPERVISIÓN,0
214,P-1695,4. COSTO SUPERVISIÓN,0
83,P-1656,1. INGRESO,0
34,P-1626,4. COSTO SUPERVISIÓN,0
294,P-1718,4. COSTO SUPERVISIÓN,0


In [317]:
# TESTING VISUAL 1

print(df_ytd_cerrado_1[df_ytd_cerrado_1["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_ytd_cerrado_1[df_ytd_cerrado_1["CODIGO_PROYECTO"] == "1. TOTAL"]

473614937


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
0,1. TOTAL,3. COSTO MANO DE OBRA,-478147965
1,1. TOTAL,2. COSTO MATERIALES,-605032841
2,1. TOTAL,4. COSTO SUPERVISIÓN,-22566666
3,1. TOTAL,1. INGRESO,1579362409


In [318]:
# Se crea Pivote para reducir las dimensiones

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ytd_cerrado_1 : ", df_ytd_cerrado_1.shape)
df_ytd_cerrado_2 = pd.pivot_table(df_ytd_cerrado_1, values=['TOTAL'], index=['CODIGO_PROYECTO','TIPO_CUENTA'], aggfunc='sum')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ytd_cerrado_2 : ", df_ytd_cerrado_2.shape)

# Se renombra campo "TOTAL"
df_ytd_cerrado_2 = df_ytd_cerrado_2.rename(columns={"TOTAL": "TOTAL YTD"})

df_ytd_cerrado_2 = pd.DataFrame(df_ytd_cerrado_2.to_records())
df_ytd_cerrado_2.sample(10)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ytd_cerrado_1 :  (340, 3)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ytd_cerrado_2 :  (340, 1)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL YTD
93,P-1661,2. COSTO MATERIALES,3200
34,P-1626,3. COSTO MANO DE OBRA,0
305,P-1721,2. COSTO MATERIALES,-21492393
104,P-1666,1. INGRESO,0
276,P-1714,1. INGRESO,20583182
132,P-1674,1. INGRESO,0
335,P-1728,4. COSTO SUPERVISIÓN,0
157,P-1681,2. COSTO MATERIALES,0
82,P-1656,3. COSTO MANO DE OBRA,0
282,P-1715,3. COSTO MANO DE OBRA,-16935839


In [319]:
# Exploración y tratamiento de dataframe: df_ytd_cerrado df_libro_mayor_old3

# Se crea una copia del dataframe 
print("Dimensiones del dataframe df_libro_mayor_old3 : ", df_libro_mayor_old3.shape)
df_libro_mayor_old4 = df_libro_mayor_old3.copy()
print("Dimensiones del dataframe df_libro_mayor_old4 : ", df_libro_mayor_old4.shape)
df_libro_mayor_old4.sample(10)

Dimensiones del dataframe df_libro_mayor_old3 :  (1741, 9)
Dimensiones del dataframe df_libro_mayor_old4 :  (1741, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
1468,P-1700,2. COSTO MATERIALES,2022-10-01,2022.00,10.00,0,3558488,-3558488,0.00
1585,P-1713,2. COSTO MATERIALES,2022-10-01,2022.00,10.00,0,179224,-179224,0.00
1283,P-1690,3. COSTO MANO DE OBRA,2022-04-01,2022.00,4.00,0,380485,-380485,0.00
1617,S-1601,2. COSTO MATERIALES,2021-02-01,2021.00,2.00,0,2267992,-2267992,0.00
1619,S-1601,2. COSTO MATERIALES,2021-04-01,2021.00,4.00,47200,970,46230,0.00
1660,1. TOTAL,2. COSTO MATERIALES,2021-05-01,2021.00,5.00,801384,25433594,-24632210,0.00
1116,P-1682,2. COSTO MATERIALES,2022-11-01,2022.00,11.00,0,3748163,-3748163,0.00
82,P-1615,2. COSTO MATERIALES,2021-04-01,2021.00,4.00,0,2224432,-2224432,0.00
180,P-1637,2. COSTO MATERIALES,2021-06-01,2021.00,6.00,27395,0,27395,0.00
1496,P-1701,3. COSTO MANO DE OBRA,2022-11-01,2022.00,11.00,0,5795208,-5795208,0.00


In [320]:
# Se realiza limpieza del dataframe para obtener solo las columnas utiles.

df_libro_mayor_old4 = df_libro_mayor_old4[df_libro_mayor_old4['CODIGO_PROYECTO'] != '']
df_libro_mayor_old4 = df_libro_mayor_old4[df_libro_mayor_old4['TIPO_CUENTA'] != 'OTRA']
df_libro_mayor_old4 = df_libro_mayor_old4[df_libro_mayor_old4['TIPO_CUENTA'] != '']
df_libro_mayor_old4 = df_libro_mayor_old4.dropna(subset=['TIPO_CUENTA'])
df_libro_mayor_old4 = df_libro_mayor_old4.dropna(subset=['CODIGO_PROYECTO'])

print("Dimensiones del dataframe df_libro_mayor_old4 : ", df_libro_mayor_old4.shape)
df_libro_mayor_old4.sample(10)

Dimensiones del dataframe df_libro_mayor_old4 :  (1680, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
1118,P-1682,3. COSTO MANO DE OBRA,2022-08-01,2022.00,8.00,0,328164,-328164,0.00
621,P-1664,2. COSTO MATERIALES,2022-05-01,2022.00,5.00,53898,0,53898,0.00
1600,P-1714,3. COSTO MANO DE OBRA,2022-10-01,2022.00,10.00,0,2047076,-2047076,0.00
1543,P-1704,3. COSTO MANO DE OBRA,2022-12-01,2022.00,12.00,0,12959513,-12959513,0.00
61,P-1611,2. COSTO MATERIALES,2021-08-01,2021.00,8.00,0,4459,-4459,0.00
1606,P-1716,2. COSTO MATERIALES,2022-12-01,2022.00,12.00,0,4855555,-4855555,0.00
846,P-1671,2. COSTO MATERIALES,2022-02-01,2022.00,2.00,815230,0,815230,0.00
888,P-1672,3. COSTO MANO DE OBRA,2021-05-01,2021.00,5.00,0,0,0,0.00
115,P-1623,2. COSTO MATERIALES,2021-10-01,2021.00,10.00,0,21192,-21192,0.00
1181,P-1685,2. COSTO MATERIALES,2022-11-01,2022.00,11.00,0,41347,-41347,0.00


In [321]:
# TESTING VISUAL 1

print(df_libro_mayor_old4[df_libro_mayor_old4["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_libro_mayor_old4[df_libro_mayor_old4["CODIGO_PROYECTO"] == "1. TOTAL"].head(5)

1369937537


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
1632,1. TOTAL,1. INGRESO,2021-01-01,2021.00,1.00,153082726,51479131,101603595,0.00
1633,1. TOTAL,1. INGRESO,2021-02-01,2021.00,2.00,202772310,71750206,131022104,0.00
1634,1. TOTAL,1. INGRESO,2021-03-01,2021.00,3.00,227194679,39876627,187318052,0.00
1635,1. TOTAL,1. INGRESO,2021-04-01,2021.00,4.00,165417555,36206601,129210954,0.00
1636,1. TOTAL,1. INGRESO,2021-05-01,2021.00,5.00,113310222,10911630,102398592,0.00


In [322]:
# Se crea Pivote para reducir las dimensiones

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_libro_mayor_old4 : ", df_libro_mayor_old4.shape)
df_libro_mayor_old5 = pd.pivot_table(df_libro_mayor_old4, values=['TOTAL'], index=['CODIGO_PROYECTO','TIPO_CUENTA'], aggfunc='sum')

# Se renombra campo "TOTAL"
df_libro_mayor_old5 = df_libro_mayor_old5.rename(columns={"TOTAL": "TOTAL LB OLD"})

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor_old5 : ", df_libro_mayor_old5.shape)

df_libro_mayor_old5 = pd.DataFrame(df_libro_mayor_old5.to_records())
df_libro_mayor_old5.sample(10)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_libro_mayor_old4 :  (1680, 9)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor_old5 :  (253, 1)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL LB OLD
9,P-1611,1. INGRESO,4173520
179,P-1690,3. COSTO MANO DE OBRA,-16380832
44,P-1638,2. COSTO MATERIALES,-4861326
233,P-1712,2. COSTO MATERIALES,-26612966
162,P-1685,1. INGRESO,48965677
81,P-1655,2. COSTO MATERIALES,120177
56,P-1643,2. COSTO MATERIALES,-102685
93,P-1660,2. COSTO MATERIALES,-12219084
107,P-1666,2. COSTO MATERIALES,-25105908
79,P-1654,2. COSTO MATERIALES,-1142947


In [323]:
# TESTING VISUAL 1

print(df_libro_mayor_old5[df_libro_mayor_old5["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL LB OLD"].sum())
df_libro_mayor_old5[df_libro_mayor_old5["CODIGO_PROYECTO"] == "1. TOTAL"]

1369937537


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL LB OLD
0,1. TOTAL,1. INGRESO,3185711545
1,1. TOTAL,2. COSTO MATERIALES,-1152612025
2,1. TOTAL,3. COSTO MANO DE OBRA,-611212228
3,1. TOTAL,4. COSTO SUPERVISIÓN,-51949755


In [324]:
# TESTING VISUAL 2

print(df_libro_mayor_old5[df_libro_mayor_old5["CODIGO_PROYECTO"] == "P-1615"]["TOTAL LB OLD"].sum())
df_libro_mayor_old5[df_libro_mayor_old5["CODIGO_PROYECTO"] == "P-1615"].head(5)

31342023


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL LB OLD
14,P-1615,1. INGRESO,56248814
15,P-1615,2. COSTO MATERIALES,-23003962
16,P-1615,3. COSTO MANO DE OBRA,-1902829


#### 4.7.2 - Concatenación de Dataframes

In [325]:
# Se considera df_libro_mayor5_base2 como base para construir el dataframe

df_ptd_cerrado = df_libro_mayor5_base2.copy()

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado : ", df_ptd_cerrado.shape)
df_ptd_cerrado.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado :  (340, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
291,P-1693,4. COSTO SUPERVISIÓN,0,VOLTA V0800,ECOBIO SPA
230,P-1660,1. INGRESO,0,CORONEL ALVARADO,LARRAIN PRIETO RISOPATRON S.A.
60,P-1686,2. COSTO MATERIALES,0,MACUL ICALMA,EMPRESA CONSTRUCTORA INGEVEC S.A.
89,P-1697,3. COSTO MANO DE OBRA,0,ESTOCOLMO,CONSTRUCTORA FORTALEZA S.A
88,P-1696,1. INGRESO,0,SANTA VICTORIA - TERRACORP,CONSTRUCTORA TERRACORP S.A.


In [326]:
# MERGE 1 - df_ytd_cerrado2

# Se hace merge con los codigos de proyecto.

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_pendiente2 : ", df_ptd_cerrado.shape)

df_ptd_cerrado = pd.merge(df_ptd_cerrado, df_ytd_cerrado_2, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado : ", df_ptd_cerrado.shape)

df_ptd_cerrado.sample(10)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_pendiente2 :  (340, 5)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD
321,P-1720,4. COSTO SUPERVISIÓN,0,NaN,NaN,0
130,P-1714,3. COSTO MANO DE OBRA,0,DEPARTAMENTAL II,CONSTRUCTORA FUERZA SUR DOS SPA,-8257826
11,P-1641,2. COSTO MATERIALES,0,INACAP TEMUCO,DESARROLLOS CONSTRUCTIVOS AXIS SOCIEDAD ANONIMA,-369720
216,P-1648,3. COSTO MANO DE OBRA,0,ETAPA 2 CD PERALILLO TW,INMOBILIARIA Y RENTAS LOGÍSTICA S.A.,0
4,P-1611,2. COSTO MATERIALES,0,EDIFICIO PLAZA EGAÑA,EBCO S.A.,6648
270,P-1681,2. COSTO MATERIALES,0,CD ROSEN (CCTV PROVISORIO),COLCHONES ROSEN SAIC,0
81,P-1694,2. COSTO MATERIALES,0,ECO HUEMUL,CONSTRUCTORA ICF SPA,-22032871
134,P-1715,2. COSTO MATERIALES,0,CD LO ECHEVERS,QUEZADA Y BOETSCH,-27886461
218,P-1649,1. INGRESO,0,INACAP VITACURA E2,CONSTRUCTORA CUEVAS Y PURCELL S.A.,0
188,P-1620,4. COSTO SUPERVISIÓN,0,"OF APOQUINDO- HABILITACIONES, INTEGRACION Y AD...",CONSTRUCTORA INARCO S.A.,0


In [327]:
# MERGE 2 - df_ytd_cerrado2

# Se hace merge con los codigos de proyecto.

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado : ", df_ptd_cerrado.shape)

df_ptd_cerrado = pd.merge(df_ptd_cerrado, df_libro_mayor_old5, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado : ", df_ptd_cerrado.shape)

df_ptd_cerrado.sample(10)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado :  (340, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado :  (340, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD
38,P-1675,2. COSTO MATERIALES,0,PALENA II,CONSTRUCTORA MYA SPA.,0,-7984413.00
90,P-1697,2. COSTO MATERIALES,0,ESTOCOLMO,CONSTRUCTORA FORTALEZA S.A,-17660281,-10818529.00
79,P-1693,2. COSTO MATERIALES,0,VOLTA V0800,ECOBIO SPA,103024,-89715133.00
29,P-1668,2. COSTO MATERIALES,0,QUILLAYES PLAZA,E.MOLINA MOREL CONSTRUCTORA S.A.,146500,-14171213.00
204,P-1642,1. INGRESO,0,CORINTO,EMP. CONSTRUCTORA INGENIEROS S.A.,0,7163026.00
275,P-1683,3. COSTO MANO DE OBRA,0,CD BP-A05,EBCO S.A.,0,-287910.00
114,P-1706,1. INGRESO,0,CALIFORNIA,CONSTRUCTORA E INVERSIONES VITAL SPA.,36302208,0.00
288,P-1691,4. COSTO SUPERVISIÓN,0,ROSARIO ROSALES,CONSTRUCTORA E INMOBILIARIA PRADOVERDE LTDA.,0,NaN
84,P-1695,2. COSTO MATERIALES,0,EDIFICIO ESTORIL,EMPRESA CONSTRUCTORA BRAVO E IZQUIERDO,-18940946,-6706085.00
322,P-1721,4. COSTO SUPERVISIÓN,0,STRIP CENTER LUIS PASTEUR,INMOBILIARIA LUIS PASTEUR S.A.,0,NaN


In [328]:
# Se estandarizan columnas de Totales y se agrega columna TOTAL 

# Reemplazar NaN por cero en las columnas "TOTAL YTD" y "TOTAL LB OLD"
df_ptd_cerrado["TOTAL YTD"] = df_ptd_cerrado["TOTAL YTD"].fillna(0)
df_ptd_cerrado["TOTAL LB OLD"] = df_ptd_cerrado["TOTAL LB OLD"].fillna(0)

# Crear la nueva columna "TOTAL" que suma "TOTAL YTD" y "TOTAL LB OLD"
df_ptd_cerrado["TOTAL"] = df_ptd_cerrado["TOTAL YTD"] + df_ptd_cerrado["TOTAL LB OLD"]
df_ptd_cerrado.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
317,P-1719,2. COSTO MATERIALES,0,CD METALPAR,EBCO S.A.,0,0.00,0.00
228,P-1658,3. COSTO MANO DE OBRA,0,VIVIPRA,VICHERAT Y PRADENAS LTDA.,0,-319859.00,-319859.00
16,P-1645,1. INGRESO,0,LOS TRES ANTONIOS,SOCIEDAD CONSTRUCTORA ECHAVARRI HERMANOS LIMITADA,0,17327418.00,17327418.00
240,P-1664,3. COSTO MANO DE OBRA,0,MK II PCI,COMERCIAL K SPA.,0,0.00,0.00
214,P-1645,4. COSTO SUPERVISIÓN,0,LOS TRES ANTONIOS,SOCIEDAD CONSTRUCTORA ECHAVARRI HERMANOS LIMITADA,0,0.00,0.00


In [329]:
# TESTING VISUAL 1

print(df_ptd_cerrado[df_ptd_cerrado["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL YTD"].sum())
print(df_ptd_cerrado[df_ptd_cerrado["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL LB OLD"].sum())
print(df_ptd_cerrado[df_ptd_cerrado["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())

df_ptd_cerrado[df_ptd_cerrado["CODIGO_PROYECTO"] == "1. TOTAL"]

473614937
1369937537.0
1843552474.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-22566666,-51949755.00,-74516421.00
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,1579362409,3185711545.00,4765073954.00
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-605032841,-1152612025.00,-1757644866.00
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-478147965,-611212228.00,-1089360193.00


In [330]:
# TESTING VISUAL 2

print(df_ptd_cerrado[df_ptd_cerrado["CODIGO_PROYECTO"] == "P-1727"]["TOTAL YTD"].sum())
print(df_ptd_cerrado[df_ptd_cerrado["CODIGO_PROYECTO"] == "P-1727"]["TOTAL LB OLD"].sum())
print(df_ptd_cerrado[df_ptd_cerrado["CODIGO_PROYECTO"] == "P-1727"]["TOTAL"].sum())

df_ptd_cerrado[df_ptd_cerrado["CODIGO_PROYECTO"] == "P-1727"]

6745640
0.0
6745640.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
162,P-1727,3. COSTO MANO DE OBRA,0,NaN,NaN,-8169893,0.00,-8169893.00
163,P-1727,2. COSTO MATERIALES,0,NaN,NaN,-13655419,0.00,-13655419.00
164,P-1727,1. INGRESO,0,NaN,NaN,28570952,0.00,28570952.00
330,P-1727,4. COSTO SUPERVISIÓN,0,NaN,NaN,0,0.00,0.00


#### 4.7.3 - SE AGREGAN FILAS DE 1.TOTAL

In [331]:
# Se eliminan Totales, excepto cuando se trata de COSTO_SUPERVISIÓN

print("DIMENSIONES ANTES DE TRATAMIENTO DEL DATAFRAME: df_ptd_cerrado : ", df_ptd_cerrado.shape)

df_ptd_cerrado2 = df_ptd_cerrado[~((df_ptd_cerrado["CODIGO_PROYECTO"] == "1. TOTAL") & (df_ptd_cerrado["TIPO_CUENTA"] != "4. COSTO SUPERVISIÓN"))]

print("DIMENSIONES DESPUÉS DE TRATAMIENTO DEL DATAFRAME df_ptd_cerrado2 : ", df_ptd_cerrado2.shape)

DIMENSIONES ANTES DE TRATAMIENTO DEL DATAFRAME: df_ptd_cerrado :  (340, 8)
DIMENSIONES DESPUÉS DE TRATAMIENTO DEL DATAFRAME df_ptd_cerrado2 :  (337, 8)


In [332]:
# TESTING VISUAL 2


print(df_ptd_cerrado2[df_ptd_cerrado2["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL YTD"].sum())
print(df_ptd_cerrado2[df_ptd_cerrado2["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL LB OLD"].sum())
print(df_ptd_cerrado2[df_ptd_cerrado2["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())

df_ptd_cerrado2[df_ptd_cerrado2["CODIGO_PROYECTO"] == "1. TOTAL"]

-22566666
-51949755.0
-74516421.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-22566666,-51949755.00,-74516421.00


In [333]:
# Calcular la suma de los valores totales para cada tipo de cuenta

print("DIMENSIONES ANTES DE TRATAMIENTO DEL DATAFRAME: df_ptd_cerrado2 : ", df_ptd_cerrado2.shape)

df_total = df_ptd_cerrado2.groupby('TIPO_CUENTA')['TOTAL'].sum().reset_index()
df_total['CODIGO_PROYECTO'] = '1. TOTAL'

# Agregar las filas adicionales al DataFrame
df_ptd_cerrado2 = pd.concat([df_ptd_cerrado2, df_total], ignore_index=True)

# Modificar las columnas "NOMBRE_OBRA" y "NOMBRE_CLIENTE" para "1. TOTAL"
df_ptd_cerrado2.loc[df_ptd_cerrado2['CODIGO_PROYECTO'] == '1. TOTAL', ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'

print("DIMENSIONES DESPUÉS DE TRATAMIENTO DEL DATAFRAME df_ptd_cerrado2 : ", df_ptd_cerrado2.shape)

df_ptd_cerrado2.sample(5)  

DIMENSIONES ANTES DE TRATAMIENTO DEL DATAFRAME: df_ptd_cerrado2 :  (337, 8)
DIMENSIONES DESPUÉS DE TRATAMIENTO DEL DATAFRAME df_ptd_cerrado2 :  (341, 8)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
38,P-1675,2. COSTO MATERIALES,0.00,PALENA II,CONSTRUCTORA MYA SPA.,0.00,-7984413.00,-7984413.00
264,P-1676,1. INGRESO,0.00,LO CAMPINO,EBCO S.A.,0.00,149103751.00,149103751.00
61,P-1686,1. INGRESO,0.00,MACUL ICALMA,EMPRESA CONSTRUCTORA INGEVEC S.A.,31888582.00,67737824.00,99626406.00
132,P-1714,1. INGRESO,0.00,DEPARTAMENTAL II,CONSTRUCTORA FUERZA SUR DOS SPA,20583182.00,34349625.00,54932807.00
160,P-1726,2. COSTO MATERIALES,0.00,NaN,NaN,-23539593.00,0.00,-23539593.00


In [334]:
# TESTING VISUAL 1

print(df_ptd_cerrado2[df_ptd_cerrado2["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_ptd_cerrado2[df_ptd_cerrado2["CODIGO_PROYECTO"] == "1. TOTAL"].head(5)

1591529635.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-22566666.00,-51949755.00,-74516421.00
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,NaN,NaN,4503192575.00
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-1686220937.00
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-1076409161.00
340,1. TOTAL,4. COSTO SUPERVISIÓN,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-74516421.00


In [335]:
# Se eliminan COSTO SUPERVISION INCORRECTO

df_ptd_cerrado2 = df_ptd_cerrado2[~((df_ptd_cerrado2['CODIGO_PROYECTO'] == '1. TOTAL') & (df_ptd_cerrado2['TIPO_CUENTA'] == '4. COSTO SUPERVISIÓN') & (df_ptd_cerrado2['TOTAL_LB'].isna()))]
 
# TESTING VISUAL 2

print(df_ptd_cerrado2[df_ptd_cerrado2["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_ptd_cerrado2[df_ptd_cerrado2["CODIGO_PROYECTO"] == "1. TOTAL"].head(5)

1666046056.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-22566666.00,-51949755.00,-74516421.00
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,NaN,NaN,4503192575.00
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-1686220937.00
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-1076409161.00


In [336]:
# TESTING VISUAL 2

print(df_ptd_cerrado2[df_ptd_cerrado2["CODIGO_PROYECTO"] == "P-1727"]["TOTAL"].sum())
df_ptd_cerrado2[df_ptd_cerrado2["CODIGO_PROYECTO"] == "P-1727"].head(5)

6745640.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
162,P-1727,3. COSTO MANO DE OBRA,0.00,NaN,NaN,-8169893.00,0.00,-8169893.00
163,P-1727,2. COSTO MATERIALES,0.00,NaN,NaN,-13655419.00,0.00,-13655419.00
164,P-1727,1. INGRESO,0.00,NaN,NaN,28570952.00,0.00,28570952.00
330,P-1727,4. COSTO SUPERVISIÓN,0.00,NaN,NaN,0.00,0.00,0.00


#### 4.7.4 - AGREGACIÓN COLUMNA MARGEN

In [337]:
#CREACIÓN DE MARGENES

df_ptd_cerrado3 = df_ptd_cerrado2.copy()


# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_ptd_cerrado3.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)


print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_ptd_cerrado3 : ", df_ptd_cerrado3.shape)

df_ptd_cerrado3 = pd.merge(df_ptd_cerrado3, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado3 : ", df_ptd_cerrado3.shape)
df_ptd_cerrado3.sample(5)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_ptd_cerrado3 :  (340, 8)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado3 :  (340, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL,MARGEN
302,P-1704,4. COSTO SUPERVISIÓN,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0.00,0.00,0.00,0.50
83,P-1695,3. COSTO MANO DE OBRA,0.00,EDIFICIO ESTORIL,EMPRESA CONSTRUCTORA BRAVO E IZQUIERDO,-5042385.00,-8121303.00,-13163688.00,0.11
153,P-1724,3. COSTO MANO DE OBRA,0.00,NaN,NaN,-506066.00,0.00,-506066.00,0.72
61,P-1686,1. INGRESO,0.00,MACUL ICALMA,EMPRESA CONSTRUCTORA INGEVEC S.A.,31888582.00,67737824.00,99626406.00,0.26
319,P-1719,4. COSTO SUPERVISIÓN,0.00,CD METALPAR,EBCO S.A.,0.00,0.00,0.00,1.00


In [338]:
# TESTING VISUAL 1

print(df_ptd_cerrado3[df_ptd_cerrado3["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_ptd_cerrado3[df_ptd_cerrado3["CODIGO_PROYECTO"] == "1. TOTAL"].head(5)

1666046056.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL,MARGEN
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-22566666.00,-51949755.00,-74516421.00,0.37
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,NaN,NaN,4503192575.00,0.37
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-1686220937.00,0.37
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-1076409161.00,0.37


In [339]:
# TESTING VISUAL 2

print(df_ptd_cerrado3[df_ptd_cerrado3["CODIGO_PROYECTO"] == "P-1727"]["TOTAL"].sum())
df_ptd_cerrado3[df_ptd_cerrado3["CODIGO_PROYECTO"] == "P-1727"].head(5)

6745640.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL,MARGEN
162,P-1727,3. COSTO MANO DE OBRA,0.00,NaN,NaN,-8169893.00,0.00,-8169893.00,0.24
163,P-1727,2. COSTO MATERIALES,0.00,NaN,NaN,-13655419.00,0.00,-13655419.00,0.24
164,P-1727,1. INGRESO,0.00,NaN,NaN,28570952.00,0.00,28570952.00,0.24
330,P-1727,4. COSTO SUPERVISIÓN,0.00,NaN,NaN,0.00,0.00,0.00,0.24


In [340]:
df_ptd_cerrado3.to_csv('archivos_respaldo/df_ptd_cerrado3.csv', index = False)

### 4.8 - **MERGE PARA -> PTD CERRADO (PERSPECTIVA MES ANTERIOR)
Cruce entre YTD_Cerrado + Libros mayores de años anteriores.

#### 4.8.1 - EXPLORACIÓN Y PREPARACIÓN DE DATAFRAMES

In [341]:
# Exploración Dataframe para el dataframe que proviene del Libro Mayor 2023.
# Dataframe : df_ytd_cerrado2 proveniente del punto 4.3 - MERGE PARA -> YTD AÑO EN CURSO HASTA MES CERRADO PERSPECTIVA MES ANTERIOR

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ytd_cerrado_2M_3 : ", df_ytd_cerrado_2M_3.shape)

df_ytd_cerrado_2M_3_1 = df_ytd_cerrado_2M_3.copy()
df_ytd_cerrado_2M_3_1.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ytd_cerrado_2M_3 :  (340, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
83,P-1695,3. COSTO MANO DE OBRA,0,EDIFICIO ESTORIL,EMPRESA CONSTRUCTORA BRAVO E IZQUIERDO,-5042385,0.03
167,P-1729,2. COSTO MATERIALES,0,NaN,NaN,0,0.00
92,P-1698,2. COSTO MATERIALES,0,CD ANDES,EBCO S.A.,924,1.00
96,P-1699,1. INGRESO,0,VILLA PANAMERICANA LOTE B,EMPRESA CONSTRUCTORA DLP S.A.,73159443,0.44
162,P-1727,3. COSTO MANO DE OBRA,0,NaN,NaN,-4714680,0.23


In [342]:
# TESTING VISUAL 1

print(df_ytd_cerrado_2M_3_1[df_ytd_cerrado_2M_3_1["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_ytd_cerrado_2M_3_1[df_ytd_cerrado_2M_3_1["CODIGO_PROYECTO"] == "1. TOTAL"]

301368788


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-17923405,0.27
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,1118563896,0.27
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-435497453,0.27
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-363774250,0.27


In [343]:
# Se crea Pivote para reducir las dimensiones

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ytd_cerrado_2M_3_1   : ", df_ytd_cerrado_2M_3_1.shape)
df_ytd_cerrado_2M_3_2 = pd.pivot_table(df_ytd_cerrado_2M_3_1, values=['TOTAL'], index=['CODIGO_PROYECTO','TIPO_CUENTA'], aggfunc='sum')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ytd_cerrado_2M_3_2 : ", df_ytd_cerrado_2M_3_2.shape)

# Se renombra campo "TOTAL"
df_ytd_cerrado_2M_3_2 = df_ytd_cerrado_2M_3_2.rename(columns={"TOTAL": "TOTAL YTD"})

df_ytd_cerrado_2M_3_2 = pd.DataFrame(df_ytd_cerrado_2M_3_2.to_records())
df_ytd_cerrado_2M_3_2.sample(10)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ytd_cerrado_2M_3_1   :  (340, 7)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ytd_cerrado_2M_3_2 :  (340, 1)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL YTD
189,P-1689,2. COSTO MATERIALES,26522
179,P-1686,4. COSTO SUPERVISIÓN,0
152,P-1680,1. INGRESO,63068231
162,P-1682,3. COSTO MANO DE OBRA,-9905519
335,P-1728,4. COSTO SUPERVISIÓN,0
37,P-1632,2. COSTO MATERIALES,0
6,P-1557,3. COSTO MANO DE OBRA,-945949
119,P-1669,4. COSTO SUPERVISIÓN,0
243,P-1703,4. COSTO SUPERVISIÓN,0
251,P-1705,4. COSTO SUPERVISIÓN,0


In [344]:
# Exploración y tratamiento de dataframe: df_ytd_cerrado df_libro_mayor_old3
# Se utiliza el dataframe creado previamente en el punto: 4.7.1 : df_libro_mayor_old5 (NO ES NECESARIO TRATAR NUEVAMENTE LOS DATOS DEL LIBRO MAYOR DE AÑOS ANTRIORES)

print("Dimensiones del dataframe df_libro_mayor_old5 : ", df_libro_mayor_old5.shape)
df_libro_mayor_old5.sample(10)

Dimensiones del dataframe df_libro_mayor_old5 :  (253, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL LB OLD
243,P-1716,2. COSTO MATERIALES,-4855555
127,P-1673,1. INGRESO,26810752
222,P-1705,1. INGRESO,1
206,P-1699,3. COSTO MANO DE OBRA,-1005045
240,P-1714,3. COSTO MANO DE OBRA,-8044526
187,P-1693,2. COSTO MATERIALES,-89715133
167,P-1686,3. COSTO MANO DE OBRA,-21817526
19,P-1617,2. COSTO MATERIALES,51814
103,P-1665,1. INGRESO,86633939
87,P-1658,1. INGRESO,30794481


#### 4.8.2 - Concatenación de Dataframes

In [345]:
# Se considera df_libro_mayor5_base2 como base para construir el dataframe

df_ptd_cerrado_anterior = df_libro_mayor5_base2.copy()

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior : ", df_ptd_cerrado_anterior.shape)
df_ptd_cerrado_anterior.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior :  (340, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
109,P-1705,3. COSTO MANO DE OBRA,0,LOS CLARINES,CONSTRUCTORA INARCO S.A.
70,P-1690,2. COSTO MATERIALES,0,ONGOLMO - HOME CONDOMINIO,CANTAUCO CONSTRUCCION LTDA.
98,P-1700,2. COSTO MATERIALES,0,CD ORIZON,EBCO S.A.
157,P-1725,2. COSTO MATERIALES,0,LO CAÑAS,CONSTRUCTORA BRISAS DE BATUCO
193,P-1632,3. COSTO MANO DE OBRA,0,CHORRILLOS CLOROX,CONSTRUCTORA SUKSA S.A.


In [346]:
# MERGE 1 - df_ytd_cerrado_2M_3_2

# Se hace merge con los codigos de proyecto.

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ytd_cerrado_2M_3_2 : ", df_ytd_cerrado_2M_3_2.shape)

df_ptd_cerrado_anterior = pd.merge(df_ptd_cerrado_anterior, df_ytd_cerrado_2M_3_2, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior : ", df_ptd_cerrado_anterior.shape)

df_ptd_cerrado_anterior.sample(10)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ytd_cerrado_2M_3_2 :  (340, 3)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD
183,P-1612,1. INGRESO,0,BARRIO PARQUE SANTIAGO 7,CONSTRUCTORA ALTURAS DEL SUR S.A,0
220,P-1649,4. COSTO SUPERVISIÓN,0,INACAP VITACURA E2,CONSTRUCTORA CUEVAS Y PURCELL S.A.,0
137,P-1716,2. COSTO MATERIALES,0,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,-23165972
270,P-1681,2. COSTO MATERIALES,0,CD ROSEN (CCTV PROVISORIO),COLCHONES ROSEN SAIC,0
74,P-1691,1. INGRESO,0,ROSARIO ROSALES,CONSTRUCTORA E INMOBILIARIA PRADOVERDE LTDA.,2124157
152,P-1723,1. INGRESO,0,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,18134971
217,P-1648,4. COSTO SUPERVISIÓN,0,ETAPA 2 CD PERALILLO TW,INMOBILIARIA Y RENTAS LOGÍSTICA S.A.,0
143,P-1719,1. INGRESO,0,CD METALPAR,EBCO S.A.,131348
116,P-1707,2. COSTO MATERIALES,0,LAS TRANQUERAS,CONSTRUCTORA SANTOLAYA LTDA.,-2860953
287,P-1690,4. COSTO SUPERVISIÓN,0,ONGOLMO - HOME CONDOMINIO,CANTAUCO CONSTRUCCION LTDA.,0


In [347]:
# MERGE 2 - df_ytd_cerrado2

# Se hace merge con los codigos de proyecto.

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior : ", df_ptd_cerrado_anterior.shape)

df_ptd_cerrado_anterior = pd.merge(df_ptd_cerrado_anterior, df_libro_mayor_old5, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior : ", df_ptd_cerrado_anterior.shape)

df_ptd_cerrado_anterior.sample(10)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior :  (340, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior :  (340, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD
172,P-1591,2. COSTO MATERIALES,0,EDIFICIO ESTORIL,EBCO S.A.,0,NaN
11,P-1641,2. COSTO MATERIALES,0,INACAP TEMUCO,DESARROLLOS CONSTRUCTIVOS AXIS SOCIEDAD ANONIMA,-369720,-669050.00
189,P-1626,1. INGRESO,0,CLINICA BIO BIO,CONSTRUCTORA LYD S.A.,0,75533.00
1,P-1591,3. COSTO MANO DE OBRA,0,EDIFICIO ESTORIL,EBCO S.A.,-2483998,NaN
224,P-1656,1. INGRESO,0,MIRADOR CLUB HÍPICO,CONSTRUCTORA VITAL EDIFICACION SPA,0,16765455.00
306,P-1708,4. COSTO SUPERVISIÓN,0,ECO ESPAÑA,CONSTRUCTORA ICF SPA,0,NaN
192,P-1632,1. INGRESO,0,CHORRILLOS CLOROX,CONSTRUCTORA SUKSA S.A.,0,0.00
119,P-1708,2. COSTO MATERIALES,0,ECO ESPAÑA,CONSTRUCTORA ICF SPA,-125100,NaN
20,P-1656,2. COSTO MATERIALES,0,MIRADOR CLUB HÍPICO,CONSTRUCTORA VITAL EDIFICACION SPA,0,-7997069.00
97,P-1700,3. COSTO MANO DE OBRA,0,CD ORIZON,EBCO S.A.,-4264241,-27789610.00


In [348]:
# Se estandarizan columnas de Totales y se agrega columna TOTAL 

# Reemplazar NaN por cero en las columnas "TOTAL YTD" y "TOTAL LB OLD"
df_ptd_cerrado_anterior["TOTAL YTD"] = df_ptd_cerrado_anterior["TOTAL YTD"].fillna(0)
df_ptd_cerrado_anterior["TOTAL LB OLD"] = df_ptd_cerrado_anterior["TOTAL LB OLD"].fillna(0)

# Crear la nueva columna "TOTAL" que suma "TOTAL YTD" y "TOTAL LB OLD"
df_ptd_cerrado_anterior["TOTAL"] = df_ptd_cerrado_anterior["TOTAL YTD"] + df_ptd_cerrado_anterior["TOTAL LB OLD"]
df_ptd_cerrado_anterior.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
172,P-1591,2. COSTO MATERIALES,0,EDIFICIO ESTORIL,EBCO S.A.,0,0.00,0.00
150,P-1723,3. COSTO MANO DE OBRA,0,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,-10269821,0.00,-10269821.00
132,P-1714,1. INGRESO,0,DEPARTAMENTAL II,CONSTRUCTORA FUERZA SUR DOS SPA,18147027,34349625.00,52496652.00
26,P-1666,2. COSTO MATERIALES,0,CD AXIONLONG,EBCO S.A.,0,-25105908.00,-25105908.00
116,P-1707,2. COSTO MATERIALES,0,LAS TRANQUERAS,CONSTRUCTORA SANTOLAYA LTDA.,-2860953,-7486796.00,-10347749.00


In [349]:
# TESTING VISUAL 1

print(df_ptd_cerrado_anterior[df_ptd_cerrado_anterior["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL YTD"].sum())
print(df_ptd_cerrado_anterior[df_ptd_cerrado_anterior["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL LB OLD"].sum())
print(df_ptd_cerrado_anterior[df_ptd_cerrado_anterior["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())

df_ptd_cerrado_anterior[df_ptd_cerrado_anterior["CODIGO_PROYECTO"] == "1. TOTAL"]

301368788
1369937537.0
1671306325.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-17923405,-51949755.00,-69873160.00
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,1118563896,3185711545.00,4304275441.00
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-435497453,-1152612025.00,-1588109478.00
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-363774250,-611212228.00,-974986478.00


In [350]:
# TESTING VISUAL 2

print(df_ptd_cerrado_anterior[df_ptd_cerrado_anterior["CODIGO_PROYECTO"] == "P-1704"]["TOTAL YTD"].sum())
print(df_ptd_cerrado_anterior[df_ptd_cerrado_anterior["CODIGO_PROYECTO"] == "P-1704"]["TOTAL LB OLD"].sum())
print(df_ptd_cerrado_anterior[df_ptd_cerrado_anterior["CODIGO_PROYECTO"] == "P-1704"]["TOTAL"].sum())

df_ptd_cerrado_anterior[df_ptd_cerrado_anterior["CODIGO_PROYECTO"] == "P-1704"]

38892837
252119092.0
291011929.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
106,P-1704,3. COSTO MANO DE OBRA,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-44701591,-87942380.00,-132643971.00
107,P-1704,2. COSTO MATERIALES,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-58121771,-125594139.00,-183715910.00
108,P-1704,1. INGRESO,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,141716199,465655611.00,607371810.00
302,P-1704,4. COSTO SUPERVISIÓN,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.00,0.00


#### 4.8.3 - SE AGREGAN FILAS DE 1.TOTAL

In [351]:
# Se eliminan Totales, excepto cuando se trata de COSTO_SUPERVISIÓN

print("DIMENSIONES ANTES DE TRATAMIENTO DEL DATAFRAME: df_ptd_cerrado_anterior : ", df_ptd_cerrado_anterior.shape)

df_ptd_cerrado_anterior2 = df_ptd_cerrado_anterior[~((df_ptd_cerrado_anterior["CODIGO_PROYECTO"] == "1. TOTAL") & (df_ptd_cerrado_anterior["TIPO_CUENTA"] != "4. COSTO SUPERVISIÓN"))]

print("DIMENSIONES DESPUÉS DE TRATAMIENTO DEL DATAFRAME df_ptd_cerrado_anterior2 : ", df_ptd_cerrado_anterior2.shape)

DIMENSIONES ANTES DE TRATAMIENTO DEL DATAFRAME: df_ptd_cerrado_anterior :  (340, 8)
DIMENSIONES DESPUÉS DE TRATAMIENTO DEL DATAFRAME df_ptd_cerrado_anterior2 :  (337, 8)


In [352]:
# TESTING VISUAL 2


print(df_ptd_cerrado_anterior2[df_ptd_cerrado_anterior2["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL YTD"].sum())
print(df_ptd_cerrado_anterior2[df_ptd_cerrado_anterior2["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL LB OLD"].sum())
print(df_ptd_cerrado_anterior2[df_ptd_cerrado_anterior2["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())

df_ptd_cerrado_anterior2[df_ptd_cerrado_anterior2["CODIGO_PROYECTO"] == "1. TOTAL"]

-17923405
-51949755.0
-69873160.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-17923405,-51949755.00,-69873160.00


In [353]:
# Calcular la suma de los valores totales para cada tipo de cuenta

print("DIMENSIONES ANTES DE TRATAMIENTO DEL DATAFRAME: df_ptd_cerrado_anterior2 : ", df_ptd_cerrado_anterior2.shape)

df_total = df_ptd_cerrado_anterior2.groupby('TIPO_CUENTA')['TOTAL'].sum().reset_index()
df_total['CODIGO_PROYECTO'] = '1. TOTAL'

# Agregar las filas adicionales al DataFrame
df_ptd_cerrado_anterior2 = pd.concat([df_ptd_cerrado_anterior2, df_total], ignore_index=True)

# Modificar las columnas "NOMBRE_OBRA" y "NOMBRE_CLIENTE" para "1. TOTAL"
df_ptd_cerrado_anterior2.loc[df_ptd_cerrado_anterior2['CODIGO_PROYECTO'] == '1. TOTAL', ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'

print("DIMENSIONES DESPUÉS DE TRATAMIENTO DEL DATAFRAME df_ptd_cerrado_anterior2 : ", df_ptd_cerrado_anterior2.shape)

df_ptd_cerrado_anterior2.sample(5)  

DIMENSIONES ANTES DE TRATAMIENTO DEL DATAFRAME: df_ptd_cerrado_anterior2 :  (337, 8)
DIMENSIONES DESPUÉS DE TRATAMIENTO DEL DATAFRAME df_ptd_cerrado_anterior2 :  (341, 8)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
257,P-1673,3. COSTO MANO DE OBRA,0.00,INACAP TEMUCO - ETAPA A,UNIVERSIDAD TECNOLOGICA DE CHILE INACAP,0.00,0.00,0.00
317,P-1719,2. COSTO MATERIALES,0.00,CD METALPAR,EBCO S.A.,0.00,0.00,0.00
116,P-1707,2. COSTO MATERIALES,0.00,LAS TRANQUERAS,CONSTRUCTORA SANTOLAYA LTDA.,-2860953.00,-7486796.00,-10347749.00
309,P-1713,4. COSTO SUPERVISIÓN,0.00,RESERVA SAN FRANCISCO,EMPRESA NACIONAL DE CONSTRUCCIÓN LAS VERTIENTE...,0.00,0.00,0.00
137,P-1716,2. COSTO MATERIALES,0.00,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,-23165972.00,-4855555.00,-28021527.00


In [354]:
# TESTING VISUAL 1

print(df_ptd_cerrado_anterior2[df_ptd_cerrado_anterior2["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_ptd_cerrado_anterior2[df_ptd_cerrado_anterior2["CODIGO_PROYECTO"] == "1. TOTAL"].head(5)

1423926747.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-17923405.00,-51949755.00,-69873160.00
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,NaN,NaN,4042394062.00
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-1516685549.00
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-962035446.00
340,1. TOTAL,4. COSTO SUPERVISIÓN,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-69873160.00


In [355]:
# Se eliminan COSTO SUPERVISION INCORRECTO

df_ptd_cerrado_anterior2 = df_ptd_cerrado_anterior2[~((df_ptd_cerrado_anterior2['CODIGO_PROYECTO'] == '1. TOTAL') & (df_ptd_cerrado_anterior2['TIPO_CUENTA'] == '4. COSTO SUPERVISIÓN') & (df_ptd_cerrado_anterior2['TOTAL_LB'].isna()))]
 
# TESTING VISUAL 2

print(df_ptd_cerrado_anterior2[df_ptd_cerrado_anterior2["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_ptd_cerrado_anterior2[df_ptd_cerrado_anterior2["CODIGO_PROYECTO"] == "P-1704"].head(5)

1493799907.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
106,P-1704,3. COSTO MANO DE OBRA,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-44701591.00,-87942380.00,-132643971.00
107,P-1704,2. COSTO MATERIALES,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-58121771.00,-125594139.00,-183715910.00
108,P-1704,1. INGRESO,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,141716199.00,465655611.00,607371810.00
302,P-1704,4. COSTO SUPERVISIÓN,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0.00,0.00,0.00


#### 4.8.4 - AGREGACIÓN COLUMNA MARGEN

In [356]:
#CREACIÓN DE MARGENES

df_ptd_cerrado_anterior3 = df_ptd_cerrado_anterior2.copy()


# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_ptd_cerrado_anterior3.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)


print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior3 : ", df_ptd_cerrado_anterior3.shape)

df_ptd_cerrado_anterior3 = pd.merge(df_ptd_cerrado_anterior3, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior3 : ", df_ptd_cerrado_anterior3.shape)
df_ptd_cerrado_anterior3.sample(5)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior3 :  (340, 8)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior3 :  (340, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL,MARGEN
244,P-1666,4. COSTO SUPERVISIÓN,0.00,CD AXIONLONG,EBCO S.A.,0.00,0.00,0.00,0.63
292,P-1694,4. COSTO SUPERVISIÓN,0.00,ECO HUEMUL,CONSTRUCTORA ICF SPA,0.00,0.00,0.00,0.20
136,P-1716,3. COSTO MANO DE OBRA,0.00,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,-20322771.00,-4333084.00,-24655855.00,0.55
61,P-1686,1. INGRESO,0.00,MACUL ICALMA,EMPRESA CONSTRUCTORA INGEVEC S.A.,20080642.00,67737824.00,87818466.00,0.23
139,P-1717,2. COSTO MATERIALES,0.00,ECO QUILÍN I,CONSTRUCTORA ICF SPA,0.00,0.00,0.00,0.00


In [357]:
# TESTING VISUAL 1

print(df_ptd_cerrado_anterior3[df_ptd_cerrado_anterior3["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_ptd_cerrado_anterior3[df_ptd_cerrado_anterior3["CODIGO_PROYECTO"] == "1. TOTAL"].head(5)

1493799907.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL,MARGEN
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-17923405.00,-51949755.00,-69873160.00,0.37
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,NaN,NaN,4042394062.00,0.37
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-1516685549.00,0.37
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-962035446.00,0.37


In [358]:
# TESTING VISUAL 2

print(df_ptd_cerrado_anterior3[df_ptd_cerrado_anterior3["CODIGO_PROYECTO"] == "P-1727"]["TOTAL"].sum())
df_ptd_cerrado_anterior3[df_ptd_cerrado_anterior3["CODIGO_PROYECTO"] == "P-1704"].head(5)

2536455.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL,MARGEN
106,P-1704,3. COSTO MANO DE OBRA,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-44701591.00,-87942380.00,-132643971.00,0.48
107,P-1704,2. COSTO MATERIALES,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-58121771.00,-125594139.00,-183715910.00,0.48
108,P-1704,1. INGRESO,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,141716199.00,465655611.00,607371810.00,0.48
302,P-1704,4. COSTO SUPERVISIÓN,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0.00,0.00,0.00,0.48


In [359]:
df_ptd_cerrado_anterior3.to_csv('archivos_respaldo/df_ptd_cerrado_anterior3.csv', index = False)

### 4.9 - MERGE PTD + MES EN CURSO (PROYECTADO)
Cruce entre df_ptd_cerrado3 + mes_en_curso_proy5

NOTA IMPORTANTE:
Ambos dataframes vienen depurados de los MERGES anteriores y no es necesario trabajar con el df_libro_mayor5_base2, por lo tanto solo se realiza concatenación y se crean las filas 1. TOTAL y el MARGEN. 

#### 4.9.1 - EXPLORACIÓN Y PREPARACIÓN DE DATAFRAMES

In [360]:
# TRATAMIENTO DATAFRAME df_ptd_cerrado3_1

# Se realiza una copia del dataframe 
df_ptd_cerrado3_1 = df_ptd_cerrado3.copy()

# Se Seleccionan columnas utiles:

df_ptd_cerrado3_1 = df_ptd_cerrado3_1[["CODIGO_PROYECTO", "TIPO_CUENTA", "NOMBRE_OBRA", "NOMBRE_CLIENTE", "TOTAL"]]

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME  : df_ptd_cerrado3   : ", df_ptd_cerrado3.shape)
print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ptd_cerrado3_1 : ", df_ptd_cerrado3_1.shape)

# Se renombra campo "TOTAL"
df_ptd_cerrado3_1 = df_ptd_cerrado3_1.rename(columns={"TOTAL": "TOTAL PTD"})

df_ptd_cerrado3_1.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME  : df_ptd_cerrado3   :  (340, 9)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ptd_cerrado3_1 :  (340, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD
8,P-1632,2. COSTO MATERIALES,CHORRILLOS CLOROX,CONSTRUCTORA SUKSA S.A.,312558.00
150,P-1723,3. COSTO MANO DE OBRA,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,-25834725.00
83,P-1695,3. COSTO MANO DE OBRA,EDIFICIO ESTORIL,EMPRESA CONSTRUCTORA BRAVO E IZQUIERDO,-13163688.00
29,P-1668,2. COSTO MATERIALES,QUILLAYES PLAZA,E.MOLINA MOREL CONSTRUCTORA S.A.,-14024713.00
51,P-1682,3. COSTO MANO DE OBRA,PLAZA CEPPI,EMP. CONSTRUCTORA INGENIEROS S.A.,-21010033.00


In [361]:
# TRATAMIENTO DATAFRAME df_ptd_cerrado3_1

# Se realiza una copia del dataframe 
mes_en_curso_proy5_1 = mes_en_curso_proy5.copy()

# Se Seleccionan columnas utiles:

mes_en_curso_proy5_1 = mes_en_curso_proy5_1[["CODIGO_PROYECTO", "TIPO_CUENTA", "TOTAL"]]

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME  : mes_en_curso_proy5  : ", mes_en_curso_proy5.shape)
print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy5_1 : ", mes_en_curso_proy5_1.shape)

# Se renombra campo "TOTAL"
mes_en_curso_proy5_1 = mes_en_curso_proy5_1.rename(columns={"TOTAL": "TOTAL MEC PROY"})

mes_en_curso_proy5_1.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME  : mes_en_curso_proy5  :  (340, 7)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy5_1 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL MEC PROY
279,P-1685,3. COSTO MANO DE OBRA,0
212,P-1644,4. COSTO SUPERVISIÓN,0
189,P-1626,1. INGRESO,0
53,P-1682,1. INGRESO,0
291,P-1693,4. COSTO SUPERVISIÓN,0


#### 4.9.2 - MERGE DE DATAFRAMES 

In [362]:
# Se hace merge manteniendo como base df_ptd_cerrado3_1 que viene del total de proyectos existentes en el libro mayor 2023

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado3_1 : ", df_ptd_cerrado3_1.shape)

df_ptd_cerrado_y_proyectado = pd.merge(df_ptd_cerrado3_1, mes_en_curso_proy5_1, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado : ", df_ptd_cerrado_y_proyectado.shape)

df_ptd_cerrado_y_proyectado.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado3_1 :  (340, 5)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY
124,P-1712,3. COSTO MANO DE OBRA,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,-32290414.00,-5436047
154,P-1724,2. COSTO MATERIALES,NaN,NaN,-7660814.00,0
119,P-1708,2. COSTO MATERIALES,ECO ESPAÑA,CONSTRUCTORA ICF SPA,-125100.00,-4200000
37,P-1674,2. COSTO MATERIALES,OFICINAS BEIERSDORF CHORRILLOS,CONSTRUCTORA SUKSA S.A.,-8601126.00,0
72,P-1691,3. COSTO MANO DE OBRA,ROSARIO ROSALES,CONSTRUCTORA E INMOBILIARIA PRADOVERDE LTDA.,-17223414.00,0


In [363]:
# Se estandarizan columnas de Totales y se agrega columna TOTAL 

# Reemplazar NaN por cero en las columnas "TOTAL YTD" y "TOTAL LB OLD"
df_ptd_cerrado_y_proyectado["TOTAL PTD"] = df_ptd_cerrado_y_proyectado["TOTAL PTD"].fillna(0)
df_ptd_cerrado_y_proyectado["TOTAL MEC PROY"] = df_ptd_cerrado_y_proyectado["TOTAL MEC PROY"].fillna(0)

# Crear la nueva columna "TOTAL" que suma "TOTAL YTD" y "TOTAL LB OLD"
df_ptd_cerrado_y_proyectado["TOTAL"] = df_ptd_cerrado_y_proyectado["TOTAL PTD"] + df_ptd_cerrado_y_proyectado["TOTAL MEC PROY"]
df_ptd_cerrado_y_proyectado.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL
273,P-1682,4. COSTO SUPERVISIÓN,PLAZA CEPPI,EMP. CONSTRUCTORA INGENIEROS S.A.,0.00,0,0.00
298,P-1699,4. COSTO SUPERVISIÓN,VILLA PANAMERICANA LOTE B,EMPRESA CONSTRUCTORA DLP S.A.,0.00,0,0.00
183,P-1612,1. INGRESO,BARRIO PARQUE SANTIAGO 7,CONSTRUCTORA ALTURAS DEL SUR S.A,0.00,0,0.00
263,P-1675,4. COSTO SUPERVISIÓN,PALENA II,CONSTRUCTORA MYA SPA.,0.00,0,0.00
313,P-1717,1. INGRESO,ECO QUILÍN I,CONSTRUCTORA ICF SPA,0.00,1000000,1000000.00


In [364]:
# TESTING VISUAL 1

print(df_ptd_cerrado_y_proyectado[df_ptd_cerrado_y_proyectado["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_ptd_cerrado_y_proyectado[df_ptd_cerrado_y_proyectado["CODIGO_PROYECTO"] == "1. TOTAL"].head(5)

1794293274.0


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,1. TOTAL,1. TOTAL,-74516421.00,-4643261,-79159682.00
337,1. TOTAL,1. INGRESO,1. TOTAL,1. TOTAL,4503192575.00,423800000,4926992575.00
338,1. TOTAL,2. COSTO MATERIALES,1. TOTAL,1. TOTAL,-1686220937.00,-176535806,-1862756743.00
339,1. TOTAL,3. COSTO MANO DE OBRA,1. TOTAL,1. TOTAL,-1076409161.00,-114373715,-1190782876.00


In [365]:
# TESTING VISUAL 2

print(df_ptd_cerrado_y_proyectado[df_ptd_cerrado_y_proyectado["CODIGO_PROYECTO"] == "P-1727"]["TOTAL"].sum())
df_ptd_cerrado_y_proyectado[df_ptd_cerrado_y_proyectado["CODIGO_PROYECTO"] == "P-1727"].head(5)

19790427.0


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL
162,P-1727,3. COSTO MANO DE OBRA,NaN,NaN,-8169893.00,-3455213,-11625106.00
163,P-1727,2. COSTO MATERIALES,NaN,NaN,-13655419.00,-10000000,-23655419.00
164,P-1727,1. INGRESO,NaN,NaN,28570952.00,26500000,55070952.00
330,P-1727,4. COSTO SUPERVISIÓN,NaN,NaN,0.00,0,0.00


#### 4.9.3 - AGREGACIÓN COLUMNA MARGEN

In [366]:
#CREACIÓN DE MARGENES

df_ptd_cerrado_y_proyectado2 = df_ptd_cerrado_y_proyectado.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_ptd_cerrado_y_proyectado2.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)


print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado2 : ", df_ptd_cerrado_y_proyectado2.shape)

df_ptd_cerrado_y_proyectado2 = pd.merge(df_ptd_cerrado_y_proyectado2, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado2 : ", df_ptd_cerrado_y_proyectado2.shape)
df_ptd_cerrado_y_proyectado2.sample(10)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado2 :  (340, 7)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado2 :  (340, 8)


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL,MARGEN
88,P-1696,1. INGRESO,SANTA VICTORIA - TERRACORP,CONSTRUCTORA TERRACORP S.A.,75515364.00,0,75515364.00,0.12
20,P-1656,2. COSTO MATERIALES,MIRADOR CLUB HÍPICO,CONSTRUCTORA VITAL EDIFICACION SPA,-7985819.00,0,-7985819.00,0.47
297,P-1698,4. COSTO SUPERVISIÓN,CD ANDES,EBCO S.A.,0.00,0,0.00,0.36
168,P-1557,1. INGRESO,ROSARIO ROSALES,EBCO S.A.,0.00,0,0.00,0.00
316,P-1718,4. COSTO SUPERVISIÓN,PATIO COMIDAS,EBCO S.A.,0.00,0,0.00,0.52
310,P-1714,4. COSTO SUPERVISIÓN,DEPARTAMENTAL II,CONSTRUCTORA FUERZA SUR DOS SPA,0.00,0,0.00,0.02
222,P-1653,3. COSTO MANO DE OBRA,EDUARDO DE LA BARRA,CONSTRUCTORA E INVERSIONES VITAL SPA.,0.00,0,0.00,0.59
262,P-1675,3. COSTO MANO DE OBRA,PALENA II,CONSTRUCTORA MYA SPA.,-4557877.00,0,-4557877.00,0.27
118,P-1708,3. COSTO MANO DE OBRA,ECO ESPAÑA,CONSTRUCTORA ICF SPA,-1679752.00,-208532,-1888284.00,-14.84
252,P-1670,3. COSTO MANO DE OBRA,MK II SEGURIDAD Y REDES,COMERCIAL K SPA.,-431689.00,0,-431689.00,0.65


In [367]:
# TESTING VISUAL 1

print(df_ptd_cerrado_y_proyectado2[df_ptd_cerrado_y_proyectado2["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_ptd_cerrado_y_proyectado2[df_ptd_cerrado_y_proyectado2["CODIGO_PROYECTO"] == "1. TOTAL"].head(5)

1794293274.0


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL,MARGEN
336,1. TOTAL,4. COSTO SUPERVISIÓN,1. TOTAL,1. TOTAL,-74516421.00,-4643261,-79159682.00,0.36
337,1. TOTAL,1. INGRESO,1. TOTAL,1. TOTAL,4503192575.00,423800000,4926992575.00,0.36
338,1. TOTAL,2. COSTO MATERIALES,1. TOTAL,1. TOTAL,-1686220937.00,-176535806,-1862756743.00,0.36
339,1. TOTAL,3. COSTO MANO DE OBRA,1. TOTAL,1. TOTAL,-1076409161.00,-114373715,-1190782876.00,0.36


In [368]:
# TESTING VISUAL 2

print(df_ptd_cerrado_y_proyectado2[df_ptd_cerrado_y_proyectado2["CODIGO_PROYECTO"] == "P-1727"]["TOTAL"].sum())
df_ptd_cerrado_y_proyectado2[df_ptd_cerrado_y_proyectado2["CODIGO_PROYECTO"] == "P-1727"].head(5)

19790427.0


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL,MARGEN
162,P-1727,3. COSTO MANO DE OBRA,NaN,NaN,-8169893.00,-3455213,-11625106.00,0.36
163,P-1727,2. COSTO MATERIALES,NaN,NaN,-13655419.00,-10000000,-23655419.00,0.36
164,P-1727,1. INGRESO,NaN,NaN,28570952.00,26500000,55070952.00,0.36
330,P-1727,4. COSTO SUPERVISIÓN,NaN,NaN,0.00,0,0.00,0.36


In [369]:
df_ptd_cerrado_y_proyectado2.to_csv('archivos_respaldo/df_ptd_cerrado_y_proyectado2.csv', index = False)

### 4.10 - MERGE PTD + MES EN CURSO (PROYECTADO) (PERSPECTIVA MES ANTERIOR)
Cruce entre df_ptd_cerrado_anterior3 + mes_en_curso_proy_2M_5

NOTA IMPORTANTE:
Ambos dataframes vienen depurados de los MERGES anteriores y no es necesario trabajar con el df_libro_mayor5_base2, por lo tanto solo se realiza concatenación y se crean las filas 1. TOTAL y el MARGEN. 

#### 4.10.1 - EXPLORACIÓN Y PREPARACIÓN DE DATAFRAMES

In [370]:
# TRATAMIENTO DATAFRAME df_ptd_cerrado3_1

# Se realiza una copia del dataframe 
df_ptd_cerrado_anterior3_1 = df_ptd_cerrado_anterior3.copy()

# Se Seleccionan columnas utiles:

df_ptd_cerrado_anterior3_1 = df_ptd_cerrado_anterior3_1[["CODIGO_PROYECTO", "TIPO_CUENTA", "NOMBRE_OBRA", "NOMBRE_CLIENTE", "TOTAL"]]

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME  : df_ptd_cerrado_anterior3   : ", df_ptd_cerrado_anterior3.shape)
print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ptd_cerrado_anterior3_1 : ", df_ptd_cerrado_anterior3_1.shape)

# Se renombra campo "TOTAL"
df_ptd_cerrado_anterior3_1 = df_ptd_cerrado_anterior3_1.rename(columns={"TOTAL": "TOTAL PTD"})

df_ptd_cerrado_anterior3_1.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME  : df_ptd_cerrado_anterior3   :  (340, 9)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ptd_cerrado_anterior3_1 :  (340, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD
254,P-1672,3. COSTO MANO DE OBRA,STA CLARA (CCDD-TELECOM),CONSTRUCTORA UPC S.A.,-46046794.00
187,P-1620,3. COSTO MANO DE OBRA,"OF APOQUINDO- HABILITACIONES, INTEGRACION Y AD...",CONSTRUCTORA INARCO S.A.,0.00
0,P-1557,3. COSTO MANO DE OBRA,ROSARIO ROSALES,EBCO S.A.,-945949.00
124,P-1712,3. COSTO MANO DE OBRA,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,-26854367.00
311,P-1715,4. COSTO SUPERVISIÓN,CD LO ECHEVERS,QUEZADA Y BOETSCH,0.00


In [371]:
# TRATAMIENTO DATAFRAME mes en curso proyectado

# Se realiza una copia del dataframe 
mes_en_curso_proy_2M_5_1 = mes_en_curso_proy_2M_5.copy()

# Se Seleccionan columnas utiles:

mes_en_curso_proy_2M_5_1 = mes_en_curso_proy_2M_5_1[["CODIGO_PROYECTO", "TIPO_CUENTA", "TOTAL"]]

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME  : mes_en_curso_proy_2M_5   : ", mes_en_curso_proy_2M_5.shape)
print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy_2M_5_1 : ", mes_en_curso_proy_2M_5_1.shape)

# Se renombra campo "TOTAL"
mes_en_curso_proy_2M_5_1 = mes_en_curso_proy_2M_5_1.rename(columns={"TOTAL": "TOTAL MEC PROY"})

mes_en_curso_proy_2M_5_1.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME  : mes_en_curso_proy_2M_5   :  (340, 7)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy_2M_5_1 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL MEC PROY
244,P-1666,4. COSTO SUPERVISIÓN,0.00
265,P-1676,4. COSTO SUPERVISIÓN,0.00
314,P-1717,3. COSTO MANO DE OBRA,0.00
158,P-1725,1. INGRESO,19181571.00
303,P-1705,4. COSTO SUPERVISIÓN,0.00


#### 4.10.2 - MERGE DE DATAFRAMES 

In [372]:
# Se hace merge manteniendo como base df_ptd_cerrado3_1 que viene del total de proyectos existentes en el libro mayor 2023

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior3_1 : ", df_ptd_cerrado_anterior3_1.shape)

df_ptd_cerrado_y_proyectado_ant = pd.merge(df_ptd_cerrado_anterior3_1, mes_en_curso_proy_2M_5_1, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant : ", df_ptd_cerrado_y_proyectado_ant.shape)

df_ptd_cerrado_y_proyectado_ant.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior3_1 :  (340, 5)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY
84,P-1695,2. COSTO MATERIALES,EDIFICIO ESTORIL,EMPRESA CONSTRUCTORA BRAVO E IZQUIERDO,-17009933.00,-10500000.00
256,P-1673,1. INGRESO,INACAP TEMUCO - ETAPA A,UNIVERSIDAD TECNOLOGICA DE CHILE INACAP,26810752.00,0.00
183,P-1612,1. INGRESO,BARRIO PARQUE SANTIAGO 7,CONSTRUCTORA ALTURAS DEL SUR S.A,0.00,0.00
192,P-1632,1. INGRESO,CHORRILLOS CLOROX,CONSTRUCTORA SUKSA S.A.,0.00,0.00
21,P-1658,2. COSTO MATERIALES,VIVIPRA,VICHERAT Y PRADENAS LTDA.,-11357166.00,0.00


In [373]:
# Se estandarizan columnas de Totales y se agrega columna TOTAL 

# Reemplazar NaN por cero en las columnas "TOTAL YTD" y "TOTAL LB OLD"
df_ptd_cerrado_y_proyectado_ant["TOTAL PTD"] = df_ptd_cerrado_y_proyectado_ant["TOTAL PTD"].fillna(0)
df_ptd_cerrado_y_proyectado_ant["TOTAL MEC PROY"] = df_ptd_cerrado_y_proyectado_ant["TOTAL MEC PROY"].fillna(0)

# Crear la nueva columna "TOTAL" que suma "TOTAL YTD" y "TOTAL LB OLD"
df_ptd_cerrado_y_proyectado_ant["TOTAL"] = df_ptd_cerrado_y_proyectado_ant["TOTAL PTD"] + df_ptd_cerrado_y_proyectado_ant["TOTAL MEC PROY"]
df_ptd_cerrado_y_proyectado_ant.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL
137,P-1716,2. COSTO MATERIALES,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,-28021527.00,0.00,-28021527.00
333,P-1729,1. INGRESO,NaN,NaN,0.00,0.00,0.00
208,P-1643,3. COSTO MANO DE OBRA,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,0.00,0.00,0.00
102,P-1701,1. INGRESO,EDIFICIO ZENTENO,CONSTRUCTORA GREVIA SPA.,79529092.00,0.00,79529092.00
37,P-1674,2. COSTO MATERIALES,OFICINAS BEIERSDORF CHORRILLOS,CONSTRUCTORA SUKSA S.A.,-8601126.00,0.00,-8601126.00


In [374]:
# TESTING VISUAL 1

print(df_ptd_cerrado_y_proyectado_ant[df_ptd_cerrado_y_proyectado_ant["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_ptd_cerrado_y_proyectado_ant[df_ptd_cerrado_y_proyectado_ant["CODIGO_PROYECTO"] == "1. TOTAL"].head(5)

1626474125.3564873


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,1. TOTAL,1. TOTAL,-69873160.00,-5096121.00,-74969281.00
337,1. TOTAL,1. INGRESO,1. TOTAL,1. TOTAL,4042394062.00,461955807.36,4504349869.36
338,1. TOTAL,2. COSTO MATERIALES,1. TOTAL,1. TOTAL,-1516685549.00,-212300000.00,-1728985549.00
339,1. TOTAL,3. COSTO MANO DE OBRA,1. TOTAL,1. TOTAL,-962035446.00,-111885468.00,-1073920914.00


In [375]:
# TESTING VISUAL 2

print(df_ptd_cerrado_y_proyectado_ant[df_ptd_cerrado_y_proyectado_ant["CODIGO_PROYECTO"] == "P-1727"]["TOTAL"].sum())
df_ptd_cerrado_y_proyectado_ant[df_ptd_cerrado_y_proyectado_ant["CODIGO_PROYECTO"] == "P-1727"].head(5)

7086615.0


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL
162,P-1727,3. COSTO MANO DE OBRA,NaN,NaN,-4714680.00,-3205300.00,-7919980.00
163,P-1727,2. COSTO MATERIALES,NaN,NaN,-3753865.00,-9500000.00,-13253865.00
164,P-1727,1. INGRESO,NaN,NaN,11005000.00,17255460.00,28260460.00
330,P-1727,4. COSTO SUPERVISIÓN,NaN,NaN,0.00,0.00,0.00


#### 4.10.3 - AGREGACIÓN COLUMNA MARGEN

In [376]:
#CREACIÓN DE MARGENES

df_ptd_cerrado_y_proyectado_ant2 = df_ptd_cerrado_y_proyectado_ant.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_ptd_cerrado_y_proyectado_ant2.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)


print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant2 : ", df_ptd_cerrado_y_proyectado_ant2.shape)

df_ptd_cerrado_y_proyectado_ant2 = pd.merge(df_ptd_cerrado_y_proyectado_ant2, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant2 : ", df_ptd_cerrado_y_proyectado_ant2.shape)
df_ptd_cerrado_y_proyectado_ant2.sample(10)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant2 :  (340, 7)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant2 :  (340, 8)


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL,MARGEN
53,P-1682,1. INGRESO,PLAZA CEPPI,EMP. CONSTRUCTORA INGENIEROS S.A.,55359285.00,18808581.00,74167866.00,0.35
262,P-1675,3. COSTO MANO DE OBRA,PALENA II,CONSTRUCTORA MYA SPA.,-4557877.00,0.00,-4557877.00,0.27
332,P-1728,4. COSTO SUPERVISIÓN,NaN,NaN,0.00,0.00,0.00,0.53
255,P-1672,4. COSTO SUPERVISIÓN,STA CLARA (CCDD-TELECOM),CONSTRUCTORA UPC S.A.,0.00,0.00,0.00,0.32
79,P-1693,2. COSTO MATERIALES,VOLTA V0800,ECOBIO SPA,-89612109.00,0.00,-89612109.00,0.15
242,P-1666,1. INGRESO,CD AXIONLONG,EBCO S.A.,72760796.00,0.00,72760796.00,0.63
229,P-1658,4. COSTO SUPERVISIÓN,VIVIPRA,VICHERAT Y PRADENAS LTDA.,0.00,0.00,0.00,0.62
158,P-1725,1. INGRESO,LO CAÑAS,CONSTRUCTORA BRISAS DE BATUCO,8921780.00,19181571.00,28103351.00,0.47
191,P-1626,4. COSTO SUPERVISIÓN,CLINICA BIO BIO,CONSTRUCTORA LYD S.A.,0.00,0.00,0.00,10.03
327,P-1724,4. COSTO SUPERVISIÓN,NaN,NaN,0.00,0.00,0.00,0.72


In [377]:
# TESTING VISUAL 1

print(df_ptd_cerrado_y_proyectado_ant2[df_ptd_cerrado_y_proyectado_ant2["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_ptd_cerrado_y_proyectado_ant2[df_ptd_cerrado_y_proyectado_ant2["CODIGO_PROYECTO"] == "1. TOTAL"].head(5)

1626474125.3564873


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL,MARGEN
336,1. TOTAL,4. COSTO SUPERVISIÓN,1. TOTAL,1. TOTAL,-69873160.00,-5096121.00,-74969281.00,0.36
337,1. TOTAL,1. INGRESO,1. TOTAL,1. TOTAL,4042394062.00,461955807.36,4504349869.36,0.36
338,1. TOTAL,2. COSTO MATERIALES,1. TOTAL,1. TOTAL,-1516685549.00,-212300000.00,-1728985549.00,0.36
339,1. TOTAL,3. COSTO MANO DE OBRA,1. TOTAL,1. TOTAL,-962035446.00,-111885468.00,-1073920914.00,0.36


In [378]:
# TESTING VISUAL 2

print(df_ptd_cerrado_y_proyectado_ant[df_ptd_cerrado_y_proyectado_ant["CODIGO_PROYECTO"] == "P-1704"]["TOTAL"].sum())
df_ptd_cerrado_y_proyectado_ant[df_ptd_cerrado_y_proyectado_ant["CODIGO_PROYECTO"] == "P-1704"].head(5)

321376053.27884996


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL
106,P-1704,3. COSTO MANO DE OBRA,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-132643971.00,-14242535.00,-146886506.00
107,P-1704,2. COSTO MATERIALES,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-183715910.00,-24000000.00,-207715910.00
108,P-1704,1. INGRESO,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,607371810.00,68606659.28,675978469.28
302,P-1704,4. COSTO SUPERVISIÓN,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0.00,0.00,0.00


In [379]:
df_ptd_cerrado_y_proyectado_ant.to_csv('archivos_respaldo/df_ptd_cerrado_y_proyectado_ant.csv', index = False)

### 4.11 MERGE PRESUPUESTO
Se realiza mergen entre los datos que provienen de la planilla de presupuesto y tabla de presupuestos.

#### 4.11.1 CREACIÓN Y TRATAMIENTO INICIAL DE DATAFRAMES A CONCANTENAR.

In [380]:
print("DIMENSIONES DATAFRAME *df_tabla_presupuesto3* :", df_tabla_presupuesto3.shape)
df_tabla_presupuesto4 = df_tabla_presupuesto3.copy()
print("DIMENSIONES DATAFRAME *df_presupuesto2* :", df_presupuesto2.shape)
df_presupuesto3 = df_presupuesto2.copy()

DIMENSIONES DATAFRAME *df_tabla_presupuesto3* : (388, 5)
DIMENSIONES DATAFRAME *df_presupuesto2* : (70, 4)


In [381]:
df_tabla_presupuesto4 = df_tabla_presupuesto4[['CODIGO_PROYECTO', 'TIPO_COSTO', 'TOTAL', 'ORIGEN']]
df_tabla_presupuesto4.rename(columns={'TIPO_COSTO': 'TIPO_CUENTA'}, inplace=True)

df_tabla_presupuesto4.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,ORIGEN
183,P-1663,2. COSTO MATERIALES,-28692183.00,TABLA_PRESUPUESTO
14,P-1606,4. COSTO SUPERVISIÓN,-658245.00,TABLA_PRESUPUESTO
305,P-1705,4. COSTO SUPERVISIÓN,-3523114.00,TABLA_PRESUPUESTO
160,P-1655,3. COSTO MANO DE OBRA,-5589798.00,TABLA_PRESUPUESTO
273,P-1693,3. COSTO MANO DE OBRA,-11371251.00,TABLA_PRESUPUESTO
293,P-1701,3. COSTO MANO DE OBRA,-28379446.00,TABLA_PRESUPUESTO
65,P-1623,4. COSTO SUPERVISIÓN,-1463910.00,TABLA_PRESUPUESTO
134,P-1646,4. COSTO SUPERVISIÓN,-1226580.00,TABLA_PRESUPUESTO
246,P-1684,2. COSTO MATERIALES,-59464726.00,TABLA_PRESUPUESTO
268,P-1691,4. COSTO SUPERVISIÓN,-2099697.00,TABLA_PRESUPUESTO


In [382]:
df_presupuesto3.rename(columns={'VALOR': 'TOTAL'}, inplace=True)
df_presupuesto3.sample(10)

,CODIGO_PROYECTO,TOTAL,TIPO_CUENTA,ORIGEN
39,P-1695,94918400.00,1. INGRESO,PLANILLA PRESUPUESTO
14,P-1670,202553946.00,1. INGRESO,PLANILLA PRESUPUESTO
7,P-1656,36463980.00,1. INGRESO,PLANILLA PRESUPUESTO
48,P-1704,1058718831.87,1. INGRESO,PLANILLA PRESUPUESTO
43,P-1699,406744605.61,1. INGRESO,PLANILLA PRESUPUESTO
29,P-1685,153894009.96,1. INGRESO,PLANILLA PRESUPUESTO
53,P-1709,105853124.47,1. INGRESO,PLANILLA PRESUPUESTO
55,P-1713,123509706.24,1. INGRESO,PLANILLA PRESUPUESTO
31,P-1687,25285885.04,1. INGRESO,PLANILLA PRESUPUESTO
51,P-1707,67945807.16,1. INGRESO,PLANILLA PRESUPUESTO


#### 4.11.2 SE CONCATENA LOS DATAFRAMES

In [383]:
# Imprimir las dimensiones de los DataFrames
print("Dimensiones de df_tabla_presupuesto4:", df_tabla_presupuesto4.shape)
print("Dimensiones de df_presupuesto3:", df_presupuesto3.shape)

# Concatenar los DataFrames
df_presupuesto_final = pd.concat([df_tabla_presupuesto4, df_presupuesto3], ignore_index=True)
print("Dimensiones de df_presupuesto_final:", df_presupuesto_final.shape)

df_presupuesto_final.sample(10)

Dimensiones de df_tabla_presupuesto4: (388, 4)
Dimensiones de df_presupuesto3: (70, 4)
Dimensiones de df_presupuesto_final: (458, 4)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,ORIGEN
129,P-1645,2. COSTO MATERIALES,-8503715.00,TABLA_PRESUPUESTO
356,P-1725,2. COSTO MATERIALES,-46312307.00,TABLA_PRESUPUESTO
97,P-1634,3. COSTO MANO DE OBRA,-3906066.00,TABLA_PRESUPUESTO
408,P-1677,1. INGRESO,83882285.07,PLANILLA PRESUPUESTO
190,P-1665,3. COSTO MANO DE OBRA,-22328816.00,TABLA_PRESUPUESTO
174,P-1660,2. COSTO MATERIALES,-13492166.00,TABLA_PRESUPUESTO
110,P-1638,4. COSTO SUPERVISIÓN,-1226616.00,TABLA_PRESUPUESTO
404,P-1674,1. INGRESO,28602126.00,PLANILLA PRESUPUESTO
233,P-1679,4. COSTO SUPERVISIÓN,-6052803.00,TABLA_PRESUPUESTO
378,P-Proyectos,4. COSTO SUPERVISIÓN,-133630.00,TABLA_PRESUPUESTO


#### 4.11.3 SE REALIZA MERGE CON CODIGOS DE PROYECTO

In [384]:
# Se hace merge con los codigos de proyecto.

print("Dimensiones de df_libro_mayor5_base2", df_libro_mayor5_base2.shape)

df_presupuesto_final2 = pd.merge(df_libro_mayor5_base2, df_presupuesto_final, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("Dimensiones de df_presupuesto_final2", df_presupuesto_final2.shape)
df_presupuesto_final2.sample(10)

Dimensiones de df_libro_mayor5_base2 (340, 5)
Dimensiones de df_presupuesto_final2 (340, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,ORIGEN
265,P-1676,4. COSTO SUPERVISIÓN,0,LO CAMPINO,EBCO S.A.,-8951416.00,TABLA_PRESUPUESTO
270,P-1681,2. COSTO MATERIALES,0,CD ROSEN (CCTV PROVISORIO),COLCHONES ROSEN SAIC,-4132408.00,TABLA_PRESUPUESTO
295,P-1697,4. COSTO SUPERVISIÓN,0,ESTOCOLMO,CONSTRUCTORA FORTALEZA S.A,NaN,NaN
31,P-1669,2. COSTO MATERIALES,0,DEPARTAMENTAL,EMP. CONSTRUCTORA INGENIEROS S.A.,-32905472.00,TABLA_PRESUPUESTO
38,P-1675,2. COSTO MATERIALES,0,PALENA II,CONSTRUCTORA MYA SPA.,-7361719.00,TABLA_PRESUPUESTO
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,NaN,NaN
268,P-1679,4. COSTO SUPERVISIÓN,0,JUAN SABAJ,CONSTRUCTORA GREVIA,-6052803.00,TABLA_PRESUPUESTO
278,P-1685,1. INGRESO,0,MAPA EDIFICIOS NO OPERACIONALES,CLARO VICUÑA VALENZUELA S.A.,153894009.96,PLANILLA PRESUPUESTO
57,P-1684,1. INGRESO,0,EDIFICIO GOYCOLEA,EBCO S.A.,164431253.35,PLANILLA PRESUPUESTO
90,P-1697,2. COSTO MATERIALES,0,ESTOCOLMO,CONSTRUCTORA FORTALEZA S.A,-43150692.00,TABLA_PRESUPUESTO


#### 4.11.4 SE AGREGAN FILAS DE "1.TOTAL"

In [385]:
# Calcular la suma de los valores totales para cada tipo de cuenta


df_total = df_presupuesto_final2.groupby('TIPO_CUENTA')['TOTAL'].sum().reset_index()
df_total['CODIGO_PROYECTO'] = '1. TOTAL'

# Agregar las filas adicionales al DataFrame
df_presupuesto_final2 = pd.concat([df_presupuesto_final2, df_total], ignore_index=True)

df_presupuesto_final2.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,ORIGEN
151,P-1723,2. COSTO MATERIALES,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,-147892783.00,TABLA_PRESUPUESTO
332,P-1728,4. COSTO SUPERVISIÓN,0.00,NaN,NaN,NaN,NaN
138,P-1716,1. INGRESO,0.00,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,121165036.91,PLANILLA PRESUPUESTO
24,P-1663,2. COSTO MATERIALES,0.00,OFICINAS EQUIFAX,CONSTRUCTORA VRK INGENIERIA LIMITADA,-28692183.00,TABLA_PRESUPUESTO
58,P-1685,2. COSTO MATERIALES,0.00,MAPA EDIFICIOS NO OPERACIONALES,CLARO VICUÑA VALENZUELA S.A.,-84597413.00,TABLA_PRESUPUESTO
113,P-1706,2. COSTO MATERIALES,0.00,CALIFORNIA,CONSTRUCTORA E INVERSIONES VITAL SPA.,-39538656.00,TABLA_PRESUPUESTO
192,P-1632,1. INGRESO,0.00,CHORRILLOS CLOROX,CONSTRUCTORA SUKSA S.A.,138107706.03,PLANILLA PRESUPUESTO
43,P-1677,2. COSTO MATERIALES,0.00,GALVARINO GALLARDO,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...,-35515426.00,TABLA_PRESUPUESTO
232,P-1660,4. COSTO SUPERVISIÓN,0.00,CORONEL ALVARADO,LARRAIN PRIETO RISOPATRON S.A.,-1495636.00,TABLA_PRESUPUESTO
242,P-1666,1. INGRESO,0.00,CD AXIONLONG,EBCO S.A.,NaN,NaN


In [386]:
# Modificar las columnas "NOMBRE_OBRA" y "NOMBRE_CLIENTE" para "1. TOTAL"
df_presupuesto_final2.loc[df_presupuesto_final2['CODIGO_PROYECTO'] == '1. TOTAL', ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'

df_presupuesto_final2.sample(10)      

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,ORIGEN
275,P-1683,3. COSTO MANO DE OBRA,0.00,CD BP-A05,EBCO S.A.,-8303894.00,TABLA_PRESUPUESTO
157,P-1725,2. COSTO MATERIALES,0.00,LO CAÑAS,CONSTRUCTORA BRISAS DE BATUCO,-46312307.00,TABLA_PRESUPUESTO
323,P-1722,2. COSTO MATERIALES,0.00,ECO CERRILLOS,CONSTRUCTORA ICF SPA,-124836610.00,TABLA_PRESUPUESTO
308,P-1712,4. COSTO SUPERVISIÓN,0.00,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,-5621958.00,TABLA_PRESUPUESTO
170,P-1557,4. COSTO SUPERVISIÓN,0.00,ROSARIO ROSALES,EBCO S.A.,NaN,NaN
81,P-1694,2. COSTO MATERIALES,0.00,ECO HUEMUL,CONSTRUCTORA ICF SPA,-57440940.00,TABLA_PRESUPUESTO
15,P-1645,2. COSTO MATERIALES,0.00,LOS TRES ANTONIOS,SOCIEDAD CONSTRUCTORA ECHAVARRI HERMANOS LIMITADA,-8503715.00,TABLA_PRESUPUESTO
44,P-1677,1. INGRESO,0.00,GALVARINO GALLARDO,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...,83882285.07,PLANILLA PRESUPUESTO
255,P-1672,4. COSTO SUPERVISIÓN,0.00,STA CLARA (CCDD-TELECOM),CONSTRUCTORA UPC S.A.,-8009468.00,TABLA_PRESUPUESTO
88,P-1696,1. INGRESO,0.00,SANTA VICTORIA - TERRACORP,CONSTRUCTORA TERRACORP S.A.,75985768.29,PLANILLA PRESUPUESTO


#### 4.11.5 AGREGACIÓN COLUMNA MARGEN

In [387]:
#CREACIÓN DE MARGENES

df_presupuesto_final3 = df_presupuesto_final2.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_presupuesto_final3.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)


print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_presupuesto_final3 : ", df_presupuesto_final3.shape)

df_presupuesto_final4 = pd.merge(df_presupuesto_final3, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_presupuesto_final4 : ", df_presupuesto_final4.shape)
df_presupuesto_final4.sample(10)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_presupuesto_final3 :  (344, 7)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_presupuesto_final4 :  (344, 8)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,ORIGEN,MARGEN
87,P-1696,2. COSTO MATERIALES,0.00,SANTA VICTORIA - TERRACORP,CONSTRUCTORA TERRACORP S.A.,-30348318.00,TABLA_PRESUPUESTO,0.60
300,P-1701,4. COSTO SUPERVISIÓN,0.00,EDIFICIO ZENTENO,CONSTRUCTORA GREVIA SPA.,-3113372.00,TABLA_PRESUPUESTO,0.01
197,P-1636,4. COSTO SUPERVISIÓN,0.00,SANTO DOMINGO 1410,DANACORP S.A,-1256920.00,TABLA_PRESUPUESTO,0.00
123,P-1709,1. INGRESO,0.00,CENTRO COMERCIAL PASEO SAN PEDRO,EBCO S.A.,105853124.47,PLANILLA PRESUPUESTO,0.48
116,P-1707,2. COSTO MATERIALES,0.00,LAS TRANQUERAS,CONSTRUCTORA SANTOLAYA LTDA.,-23066986.00,TABLA_PRESUPUESTO,0.37
54,P-1683,2. COSTO MATERIALES,0.00,CD BP-A05,EBCO S.A.,-29098536.60,TABLA_PRESUPUESTO,0.31
333,P-1729,1. INGRESO,0.00,NaN,NaN,NaN,NaN,0.00
231,P-1660,3. COSTO MANO DE OBRA,0.00,CORONEL ALVARADO,LARRAIN PRIETO RISOPATRON S.A.,-5998123.00,TABLA_PRESUPUESTO,0.47
224,P-1656,1. INGRESO,0.00,MIRADOR CLUB HÍPICO,CONSTRUCTORA VITAL EDIFICACION SPA,36463980.00,PLANILLA PRESUPUESTO,0.49
15,P-1645,2. COSTO MATERIALES,0.00,LOS TRES ANTONIOS,SOCIEDAD CONSTRUCTORA ECHAVARRI HERMANOS LIMITADA,-8503715.00,TABLA_PRESUPUESTO,0.35


In [388]:
# TESTING VISUAL 1

print(df_presupuesto_final4[df_presupuesto_final4["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_presupuesto_final4 = df_presupuesto_final4[~((df_presupuesto_final4['CODIGO_PROYECTO'] == '1. TOTAL') & df_presupuesto_final4['TOTAL'].isna())]
df_presupuesto_final4[df_presupuesto_final4["CODIGO_PROYECTO"] == "1. TOTAL"]

2049435901.133747


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,ORIGEN,MARGEN
340,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,7806687863.43,NaN,0.26
341,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-4375432891.30,NaN,0.26
342,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-1139100986.00,NaN,0.26
343,1. TOTAL,4. COSTO SUPERVISIÓN,NaN,1. TOTAL,1. TOTAL,-242718085.00,NaN,0.26


In [389]:
# TESTING VISUAL 2

print(df_presupuesto_final4[df_presupuesto_final4["CODIGO_PROYECTO"] == "P-1704"]["TOTAL"].sum())
df_presupuesto_final4 = df_presupuesto_final4[(df_presupuesto_final4['CODIGO_PROYECTO'] != '1. TOTAL') | (df_presupuesto_final4['ORIGEN'] != 'AGREGACIÓN TIPO_CUENTA VALOR O')]
df_presupuesto_final4[df_presupuesto_final4["CODIGO_PROYECTO"] == "P-1704"]

483525857.8694887


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,ORIGEN,MARGEN
106,P-1704,3. COSTO MANO DE OBRA,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-156095149.00,TABLA_PRESUPUESTO,0.46
107,P-1704,2. COSTO MATERIALES,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-387816472.00,TABLA_PRESUPUESTO,0.46
108,P-1704,1. INGRESO,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,1058718831.87,PLANILLA PRESUPUESTO,0.46
302,P-1704,4. COSTO SUPERVISIÓN,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-31281353.00,TABLA_PRESUPUESTO,0.46


In [390]:
df_presupuesto_final4.to_csv('archivos_respaldo/df_presupuesto_final4.csv', index = False)

### 4.12 MERGE PENDIENTE (PRESUPUESTO -PDT CERRADO)

#### 4.12.1 EXPLORACIÓN Y PREPARACIÓN DE DATAFRAMES

In [391]:
# Exploración de dataframe base "df_libro_mayor5_base2"

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME  : df_libro_mayor5_base2 : ", df_libro_mayor5_base2.shape)
df_libro_mayor5_base2.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME  : df_libro_mayor5_base2 :  (340, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
329,P-1726,4. COSTO SUPERVISIÓN,0,NaN,NaN
152,P-1723,1. INGRESO,0,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.
64,P-1687,1. INGRESO,0,ROJAS MAGALLANES,E.MOLINA MOREL CONSTRUCTORA S.A.
68,P-1689,2. COSTO MATERIALES,0,CASINO BUENAVENTURA,EBCO S.A.
91,P-1697,1. INGRESO,0,ESTOCOLMO,CONSTRUCTORA FORTALEZA S.A


In [392]:
# Exploración y tratamiento de dataframe de presupuesto "df_presupuesto_final4"

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME  : df_presupuesto_final4 : ", df_presupuesto_final4.shape)

df_presupuesto_final4_1 = df_presupuesto_final4.copy()

df_presupuesto_final4_1 = df_presupuesto_final4_1.pivot_table(values="TOTAL", index=['CODIGO_PROYECTO', 'TIPO_CUENTA'], aggfunc='sum')
df_presupuesto_final4_1 = pd.DataFrame(df_presupuesto_final4_1.to_records())

df_presupuesto_final4_1.rename(columns={'TOTAL': 'TOTAL_PREP'}, inplace=True)

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME  : df_presupuesto_final4_1 : ", df_presupuesto_final4_1.shape)

df_presupuesto_final4_1.sample(10)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME  : df_presupuesto_final4 :  (340, 8)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME  : df_presupuesto_final4_1 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_PREP
14,P-1592,3. COSTO MANO DE OBRA,0.00
162,P-1682,3. COSTO MANO DE OBRA,0.00
180,P-1687,1. INGRESO,25285885.04
59,P-1643,4. COSTO SUPERVISIÓN,0.00
256,P-1707,1. INGRESO,67945807.16
91,P-1660,4. COSTO SUPERVISIÓN,-1495636.00
284,P-1716,1. INGRESO,121165036.91
232,P-1700,1. INGRESO,149471447.02
285,P-1716,2. COSTO MATERIALES,-56794940.00
154,P-1680,3. COSTO MANO DE OBRA,-61607735.00


In [393]:
# Exploración y tratamiento de dataframe de presupuesto "df_ptd_cerrado3"

# Se Crea una copia del dataframe df_ptd_cerrado3 para trabajarlo 

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME  : df_ptd_cerrado3 : ", df_ptd_cerrado3.shape)

df_ptd_cerrado3_1= df_ptd_cerrado3.drop(["NOMBRE_OBRA","NOMBRE_CLIENTE","TOTAL_LB","TOTAL YTD","TOTAL LB OLD","MARGEN"], axis=1)

df_ptd_cerrado3_2 = df_ptd_cerrado3_1.pivot_table(values="TOTAL", index=['CODIGO_PROYECTO', 'TIPO_CUENTA'], aggfunc='sum')
df_ptd_cerrado3_2 = pd.DataFrame(df_ptd_cerrado3_2.to_records())

print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_1 : ", df_ptd_cerrado3_1.shape)
df_ptd_cerrado3_2.rename(columns={'TOTAL': 'TOTAL_PDT'}, inplace=True)

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME  : df_ptd_cerrado3_2 : ", df_ptd_cerrado3_2.shape)

df_ptd_cerrado3_2.sample(10)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME  : df_ptd_cerrado3 :  (340, 9)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_1 :  (340, 3)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME  : df_ptd_cerrado3_2 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_PDT
245,P-1704,2. COSTO MATERIALES,-197683461.00
232,P-1700,1. INGRESO,126038809.00
50,P-1641,3. COSTO MANO DE OBRA,0.00
220,P-1697,1. INGRESO,89702258.00
150,P-1679,3. COSTO MANO DE OBRA,-187507.00
315,P-1723,4. COSTO SUPERVISIÓN,0.00
303,P-1720,4. COSTO SUPERVISIÓN,0.00
3,1. TOTAL,4. COSTO SUPERVISIÓN,-74516421.00
59,P-1643,4. COSTO SUPERVISIÓN,0.00
214,P-1695,3. COSTO MANO DE OBRA,-13163688.00


#### 4.12.2 MERGE ENTRE DATAFRAMES 

In [394]:
# MERGE df_libro_mayor5_2 y df_presupuesto_final5_2

print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_libro_mayor5_base2      : ", df_libro_mayor5_base2.shape)
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_presupuesto_final4_1    : ", df_presupuesto_final4_1.shape)
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_2          : ", df_ptd_cerrado3_2.shape)

df_pendiente = df_libro_mayor5_base2.merge(df_presupuesto_final4_1, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pendiente               : ", df_pendiente.shape)

df_pendiente.sample(5)

DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_libro_mayor5_base2      :  (340, 5)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_presupuesto_final4_1    :  (340, 3)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_2          :  (340, 3)
DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pendiente               :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP
148,P-1721,1. INGRESO,0,STRIP CENTER LUIS PASTEUR,INMOBILIARIA LUIS PASTEUR S.A.,58149422.75
133,P-1715,3. COSTO MANO DE OBRA,0,CD LO ECHEVERS,QUEZADA Y BOETSCH,-33061305.00
177,P-1603,1. INGRESO,0,FARELLONES,EMPRESA CONSTRUCTORA DLP S.A.,0.00
11,P-1641,2. COSTO MATERIALES,0,INACAP TEMUCO,DESARROLLOS CONSTRUCTIVOS AXIS SOCIEDAD ANONIMA,-30498737.00
292,P-1694,4. COSTO SUPERVISIÓN,0,ECO HUEMUL,CONSTRUCTORA ICF SPA,0.00


In [395]:
# MERGE df_pendiente y df_ptd_cerrado3_2

print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_pendiente               : ", df_pendiente.shape)
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_2          : ", df_ptd_cerrado3_2.shape)

df_pendiente2 = df_pendiente.merge(df_ptd_cerrado3_2, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pendiente2              : ", df_pendiente2.shape)
df_pendiente2.sample(5)

DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_pendiente               :  (340, 6)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_2          :  (340, 3)
DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pendiente2              :  (340, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT
280,P-1685,4. COSTO SUPERVISIÓN,0,MAPA EDIFICIOS NO OPERACIONALES,CLARO VICUÑA VALENZUELA S.A.,-2921087.00,0.00
269,P-1680,4. COSTO SUPERVISIÓN,0,NUEVA UNO,CONSTRUCTORA TECSA S.A.,-13142983.00,0.00
256,P-1673,1. INGRESO,0,INACAP TEMUCO - ETAPA A,UNIVERSIDAD TECNOLOGICA DE CHILE INACAP,0.00,26810752.00
25,P-1664,2. COSTO MATERIALES,0,MK II PCI,COMERCIAL K SPA.,-72301755.00,-50055425.00
205,P-1642,3. COSTO MANO DE OBRA,0,CORINTO,EMP. CONSTRUCTORA INGENIEROS S.A.,-2799066.00,0.00


In [396]:
# TESTING VISUAL 1
 
df_pendiente2[df_pendiente2["CODIGO_PROYECTO"]=="P-1643"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT
13,P-1643,2. COSTO MATERIALES,0,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,-49797008.30,-433653.00
207,P-1643,1. INGRESO,0,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,0.00,0.00
208,P-1643,3. COSTO MANO DE OBRA,0,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,0.00,0.00
209,P-1643,4. COSTO SUPERVISIÓN,0,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,0.00,0.00


In [397]:
# TESTING VISUAL 2
 
df_pendiente2[df_pendiente2["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-242718085.00,-74516421.00
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,7806687863.43,4503192575.00
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-4375432891.30,-1686220937.00
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-1139100986.00,-1076409161.00


#### 4.12.3 CALCULO DE COLUMNA TOTAL

El objetivo de este Dataframe es lograr identificar la diferencia entre el el presupuesto y el PDT Cerrado.

In [398]:
# Modificar los valores negativos a positivos
df_pendiente2['TOTAL_PREP2'] = np.where(df_pendiente2['TOTAL_PREP'] < 0, df_pendiente2['TOTAL_PREP'] * -1, df_pendiente2['TOTAL_PREP'])

df_pendiente2['TOTAL_PDT2'] = np.where(df_pendiente2['TOTAL_PDT'] < 0, df_pendiente2['TOTAL_PDT'] * -1, df_pendiente2['TOTAL_PDT'])

# Calcular la columna "TOTAL" con los valores positivos
df_pendiente2['TOTAL_1'] = df_pendiente2['TOTAL_PREP2'] - df_pendiente2['TOTAL_PDT2']

df_pendiente2.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT,TOTAL_PREP2,TOTAL_PDT2,TOTAL_1
115,P-1707,3. COSTO MANO DE OBRA,0,LAS TRANQUERAS,CONSTRUCTORA SANTOLAYA LTDA.,-15679431.00,-15597431.00,15679431.00,15597431.00,82000.00
240,P-1664,3. COSTO MANO DE OBRA,0,MK II PCI,COMERCIAL K SPA.,-343022.00,0.00,343022.00,0.00,343022.00
303,P-1705,4. COSTO SUPERVISIÓN,0,LOS CLARINES,CONSTRUCTORA INARCO S.A.,-3523114.00,0.00,3523114.00,0.00,3523114.00
140,P-1718,3. COSTO MANO DE OBRA,0,PATIO COMIDAS,EBCO S.A.,0.00,-35067373.00,0.00,35067373.00,-35067373.00
335,P-1729,4. COSTO SUPERVISIÓN,0,NaN,NaN,-6254611.00,0.00,6254611.00,0.00,6254611.00


In [399]:
# TESTING VISUAL 1
 
df_pendiente2[df_pendiente2["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT,TOTAL_PREP2,TOTAL_PDT2,TOTAL_1
106,P-1704,3. COSTO MANO DE OBRA,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-156095149.00,-145534854.00,156095149.00,145534854.00,10560295.00
107,P-1704,2. COSTO MATERIALES,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-387816472.00,-197683461.00,387816472.00,197683461.00,190133011.00
108,P-1704,1. INGRESO,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,1058718831.87,686431807.00,1058718831.87,686431807.00,372287024.87
302,P-1704,4. COSTO SUPERVISIÓN,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-31281353.00,0.00,31281353.00,0.00,31281353.00


In [400]:
# Se realiza estandarización de columna TOTAL considerando que los Tipos de cuenta de Costos deben estar en valor negativo.

# Crear nueva columna 'TOTAL' con el valor de 'TOTAL_PREV'
df_pendiente2['TOTAL'] = df_pendiente2['TOTAL_1']

# Aplicar condiciones para cambiar el valor a negativo
mask = (df_pendiente2['TIPO_CUENTA'].isin(['2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN'])) & (df_pendiente2['TOTAL_1'] != 0)
df_pendiente2.loc[mask, 'TOTAL'] *= -1

# Mantener el valor de 'TOTAL_PREV' como negativo si ya es negativo
df_pendiente2.loc[df_pendiente2['TOTAL_1'] < 0, 'TOTAL'] *= -1

df_pendiente2.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT,TOTAL_PREP2,TOTAL_PDT2,TOTAL_1,TOTAL
83,P-1695,3. COSTO MANO DE OBRA,0,EDIFICIO ESTORIL,EMPRESA CONSTRUCTORA BRAVO E IZQUIERDO,-19577383.00,-13163688.00,19577383.00,13163688.00,6413695.00,-6413695.00
142,P-1718,1. INGRESO,0,PATIO COMIDAS,EBCO S.A.,184014731.01,123171387.00,184014731.01,123171387.00,60843344.01,60843344.01
322,P-1721,4. COSTO SUPERVISIÓN,0,STRIP CENTER LUIS PASTEUR,INMOBILIARIA LUIS PASTEUR S.A.,0.00,0.00,0.00,0.00,0.00,0.00
19,P-1653,2. COSTO MATERIALES,0,EDUARDO DE LA BARRA,CONSTRUCTORA E INVERSIONES VITAL SPA.,-10765426.00,-5673570.00,10765426.00,5673570.00,5091856.00,-5091856.00
154,P-1724,2. COSTO MATERIALES,0,NaN,NaN,-9891221.00,-7660814.00,9891221.00,7660814.00,2230407.00,-2230407.00
300,P-1701,4. COSTO SUPERVISIÓN,0,EDIFICIO ZENTENO,CONSTRUCTORA GREVIA SPA.,-3113372.00,0.00,3113372.00,0.00,3113372.00,-3113372.00
280,P-1685,4. COSTO SUPERVISIÓN,0,MAPA EDIFICIOS NO OPERACIONALES,CLARO VICUÑA VALENZUELA S.A.,-2921087.00,0.00,2921087.00,0.00,2921087.00,-2921087.00
11,P-1641,2. COSTO MATERIALES,0,INACAP TEMUCO,DESARROLLOS CONSTRUCTIVOS AXIS SOCIEDAD ANONIMA,-30498737.00,-1038770.00,30498737.00,1038770.00,29459967.00,-29459967.00
31,P-1669,2. COSTO MATERIALES,0,DEPARTAMENTAL,EMP. CONSTRUCTORA INGENIEROS S.A.,-32905472.00,-4648895.00,32905472.00,4648895.00,28256577.00,-28256577.00
153,P-1724,3. COSTO MANO DE OBRA,0,NaN,NaN,0.00,-506066.00,0.00,506066.00,-506066.00,-506066.00


In [401]:
df_pendiente3 = df_pendiente2.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_pendiente3.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)


print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_pendiente3 : ", df_pendiente3.shape)

df_pendiente3 = pd.merge(df_pendiente3, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_pendiente3 : ", df_pendiente3.shape)
df_pendiente3.sample(5)


DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_pendiente3 :  (340, 11)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_pendiente3 :  (340, 12)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT,TOTAL_PREP2,TOTAL_PDT2,TOTAL_1,TOTAL,MARGEN
120,P-1708,1. INGRESO,0,ECO ESPAÑA,CONSTRUCTORA ICF SPA,213452539.86,392191.00,213452539.86,392191.00,213060348.86,213060348.86,0.36
273,P-1682,4. COSTO SUPERVISIÓN,0,PLAZA CEPPI,EMP. CONSTRUCTORA INGENIEROS S.A.,0.00,0.00,0.00,0.00,0.00,0.00,-5.72
128,P-1713,2. COSTO MATERIALES,0,RESERVA SAN FRANCISCO,EMPRESA NACIONAL DE CONSTRUCCIÓN LAS VERTIENTE...,-55849894.00,-2954283.00,55849894.00,2954283.00,52895611.00,-52895611.00,0.40
10,P-1639,2. COSTO MATERIALES,0,LYON 2546,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...,-18254050.00,-11381381.00,18254050.00,11381381.00,6872669.00,-6872669.00,0.41
84,P-1695,2. COSTO MATERIALES,0,EDIFICIO ESTORIL,EMPRESA CONSTRUCTORA BRAVO E IZQUIERDO,-41184361.00,-25647031.00,41184361.00,25647031.00,15537330.00,-15537330.00,0.48


In [402]:
# TESTING VISUAL 1
 
df_pendiente3[df_pendiente3["CODIGO_PROYECTO"]=="P-1727"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT,TOTAL_PREP2,TOTAL_PDT2,TOTAL_1,TOTAL,MARGEN
162,P-1727,3. COSTO MANO DE OBRA,0,NaN,NaN,0.00,-8169893.00,0.00,8169893.00,-8169893.00,-8169893.00,0.37
163,P-1727,2. COSTO MATERIALES,0,NaN,NaN,-92610838.00,-13655419.00,92610838.00,13655419.00,78955419.00,-78955419.00,0.37
164,P-1727,1. INGRESO,0,NaN,NaN,166727122.73,28570952.00,166727122.73,28570952.00,138156170.73,138156170.73,0.37
330,P-1727,4. COSTO SUPERVISIÓN,0,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.37


In [403]:
# TESTING VISUAL 2
 
df_pendiente3[df_pendiente3["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT,TOTAL_PREP2,TOTAL_PDT2,TOTAL_1,TOTAL,MARGEN
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-242718085.00,-74516421.00,242718085.00,74516421.00,168201664.00,-168201664.00,0.12
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,7806687863.43,4503192575.00,7806687863.43,4503192575.00,3303495288.43,3303495288.43,0.12
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-4375432891.30,-1686220937.00,4375432891.30,1686220937.00,2689211954.30,-2689211954.30,0.12
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-1139100986.00,-1076409161.00,1139100986.00,1076409161.00,62691825.00,-62691825.00,0.12


In [404]:
df_pendiente3.to_csv('archivos_respaldo/df_pendiente3.csv', index = False)

### 4.13 MERGE PENDIENTE (PRESUPUESTO -PDT CERRADO) (PERSPECTIVA MES ANTERIOR)

#### 4.13.1 EXPLORACIÓN Y PREPARACIÓN DE DATAFRAMES

In [405]:
# Exploración de dataframe base "df_libro_mayor5_base2"

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME  : df_libro_mayor5_base2 : ", df_libro_mayor5_base2.shape)
df_libro_mayor5_base2.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME  : df_libro_mayor5_base2 :  (340, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
304,P-1706,4. COSTO SUPERVISIÓN,0,CALIFORNIA,CONSTRUCTORA E INVERSIONES VITAL SPA.
45,P-1679,3. COSTO MANO DE OBRA,0,JUAN SABAJ,CONSTRUCTORA GREVIA
222,P-1653,3. COSTO MANO DE OBRA,0,EDUARDO DE LA BARRA,CONSTRUCTORA E INVERSIONES VITAL SPA.
99,P-1700,1. INGRESO,0,CD ORIZON,EBCO S.A.
327,P-1724,4. COSTO SUPERVISIÓN,0,NaN,NaN


In [406]:
# Exploración y tratamiento de dataframe de presupuesto "df_presupuesto_final4"

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME  : df_presupuesto_final4 : ", df_presupuesto_final4.shape)

df_presupuesto_final4_1_1 = df_presupuesto_final4.copy()

df_presupuesto_final4_1_1 = df_presupuesto_final4_1_1.pivot_table(values="TOTAL", index=['CODIGO_PROYECTO', 'TIPO_CUENTA'], aggfunc='sum')
df_presupuesto_final4_1_1 = pd.DataFrame(df_presupuesto_final4_1_1.to_records())

df_presupuesto_final4_1_1.rename(columns={'TOTAL': 'TOTAL_PREP'}, inplace=True)

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME  : df_presupuesto_final4_1_1 : ", df_presupuesto_final4_1_1.shape)

df_presupuesto_final4_1_1.sample(10)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME  : df_presupuesto_final4 :  (340, 8)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME  : df_presupuesto_final4_1_1 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_PREP
43,P-1636,4. COSTO SUPERVISIÓN,-1256920.00
79,P-1653,4. COSTO SUPERVISIÓN,-773943.00
87,P-1658,4. COSTO SUPERVISIÓN,-1908953.00
109,P-1667,2. COSTO MATERIALES,-36944607.00
52,P-1642,1. INGRESO,0.00
147,P-1677,4. COSTO SUPERVISIÓN,-4657579.00
61,P-1644,2. COSTO MATERIALES,-32945669.00
290,P-1717,3. COSTO MANO DE OBRA,0.00
157,P-1681,2. COSTO MATERIALES,-4132408.00
71,P-1648,4. COSTO SUPERVISIÓN,-8171095.00


In [407]:
# Exploración y tratamiento de dataframe de presupuesto "df_ptd_cerrado3"

# Se Crea una copia del dataframe df_ptd_cerrado3 para trabajarlo 

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME  : df_ptd_cerrado_anterior3 : ", df_ptd_cerrado_anterior3.shape)

df_ptd_cerrado3_1_1 = df_ptd_cerrado_anterior3.drop(["NOMBRE_OBRA","NOMBRE_CLIENTE","TOTAL_LB","TOTAL YTD","TOTAL LB OLD","MARGEN"], axis=1)

df_ptd_cerrado3_1_2 = df_ptd_cerrado3_1_1.pivot_table(values="TOTAL", index=['CODIGO_PROYECTO', 'TIPO_CUENTA'], aggfunc='sum')
df_ptd_cerrado3_1_2 = pd.DataFrame(df_ptd_cerrado3_1_2.to_records())

print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_1_2 : ", df_ptd_cerrado3_1_2.shape)
df_ptd_cerrado3_1_2.rename(columns={'TOTAL': 'TOTAL_PDT'}, inplace=True)

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME  : df_ptd_cerrado3_1_2 : ", df_ptd_cerrado3_1_2.shape)

df_ptd_cerrado3_1_2.sample(10)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME  : df_ptd_cerrado_anterior3 :  (340, 9)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_1_2 :  (340, 3)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME  : df_ptd_cerrado3_1_2 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_PDT
229,P-1699,2. COSTO MATERIALES,-22580828.00
147,P-1677,4. COSTO SUPERVISIÓN,0.00
96,P-1663,1. INGRESO,0.00
328,P-1727,1. INGRESO,11005000.00
43,P-1636,4. COSTO SUPERVISIÓN,0.00
38,P-1632,3. COSTO MANO DE OBRA,0.00
95,P-1661,4. COSTO SUPERVISIÓN,0.00
173,P-1685,2. COSTO MATERIALES,-16463005.00
122,P-1670,3. COSTO MANO DE OBRA,-431689.00
123,P-1670,4. COSTO SUPERVISIÓN,0.00


#### 4.13.2 MERGE ENTRE DATAFRAMES 

In [408]:
# MERGE df_libro_mayor5_2 y df_presupuesto_final5_2

print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_libro_mayor5_base2      : ", df_libro_mayor5_base2.shape)
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_presupuesto_final4_1_1  : ", df_presupuesto_final4_1_1.shape)
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_1_2        : ", df_ptd_cerrado3_1_2.shape)

df_pendiente_2 = df_libro_mayor5_base2.merge(df_presupuesto_final4_1_1, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pendiente_2             : ", df_pendiente_2.shape)

df_pendiente_2.sample(5)

DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_libro_mayor5_base2      :  (340, 5)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_presupuesto_final4_1_1  :  (340, 3)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_1_2        :  (340, 3)
DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pendiente_2             :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP
172,P-1591,2. COSTO MATERIALES,0,EDIFICIO ESTORIL,EBCO S.A.,0.00
271,P-1681,3. COSTO MANO DE OBRA,0,CD ROSEN (CCTV PROVISORIO),COLCHONES ROSEN SAIC,-2136024.00
333,P-1729,1. INGRESO,0,NaN,NaN,0.00
77,P-1692,1. INGRESO,0,JOSÉ PEREZ COTAPOS,EBCO S.A.,91720000.00
229,P-1658,4. COSTO SUPERVISIÓN,0,VIVIPRA,VICHERAT Y PRADENAS LTDA.,-1908953.00


In [409]:
# MERGE df_pendiente y df_ptd_cerrado3_2

print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_pendiente_2             : ", df_pendiente_2.shape)
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_1_2        : ", df_ptd_cerrado3_1_2.shape)

df_pendiente_2 = df_pendiente_2.merge(df_ptd_cerrado3_1_2, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pendiente_2              : ", df_pendiente_2.shape)
df_pendiente_2.sample(5)

DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_pendiente_2             :  (340, 6)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_1_2        :  (340, 3)
DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pendiente_2              :  (340, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT
311,P-1715,4. COSTO SUPERVISIÓN,0,CD LO ECHEVERS,QUEZADA Y BOETSCH,-7424606.00,0.00
181,P-1611,3. COSTO MANO DE OBRA,0,EDIFICIO PLAZA EGAÑA,EBCO S.A.,-11033925.00,0.00
189,P-1626,1. INGRESO,0,CLINICA BIO BIO,CONSTRUCTORA LYD S.A.,0.00,75533.00
32,P-1670,2. COSTO MATERIALES,0,MK II SEGURIDAD Y REDES,COMERCIAL K SPA.,-77955381.00,-56742175.00
142,P-1718,1. INGRESO,0,PATIO COMIDAS,EBCO S.A.,184014731.01,104494756.00


In [410]:
# TESTING VISUAL 1
 
df_pendiente_2[df_pendiente_2["CODIGO_PROYECTO"]=="P-1643"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT
13,P-1643,2. COSTO MATERIALES,0,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,-49797008.30,-102685.00
207,P-1643,1. INGRESO,0,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,0.00,0.00
208,P-1643,3. COSTO MANO DE OBRA,0,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,0.00,0.00
209,P-1643,4. COSTO SUPERVISIÓN,0,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,0.00,0.00


#### 4.13.3 CALCULO DE COLUMNA TOTAL

El objetivo de este Dataframe es lograr identificar la diferencia entre el el presupuesto y el PDT Cerrado.

In [411]:
# Modificar los valores negativos a positivos
df_pendiente_2['TOTAL_PREP2'] = np.where(df_pendiente_2['TOTAL_PREP'] < 0, df_pendiente_2['TOTAL_PREP'] * -1, df_pendiente_2['TOTAL_PREP'])

df_pendiente_2['TOTAL_PDT2'] = np.where(df_pendiente_2['TOTAL_PDT'] < 0, df_pendiente_2['TOTAL_PDT'] * -1, df_pendiente_2['TOTAL_PDT'])

# Calcular la columna "TOTAL" con los valores positivos
df_pendiente_2['TOTAL_1'] = df_pendiente_2['TOTAL_PREP2'] - df_pendiente_2['TOTAL_PDT2']

df_pendiente_2.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT,TOTAL_PREP2,TOTAL_PDT2,TOTAL_1
296,P-1698,3. COSTO MANO DE OBRA,0,CD ANDES,EBCO S.A.,-6815995.00,-19007691.00,6815995.00,19007691.00,-12191696.00
25,P-1664,2. COSTO MATERIALES,0,MK II PCI,COMERCIAL K SPA.,-72301755.00,-50055425.00,72301755.00,50055425.00,22246330.00
239,P-1664,1. INGRESO,0,MK II PCI,COMERCIAL K SPA.,0.00,138414890.00,0.00,138414890.00,-138414890.00
43,P-1677,2. COSTO MATERIALES,0,GALVARINO GALLARDO,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...,-35515426.00,-32468202.00,35515426.00,32468202.00,3047224.00
13,P-1643,2. COSTO MATERIALES,0,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,-49797008.30,-102685.00,49797008.30,102685.00,49694323.30


In [412]:
# TESTING VISUAL 1
 
df_pendiente_2[df_pendiente_2["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT,TOTAL_PREP2,TOTAL_PDT2,TOTAL_1
106,P-1704,3. COSTO MANO DE OBRA,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-156095149.00,-132643971.00,156095149.00,132643971.00,23451178.00
107,P-1704,2. COSTO MATERIALES,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-387816472.00,-183715910.00,387816472.00,183715910.00,204100562.00
108,P-1704,1. INGRESO,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,1058718831.87,607371810.00,1058718831.87,607371810.00,451347021.87
302,P-1704,4. COSTO SUPERVISIÓN,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-31281353.00,0.00,31281353.00,0.00,31281353.00


In [413]:
# Se realiza estandarización de columna TOTAL considerando que los Tipos de cuenta de Costos deben estar en valor negativo.

# Crear nueva columna 'TOTAL' con el valor de 'TOTAL_PREV'
df_pendiente_2['TOTAL'] = df_pendiente_2['TOTAL_1']

# Aplicar condiciones para cambiar el valor a negativo
mask = (df_pendiente_2['TIPO_CUENTA'].isin(['2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN'])) & (df_pendiente_2['TOTAL_1'] != 0)
df_pendiente_2.loc[mask, 'TOTAL'] *= -1

# Mantener el valor de 'TOTAL_PREV' como negativo si ya es negativo
df_pendiente_2.loc[df_pendiente_2['TOTAL_1'] < 0, 'TOTAL'] *= -1

df_pendiente_2.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT,TOTAL_PREP2,TOTAL_PDT2,TOTAL_1,TOTAL
25,P-1664,2. COSTO MATERIALES,0,MK II PCI,COMERCIAL K SPA.,-72301755.00,-50055425.00,72301755.00,50055425.00,22246330.00,-22246330.00
44,P-1677,1. INGRESO,0,GALVARINO GALLARDO,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...,83882285.07,90155711.00,83882285.07,90155711.00,-6273425.93,6273425.93
289,P-1692,4. COSTO SUPERVISIÓN,0,JOSÉ PEREZ COTAPOS,EBCO S.A.,-5768030.00,0.00,5768030.00,0.00,5768030.00,-5768030.00
235,P-1661,4. COSTO SUPERVISIÓN,0,SAN VICENTE DE PAUL,DANACORP S.A,-2015626.00,0.00,2015626.00,0.00,2015626.00,-2015626.00
105,P-1703,1. INGRESO,0,SANTA VICTORIA DANACORP,DANACORP S.A,56316406.63,51991271.00,56316406.63,51991271.00,4325135.63,4325135.63
256,P-1673,1. INGRESO,0,INACAP TEMUCO - ETAPA A,UNIVERSIDAD TECNOLOGICA DE CHILE INACAP,0.00,26810752.00,0.00,26810752.00,-26810752.00,26810752.00
27,P-1667,3. COSTO MANO DE OBRA,0,ECO PLACILLA,CONSTRUCTORA ICF SPA,-16552739.00,-37223140.00,16552739.00,37223140.00,-20670401.00,-20670401.00
96,P-1699,1. INGRESO,0,VILLA PANAMERICANA LOTE B,EMPRESA CONSTRUCTORA DLP S.A.,406744605.61,73159443.00,406744605.61,73159443.00,333585162.61,333585162.61
175,P-1592,3. COSTO MANO DE OBRA,0,NUEVA COSTANERA 3300,EBCO S.A.,0.00,0.00,0.00,0.00,0.00,0.00
165,P-1728,3. COSTO MANO DE OBRA,0,NaN,NaN,0.00,-630869.00,0.00,630869.00,-630869.00,-630869.00


In [414]:
df_pendiente_3 = df_pendiente_2.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_pendiente_3.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)


print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_pendiente_3 : ", df_pendiente_3.shape)

df_pendiente_3 = pd.merge(df_pendiente_3, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_pendiente_3 : ", df_pendiente_3.shape)
df_pendiente_3.sample(5)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_pendiente_3 :  (340, 11)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_pendiente_3 :  (340, 12)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT,TOTAL_PREP2,TOTAL_PDT2,TOTAL_1,TOTAL,MARGEN
203,P-1641,4. COSTO SUPERVISIÓN,0,INACAP TEMUCO,DESARROLLOS CONSTRUCTIVOS AXIS SOCIEDAD ANONIMA,-657160.00,0.00,657160.00,0.00,657160.00,-657160.00,0.00
305,P-1707,4. COSTO SUPERVISIÓN,0,LAS TRANQUERAS,CONSTRUCTORA SANTOLAYA LTDA.,-3909676.00,0.00,3909676.00,0.00,3909676.00,-3909676.00,0.46
209,P-1643,4. COSTO SUPERVISIÓN,0,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,0.00,0.00,0.00,0.00,0.00,0.00,0.00
167,P-1729,2. COSTO MATERIALES,0,NaN,NaN,-51223238.00,0.00,51223238.00,0.00,51223238.00,-51223238.00,0.00
180,P-1611,1. INGRESO,0,EDIFICIO PLAZA EGAÑA,EBCO S.A.,0.00,4173520.00,0.00,4173520.00,-4173520.00,4173520.00,-7.83


In [415]:
# TESTING VISUAL 1
 
df_pendiente_3[df_pendiente_3["CODIGO_PROYECTO"]=="P-1727"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT,TOTAL_PREP2,TOTAL_PDT2,TOTAL_1,TOTAL,MARGEN
162,P-1727,3. COSTO MANO DE OBRA,0,NaN,NaN,0.00,-4714680.00,0.00,4714680.00,-4714680.00,-4714680.00,0.40
163,P-1727,2. COSTO MATERIALES,0,NaN,NaN,-92610838.00,-3753865.00,92610838.00,3753865.00,88856973.00,-88856973.00,0.40
164,P-1727,1. INGRESO,0,NaN,NaN,166727122.73,11005000.00,166727122.73,11005000.00,155722122.73,155722122.73,0.40
330,P-1727,4. COSTO SUPERVISIÓN,0,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.40


In [416]:
# TESTING VISUAL 2
 
df_pendiente_3[df_pendiente_3["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT,TOTAL_PREP2,TOTAL_PDT2,TOTAL_1,TOTAL,MARGEN
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-242718085.00,-69873160.00,242718085.00,69873160.00,172844925.00,-172844925.00,0.15
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,7806687863.43,4042394062.00,7806687863.43,4042394062.00,3764293801.43,3764293801.43,0.15
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-4375432891.30,-1516685549.00,4375432891.30,1516685549.00,2858747342.30,-2858747342.30,0.15
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-1139100986.00,-962035446.00,1139100986.00,962035446.00,177065540.00,-177065540.00,0.15


In [417]:
df_pendiente_3.to_csv('archivos_respaldo/df_pendiente_3.csv', index = False)

### 4.14 MERGE PDT CERRADO VS (PDT + PROYECTADO)

#### 4.14.1 EXPLORACIÓN Y PREPARACIÓN DE DATAFRAMES

In [418]:
# Exploración de dataframe df_pdt_cerrado

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado3 : ", df_ptd_cerrado3.shape)

df_ptd_cerrado3_2 = df_ptd_cerrado3.copy()

df_ptd_cerrado3_2.rename(columns={'TOTAL': 'TOTAL_PDT'}, inplace=True)
df_ptd_cerrado3_2.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado3 :  (340, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL_PDT,MARGEN
298,P-1699,4. COSTO SUPERVISIÓN,0.00,VILLA PANAMERICANA LOTE B,EMPRESA CONSTRUCTORA DLP S.A.,0.00,0.00,0.00,0.43
317,P-1719,2. COSTO MATERIALES,0.00,CD METALPAR,EBCO S.A.,0.00,0.00,0.00,1.00
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-22566666.00,-51949755.00,-74516421.00,0.37
223,P-1653,4. COSTO SUPERVISIÓN,0.00,EDUARDO DE LA BARRA,CONSTRUCTORA E INVERSIONES VITAL SPA.,0.00,0.00,0.00,0.59
305,P-1707,4. COSTO SUPERVISIÓN,0.00,LAS TRANQUERAS,CONSTRUCTORA SANTOLAYA LTDA.,0.00,0.00,0.00,0.15


In [419]:
df_ptd_cerrado_y_proyectado2.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL,MARGEN
62,P-1687,3. COSTO MANO DE OBRA,ROJAS MAGALLANES,E.MOLINA MOREL CONSTRUCTORA S.A.,-7395436.00,0,-7395436.00,0.31
0,P-1557,3. COSTO MANO DE OBRA,ROSARIO ROSALES,EBCO S.A.,-1247014.00,-301065,-1548079.00,0.00
210,P-1644,1. INGRESO,MIRADOR DE SAN CRISTOBAL,EBCO S.A.,16864978.00,0,16864978.00,0.71
138,P-1716,1. INGRESO,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,125927056.00,0,125927056.00,0.53
258,P-1673,4. COSTO SUPERVISIÓN,INACAP TEMUCO - ETAPA A,UNIVERSIDAD TECNOLOGICA DE CHILE INACAP,0.00,0,0.00,0.56


In [420]:
# Exploración de dataframe presupuesto

df_ptd_cerrado_y_proyectado2_1 = df_ptd_cerrado_y_proyectado2.copy()
df_ptd_cerrado_y_proyectado2_1.rename(columns={'TOTAL': 'TOTAL_PDT_PROY'}, inplace=True)

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado2_1 : ", df_ptd_cerrado_y_proyectado2_1.shape)

df_ptd_cerrado_y_proyectado2_1 = pd.pivot_table(df_ptd_cerrado_y_proyectado2_1, values=['TOTAL_PDT_PROY'], index=['CODIGO_PROYECTO','TIPO_CUENTA'], aggfunc='sum', fill_value=0)
df_ptd_cerrado_y_proyectado2_1 = pd.DataFrame(df_ptd_cerrado_y_proyectado2_1.to_records())

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado2_1 : ", df_ptd_cerrado_y_proyectado2_1.shape)

df_ptd_cerrado_y_proyectado2_1.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado2_1 :  (340, 8)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado2_1 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_PDT_PROY
225,P-1698,2. COSTO MATERIALES,-25887377
77,P-1653,2. COSTO MATERIALES,-5673570
284,P-1716,1. INGRESO,125927056
238,P-1701,3. COSTO MANO DE OBRA,-25556198
22,P-1611,3. COSTO MANO DE OBRA,0


#### 4.14.2 MERGE ENTRE DATAFRAMES

In [421]:
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_2                  : ", df_ptd_cerrado3_2.shape)
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado2_1     : ", df_ptd_cerrado_y_proyectado2_1.shape)

df_pdt_vs_proy = df_ptd_cerrado3_2.merge(df_ptd_cerrado_y_proyectado2_1, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pdt_vs_proy           : ", df_pdt_vs_proy.shape)

df_pdt_vs_proy.sample(5)

DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_2                  :  (340, 9)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado2_1     :  (340, 3)
DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pdt_vs_proy           :  (340, 10)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL_PDT,MARGEN,TOTAL_PDT_PROY
258,P-1673,4. COSTO SUPERVISIÓN,0.00,INACAP TEMUCO - ETAPA A,UNIVERSIDAD TECNOLOGICA DE CHILE INACAP,0.00,0.00,0.00,0.56,0
94,P-1699,3. COSTO MANO DE OBRA,0.00,VILLA PANAMERICANA LOTE B,EMPRESA CONSTRUCTORA DLP S.A.,-21466214.00,-1005045.00,-22471259.00,0.43,-25637827
286,P-1689,4. COSTO SUPERVISIÓN,0.00,CASINO BUENAVENTURA,EBCO S.A.,0.00,0.00,0.00,0.51,0
35,P-1672,1. INGRESO,0.00,STA CLARA (CCDD-TELECOM),CONSTRUCTORA UPC S.A.,9435519.00,125091722.00,134527241.00,0.31,134527241
183,P-1612,1. INGRESO,0.00,BARRIO PARQUE SANTIAGO 7,CONSTRUCTORA ALTURAS DEL SUR S.A,0.00,0.00,0.00,0.00,0


#### 4.14.3 CREACIÓN DE COLUMNA % DE AVANCE

In [422]:
df_pdt_vs_proy['% AVANCE'] = df_pdt_vs_proy['TOTAL_PDT'] / df_pdt_vs_proy['TOTAL_PDT_PROY']

# Reemplazar NaN por cero
df_pdt_vs_proy['% AVANCE'] = df_pdt_vs_proy['% AVANCE'].fillna(0)

# Reemplazar -inf por cero
df_pdt_vs_proy['% AVANCE'].replace(-np.inf, 0, inplace=True)

# Reemplazar inf por cero
df_pdt_vs_proy['% AVANCE'] = df_pdt_vs_proy['% AVANCE'].replace(np.inf, 0)

df_pdt_vs_proy.head(10)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL_PDT,MARGEN,TOTAL_PDT_PROY,% AVANCE
0,P-1557,3. COSTO MANO DE OBRA,0.00,ROSARIO ROSALES,EBCO S.A.,-1247014.00,0.00,-1247014.00,0.00,-1548079,0.81
1,P-1591,3. COSTO MANO DE OBRA,0.00,EDIFICIO ESTORIL,EBCO S.A.,-4593960.00,0.00,-4593960.00,0.00,-6703922,0.69
2,P-1592,2. COSTO MATERIALES,0.00,NUEVA COSTANERA 3300,EBCO S.A.,41716.00,856004.00,897720.00,1.05,897720,1.00
3,P-1603,2. COSTO MATERIALES,0.00,FARELLONES,EMPRESA CONSTRUCTORA DLP S.A.,-30000.00,0.00,-30000.00,0.00,-30000,1.00
4,P-1611,2. COSTO MATERIALES,0.00,EDIFICIO PLAZA EGAÑA,EBCO S.A.,6648.00,596177.00,602825.00,1.14,602825,1.00
5,P-1612,2. COSTO MATERIALES,0.00,BARRIO PARQUE SANTIAGO 7,CONSTRUCTORA ALTURAS DEL SUR S.A,30258.00,280644.00,310902.00,0.00,310902,1.00
6,P-1620,2. COSTO MATERIALES,0.00,"OF APOQUINDO- HABILITACIONES, INTEGRACION Y AD...",CONSTRUCTORA INARCO S.A.,-16400.00,0.00,-16400.00,0.00,-16400,1.00
7,P-1626,2. COSTO MATERIALES,0.00,CLINICA BIO BIO,CONSTRUCTORA LYD S.A.,3460.00,678564.00,682024.00,10.03,682024,1.00
8,P-1632,2. COSTO MATERIALES,0.00,CHORRILLOS CLOROX,CONSTRUCTORA SUKSA S.A.,75965.00,236593.00,312558.00,0.00,312558,1.00
9,P-1636,2. COSTO MATERIALES,0.00,SANTO DOMINGO 1410,DANACORP S.A,0.00,-455187.00,-455187.00,0.77,-455187,1.00


In [423]:
# TESTING VISUAL 1
 
df_pdt_vs_proy[df_pdt_vs_proy["CODIGO_PROYECTO"]=="P-1727"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL_PDT,MARGEN,TOTAL_PDT_PROY,% AVANCE
162,P-1727,3. COSTO MANO DE OBRA,0.00,NaN,NaN,-8169893.00,0.00,-8169893.00,0.24,-11625106,0.70
163,P-1727,2. COSTO MATERIALES,0.00,NaN,NaN,-13655419.00,0.00,-13655419.00,0.24,-23655419,0.58
164,P-1727,1. INGRESO,0.00,NaN,NaN,28570952.00,0.00,28570952.00,0.24,55070952,0.52
330,P-1727,4. COSTO SUPERVISIÓN,0.00,NaN,NaN,0.00,0.00,0.00,0.24,0,0.00


In [424]:
# TESTING VISUAL 2
 
df_pdt_vs_proy[df_pdt_vs_proy["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL_PDT,MARGEN,TOTAL_PDT_PROY,% AVANCE
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-22566666.00,-51949755.00,-74516421.00,0.37,-79159682,0.94
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,NaN,NaN,4503192575.00,0.37,4926992575,0.91
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-1686220937.00,0.37,-1862756743,0.91
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-1076409161.00,0.37,-1190782876,0.90


In [425]:
df_pdt_vs_proy.to_csv('archivos_respaldo/df_pdt_vs_proy.csv', index = False)

### 4.15 MERGE PDT CERRADO VS (PDT + PROYECTADO) PERSPECTIVA MES ANTERIOR

#### 4.15.1 EXPLORACIÓN Y PREPARACIÓN DE DATAFRAMES

In [426]:
# Exploración de dataframe df_pdt_cerrado

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado3 : ", df_ptd_cerrado3.shape)

df_ptd_cerrado3_2_2 = df_ptd_cerrado_anterior3.copy()

df_ptd_cerrado3_2_2.rename(columns={'TOTAL': 'TOTAL_PDT'}, inplace=True)
df_ptd_cerrado3_2_2.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado3 :  (340, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL_PDT,MARGEN
316,P-1718,4. COSTO SUPERVISIÓN,0.00,PATIO COMIDAS,EBCO S.A.,0.00,0.00,0.00,0.55
184,P-1612,3. COSTO MANO DE OBRA,0.00,BARRIO PARQUE SANTIAGO 7,CONSTRUCTORA ALTURAS DEL SUR S.A,0.00,0.00,0.00,0.00
203,P-1641,4. COSTO SUPERVISIÓN,0.00,INACAP TEMUCO,DESARROLLOS CONSTRUCTIVOS AXIS SOCIEDAD ANONIMA,0.00,0.00,0.00,0.00
74,P-1691,1. INGRESO,0.00,ROSARIO ROSALES,CONSTRUCTORA E INMOBILIARIA PRADOVERDE LTDA.,2124157.00,39478959.00,41603116.00,0.16
158,P-1725,1. INGRESO,0.00,LO CAÑAS,CONSTRUCTORA BRISAS DE BATUCO,8921780.00,0.00,8921780.00,0.23


In [427]:
df_ptd_cerrado_y_proyectado_ant2.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL,MARGEN
303,P-1705,4. COSTO SUPERVISIÓN,LOS CLARINES,CONSTRUCTORA INARCO S.A.,0.00,0.00,0.00,0.05
36,P-1673,2. COSTO MATERIALES,INACAP TEMUCO - ETAPA A,UNIVERSIDAD TECNOLOGICA DE CHILE INACAP,-11666275.00,0.00,-11666275.00,0.56
205,P-1642,3. COSTO MANO DE OBRA,CORINTO,EMP. CONSTRUCTORA INGENIEROS S.A.,0.00,0.00,0.00,0.57
160,P-1726,2. COSTO MATERIALES,NaN,NaN,-14374046.00,-10000000.00,-24374046.00,0.36
336,1. TOTAL,4. COSTO SUPERVISIÓN,1. TOTAL,1. TOTAL,-69873160.00,-5096121.00,-74969281.00,0.36


In [428]:
# Exploración de dataframe presupuesto

df_ptd_cerrado_y_proyectado_ant2_2 = df_ptd_cerrado_y_proyectado_ant2.copy()
df_ptd_cerrado_y_proyectado_ant2_2.rename(columns={'TOTAL': 'TOTAL_PDT_PROY'}, inplace=True)

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant2_2 : ", df_ptd_cerrado_y_proyectado_ant2_2.shape)

df_ptd_cerrado_y_proyectado_ant2_2 = pd.pivot_table(df_ptd_cerrado_y_proyectado_ant2_2, values=['TOTAL_PDT_PROY'], index=['CODIGO_PROYECTO','TIPO_CUENTA'], aggfunc='sum', fill_value=0)
df_ptd_cerrado_y_proyectado_ant2_2 = pd.DataFrame(df_ptd_cerrado_y_proyectado_ant2_2.to_records())

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant2_2 : ", df_ptd_cerrado_y_proyectado_ant2_2.shape)

df_ptd_cerrado_y_proyectado_ant2_2.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant2_2 :  (340, 8)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant2_2 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_PDT_PROY
78,P-1653,3. COSTO MANO DE OBRA,0.00
167,P-1683,4. COSTO SUPERVISIÓN,0.00
106,P-1666,3. COSTO MANO DE OBRA,-1957926.00
325,P-1726,2. COSTO MATERIALES,-24374046.00
329,P-1727,2. COSTO MATERIALES,-13253865.00


#### 4.15.2 MERGE ENTRE DATAFRAMES

In [429]:
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_2_2                  : ", df_ptd_cerrado3_2_2.shape)
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant2_2   : ", df_ptd_cerrado_y_proyectado_ant2_2.shape)

df_pdt_vs_proy_2 = df_ptd_cerrado3_2_2.merge(df_ptd_cerrado_y_proyectado_ant2_2, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pdt_vs_proy_2           : ", df_pdt_vs_proy_2.shape)

df_pdt_vs_proy_2.sample(5)

DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_2_2                  :  (340, 9)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant2_2   :  (340, 3)
DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pdt_vs_proy_2           :  (340, 10)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL_PDT,MARGEN,TOTAL_PDT_PROY
293,P-1695,4. COSTO SUPERVISIÓN,0.00,EDIFICIO ESTORIL,EMPRESA CONSTRUCTORA BRAVO E IZQUIERDO,0.00,0.00,0.00,0.02,0.00
192,P-1632,1. INGRESO,0.00,CHORRILLOS CLOROX,CONSTRUCTORA SUKSA S.A.,0.00,0.00,0.00,0.00,0.00
323,P-1722,2. COSTO MATERIALES,0.00,ECO CERRILLOS,CONSTRUCTORA ICF SPA,0.00,0.00,0.00,0.00,0.00
223,P-1653,4. COSTO SUPERVISIÓN,0.00,EDUARDO DE LA BARRA,CONSTRUCTORA E INVERSIONES VITAL SPA.,0.00,0.00,0.00,0.59,0.00
275,P-1683,3. COSTO MANO DE OBRA,0.00,CD BP-A05,EBCO S.A.,0.00,-287910.00,-287910.00,0.55,-287910.00


#### 4.15.3 CREACIÓN DE COLUMNA % DE AVANCE

In [430]:
df_pdt_vs_proy_2['% AVANCE'] = df_pdt_vs_proy_2['TOTAL_PDT'] / df_pdt_vs_proy_2['TOTAL_PDT_PROY']

# Reemplazar NaN por cero
df_pdt_vs_proy_2['% AVANCE'] = df_pdt_vs_proy_2['% AVANCE'].fillna(0)

# Reemplazar -inf por cero
df_pdt_vs_proy_2['% AVANCE'].replace(-np.inf, 0, inplace=True)

# Reemplazar inf por cero
df_pdt_vs_proy_2['% AVANCE'] = df_pdt_vs_proy_2['% AVANCE'].replace(np.inf, 0)

df_pdt_vs_proy_2.head(10)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL_PDT,MARGEN,TOTAL_PDT_PROY,% AVANCE
0,P-1557,3. COSTO MANO DE OBRA,0.00,ROSARIO ROSALES,EBCO S.A.,-945949.00,0.00,-945949.00,0.00,-1891898.00,0.50
1,P-1591,3. COSTO MANO DE OBRA,0.00,EDIFICIO ESTORIL,EBCO S.A.,-2483998.00,0.00,-2483998.00,0.00,-4967996.00,0.50
2,P-1592,2. COSTO MATERIALES,0.00,NUEVA COSTANERA 3300,EBCO S.A.,-9821.00,856004.00,846183.00,1.05,846183.00,1.00
3,P-1603,2. COSTO MATERIALES,0.00,FARELLONES,EMPRESA CONSTRUCTORA DLP S.A.,-30000.00,0.00,-30000.00,0.00,-30000.00,1.00
4,P-1611,2. COSTO MATERIALES,0.00,EDIFICIO PLAZA EGAÑA,EBCO S.A.,6648.00,596177.00,602825.00,1.14,602825.00,1.00
5,P-1612,2. COSTO MATERIALES,0.00,BARRIO PARQUE SANTIAGO 7,CONSTRUCTORA ALTURAS DEL SUR S.A,30258.00,280644.00,310902.00,0.00,310902.00,1.00
6,P-1620,2. COSTO MATERIALES,0.00,"OF APOQUINDO- HABILITACIONES, INTEGRACION Y AD...",CONSTRUCTORA INARCO S.A.,-16400.00,0.00,-16400.00,0.00,-16400.00,1.00
7,P-1626,2. COSTO MATERIALES,0.00,CLINICA BIO BIO,CONSTRUCTORA LYD S.A.,3460.00,678564.00,682024.00,10.03,682024.00,1.00
8,P-1632,2. COSTO MATERIALES,0.00,CHORRILLOS CLOROX,CONSTRUCTORA SUKSA S.A.,0.00,236593.00,236593.00,0.00,236593.00,1.00
9,P-1636,2. COSTO MATERIALES,0.00,SANTO DOMINGO 1410,DANACORP S.A,0.00,-455187.00,-455187.00,0.77,-455187.00,1.00


In [431]:
# TESTING VISUAL 1
 
df_pdt_vs_proy_2[df_pdt_vs_proy_2["CODIGO_PROYECTO"]=="P-1727"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL_PDT,MARGEN,TOTAL_PDT_PROY,% AVANCE
162,P-1727,3. COSTO MANO DE OBRA,0.00,NaN,NaN,-4714680.00,0.00,-4714680.00,0.23,-7919980.00,0.60
163,P-1727,2. COSTO MATERIALES,0.00,NaN,NaN,-3753865.00,0.00,-3753865.00,0.23,-13253865.00,0.28
164,P-1727,1. INGRESO,0.00,NaN,NaN,11005000.00,0.00,11005000.00,0.23,28260460.00,0.39
330,P-1727,4. COSTO SUPERVISIÓN,0.00,NaN,NaN,0.00,0.00,0.00,0.23,0.00,0.00


In [432]:
# TESTING VISUAL 2
 
df_pdt_vs_proy_2[df_pdt_vs_proy_2["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL_PDT,MARGEN,TOTAL_PDT_PROY,% AVANCE
106,P-1704,3. COSTO MANO DE OBRA,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-44701591.00,-87942380.00,-132643971.00,0.48,-146886506.00,0.90
107,P-1704,2. COSTO MATERIALES,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-58121771.00,-125594139.00,-183715910.00,0.48,-207715910.00,0.88
108,P-1704,1. INGRESO,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,141716199.00,465655611.00,607371810.00,0.48,675978469.28,0.90
302,P-1704,4. COSTO SUPERVISIÓN,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0.00,0.00,0.00,0.48,0.00,0.00


In [433]:
df_pdt_vs_proy_2.to_csv('archivos_respaldo/df_pdt_vs_proy_2.csv', index = False)

### 4.16 MERGE PTD CERRADO + PROYECTADO VS PRESUPUESTO

#### 4.16.1 EXPLORACIÓN Y PREPARACIÓN DE DATAFRAMES

In [434]:
# Exploración de dataframe df_ptd_cerrado_y_proyectado2

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado2 : ", df_ptd_cerrado_y_proyectado2.shape)

df_ptd_cerrado_y_proyectado2_2 = df_ptd_cerrado_y_proyectado2.copy()

df_ptd_cerrado_y_proyectado2_2.rename(columns={'TOTAL': 'TOTAL_PDT'}, inplace=True)
df_ptd_cerrado_y_proyectado2_2.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado2 :  (340, 8)


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL_PDT,MARGEN
188,P-1620,4. COSTO SUPERVISIÓN,"OF APOQUINDO- HABILITACIONES, INTEGRACION Y AD...",CONSTRUCTORA INARCO S.A.,0.00,0,0.00,0.00
61,P-1686,1. INGRESO,MACUL ICALMA,EMPRESA CONSTRUCTORA INGEVEC S.A.,99626406.00,5000000,104626406.00,0.24
153,P-1724,3. COSTO MANO DE OBRA,NaN,NaN,-506066.00,0,-506066.00,0.72
299,P-1700,4. COSTO SUPERVISIÓN,CD ORIZON,EBCO S.A.,0.00,0,0.00,0.24
34,P-1672,2. COSTO MATERIALES,STA CLARA (CCDD-TELECOM),CONSTRUCTORA UPC S.A.,-46171517.00,0,-46171517.00,0.31


In [435]:
# Exploración de dataframe presupuesto

df_presupuesto_final4_2 = df_presupuesto_final4.copy()
df_presupuesto_final4_2.rename(columns={'TOTAL': 'TOTAL_PDT_PREP'}, inplace=True)

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_presupuesto_final4_2 : ", df_presupuesto_final4_2.shape)

df_presupuesto_final4_2 = pd.pivot_table(df_presupuesto_final4_2, values=['TOTAL_PDT_PREP'], index=['CODIGO_PROYECTO','TIPO_CUENTA'], aggfunc='sum', fill_value=0)
df_presupuesto_final4_2 = pd.DataFrame(df_presupuesto_final4_2.to_records())

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_presupuesto_final4_2 : ", df_presupuesto_final4_2.shape)

df_presupuesto_final4_2.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_presupuesto_final4_2 :  (340, 8)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_presupuesto_final4_2 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_PDT_PREP
151,P-1679,4. COSTO SUPERVISIÓN,-6052803.00
84,P-1658,1. INGRESO,0.00
99,P-1663,4. COSTO SUPERVISIÓN,-2318715.00
27,P-1612,4. COSTO SUPERVISIÓN,-2259746.00
272,P-1713,1. INGRESO,123509706.24


#### 4.16.2 MERGE ENTRE DATAFRAMES

In [436]:
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado2_2 : ", df_ptd_cerrado_y_proyectado2_2.shape)
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_presupuesto_final4_2        : ", df_presupuesto_final4_2.shape)

df_pdt_proy_vs_prep = df_ptd_cerrado_y_proyectado2_2.merge(df_presupuesto_final4_2, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pdt_proy_vs_prep           : ", df_pdt_proy_vs_prep.shape)

df_pdt_proy_vs_prep.sample(5)

DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado2_2 :  (340, 8)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_presupuesto_final4_2        :  (340, 3)
DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pdt_proy_vs_prep           :  (340, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL_PDT,MARGEN,TOTAL_PDT_PREP
39,P-1675,1. INGRESO,PALENA II,CONSTRUCTORA MYA SPA.,17151215.00,0,17151215.00,0.27,17376021.17
77,P-1692,1. INGRESO,JOSÉ PEREZ COTAPOS,EBCO S.A.,85099786.00,6800000,91899786.00,-0.03,91720000.00
283,P-1688,4. COSTO SUPERVISIÓN,SUIZA,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...,0.00,0,0.00,0.20,-1849560.00
182,P-1611,4. COSTO SUPERVISIÓN,EDIFICIO PLAZA EGAÑA,EBCO S.A.,0.00,0,0.00,1.14,-2206785.00
259,P-1674,1. INGRESO,OFICINAS BEIERSDORF CHORRILLOS,CONSTRUCTORA SUKSA S.A.,23646019.00,0,23646019.00,0.63,28602126.00


#### 4.16.3 CREACIÓN DE COLUMNA % DE AVANCE

In [437]:
df_pdt_proy_vs_prep['% AVANCE'] = df_pdt_proy_vs_prep['TOTAL_PDT'] / df_pdt_proy_vs_prep['TOTAL_PDT_PREP']

# Reemplazar NaN por cero
df_pdt_proy_vs_prep['% AVANCE'] = df_pdt_proy_vs_prep['% AVANCE'].fillna(0)

# Reemplazar -inf por cero
df_pdt_proy_vs_prep['% AVANCE'].replace(-np.inf, 0, inplace=True)

# Reemplazar inf por cero
df_pdt_proy_vs_prep['% AVANCE'] = df_pdt_proy_vs_prep['% AVANCE'].replace(np.inf, 0)

df_pdt_proy_vs_prep.head(10)

,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL_PDT,MARGEN,TOTAL_PDT_PREP,% AVANCE
0,P-1557,3. COSTO MANO DE OBRA,ROSARIO ROSALES,EBCO S.A.,-1247014.00,-301065,-1548079.00,0.00,0.00,0.00
1,P-1591,3. COSTO MANO DE OBRA,EDIFICIO ESTORIL,EBCO S.A.,-4593960.00,-2109962,-6703922.00,0.00,0.00,0.00
2,P-1592,2. COSTO MATERIALES,NUEVA COSTANERA 3300,EBCO S.A.,897720.00,0,897720.00,1.05,0.00,0.00
3,P-1603,2. COSTO MATERIALES,FARELLONES,EMPRESA CONSTRUCTORA DLP S.A.,-30000.00,0,-30000.00,0.00,-13744197.00,0.00
4,P-1611,2. COSTO MATERIALES,EDIFICIO PLAZA EGAÑA,EBCO S.A.,602825.00,0,602825.00,1.14,-24208141.00,-0.02
5,P-1612,2. COSTO MATERIALES,BARRIO PARQUE SANTIAGO 7,CONSTRUCTORA ALTURAS DEL SUR S.A,310902.00,0,310902.00,0.00,-30609079.00,-0.01
6,P-1620,2. COSTO MATERIALES,"OF APOQUINDO- HABILITACIONES, INTEGRACION Y AD...",CONSTRUCTORA INARCO S.A.,-16400.00,0,-16400.00,0.00,-11746595.00,0.00
7,P-1626,2. COSTO MATERIALES,CLINICA BIO BIO,CONSTRUCTORA LYD S.A.,682024.00,0,682024.00,10.03,-168705405.50,-0.00
8,P-1632,2. COSTO MATERIALES,CHORRILLOS CLOROX,CONSTRUCTORA SUKSA S.A.,312558.00,0,312558.00,0.00,-219387084.00,-0.00
9,P-1636,2. COSTO MATERIALES,SANTO DOMINGO 1410,DANACORP S.A,-455187.00,0,-455187.00,0.77,-11381553.00,0.04


In [438]:
# TESTING VISUAL 1
 
df_pdt_proy_vs_prep[df_pdt_proy_vs_prep["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL_PDT,MARGEN,TOTAL_PDT_PREP,% AVANCE
106,P-1704,3. COSTO MANO DE OBRA,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-145534854.00,-12890883,-158425737.00,0.49,-156095149.00,1.01
107,P-1704,2. COSTO MATERIALES,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-197683461.00,-27000000,-224683461.00,0.49,-387816472.00,0.58
108,P-1704,1. INGRESO,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,686431807.00,62000000,748431807.00,0.49,1058718831.87,0.71
302,P-1704,4. COSTO SUPERVISIÓN,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0.00,0,0.00,0.49,-31281353.00,-0.00


In [439]:
# TESTING VISUAL 2
 
df_pdt_proy_vs_prep[df_pdt_proy_vs_prep["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL_PDT,MARGEN,TOTAL_PDT_PREP,% AVANCE
106,P-1704,3. COSTO MANO DE OBRA,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-145534854.00,-12890883,-158425737.00,0.49,-156095149.00,1.01
107,P-1704,2. COSTO MATERIALES,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-197683461.00,-27000000,-224683461.00,0.49,-387816472.00,0.58
108,P-1704,1. INGRESO,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,686431807.00,62000000,748431807.00,0.49,1058718831.87,0.71
302,P-1704,4. COSTO SUPERVISIÓN,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0.00,0,0.00,0.49,-31281353.00,-0.00


In [440]:
df_pdt_proy_vs_prep.to_csv('archivos_respaldo/df_pdt_proy_vs_prep.csv', index = False)

### 4.17 MERGE PTD CERRADO + PROYECTADO VS PRESUPUESTO (PERSPECTIVA MES ANTERIOR)

#### 4.17.1 EXPLORACIÓN Y PREPARACIÓN DE DATAFRAMES

In [441]:
# Exploración de dataframe df_ptd_cerrado_y_proyectado_ant2

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant2 : ", df_ptd_cerrado_y_proyectado_ant2.shape)

df_ptd_cerrado_y_proyectado_ant2_2 = df_ptd_cerrado_y_proyectado_ant2.copy()

df_ptd_cerrado_y_proyectado_ant2_2.rename(columns={'TOTAL': 'TOTAL_PDT'}, inplace=True)
df_ptd_cerrado_y_proyectado_ant2_2.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant2 :  (340, 8)


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL_PDT,MARGEN
320,P-1720,2. COSTO MATERIALES,NaN,NaN,0.00,0.00,0.00,0.00
228,P-1658,3. COSTO MANO DE OBRA,VIVIPRA,VICHERAT Y PRADENAS LTDA.,-319859.00,0.00,-319859.00,0.62
240,P-1664,3. COSTO MANO DE OBRA,MK II PCI,COMERCIAL K SPA.,0.00,0.00,0.00,0.64
3,P-1603,2. COSTO MATERIALES,FARELLONES,EMPRESA CONSTRUCTORA DLP S.A.,-30000.00,0.00,-30000.00,0.00
219,P-1649,3. COSTO MANO DE OBRA,INACAP VITACURA E2,CONSTRUCTORA CUEVAS Y PURCELL S.A.,-220289.00,0.00,-220289.00,0.56


In [442]:
# Exploración de dataframe presupuesto

df_presupuesto_final4_2_1 = df_presupuesto_final4.copy()
df_presupuesto_final4_2_1.rename(columns={'TOTAL': 'TOTAL_PDT_PREP'}, inplace=True)

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_presupuesto_final4_2 : ", df_presupuesto_final4_2.shape)

df_presupuesto_final4_2_1 = pd.pivot_table(df_presupuesto_final4_2_1, values=['TOTAL_PDT_PREP'], index=['CODIGO_PROYECTO','TIPO_CUENTA'], aggfunc='sum', fill_value=0)
df_presupuesto_final4_2_1 = pd.DataFrame(df_presupuesto_final4_2_1.to_records())

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_presupuesto_final4_2_1 : ", df_presupuesto_final4_2_1.shape)

df_presupuesto_final4_2_1.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_presupuesto_final4_2 :  (340, 3)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_presupuesto_final4_2_1 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_PDT_PREP
253,P-1706,2. COSTO MATERIALES,-39538656.00
278,P-1714,3. COSTO MANO DE OBRA,-7835147.00
283,P-1715,4. COSTO SUPERVISIÓN,-7424606.00
72,P-1649,1. INGRESO,0.00
293,P-1718,2. COSTO MATERIALES,-67771001.00


#### 4.17.2 MERGE ENTRE DATAFRAMES

In [443]:
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant2_2 : ", df_ptd_cerrado_y_proyectado_ant2_2.shape)
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_presupuesto_final4_2_1          : ", df_presupuesto_final4_2_1.shape)

df_pdt_proy_vs_prep_2 = df_ptd_cerrado_y_proyectado_ant2_2.merge(df_presupuesto_final4_2_1, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pdt_proy_vs_prep_2           : ", df_pdt_proy_vs_prep_2.shape)

df_pdt_proy_vs_prep_2.sample(5)

DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant2_2 :  (340, 8)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_presupuesto_final4_2_1          :  (340, 3)
DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pdt_proy_vs_prep_2           :  (340, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL_PDT,MARGEN,TOTAL_PDT_PREP
235,P-1661,4. COSTO SUPERVISIÓN,SAN VICENTE DE PAUL,DANACORP S.A,0.00,0.00,0.00,0.61,-2015626.00
241,P-1664,4. COSTO SUPERVISIÓN,MK II PCI,COMERCIAL K SPA.,0.00,0.00,0.00,0.64,-76676.00
75,P-1692,3. COSTO MANO DE OBRA,JOSÉ PEREZ COTAPOS,EBCO S.A.,-40393357.00,-7576783.00,-47970140.00,-0.06,-23173688.00
319,P-1719,4. COSTO SUPERVISIÓN,CD METALPAR,EBCO S.A.,0.00,0.00,0.00,1.00,0.00
170,P-1557,4. COSTO SUPERVISIÓN,ROSARIO ROSALES,EBCO S.A.,0.00,0.00,0.00,0.00,0.00


#### 4.17.3 CREACIÓN DE COLUMNA % DE AVANCE

In [444]:
df_pdt_proy_vs_prep_2['% AVANCE'] = df_pdt_proy_vs_prep_2['TOTAL_PDT'] / df_pdt_proy_vs_prep_2['TOTAL_PDT_PREP']

# Reemplazar NaN por cero
df_pdt_proy_vs_prep_2['% AVANCE'] = df_pdt_proy_vs_prep_2['% AVANCE'].fillna(0)

# Reemplazar -inf por cero
df_pdt_proy_vs_prep_2['% AVANCE'].replace(-np.inf, 0, inplace=True)

# Reemplazar inf por cero
df_pdt_proy_vs_prep_2['% AVANCE'] = df_pdt_proy_vs_prep_2['% AVANCE'].replace(np.inf, 0)

df_pdt_proy_vs_prep_2.head(10)

,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL_PDT,MARGEN,TOTAL_PDT_PREP,% AVANCE
0,P-1557,3. COSTO MANO DE OBRA,ROSARIO ROSALES,EBCO S.A.,-945949.00,-945949.00,-1891898.00,0.00,0.00,0.00
1,P-1591,3. COSTO MANO DE OBRA,EDIFICIO ESTORIL,EBCO S.A.,-2483998.00,-2483998.00,-4967996.00,0.00,0.00,0.00
2,P-1592,2. COSTO MATERIALES,NUEVA COSTANERA 3300,EBCO S.A.,846183.00,0.00,846183.00,1.05,0.00,0.00
3,P-1603,2. COSTO MATERIALES,FARELLONES,EMPRESA CONSTRUCTORA DLP S.A.,-30000.00,0.00,-30000.00,0.00,-13744197.00,0.00
4,P-1611,2. COSTO MATERIALES,EDIFICIO PLAZA EGAÑA,EBCO S.A.,602825.00,0.00,602825.00,1.14,-24208141.00,-0.02
5,P-1612,2. COSTO MATERIALES,BARRIO PARQUE SANTIAGO 7,CONSTRUCTORA ALTURAS DEL SUR S.A,310902.00,0.00,310902.00,0.00,-30609079.00,-0.01
6,P-1620,2. COSTO MATERIALES,"OF APOQUINDO- HABILITACIONES, INTEGRACION Y AD...",CONSTRUCTORA INARCO S.A.,-16400.00,0.00,-16400.00,0.00,-11746595.00,0.00
7,P-1626,2. COSTO MATERIALES,CLINICA BIO BIO,CONSTRUCTORA LYD S.A.,682024.00,0.00,682024.00,10.03,-168705405.50,-0.00
8,P-1632,2. COSTO MATERIALES,CHORRILLOS CLOROX,CONSTRUCTORA SUKSA S.A.,236593.00,0.00,236593.00,0.00,-219387084.00,-0.00
9,P-1636,2. COSTO MATERIALES,SANTO DOMINGO 1410,DANACORP S.A,-455187.00,0.00,-455187.00,0.77,-11381553.00,0.04


In [445]:
# TESTING VISUAL 1
 
df_pdt_proy_vs_prep_2[df_pdt_proy_vs_prep_2["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL_PDT,MARGEN,TOTAL_PDT_PREP,% AVANCE
106,P-1704,3. COSTO MANO DE OBRA,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-132643971.00,-14242535.00,-146886506.00,0.48,-156095149.00,0.94
107,P-1704,2. COSTO MATERIALES,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-183715910.00,-24000000.00,-207715910.00,0.48,-387816472.00,0.54
108,P-1704,1. INGRESO,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,607371810.00,68606659.28,675978469.28,0.48,1058718831.87,0.64
302,P-1704,4. COSTO SUPERVISIÓN,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0.00,0.00,0.00,0.48,-31281353.00,-0.00


In [446]:
df_pdt_proy_vs_prep_2.to_csv('archivos_respaldo/df_pdt_proy_vs_prep_2.csv', index = False)

# AQUÍ ESTOY

## 5. - EXPORTACIÓN DE ARCHIVOS

In [447]:
# Conexión a google drive

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

# Credenciales de GOOGLE API, en archivo .json
credentials = ServiceAccountCredentials.from_json_keyfile_name('keen-extension-358919-9214486a06be.json', scope) # Your json file here

gc = gspread.authorize(credentials)

## SIEMPRE DAR ACCESO A ESTE USUARIO: test-321@keen-extension-358919.iam.gserviceaccount.com

### 5.1 EXPORTACIÓN LIBRO MAYOR PARA CUADRO FINANCIERO DE PROYECTOS

In [448]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1COX0_p3dTPNLZMwoh__0uQdOYfP_AA1mw7lfKx4SMMQ')

# select a work sheet from its name
worksheet1 = gs.worksheet('Hoja 1')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe=df_libro_mayor6, include_index=False,
include_column_header=True, resize=True)

### 5.2 EXPORTACIÓN LIBRO MAYOR YTD HASTA MES CERRADO
Dataframe: df_ytd_cerrado3

In [449]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1_53Vj0j4xnqsWDdHDdetp3JmoUHRjLlvsUyi7tQXCY4')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_actual')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_ytd_cerrado3, include_index=False,
include_column_header=True, resize=True)

### 5.2.1 EXPORTACIÓN LIBRO MAYOR YTD HASTA MES CERRADO (MES ANTERIOR) PERSPECTIVA MES ANTERIOR
Dataframe: df_ytd_cerrado_2M_3

In [450]:
gs = gc.open_by_key('1_53Vj0j4xnqsWDdHDdetp3JmoUHRjLlvsUyi7tQXCY4')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_anterior')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_ytd_cerrado_2M_3, include_index=False,
include_column_header=True, resize=True)

### 5.3 EXPORTACIÓN MES EN CURSO PROYECTADO
Dataframe: mes_en_curso2

In [451]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1xyeMVO6VL0yBCX6aZRcFxNKPH5yum2o2gF_HM2NSLIs')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_actual')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= mes_en_curso_proy5, include_index=False,
include_column_header=True, resize=True)

### 5.3.1 EXPORTACIÓN MES EN CURSO PROYECTADO (MES ANTERIOR) PERSPECTIVA MES ANTERIOR
Dataframe: mes_en_curso_2M

In [452]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1xyeMVO6VL0yBCX6aZRcFxNKPH5yum2o2gF_HM2NSLIs')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_anterior')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= mes_en_curso_proy_2M_5, include_index=False,
include_column_header=True, resize=True)

### 5.4 EXPORTACIÓN MES EN CURSO

In [453]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1k4Yf_ycqV1TjqPcS0eDGyUU_oKu9JVzYUHw9shUBGTM')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_actual')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= mes_en_curso_real5, include_index=False,
include_column_header=True, resize=True)

### 5.4.1 EXPORTACIÓN MES EN CURSO PERSPECTIVA MES ANTERIOR

In [454]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1k4Yf_ycqV1TjqPcS0eDGyUU_oKu9JVzYUHw9shUBGTM')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_anterior')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= mes_en_curso_real_2M_5, include_index=False,
include_column_header=True, resize=True)

### 5.5 EXPORTACIÓN YTD PROYECTADO

In [455]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1_39cxdPhNrDcz2WJ-t3oLQRpgQPhWwFsopAGzcmxuuk')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_actual')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_ytd_proyectado2, include_index=False,
include_column_header=True, resize=True)

### 5.5.1 EXPORTACIÓN YTD PROYECTADO PERSPECTIVA MES ANTERIOR

In [456]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1_39cxdPhNrDcz2WJ-t3oLQRpgQPhWwFsopAGzcmxuuk')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_anterior')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_ytd_proyectado_2M_2, include_index=False,
include_column_header=True, resize=True)

### 5.6 EXPORTACIÓN PTD CERRADO

In [457]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1bMRHlz-oMTZ7ccBXbEvjQ3es7mBcWxZqpVrPBXGAVjc')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_actual')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_ptd_cerrado3, include_index=False,
include_column_header=True, resize=True)

### 5.6.1 EXPORTACIÓN PTD CERRADO (PERSPECTIVA MES ANTERIOR)

In [458]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1bMRHlz-oMTZ7ccBXbEvjQ3es7mBcWxZqpVrPBXGAVjc')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_anterior')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_ptd_cerrado_anterior3, include_index=False,
include_column_header=True, resize=True)

### 5.7 EXPORTACIÓN PTD CERRADO + PROYECTADO

In [459]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('14dw0WLmoa8gDJLcOkDJor17BCy7pozNQK7L1bSvJQqs')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_actual')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_ptd_cerrado_y_proyectado2, include_index=False,
include_column_header=True, resize=True)

### 5.7.1 EXPORTACIÓN PTD CERRADO + PROYECTADO (PERSPECTIVA MES ANTERIOR)

In [460]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('14dw0WLmoa8gDJLcOkDJor17BCy7pozNQK7L1bSvJQqs')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_anterior')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_ptd_cerrado_y_proyectado_ant2, include_index=False,
include_column_header=True, resize=True)

### 5.8 EXPORTACIÓN PRESUPUESTO

In [461]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1yTtFi7x0rc_FU6xb-FrdQGKndttaf0dg3ENDyOO52eM')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_actual')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_presupuesto_final4, include_index=False,
include_column_header=True, resize=True)

### 5.9 EXPORTACIÓN PENDIENTE

In [462]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1qLDVshnNP-whjDaMYi1N4QSeeD6AK4EVSQtQx6Y438o')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_actual')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_pendiente3, include_index=False,
include_column_header=True, resize=True)

### 5.10 EXPORTACIÓN PENDIENTE (PERSPECTIVA MES ANTERIOR)

In [463]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1qLDVshnNP-whjDaMYi1N4QSeeD6AK4EVSQtQx6Y438o')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_anterior')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_pendiente_3, include_index=False,
include_column_header=True, resize=True)

### 5.11 EXPORTACIÓN PTD Cerrado vs PTD Cerrado + Proyectado

In [464]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1ZsZGZ5uzyEK7rOqHXER0MYjKbrbiLK9cdeYlXk4t8oE')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_actual')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_pdt_vs_proy, include_index=False,
include_column_header=True, resize=True)

### 5.12 EXPORTACIÓN PTD Cerrado vs PTD Cerrado + Proyectado (PERSPECTIVA MES ANTERIOR)

In [465]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1ZsZGZ5uzyEK7rOqHXER0MYjKbrbiLK9cdeYlXk4t8oE')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_anterior')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_pdt_vs_proy_2, include_index=False,
include_column_header=True, resize=True)

### 5.13 EXPORTACIÓN  PTD Cerrado + Proyectado vs Presupuesto

In [466]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1yqyEJ4mkcaS_kI7rrNZ8reRxqFhWdbhBtvDcSeWxkPo')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_actual')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_pdt_proy_vs_prep, include_index=False,
include_column_header=True, resize=True)

### 5.13 EXPORTACIÓN  PTD Cerrado + Proyectado vs Presupuesto (PERSPECTIVA ANTERIOR)

In [467]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1yqyEJ4mkcaS_kI7rrNZ8reRxqFhWdbhBtvDcSeWxkPo')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_anterior')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_pdt_proy_vs_prep_2, include_index=False,
include_column_header=True, resize=True)